# Import Packages

In [1]:
import json
import requests
from bs4 import BeautifulSoup as BS
import regex as re
import string
import time
import pandas as pd
import math

# Parse Quick List of Ted Talks
### Get date, speaker, title, url, and length, return as dataframe

In [2]:
def parse_talk_list(last_page):
    all_dates = []
    all_speakers = []
    all_titles = []
    all_links = []
    all_lengths = []
    
    for p in range(1, last_page + 1):
        url = "https://www.ted.com/talks/quick-list?page=" + str(p)
        try:
            print('You are trying to parse page ' + str(p))
            page = requests.get(url)
            content = BS(page.content, 'html.parser')
            print('Successfully parsed page ' + str(p))
            
            # Get date of talk
            dates = content.find_all(class_ = 'meta')
            date_str = [x.text.strip('\n') for x in dates if ' ' in x.text]
            all_dates.extend(date_str)
            
            
            # Get speaker
            speaker_title = content.find_all(class_ = 'l3')
            speaker_str = [x.text.strip('\n').split(': ')[0] for x in speaker_title]
            all_speakers.extend(speaker_str)
            
            # Get title
            title_str = [x.text.strip('\n').split(': ')[1:] for x in speaker_title]
            for x in title_str:
                title = ''
                for string in x:
                    title += string
                all_titles.append(title)
            
            
            # Get url
            urls = content.find_all('a', href = True)
            links = [url['href'] for url in urls if url['href'].startswith('/talks/') and 'page=' not in url['href']]
            all_links.extend(links)
            
            # Get length
            length = content.find_all(class_ = "col-xs-1")
            length_str = [re.sub(r"\s+", '', x.text) for x in length if ':' in x.text or ('h' and 'm' in x.text)]
            length_str = [str(int(x.split('h')[0])*60 + int(x.split('h')[1].split('m')[0])) + ':00' if 'h' in x else x for x in length_str]
            all_lengths.extend(length_str)
            
            # Let program pause to prevent too many calls
            time.sleep(3)
        except:
            print('You failed to parse page' + p)
            
    
    return pd.DataFrame(list(zip(all_dates, all_speakers, all_titles, all_links, all_lengths)), columns = ['date', 'speaker', 'title', 'url', 'length'])

In [3]:
talk_df = parse_talk_list(118)

You are trying to parse page 1
Successfully parsed page 1
You are trying to parse page 2
Successfully parsed page 2
You are trying to parse page 3
Successfully parsed page 3
You are trying to parse page 4
Successfully parsed page 4
You are trying to parse page 5
Successfully parsed page 5
You are trying to parse page 6
Successfully parsed page 6
You are trying to parse page 7
Successfully parsed page 7
You are trying to parse page 8
Successfully parsed page 8
You are trying to parse page 9
Successfully parsed page 9
You are trying to parse page 10
Successfully parsed page 10
You are trying to parse page 11
Successfully parsed page 11
You are trying to parse page 12
Successfully parsed page 12
You are trying to parse page 13
Successfully parsed page 13
You are trying to parse page 14
Successfully parsed page 14
You are trying to parse page 15
Successfully parsed page 15
You are trying to parse page 16
Successfully parsed page 16
You are trying to parse page 17
Successfully parsed page 1

In [4]:
talk_df.shape

(4244, 5)

In [33]:
talk_df.head()

,date,speaker,title,url,length
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49


In [5]:
# Save dataframe to csv
talk_df.to_csv('ted_links.csv')

# Get Description of Talk Using URLs
### Return as list of talk descriptions

In [155]:
def get_descr(start, end):
    talk_descr = []
    for url in talk_df['url'][start:end]:
        link = "https://www.ted.com" + url
        try:
            page = requests.get(link)
            content = BS(page.content, 'html.parser')
            talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
            print('appended description for ' + link)
            time.sleep(3)
        except:
            print('unable to append description @ ' + link)
    return talk_descr

In [ ]:
talk_descr1000 = get_descr(0, 1000)
talk_descr2000 = get_descr(1000, 2000)
talk_descr3000 = get_descr(3000, 4000)
talk_descr4000 = get_descr(4000, len(talk_df))

In [18]:
talk_descr = []
links_appended = 0
for url in talk_df['url'][0:250]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)

appended description for https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator
appended description for https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol
appended description for https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women
appended description for https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear
appended description for https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success
appended description for https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time
appended description for https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism
appended description for https://www.ted.com/talks/nick_bostrom_how_civilization_could_destroy_itself_and_4_ways_we_could_prevent_it
appended description for https://www.ted.com/talks/jasmine_crowe_what_we_re_getting_wrong_in_the_fight_to_end_hunger
a

appended description for https://www.ted.com/talks/paul_a_kramer_our_immigration_conversation_is_broken_here_s_how_to_have_a_better_one
appended description for https://www.ted.com/talks/laura_rovner_what_happens_to_people_in_solitary_confinement
appended description for https://www.ted.com/talks/lucas_husted_game_theory_challenge_can_you_predict_human_behavior
appended description for https://www.ted.com/talks/tashka_and_laura_yawanawa_the_amazon_belongs_to_humanity_let_s_protect_it_together
appended description for https://www.ted.com/talks/david_carroll_how_i_sued_cambridge_analytica_over_my_personal_data
appended description for https://www.ted.com/talks/bhakti_sharma_what_open_water_swimming_taught_me_about_resilience
appended description for https://www.ted.com/talks/bill_keegan_the_taino_myth_of_the_cursed_creator
appended description for https://www.ted.com/talks/bob_langert_the_business_case_for_working_with_your_toughest_critics
appended description for https://www.ted.com/ta

appended description for https://www.ted.com/talks/mitchell_katz_what_the_us_health_care_system_assumes_about_you
appended description for https://www.ted.com/talks/densho_ugly_history_japanese_american_incarceration_camps
appended description for https://www.ted.com/talks/moreangels_mbizah_how_community_led_conservation_can_save_wildlife
appended description for https://www.ted.com/talks/mohammad_modarres_why_you_should_shop_at_your_local_farmers_market
appended description for https://www.ted.com/talks/alex_rosenthal_the_prison_break_think_like_a_coder_ep_1
appended description for https://www.ted.com/talks/sarah_sze_how_we_experience_time_and_memory_through_art
appended description for https://www.ted.com/talks/antonio_garcia_vicente_porque_le_enseno_a_ninos_como_yo_a_programar
appended description for https://www.ted.com/talks/muthoni_drummer_queen_creativity_builds_nations
appended description for https://www.ted.com/talks/sam_van_aken_how_one_tree_grows_40_different_kinds_of_frui

appended description for https://www.ted.com/talks/soraya_field_fiorio_the_murder_of_ancient_alexandria_s_greatest_scholar
appended description for https://www.ted.com/talks/marc_bamuthi_joseph_you_have_the_rite
appended description for https://www.ted.com/talks/margaret_heffernan_the_human_skills_we_need_in_an_unpredictable_world
appended description for https://www.ted.com/talks/victor_vescovo_what_s_at_the_bottom_of_the_ocean_and_how_we_re_getting_there
appended description for https://www.ted.com/talks/nanfu_wang_what_it_was_like_to_grow_up_under_china_s_one_child_policy
appended description for https://www.ted.com/talks/nicola_sturgeon_what_brexit_means_for_scotland
appended description for https://www.ted.com/talks/melissa_marquez_what_sharks_taught_me_about_being_a_scientist
appended description for https://www.ted.com/talks/ethan_mann_how_sharks_could_inspire_a_new_generation_of_medical_devices
appended description for https://www.ted.com/talks/paul_cater_deaton_are_our_cultura

In [19]:
len(talk_descr)

250

In [11]:
for url in talk_df['url'][250:500]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)

appended description for https://www.ted.com/talks/ella_al_shamahi_the_fascinating_and_dangerous_places_scientists_aren_t_exploring
appended description for https://www.ted.com/talks/sharmeen_obaid_chinoy_how_film_transforms_the_way_we_see_the_world
appended description for https://www.ted.com/talks/amy_padnani_how_we_re_honoring_people_overlooked_by_history
appended description for https://www.ted.com/talks/jamie_paik_origami_robots_that_reshape_and_transform_themselves
appended description for https://www.ted.com/talks/emma_bryce_what_is_hpv_and_how_can_you_protect_yourself_from_it
appended description for https://www.ted.com/talks/rick_doblin_the_future_of_psychedelic_assisted_psychotherapy
appended description for https://www.ted.com/talks/emma_lazarus_new_colossus
appended description for https://www.ted.com/talks/barbara_j_king_grief_and_love_in_the_animal_kingdom
appended description for https://www.ted.com/talks/jasmine_cho_cookies_as_a_form_of_activism
appended description for

appended description for https://www.ted.com/talks/alex_gendler_why_should_you_read_the_master_and_margarita
appended description for https://www.ted.com/talks/erika_hamden_what_it_takes_to_launch_a_telescope
appended description for https://www.ted.com/talks/baratunde_thurston_how_to_deconstruct_racism_one_headline_at_a_time
appended description for https://www.ted.com/talks/maeve_higgins_why_the_good_immigrant_is_a_bad_narrative
appended description for https://www.ted.com/talks/galit_ariel_how_ar_can_make_us_feel_more_connected_to_the_world
appended description for https://www.ted.com/talks/beth_mortimer_and_tarje_nissen_meyer_the_enigmatic_language_of_elephants
appended description for https://www.ted.com/talks/nivruti_rai_an_open_source_database_to_create_guardian_angel_ai
appended description for https://www.ted.com/talks/chitra_aiyar_how_to_build_community_when_you_feel_isolated
appended description for https://www.ted.com/talks/stephen_doyle_art_that_brings_words_to_life
append

appended description for https://www.ted.com/talks/danielle_n_lee_how_hip_hop_helps_us_understand_science
appended description for https://www.ted.com/talks/sheperd_doeleman_inside_the_black_hole_image_that_made_history
appended description for https://www.ted.com/talks/jack_dorsey_how_twitter_needs_to_change
appended description for https://www.ted.com/talks/sarvesh_subramanian_global_warming_the_app
appended description for https://www.ted.com/talks/carole_cadwalladr_facebook_s_role_in_brexit_and_the_threat_to_democracy
appended description for https://www.ted.com/talks/christoph_keplinger_the_artificial_muscles_that_will_power_robots_of_the_future
appended description for https://www.ted.com/talks/chika_okoro_how_colorism_shapes_our_standards_of_beauty
appended description for https://www.ted.com/talks/harald_eia_where_in_the_world_is_it_easiest_to_get_rich
appended description for https://www.ted.com/talks/maria_golubeva_why_medieval_politics_was_nothing_like_game_of_thrones
append

appended description for https://www.ted.com/talks/dr_turi_king_king_richard_iii_solving_a_500_year_old_cold_case
appended description for https://www.ted.com/talks/keith_lowe_why_we_need_to_stop_obsessing_over_world_war_ii
appended description for https://www.ted.com/talks/juliet_brophy_how_a_new_species_of_ancestors_is_changing_our_theory_of_human_evolution
appended description for https://www.ted.com/talks/jordana_moore_saggese_the_chaotic_brilliance_of_artist_jean_michel_basquiat
appended description for https://www.ted.com/talks/farida_nabourema_is_your_country_at_risk_of_becoming_a_dictatorship_here_s_how_to_know
appended description for https://www.ted.com/talks/pat_crowley_farming_insects_to_save_the_world
appended description for https://www.ted.com/talks/karl_skjonnemand_the_self_assembling_computer_chips_of_the_future
appended description for https://www.ted.com/talks/daniel_finkel_can_you_solve_the_jail_break_riddle
appended description for https://www.ted.com/talks/lindy_l

In [20]:
talk_descr.extend(talk_descr_)

In [21]:
len(talk_descr)

500

In [26]:
for url in talk_df['url'][500:750]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)

appended description for https://www.ted.com/talks/yannick_roudaut_how_today_s_truths_shape_tomorrow_s_possibilities
appended description for https://www.ted.com/talks/michael_benton_mass_extinctions_and_the_future_of_life_on_earth
appended description for https://www.ted.com/talks/lindiwe_mazibuko_why_the_african_diaspora_is_crucial_to_the_continent_s_future
appended description for https://www.ted.com/talks/juan_enriquez_the_age_of_genetic_wonder_nov_2019
appended description for https://www.ted.com/talks/anne_gaskett_the_sexual_deception_of_orchids
appended description for https://www.ted.com/talks/aja_monet_and_phillip_agnew_a_love_story_about_the_power_of_art_as_organizing
appended description for https://www.ted.com/talks/glenn_cantave_vr_ar_for_social_change
appended description for https://www.ted.com/talks/steven_petrow_3_ways_to_practice_civility
appended description for https://www.ted.com/talks/christina_greer_notes_of_a_native_son_the_world_according_to_james_baldwin
appen

appended description for https://www.ted.com/talks/tony_luciani_a_mother_and_son_s_photographic_journey_through_dementia
appended description for https://www.ted.com/talks/matt_mullenweg_why_working_from_home_is_good_for_business
appended description for https://www.ted.com/talks/maryn_mckenna_how_antibiotics_changed_our_food_and_how_we_ll_change_it_back
appended description for https://www.ted.com/talks/shohini_ghose_a_beginner_s_guide_to_quantum_computing
appended description for https://www.ted.com/talks/alex_gendler_can_you_solve_the_multiplying_rabbits_riddle
appended description for https://www.ted.com/talks/karissa_sanbonmatsu_the_biology_of_gender_from_dna_to_the_brain
appended description for https://www.ted.com/talks/danae_wolfe_ew_to_awe_your_view_of_bugs_may_never_be_the_same
appended description for https://www.ted.com/talks/renzo_vitale_what_should_electric_cars_sound_like
appended description for https://www.ted.com/talks/chiki_sarkar_how_india_s_smartphone_revolution_is

appended description for https://www.ted.com/talks/julia_dhar_how_to_disagree_productively_and_find_common_ground
appended description for https://www.ted.com/talks/arindam_bhattacharya_globalization_isn_t_declining_it_s_transforming
appended description for https://www.ted.com/talks/frank_muller_pierstorff_circular_escape
appended description for https://www.ted.com/talks/viola_llewellyn_a_new_model_of_microfinance_for_africa_and_beyond
appended description for https://www.ted.com/talks/a_dara_dotz_an_ingenious_solution_for_aid_in_disaster_zones
appended description for https://www.ted.com/talks/jim_whitehurst_what_i_learned_from_giving_up_everything_i_knew_as_a_leader
appended description for https://www.ted.com/talks/francois_candelon_how_do_we_stop_the_massive_inequality_dominating_the_world_economy
appended description for https://www.ted.com/talks/alexander_belcredi_how_a_long_forgotten_virus_could_help_us_solve_the_antibiotics_crisis
appended description for https://www.ted.com/

appended description for https://www.ted.com/talks/ashwini_anburajan_how_cryptocurrency_can_help_start_ups_get_investment_capital
appended description for https://www.ted.com/talks/alex_honnold_how_i_climbed_a_3_000_foot_vertical_cliff_without_ropes
appended description for https://www.ted.com/talks/mindy_fullilove_how_we_start_envisioning_a_future_where_all_people_are_created_and_live_as_equals
appended description for https://www.ted.com/talks/michel_dugon_the_secrets_of_spider_venom
appended description for https://www.ted.com/talks/camille_a_brown_new_second_line
appended description for https://www.ted.com/talks/magatte_wade_why_it_s_too_hard_to_start_a_business_in_africa_and_how_to_change_it
appended description for https://www.ted.com/talks/joshua_w_pate_the_fascinating_science_of_phantom_limbs
appended description for https://www.ted.com/talks/melinda_epler_3_ways_to_be_a_better_ally_in_the_workplace
appended description for https://www.ted.com/talks/christine_porath_why_being_

In [27]:
len(talk_descr)

750

In [28]:
for url in talk_df['url'][750:1000]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)

appended description for https://www.ted.com/talks/luke_sital_singh_afterneath_killing_me
appended description for https://www.ted.com/talks/chip_colwell_why_museums_are_returning_cultural_treasures
appended description for https://www.ted.com/talks/fabio_pacucci_could_the_earth_be_swallowed_by_a_black_hole
appended description for https://www.ted.com/talks/kaitlyn_sadtler_how_we_could_teach_our_bodies_to_heal_faster
appended description for https://www.ted.com/talks/niels_van_namen_why_the_hospital_of_the_future_will_be_your_own_home
appended description for https://www.ted.com/talks/kate_stone_the_press_trampled_on_my_privacy_here_s_how_i_took_back_my_story
appended description for https://www.ted.com/talks/scott_peeples_why_should_you_read_edgar_allan_poe
appended description for https://www.ted.com/talks/pat_ferrucci_sports_journalists_stereotype_athletes_by_race_and_we_do_too
appended description for https://www.ted.com/talks/sian_leah_beilock_why_we_choke_under_pressure_and_how_t

appended description for https://www.ted.com/talks/mark_robinson_why_is_aristophanes_called_the_father_of_comedy
appended description for https://www.ted.com/talks/stephen_deberry_why_the_wrong_side_of_the_tracks_is_usually_the_east_side_of_cities
appended description for https://www.ted.com/talks/chetna_gala_sinha_how_women_in_rural_india_turned_courage_into_capital
appended description for https://www.ted.com/talks/yelle_interpassion_ba_in
appended description for https://www.ted.com/talks/walter_hood_how_urban_spaces_can_preserve_history_and_build_community
appended description for https://www.ted.com/talks/hasini_jayatilaka_how_cancer_cells_communicate_and_how_we_can_slow_them_down
appended description for https://www.ted.com/talks/puqun_li_zen_koans_unsolvable_enigmas_designed_to_break_your_brain
appended description for https://www.ted.com/talks/dk_osseo_asare_what_a_scrapyard_in_ghana_can_teach_us_about_innovation
appended description for https://www.ted.com/talks/mary_maker_why

appended description for https://www.ted.com/talks/gordon_hamilton_can_you_solve_the_mondrian_squares_riddle
appended description for https://www.ted.com/talks/travis_rieder_the_agony_of_opioid_withdrawal_and_what_doctors_should_tell_patients_about_it
appended description for https://www.ted.com/talks/elizabeth_cox_what_causes_hallucinations
appended description for https://www.ted.com/talks/andrew_youn_what_if_we_supported_millions_of_african_farmers_in_growing_more_food
appended description for https://www.ted.com/talks/karen_j_meech_the_story_of_oumuamua_the_first_visitor_from_another_star_system
appended description for https://www.ted.com/talks/jackie_gartner_schmidt_phd_your_voice_says_a_lot_about_you_whether_you_like_it_or_not
appended description for https://www.ted.com/talks/ian_firth_bridges_should_be_beautiful
appended description for https://www.ted.com/talks/paul_rucker_the_symbols_of_systemic_racism_and_how_to_take_away_their_power
appended description for https://www.ted

appended description for https://www.ted.com/talks/kirsty_duncan_scientists_must_be_free_to_learn_to_speak_and_to_challenge
appended description for https://www.ted.com/talks/rola_hallam_the_doctors_nurses_and_aid_workers_rebuilding_syria
appended description for https://www.ted.com/talks/sarah_montana_the_real_risk_of_forgiveness_and_why_it_s_worth_it
appended description for https://www.ted.com/talks/kate_raworth_a_healthy_economy_should_be_designed_to_thrive_not_grow
appended description for https://www.ted.com/talks/emily_nagoski_the_truth_about_unwanted_arousal
appended description for https://www.ted.com/talks/thandiswa_mazwai_iyeza_zabalaza
appended description for https://www.ted.com/talks/lb_hannahs_what_it_s_like_to_be_a_transgender_dad
appended description for https://www.ted.com/talks/simone_giertz_why_you_should_make_useless_things
appended description for https://www.ted.com/talks/sarah_murray_a_playful_solution_to_the_housing_crisis
appended description for https://www.t

In [29]:
for url in talk_df['url'][1000:1250]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
appended description for https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
appended description for https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
appended description for https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
appended description for https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
appended description for https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
appended description for https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment
appended description for https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell
appended description for https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_

appended description for https://www.ted.com/talks/marc_bamuthi_joseph_what_soccer_can_teach_us_about_freedom
appended description for https://www.ted.com/talks/minda_dentler_what_i_learned_when_i_conquered_the_world_s_toughest_triathlon
appended description for https://www.ted.com/talks/bill_bernat_how_to_connect_with_depressed_friends
appended description for https://www.ted.com/talks/felice_belle_and_jennifer_murphy_how_we_became_sisters
appended description for https://www.ted.com/talks/shameem_akhtar_to_learn_is_to_be_free
appended description for https://www.ted.com/talks/dustin_schroeder_how_we_look_kilometers_below_the_antarctic_ice_sheet
appended description for https://www.ted.com/talks/shawn_martinbrough_how_one_black_panther_comic_artist_is_changing_perceptions
appended description for https://www.ted.com/talks/wendy_suzuki_the_brain_changing_benefits_of_exercise
appended description for https://www.ted.com/talks/chuck_nice_a_funny_look_at_the_unintended_consequences_of_tec

appended description for https://www.ted.com/talks/laxmi_parida_what_if_the_best_fertilizer_was_an_algorithm
appended description for https://www.ted.com/talks/juliane_gallina_a_library_of_minds
appended description for https://www.ted.com/talks/james_kozloski_why_the_brain_is_simpler_than_we_think
appended description for https://www.ted.com/talks/ise_lyfe_we_are_not_mud
appended description for https://www.ted.com/talks/bob_nease_how_to_trick_yourself_into_good_behavior
appended description for https://www.ted.com/talks/andrew_arruda_the_world_s_first_ai_legal_assistant
appended description for https://www.ted.com/talks/valarie_kaur_3_lessons_of_revolutionary_love_in_a_time_of_rage
appended description for https://www.ted.com/talks/tito_deler_my_fine_reward
appended description for https://www.ted.com/talks/zachariah_mampilly_how_protest_is_redefining_democracy_around_the_world
appended description for https://www.ted.com/talks/mohamad_jebara_this_company_pays_kids_to_do_their_math_h

appended description for https://www.ted.com/talks/deb_willis_and_hank_willis_thomas_a_mother_and_son_united_by_love_and_art
appended description for https://www.ted.com/talks/yvette_alberdingk_thijm_the_power_of_citizen_video_to_create_undeniable_truths
appended description for https://www.ted.com/talks/tasha_eurich_increase_your_self_awareness_with_one_simple_fix
appended description for https://www.ted.com/talks/natalie_fee_why_plastic_pollution_is_personal
appended description for https://www.ted.com/talks/joel_jackson_a_vehicle_built_in_africa_for_africa
appended description for https://www.ted.com/talks/tiffany_watt_smith_the_history_of_human_emotions
appended description for https://www.ted.com/talks/javed_akhtar_the_gift_of_words
appended description for https://www.ted.com/talks/atul_gawande_want_to_get_great_at_something_get_a_coach
appended description for https://www.ted.com/talks/angela_wang_how_china_is_changing_the_future_of_shopping
appended description for https://www.

In [31]:
len(talk_descr)

1249

In [35]:
for i in range(1000, 1250):
    if talk_df['speaker'][i] == 'James Orsulak':
        print(i)

1220


In [37]:
talk_descr[1221:] = talk_descr[1220:]

In [40]:
link = "https://www.ted.com" + talk_df['url'][1220]
page = requests.get(link)
content = BS(page.content, 'html.parser')

In [43]:
talk_descr[1220] = content.find_all('meta', property = 'og:description')[0]['content']

In [44]:
len(talk_descr)

1250

In [45]:
for url in talk_df['url'][1250:1500]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/wolf_a_marriage_of_opposites_krump_meets_mozart
appended description for https://www.ted.com/talks/philipp_gerbert_the_basics_of_ai_for_business
appended description for https://www.ted.com/talks/kevin_bethune_the_4_superpowers_of_design
appended description for https://www.ted.com/talks/erica_joy_baker_how_do_we_bridge_the_anxiety_gap_at_work
appended description for https://www.ted.com/talks/michael_ringel_external_innovation_basics_from_an_r_d_expert
appended description for https://www.ted.com/talks/justin_baldoni_why_i_m_done_trying_to_be_man_enough
appended description for https://www.ted.com/talks/luvvie_ajayi_get_comfortable_with_being_uncomfortable
appended description for https://www.ted.com/talks/g_t_bynum_a_republican_mayor_s_plan_to_replace_partisanship_with_policy
appended description for https://www.ted.com/talks/sebastian_thrun_and_chris_anderson_what_ai_is_and_isn_t
appended description for https://www.ted.com/talks/na

appended description for https://www.ted.com/talks/helen_pearson_lessons_from_the_longest_study_on_human_development
appended description for https://www.ted.com/talks/euna_lee_what_i_learned_as_a_prisoner_in_north_korea
appended description for https://www.ted.com/talks/anna_heringer_the_warmth_and_wisdom_of_mud_buildings
appended description for https://www.ted.com/talks/julio_gil_future_tech_will_give_you_the_benefits_of_city_life_anywhere
appended description for https://www.ted.com/talks/nabila_alibhai_why_people_of_different_faiths_are_painting_their_houses_of_worship_yellow
appended description for https://www.ted.com/talks/mei_lin_neo_the_fascinating_secret_lives_of_giant_clams
appended description for https://www.ted.com/talks/anindya_kundu_the_boost_students_need_to_overcome_obstacles
appended description for https://www.ted.com/talks/karoliina_korppoo_how_a_video_game_might_help_us_build_better_cities
appended description for https://www.ted.com/talks/theo_e_j_wilson_a_black

appended description for https://www.ted.com/talks/noah_feldman_hamilton_vs_madison_and_the_birth_of_american_partisanship
appended description for https://www.ted.com/talks/tricia_wang_the_human_insights_missing_from_big_data
appended description for https://www.ted.com/talks/anil_seth_your_brain_hallucinates_your_conscious_reality
appended description for https://www.ted.com/talks/kate_marvel_can_clouds_buy_us_more_time_to_solve_climate_change
appended description for https://www.ted.com/talks/adam_alter_why_our_screens_make_us_less_happy
appended description for https://www.ted.com/talks/liz_hajek_what_rivers_can_tell_us_about_the_earth_s_history
appended description for https://www.ted.com/talks/jorge_ramos_why_journalists_have_an_obligation_to_challenge_power
appended description for https://www.ted.com/talks/rabbi_lord_jonathan_sacks_how_we_can_face_the_future_without_fear_together
appended description for https://www.ted.com/talks/manu_prakash_lifesaving_scientific_tools_made_of

appended description for https://www.ted.com/talks/jonathan_marks_in_praise_of_conflict
appended description for https://www.ted.com/talks/emma_bryce_why_do_we_itch
appended description for https://www.ted.com/talks/ari_wallach_3_ways_to_plan_for_the_very_long_term
appended description for https://www.ted.com/talks/giorgia_lupi_how_we_can_find_ourselves_in_data
appended description for https://www.ted.com/talks/david_r_williams_how_racism_makes_us_sick
appended description for https://www.ted.com/talks/sebastian_bortnik_the_conversation_we_re_not_having_about_digital_child_abuse
appended description for https://www.ted.com/talks/katie_bouman_how_to_take_a_picture_of_a_black_hole
appended description for https://www.ted.com/talks/gretchen_carlson_david_brooks_political_common_ground_in_a_polarized_united_states
appended description for https://www.ted.com/talks/casey_brown_know_your_worth_and_then_ask_for_it
appended description for https://www.ted.com/talks/lisa_winer_can_you_solve_the

In [46]:
len(talk_descr)

1500

In [47]:
for url in talk_df['url'][1500:1750]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/karina_galperin_should_we_simplify_spelling
appended description for https://www.ted.com/talks/deanna_pucciarelli_the_history_of_chocolate
appended description for https://www.ted.com/talks/peggy_orenstein_what_young_women_believe_about_their_own_sexual_pleasure
appended description for https://www.ted.com/talks/carrie_nugent_adventures_of_an_asteroid_hunter
appended description for https://www.ted.com/talks/caitlin_doughty_a_burial_practice_that_nourishes_the_planet
appended description for https://www.ted.com/talks/amy_adkins_the_myth_of_icarus_and_daedalus
appended description for https://www.ted.com/talks/john_koenig_beautiful_new_words_to_describe_obscure_emotions
appended description for https://www.ted.com/talks/joy_buolamwini_how_i_m_fighting_bias_in_algorithms
appended description for https://www.ted.com/talks/jude_kelly_why_women_should_tell_the_stories_of_humanity
appended description for https://www.ted.com/talks/caroline_p

appended description for https://www.ted.com/talks/jia_jiang_what_i_learned_from_100_days_of_rejection
appended description for https://www.ted.com/talks/natalie_panek_let_s_clean_up_the_space_junk_orbiting_earth
appended description for https://www.ted.com/talks/tiq_milan_and_kim_katrin_milan_a_queer_vision_of_love_and_marriage
appended description for https://www.ted.com/talks/kate_adams_4_larger_than_life_lessons_from_soap_operas
appended description for https://www.ted.com/talks/ryan_gravel_how_an_old_loop_of_railroads_is_changing_the_face_of_a_city
appended description for https://www.ted.com/talks/judd_a_schorr_can_you_solve_the_airplane_riddle
appended description for https://www.ted.com/talks/danny_dorling_maps_that_show_us_who_we_are_not_just_where_we_are
appended description for https://www.ted.com/talks/alicia_garza_patrisse_cullors_and_opal_tometi_an_interview_with_the_founders_of_black_lives_matter
appended description for https://www.ted.com/talks/joe_lassiter_we_need_nuc

appended description for https://www.ted.com/talks/nwhunter_how_do_contraceptives_work
appended description for https://www.ted.com/talks/eric_liu_there_s_no_such_thing_as_not_voting
appended description for https://www.ted.com/talks/abigail_marsh_why_some_people_are_more_altruistic_than_others
appended description for https://www.ted.com/talks/michael_murphy_architecture_that_s_built_to_heal
appended description for https://www.ted.com/talks/michael_shellenberger_how_fear_of_nuclear_power_is_hurting_the_environment
appended description for https://www.ted.com/talks/julie_lythcott_haims_how_to_raise_successful_kids_without_over_parenting
appended description for https://www.ted.com/talks/neha_narula_the_future_of_money
appended description for https://www.ted.com/talks/franz_freudenthal_a_new_way_to_heal_hearts_without_surgery
appended description for https://www.ted.com/talks/david_camarillo_why_helmets_don_t_prevent_concussions_and_what_might
appended description for https://www.ted.

appended description for https://www.ted.com/talks/sajay_samuel_how_college_loans_exploit_students_for_profit
appended description for https://www.ted.com/talks/shaolan_the_chinese_zodiac_explained
appended description for https://www.ted.com/talks/ewandro_magalhaes_how_interpreters_juggle_two_languages_at_once
appended description for https://www.ted.com/talks/amit_sood_every_piece_of_art_you_ve_ever_wanted_to_see_up_close_and_searchable
appended description for https://www.ted.com/talks/cedric_villani_what_s_so_sexy_about_math
appended description for https://www.ted.com/talks/stephen_wilkes_the_passing_of_time_caught_in_a_single_photo
appended description for https://www.ted.com/talks/norman_lear_an_entertainment_icon_on_living_a_life_of_meaning
appended description for https://www.ted.com/talks/samantha_nutt_the_real_harm_of_the_global_arms_trade
appended description for https://www.ted.com/talks/dennis_shasha_can_you_solve_the_control_room_riddle
appended description for https://w

In [48]:
len(talk_descr)

1750

In [49]:
for url in talk_df['url'][1750:2000]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/sebastian_junger_our_lonely_society_makes_it_hard_to_come_home_from_war
appended description for https://www.ted.com/talks/laura_indolfi_good_news_in_the_fight_against_pancreatic_cancer
appended description for https://www.ted.com/talks/kang_lee_can_you_really_tell_if_a_kid_is_lying
appended description for https://www.ted.com/talks/sara_garofalo_the_psychology_behind_irrational_decisions
appended description for https://www.ted.com/talks/sangeeta_bhatia_this_tiny_particle_could_roam_your_body_to_find_tumors
appended description for https://www.ted.com/talks/sanford_biggers_an_artist_s_unflinching_look_at_racial_violence
appended description for https://www.ted.com/talks/kanawat_senanan_how_computer_memory_works
appended description for https://www.ted.com/talks/uri_hasson_this_is_your_brain_on_communication
appended description for https://www.ted.com/talks/jennifer_kahn_gene_editing_can_now_change_an_entire_species_forever
appended d

appended description for https://www.ted.com/talks/dalia_mogahed_what_it_s_like_to_be_muslim_in_america
appended description for https://www.ted.com/talks/al_gore_the_case_for_optimism_on_climate_change
appended description for https://www.ted.com/talks/raffaello_d_andrea_meet_the_dazzling_flying_machines_of_the_future
appended description for https://www.ted.com/talks/allan_adams_what_the_discovery_of_gravitational_waves_means
appended description for https://www.ted.com/talks/shonda_rhimes_my_year_of_saying_yes_to_everything
appended description for https://www.ted.com/talks/angela_koine_flynn_the_science_of_skin_color
appended description for https://www.ted.com/talks/celeste_headlee_10_ways_to_have_a_better_conversation
appended description for https://www.ted.com/talks/jocelyne_bloch_the_brain_may_be_able_to_repair_itself_with_help
appended description for https://www.ted.com/talks/dorothy_roberts_the_problem_with_race_based_medicine
appended description for https://www.ted.com/ta

appended description for https://www.ted.com/talks/andreas_ekstrom_the_moral_bias_behind_your_search_results
appended description for https://www.ted.com/talks/madhumita_murgia_how_stress_affects_your_brain
appended description for https://www.ted.com/talks/jenni_chang_and_lisa_dazols_this_is_what_lgbt_life_is_like_around_the_world
appended description for https://www.ted.com/talks/kaki_king_a_musical_escape_into_a_world_of_light_and_color
appended description for https://www.ted.com/talks/harald_haas_forget_wi_fi_meet_the_new_li_fi_internet
appended description for https://www.ted.com/talks/patricia_medici_the_coolest_animal_you_know_nothing_about_and_how_we_can_save_it
appended description for https://www.ted.com/talks/jeffrey_siegel_what_makes_muscles_grow
appended description for https://www.ted.com/talks/suzanne_duncan_the_dark_side_of_storytelling
appended description for https://www.ted.com/talks/suzanne_case_could_moby_dick_prevent_the_next_financial_crisis
appended description

appended description for https://www.ted.com/talks/douglas_beal_an_alternative_to_gdp_that_factors_in_our_wellbeing
appended description for https://www.ted.com/talks/vikram_bhalla_family_businesses_are_here_to_stay_and_thrive
appended description for https://www.ted.com/talks/shalini_unnikrishnan_we_need_to_turn_our_response_to_crisis_inside_out
appended description for https://www.ted.com/talks/sarah_willersdorf_what_brands_can_learn_from_online_dating
appended description for https://www.ted.com/talks/philip_krinks_finding_your_personal_mission_in_life
appended description for https://www.ted.com/talks/patty_mccord_hr_lessons_from_the_world_of_silicon_valley_start_ups
appended description for https://www.ted.com/talks/mathieu_lamiaux_imagining_a_new_future_for_health_systems_in_africa
appended description for https://www.ted.com/talks/markus_lorenz_industry_4_0_how_intelligent_machines_will_transform_everything_we_know
appended description for https://www.ted.com/talks/luc_de_braban

In [50]:
len(talk_descr)

2000

In [51]:
for url in talk_df['url'][2000:2250]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
appended description for https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
appended description for https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
appended description for https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
appended description for https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
appended description for https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
appended description for https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
appended description for https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america
appended description for https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power


appended description for https://www.ted.com/talks/alice_goffman_how_we_re_priming_some_kids_for_college_and_others_for_prison
appended description for https://www.ted.com/talks/sophie_scott_why_we_laugh
appended description for https://www.ted.com/talks/greg_gage_how_to_control_someone_else_s_arm_with_your_brain
appended description for https://www.ted.com/talks/nick_bostrom_what_happens_when_our_computers_get_smarter_than_we_are
appended description for https://www.ted.com/talks/nizar_ibrahim_how_we_unearthed_the_spinosaurus
appended description for https://www.ted.com/talks/eleanor_nelsen_how_to_unboil_an_egg
appended description for https://www.ted.com/talks/clint_smith_how_to_raise_a_black_son_in_america
appended description for https://www.ted.com/talks/chris_milk_how_virtual_reality_can_create_the_ultimate_empathy_machine
appended description for https://www.ted.com/talks/emma_bryce_what_really_happens_to_the_plastic_you_throw_away
appended description for https://www.ted.com/ta

appended description for https://www.ted.com/talks/miguel_nicolelis_brain_to_brain_communication_has_arrived_how_we_did_it
appended description for https://www.ted.com/talks/morgana_bailey_the_danger_of_hiding_who_you_are
appended description for https://www.ted.com/talks/joe_madiath_better_toilets_better_life
appended description for https://www.ted.com/talks/sarah_bergbreiter_why_i_make_robots_the_size_of_a_grain_of_rice
appended description for https://www.ted.com/talks/matthieu_ricard_how_to_let_altruism_be_your_guide
appended description for https://www.ted.com/talks/cristina_domenech_poetry_that_frees_the_soul
appended description for https://www.ted.com/talks/robert_muggah_how_to_protect_fast_growing_cities_from_failing
appended description for https://www.ted.com/talks/robert_swan_let_s_save_the_last_pristine_continent
appended description for https://www.ted.com/talks/navi_radjou_creative_problem_solving_in_the_face_of_extreme_limits
appended description for https://www.ted.co

appended description for https://www.ted.com/talks/alexander_mclean_restoring_hope_and_dignity_to_the_justice_system
appended description for https://www.ted.com/talks/abbey_williams_my_ghost_town_road_trip
appended description for https://www.ted.com/talks/will_marshall_tiny_satellites_show_us_the_earth_as_it_changes_in_near_real_time
appended description for https://www.ted.com/talks/david_grady_how_to_save_the_world_or_at_least_yourself_from_bad_meetings
appended description for https://www.ted.com/talks/vincent_moon_and_nana_vasconcelos_hidden_music_rituals_around_the_world
appended description for https://www.ted.com/talks/alex_gendler_why_elephants_never_forget
appended description for https://www.ted.com/talks/leana_wen_what_your_doctor_won_t_disclose
appended description for https://www.ted.com/talks/ethan_nadelmann_why_we_need_to_end_the_war_on_drugs
appended description for https://www.ted.com/talks/michael_green_what_the_social_progress_index_can_reveal_about_your_country
ap

In [54]:
len(talk_descr)

2750

In [53]:
for url in talk_df['url'][2250:2750]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/joy_sun_should_you_donate_differently
appended description for https://www.ted.com/talks/fred_swaniker_the_leaders_who_ruined_africa_and_the_generation_who_can_fix_it
appended description for https://www.ted.com/talks/john_varney_a_different_way_to_visualize_rhythm
appended description for https://www.ted.com/talks/susan_etlinger_what_do_we_do_with_all_this_big_data
appended description for https://www.ted.com/talks/kitra_cahana_my_father_locked_in_his_body_but_soaring_free
appended description for https://www.ted.com/talks/melissa_fleming_let_s_help_refugees_thrive_not_just_survive
appended description for https://www.ted.com/talks/jorge_soto_the_future_of_early_cancer_detection
appended description for https://www.ted.com/talks/chad_orzel_schrodinger_s_cat_a_thought_experiment_in_quantum_mechanics
appended description for https://www.ted.com/talks/myriam_sidibe_the_simple_power_of_hand_washing
appended description for https://www.ted

appended description for https://www.ted.com/talks/ze_frank_are_you_human
appended description for https://www.ted.com/talks/lieven_scheire_how_quantum_mechanics_explains_global_warming
appended description for https://www.ted.com/talks/heather_barnett_what_humans_can_learn_from_semi_intelligent_slime
appended description for https://www.ted.com/talks/shih_chieh_huang_sculptures_that_d_be_at_home_in_the_deep_sea
appended description for https://www.ted.com/talks/nikolai_begg_a_tool_to_fix_one_of_the_most_dangerous_moments_in_surgery
appended description for https://www.ted.com/talks/david_chalmers_how_do_you_explain_consciousness
appended description for https://www.ted.com/talks/david_kwong_two_nerdy_obsessions_meet_and_it_s_magic
appended description for https://www.ted.com/talks/claudia_aguirre_what_makes_tattoos_permanent
appended description for https://www.ted.com/talks/karima_bennoune_when_people_of_muslim_heritage_challenge_fundamentalism
appended description for https://www.te

appended description for https://www.ted.com/talks/jennifer_senior_for_parents_happiness_is_a_very_high_bar
appended description for https://www.ted.com/talks/matt_anticole_what_s_the_difference_between_accuracy_and_precision
appended description for https://www.ted.com/talks/david_brooks_should_you_live_for_your_resume_or_your_eulogy
appended description for https://www.ted.com/talks/gabby_giffords_and_mark_kelly_be_passionate_be_courageous_be_your_best
appended description for https://www.ted.com/talks/david_sengeh_the_sore_problem_of_prosthetic_limbs
appended description for https://www.ted.com/talks/louie_schwartzberg_hidden_miracles_of_the_natural_world
appended description for https://www.ted.com/talks/george_zaidan_why_is_ketchup_so_hard_to_pour
appended description for https://www.ted.com/talks/christopher_emdin_teach_teachers_how_to_create_magic
appended description for https://www.ted.com/talks/amanda_burden_how_public_spaces_make_cities_work
appended description for https://

appended description for https://www.ted.com/talks/del_harvey_protecting_privacy_at_twitter
appended description for https://www.ted.com/talks/alan_iny_reigniting_creativity_in_business
appended description for https://www.ted.com/talks/wendy_woods_collective_cooperation_in_the_social_sector
appended description for https://www.ted.com/talks/paula_johnson_his_and_hers_health_care
appended description for https://www.ted.com/talks/shereen_el_feki_a_little_told_tale_of_sex_and_sensuality
appended description for https://www.ted.com/talks/guy_hoffman_robots_with_soul
appended description for https://www.ted.com/talks/jeff_dekofsky_the_infinite_hotel_paradox
appended description for https://www.ted.com/talks/luke_syson_how_i_learned_to_stop_worrying_and_love_useless_art
appended description for https://www.ted.com/talks/sheryl_sandberg_so_we_leaned_in_now_what
appended description for https://www.ted.com/talks/mark_kendall_demo_a_needle_free_vaccine_patch_that_s_safer_and_way_cheaper
appen

appended description for https://www.ted.com/talks/george_zaidan_and_charles_morton_the_uncertain_location_of_electrons
appended description for https://www.ted.com/talks/jeff_speck_the_walkable_city
appended description for https://www.ted.com/talks/andrew_fitzgerald_adventures_in_twitter_fiction
appended description for https://www.ted.com/talks/gary_slutkin_let_s_treat_violence_like_a_contagious_disease
appended description for https://www.ted.com/talks/trita_parsi_iran_and_israel_peace_is_possible
appended description for https://www.ted.com/talks/mikael_cho_the_science_of_stage_fright_and_how_to_overcome_it
appended description for https://www.ted.com/talks/janette_sadik_khan_new_york_s_streets_not_so_mean_any_more
appended description for https://www.ted.com/talks/michael_sandel_why_we_shouldn_t_trust_markets_with_our_civic_life
appended description for https://www.ted.com/talks/michael_porter_the_case_for_letting_business_solve_social_problems
appended description for https://ww

appended description for https://www.ted.com/talks/rodney_brooks_why_we_will_rely_on_robots
appended description for https://www.ted.com/talks/joy_lin_if_superpowers_were_real_invisibility
appended description for https://www.ted.com/talks/joy_lin_if_superpowers_were_real_flight
appended description for https://www.ted.com/talks/joy_lin_if_superpowers_were_real_immortality
appended description for https://www.ted.com/talks/joy_lin_if_superpowers_were_real_super_strength
appended description for https://www.ted.com/talks/joy_lin_if_superpowers_were_real_super_speed
appended description for https://www.ted.com/talks/joy_lin_if_superpowers_were_real_body_mass
appended description for https://www.ted.com/talks/michael_archer_how_we_ll_resurrect_the_gastric_brooding_frog_the_tasmanian_tiger
appended description for https://www.ted.com/talks/bob_mankoff_anatomy_of_a_new_yorker_cartoon
appended description for https://www.ted.com/talks/marjee_chmiel_and_trevor_owens_is_there_a_center_of_the_u

appended description for https://www.ted.com/talks/tom_whyntie_the_beginning_of_the_universe_for_beginners
appended description for https://www.ted.com/talks/keller_rinaudo_a_mini_robot_powered_by_your_phone
appended description for https://www.ted.com/talks/freeman_hrabowski_4_pillars_of_college_success_in_science
appended description for https://www.ted.com/talks/ken_jennings_watson_jeopardy_and_me_the_obsolete_know_it_all
appended description for https://www.ted.com/talks/skylar_tibbits_the_emergence_of_4d_printing
appended description for https://www.ted.com/talks/lawrence_lessig_we_the_people_and_the_republic_we_must_reclaim
appended description for https://www.ted.com/talks/sanjay_dastoor_a_skateboard_with_a_boost
appended description for https://www.ted.com/talks/kees_moeliker_how_a_dead_duck_changed_my_life
appended description for https://www.ted.com/talks/katherine_kuchenbecker_the_technology_of_touch
appended description for https://www.ted.com/talks/colin_camerer_when_you_r

In [55]:
for url in talk_df['url'][2750:3000]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/makiko_eda_the_corporation_as_agent_of_cultural_fusion
appended description for https://www.ted.com/talks/lorie_wigle_weaving_a_strong_smart_grid
appended description for https://www.ted.com/talks/gary_niekerk_chips_conflict_and_the_congo
appended description for https://www.ted.com/talks/erika_debenedictis_strategies_for_fuel_efficient_space_travel
appended description for https://www.ted.com/talks/achin_bhowmik_interactive_experiences_with_perceptual_computing
appended description for https://www.ted.com/talks/steve_brown_why_machines_must_make_us_better_humans
appended description for https://www.ted.com/talks/richard_turere_my_invention_that_made_peace_with_lions
appended description for https://www.ted.com/talks/jeff_dekofsky_euclid_s_puzzling_parallel_postulate
appended description for https://www.ted.com/talks/mark_shaw_one_very_dry_demo
appended description for https://www.ted.com/talks/jessica_green_we_re_covered_in_germs_let_

appended description for https://www.ted.com/talks/robin_chase_excuse_me_may_i_rent_your_car
appended description for https://www.ted.com/talks/ben_saunders_why_bother_leaving_the_house
appended description for https://www.ted.com/talks/christopher_warner_situational_irony_the_opposite_of_what_you_think
appended description for https://www.ted.com/talks/marcus_byrne_the_dance_of_the_dung_beetle
appended description for https://www.ted.com/talks/maz_jobrani_a_saudi_an_indian_and_an_iranian_walk_into_a_qatari_bar
appended description for https://www.ted.com/talks/markham_nolan_how_to_separate_fact_and_fiction_online
appended description for https://www.ted.com/talks/ellen_t_hoen_pool_medical_patents_save_lives
appended description for https://www.ted.com/talks/lemon_andersen_please_don_t_take_my_air_jordans
appended description for https://www.ted.com/talks/nina_tandon_could_tissue_engineering_mean_personalized_medicine
appended description for https://www.ted.com/talks/jeff_smith_lesson

appended description for https://www.ted.com/talks/vikram_patel_mental_health_for_all_by_involving_all
appended description for https://www.ted.com/talks/scott_fraser_why_eyewitnesses_get_it_wrong
appended description for https://www.ted.com/talks/kent_larson_brilliant_designs_to_fit_more_people_in_every_city
appended description for https://www.ted.com/talks/jonathan_trent_energy_from_floating_algae_pods
appended description for https://www.ted.com/talks/antony_gormley_sculpted_space_within_and_without
appended description for https://www.ted.com/talks/shyam_sankar_the_rise_of_human_computer_cooperation
appended description for https://www.ted.com/talks/robert_neuwirth_the_power_of_the_informal_economy
appended description for https://www.ted.com/talks/rob_legato_the_art_of_creating_awe
appended description for https://www.ted.com/talks/timothy_prestero_design_for_people_not_awards
appended description for https://www.ted.com/talks/jon_ronson_strange_answers_to_the_psychopath_test
app

appended description for https://www.ted.com/talks/william_noel_revealing_the_lost_codex_of_archimedes


In [56]:
len(talk_descr)

3000

In [57]:
for url in talk_df['url'][3000:3500]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
appended description for https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
appended description for https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
appended description for https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
appended description for https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
appended description for https://www.ted.com/talks/hans_rosling_religions_and_babies
appended description for https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
appended description for https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen
appended description for https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief
appended description for https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out
appended description for https://www.t

appended description for https://www.ted.com/talks/peter_diamandis_abundance_is_our_future
appended description for https://www.ted.com/talks/daniel_pauly_the_ocean_s_shifting_baseline
appended description for https://www.ted.com/talks/paul_snelgrove_a_census_of_the_ocean
appended description for https://www.ted.com/talks/kevin_allocca_why_videos_go_viral
appended description for https://www.ted.com/talks/jeffrey_kluger_the_sibling_bond
appended description for https://www.ted.com/talks/avi_rubin_all_your_devices_can_be_hacked
appended description for https://www.ted.com/talks/tan_le_my_immigration_story
appended description for https://www.ted.com/talks/shlomo_benartzi_saving_for_tomorrow_tomorrow
appended description for https://www.ted.com/talks/shilo_shiv_suleman_using_tech_to_enable_dreaming
appended description for https://www.ted.com/talks/chris_bliss_comedy_is_translation
appended description for https://www.ted.com/talks/neil_macgregor_2600_years_of_history_in_one_object
appen

appended description for https://www.ted.com/talks/damon_horowitz_philosophy_in_prison
appended description for https://www.ted.com/talks/amy_purdy_living_beyond_limits
appended description for https://www.ted.com/talks/britta_riley_a_garden_in_my_apartment
appended description for https://www.ted.com/talks/joe_sabia_the_technology_of_storytelling
appended description for https://www.ted.com/talks/peter_fankhauser_meet_rezero_the_dancing_ballbot
appended description for https://www.ted.com/talks/phil_plait_how_to_defend_earth_from_asteroids
appended description for https://www.ted.com/talks/natalie_warne_being_young_and_making_an_impact
appended description for https://www.ted.com/talks/gabe_zichermann_how_games_make_kids_smarter
appended description for https://www.ted.com/talks/robin_ince_science_versus_wonder
appended description for https://www.ted.com/talks/cynthia_kenyon_experiments_that_hint_of_longer_lives
appended description for https://www.ted.com/talks/thomas_suarez_a_12_ye

appended description for https://www.ted.com/talks/rory_stewart_time_to_end_the_war_in_afghanistan
appended description for https://www.ted.com/talks/markus_fischer_a_robot_that_flies_like_a_bird
appended description for https://www.ted.com/talks/kevin_slavin_how_algorithms_shape_our_world
appended description for https://www.ted.com/talks/thandie_newton_embracing_otherness_embracing_myself
appended description for https://www.ted.com/talks/mikko_hypponen_fighting_viruses_defending_the_net
appended description for https://www.ted.com/talks/nadia_al_sakkaf_see_yemen_through_my_eyes
appended description for https://www.ted.com/talks/tim_harford_trial_error_and_the_god_complex
appended description for https://www.ted.com/talks/maajid_nawaz_a_global_culture_to_fight_extremism
appended description for https://www.ted.com/talks/rebecca_mackinnon_let_s_take_back_the_internet
appended description for https://www.ted.com/talks/nina_tandon_caring_for_engineered_tissue
appended description for ht

appended description for https://www.ted.com/talks/ralph_langner_cracking_stuxnet_a_21st_century_cyber_weapon
appended description for https://www.ted.com/talks/patricia_ryan_don_t_insist_on_english
appended description for https://www.ted.com/talks/claron_mcfadden_singing_the_primal_mystery
appended description for https://www.ted.com/talks/eythor_bender_human_exoskeletons_for_war_and_healing
appended description for https://www.ted.com/talks/paul_root_wolpe_it_s_time_to_question_bio_engineering
appended description for https://www.ted.com/talks/isabel_behncke_evolution_s_gift_of_play_from_bonobo_apes_to_humans
appended description for https://www.ted.com/talks/hans_rosling_the_magic_washing_machine
appended description for https://www.ted.com/talks/sarah_kay_if_i_should_have_a_daughter
appended description for https://www.ted.com/talks/rogier_van_der_heide_why_light_needs_darkness
appended description for https://www.ted.com/talks/mark_bezos_a_life_lesson_from_a_volunteer_firefighter

unable to append description @ https://www.ted.com/talks/kristina_gjerde_making_law_on_the_high_seas
appended description for https://www.ted.com/talks/john_hardy_my_green_school_dream
unable to append description @ https://www.ted.com/talks/shimon_schocken_what_a_bike_ride_can_teach_you
unable to append description @ https://www.ted.com/talks/denis_dutton_a_darwinian_theory_of_beauty
appended description for https://www.ted.com/talks/conrad_wolfram_teaching_kids_real_math_with_computers
appended description for https://www.ted.com/talks/eric_berlow_simplifying_complexity
appended description for https://www.ted.com/talks/auret_van_heerden_making_global_labor_fair
appended description for https://www.ted.com/talks/aaron_huey_america_s_native_prisoners_of_war
appended description for https://www.ted.com/talks/stefan_wolff_the_path_to_ending_ethnic_conflicts
appended description for https://www.ted.com/talks/emily_pilloton_teaching_design_for_change
appended description for https://www.t

In [58]:
len(talk_descr)

3500

In [59]:
for url in talk_df['url'][3500:4000]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/diane_j_savino_the_case_for_same_sex_marriage
appended description for https://www.ted.com/talks/sheryl_wudunn_our_century_s_greatest_injustice
appended description for https://www.ted.com/talks/jason_clay_how_big_brands_can_help_save_biodiversity
appended description for https://www.ted.com/talks/lewis_pugh_my_mind_shifting_everest_swim
appended description for https://www.ted.com/talks/laurie_santos_a_monkey_economy_as_irrational_as_ours
appended description for https://www.ted.com/talks/john_delaney_wiring_an_interactive_ocean
appended description for https://www.ted.com/talks/jeff_bezos_gifts_vs_choices
appended description for https://www.ted.com/talks/susan_shaw_the_oil_spill_s_toxic_trade_off
appended description for https://www.ted.com/talks/sheena_iyengar_the_art_of_choosing
appended description for https://www.ted.com/talks/kevin_stone_the_bio_future_of_joint_replacement
appended description for https://www.ted.com/talks/tan_

appended description for https://www.ted.com/talks/jonathan_drori_every_pollen_grain_has_a_story
appended description for https://www.ted.com/talks/dennis_hong_my_seven_species_of_robot_and_how_we_created_them
appended description for https://www.ted.com/talks/dean_kamen_the_emotion_behind_invention
appended description for https://www.ted.com/talks/elizabeth_pisani_sex_drugs_and_hiv_let_s_get_rational
appended description for https://www.ted.com/talks/jesse_schell_when_games_invade_real_life
appended description for https://www.ted.com/talks/adora_svitak_what_adults_can_learn_from_kids
appended description for https://www.ted.com/talks/derek_sivers_how_to_start_a_movement
appended description for https://www.ted.com/talks/kirk_citron_and_now_the_real_news
appended description for https://www.ted.com/talks/shukla_bose_teaching_one_child_at_a_time
appended description for https://www.ted.com/talks/kevin_bales_how_to_combat_modern_slavery
appended description for https://www.ted.com/talk

appended description for https://www.ted.com/talks/loretta_napoleoni_the_intricate_economics_of_terrorism
appended description for https://www.ted.com/talks/shereen_el_feki_pop_culture_in_the_arab_world
appended description for https://www.ted.com/talks/thulasiraj_ravilla_how_low_cost_eye_care_can_be_world_class
appended description for https://www.ted.com/talks/marc_pachter_the_art_of_the_interview
appended description for https://www.ted.com/talks/rory_bremner_a_one_man_world_summit
appended description for https://www.ted.com/talks/sunitha_krishnan_the_fight_against_sex_slavery
appended description for https://www.ted.com/talks/scott_kim_the_art_of_puzzles
appended description for https://www.ted.com/talks/anupam_mishra_the_ancient_ingenuity_of_water_harvesting
appended description for https://www.ted.com/talks/andrea_ghez_the_hunt_for_a_supermassive_black_hole
appended description for https://www.ted.com/talks/gordon_brown_global_ethic_vs_national_interest
appended description for 

appended description for https://www.ted.com/talks/olafur_eliasson_playing_with_space_and_light
appended description for https://www.ted.com/talks/stewart_brand_4_environmental_heresies
appended description for https://www.ted.com/talks/kary_mullis_a_next_gen_cure_for_killer_infections
appended description for https://www.ted.com/talks/sophal_ear_escaping_the_khmer_rouge
appended description for https://www.ted.com/talks/tom_wujec_3_ways_the_brain_creates_meaning
appended description for https://www.ted.com/talks/eames_demetrios_the_design_genius_of_charles_ray_eames
appended description for https://www.ted.com/talks/daniel_libeskind_17_words_of_architectural_inspiration
appended description for https://www.ted.com/talks/gever_tulley_life_lessons_through_tinkering
appended description for https://www.ted.com/talks/arthur_benjamin_teach_statistics_before_calculus
appended description for https://www.ted.com/talks/ray_zahab_my_trek_to_the_south_pole
appended description for https://www.t

appended description for https://www.ted.com/talks/don_norman_3_ways_good_design_makes_you_happy
appended description for https://www.ted.com/talks/eric_lewis_piano_jazz_that_rocks
appended description for https://www.ted.com/talks/mike_rowe_learning_from_dirty_jobs
appended description for https://www.ted.com/talks/nalini_nadkarni_conserving_the_canopy
appended description for https://www.ted.com/talks/willie_smits_how_to_restore_a_rainforest
appended description for https://www.ted.com/talks/brenda_laurel_why_not_make_video_games_for_girls
appended description for https://www.ted.com/talks/evan_williams_the_voices_of_twitter_users
appended description for https://www.ted.com/talks/miru_kim_my_underground_art_explorations
appended description for https://www.ted.com/talks/richard_pyle_a_dive_into_the_reef_s_twilight_zone
appended description for https://www.ted.com/talks/charles_moore_seas_of_plastic
appended description for https://www.ted.com/talks/ed_ulbrich_how_benjamin_button_got

appended description for https://www.ted.com/talks/garrett_lisi_an_8_dimensional_model_of_the_universe
appended description for https://www.ted.com/talks/james_burchfield_playing_invisible_turntables
appended description for https://www.ted.com/talks/steven_johnson_the_web_as_a_city
appended description for https://www.ted.com/talks/doris_kearns_goodwin_lessons_from_past_presidents
appended description for https://www.ted.com/talks/david_perry_are_games_better_than_life
appended description for https://www.ted.com/talks/james_nachtwey_moving_photos_of_extreme_drug_resistant_tb
appended description for https://www.ted.com/talks/liz_diller_the_blur_building_and_other_tech_empowered_architecture
appended description for https://www.ted.com/talks/noah_feldman_politics_and_religion_are_technologies
appended description for https://www.ted.com/talks/stefan_sagmeister_things_i_ve_learned_in_my_life_so_far
appended description for https://www.ted.com/talks/rodney_brooks_robots_will_invade_our_

In [60]:
len(talk_descr)

4000

In [61]:
for url in talk_df['url'][4000:]:
    link = "https://www.ted.com" + url
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_descr.append(content.find_all('meta', property = 'og:description')[0]['content'])
        print('appended description for ' + link)
        links_appended += 1
        # print('appended link #' + links_appended)
        time.sleep(3)
    except:
        print('unable to append description @ ' + link)
        talk_descr.append('DID NOT APPEND')

appended description for https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
appended description for https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
appended description for https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
appended description for https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
appended description for https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
appended description for https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
appended description for https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
appended description for https://www.ted.com/talks/rokia_traore_kounandi
appended description for https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq
appended description for https://www.ted.com/talks/susan_blackmore_memes_and_temes
appended description for https://www.ted.com/talks/paul_collier_the_bottom_b

appended description for https://www.ted.com/talks/matthieu_ricard_the_habits_of_happiness
appended description for https://www.ted.com/talks/sherwin_nuland_how_electroshock_therapy_changed_me
appended description for https://www.ted.com/talks/eleni_gabre_madhin_a_commodities_exchange_for_ethiopia
appended description for https://www.ted.com/talks/vs_ramachandran_3_clues_to_understanding_your_brain
appended description for https://www.ted.com/talks/jan_chipchase_the_anthropology_of_mobile_phones
appended description for https://www.ted.com/talks/maira_kalman_the_illustrated_woman
appended description for https://www.ted.com/talks/hod_lipson_building_self_aware_robots
appended description for https://www.ted.com/talks/richard_branson_life_at_30_000_feet
appended description for https://www.ted.com/talks/kenichi_ebina_my_magic_moves
appended description for https://www.ted.com/talks/carolyn_porco_this_is_saturn
appended description for https://www.ted.com/talks/paul_maccready_a_flight_on

appended description for https://www.ted.com/talks/janine_benyus_biomimicry_s_surprising_lessons_from_nature_s_engineers
appended description for https://www.ted.com/talks/al_seckel_visual_illusions_that_show_how_we_mis_think
appended description for https://www.ted.com/talks/janine_benyus_biomimicry_s_surprising_lessons_from_nature_s_engineers
appended description for https://www.ted.com/talks/paul_bennett_design_is_in_the_details
appended description for https://www.ted.com/talks/james_nachtwey_my_wish_let_my_photographs_bear_witness
appended description for https://www.ted.com/talks/bill_clinton_my_wish_rebuilding_rwanda
appended description for https://www.ted.com/talks/e_o_wilson_my_wish_build_the_encyclopedia_of_life
appended description for https://www.ted.com/talks/carl_honore_in_praise_of_slowness
appended description for https://www.ted.com/talks/neil_gershenfeld_unleash_your_creativity_in_a_fab_lab
appended description for https://www.ted.com/talks/saul_griffith_everyday_inv

In [62]:
len(talk_descr)

4244

In [89]:
talk_df['summ'] = talk_descr

In [68]:
talk_df.to_csv('talks_with_summ.csv')

In [91]:
# Check for any summaries that were not appended, could be due to lost internet connection
# Rerun code
for index in range(0, len(talk_df)):
    if talk_df['summ'][index] == 'DID NOT APPEND':
        url = talk_df['url'][index]
        link = "https://www.ted.com" + url
        print(link)
        try:
            page = requests.get(link)
            content = BS(page.content, 'html.parser')
            talk_df['summ'][index] = content.find_all('meta', property = 'og:description')[0]['content']
            print('appended description for ' + link)
            links_appended += 1
            # print('appended link #' + links_appended)
            time.sleep(3)
        except:
            print('unable to append description @ ' + link)

https://www.ted.com/talks/anne_curzan_what_makes_a_word_real
appended description for https://www.ted.com/talks/anne_curzan_what_makes_a_word_real
https://www.ted.com/talks/dan_gilbert_the_psychology_of_your_future_self
appended description for https://www.ted.com/talks/dan_gilbert_the_psychology_of_your_future_self
https://www.ted.com/talks/karen_d_davis_how_does_your_brain_respond_to_pain
appended description for https://www.ted.com/talks/karen_d_davis_how_does_your_brain_respond_to_pain
https://www.ted.com/talks/suheir_hammad_poems_of_war_peace_women_power
appended description for https://www.ted.com/talks/suheir_hammad_poems_of_war_peace_women_power
https://www.ted.com/talks/johanna_blakley_social_media_and_the_end_of_gender
appended description for https://www.ted.com/talks/johanna_blakley_social_media_and_the_end_of_gender
https://www.ted.com/talks/dale_dougherty_we_are_makers
appended description for https://www.ted.com/talks/dale_dougherty_we_are_makers
https://www.ted.com/talk

In [92]:
(talk_df['summ'] == 'DID NOT APPEND').value_counts()

False    4244
Name: summ, dtype: int64

In [93]:
print(links_appended)

40


In [94]:
talk_df.head()

,date,speaker,title,url,length,summ
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma..."
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...


In [95]:
talk_df.to_csv('talks_with_summ.csv')

# Get tags for each talk

In [2]:
import pandas as pd
talk_df = pd.read_csv('talks_with_summ.csv', index_col = 0)

In [3]:
talk_df.head()

,date,speaker,title,url,length,summ
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma..."
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...


In [152]:
def get_tags(start, end):
    talk_tags = []
    for index in range(start, end):
        link = "https://www.ted.com" + talk_df['url'][index]
        try:
            page = requests.get(link)
            content = BS(page.content, 'html.parser')
            talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
            print('tags appended for ' + link)
            time.sleep(2)
        except:
            print('NO TAGS APPENDED @ ' + link)
            talk_tags.append('NO TAGS APPENDED')
    return talk_tags

In [4]:
talk_tags = []
for index in range(500):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator
tags appended for https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol
tags appended for https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women
tags appended for https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear
tags appended for https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success
tags appended for https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time
tags appended for https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism
tags appended for https://www.ted.com/talks/nick_bostrom_how_civilization_could_destroy_itself_and_4_ways_we_could_prevent_it
tags appended for https://www.ted.com/talks/jasmine_crowe_what_we_re_getting_wrong_in_the_fight_to_end_hunger
tags appended for https://www.ted.com/talks/sinan_aral_how_we_ca

tags appended for https://www.ted.com/talks/bhakti_sharma_what_open_water_swimming_taught_me_about_resilience
tags appended for https://www.ted.com/talks/bill_keegan_the_taino_myth_of_the_cursed_creator
tags appended for https://www.ted.com/talks/bob_langert_the_business_case_for_working_with_your_toughest_critics
tags appended for https://www.ted.com/talks/sara_jane_dunn_the_next_software_revolution_programming_biological_cells
tags appended for https://www.ted.com/talks/lori_gottlieb_how_changing_your_story_can_change_your_life
tags appended for https://www.ted.com/talks/chip_berlet_are_the_illuminati_real
tags appended for https://www.ted.com/talks/daniel_streicker_what_vaccinating_vampire_bats_can_teach_us_about_pandemics
tags appended for https://www.ted.com/talks/jess_kutch_what_productive_conflict_can_offer_a_workplace
tags appended for https://www.ted.com/talks/jacob_soboroff_why_do_americans_vote_on_tuesdays
tags appended for https://www.ted.com/talks/toby_kiers_lessons_from_f

tags appended for https://www.ted.com/talks/will_hurd_a_wall_won_t_solve_america_s_border_problems
tags appended for https://www.ted.com/talks/yaniv_erlich_how_we_re_building_the_world_s_largest_family_tree
tags appended for https://www.ted.com/talks/laura_wright_why_should_you_read_the_god_of_small_things_by_arundhati_roy
tags appended for https://www.ted.com/talks/emily_nagoski_how_couples_can_sustain_a_strong_sexual_connection_for_a_lifetime
tags appended for https://www.ted.com/talks/hanna_ilona_harmavaara_the_myth_of_the_sampo_an_infinite_source_of_fortune_and_greed
tags appended for https://www.ted.com/talks/tim_flannery_can_seaweed_help_curb_global_warming
tags appended for https://www.ted.com/talks/safeena_husain_a_bold_plan_to_empower_1_6_million_out_of_school_girls_in_india
tags appended for https://www.ted.com/talks/sonaar_luthra_we_need_to_track_the_world_s_water_like_we_track_the_weather
tags appended for https://www.ted.com/talks/richard_bona_tumba_la_nyama_mulema
tags ap

tags appended for https://www.ted.com/talks/albert_laszlo_barabasi_the_real_relationship_between_your_age_and_your_chance_of_success
tags appended for https://www.ted.com/talks/ilan_stavans_romance_and_revolution_the_poetry_of_pablo_neruda
tags appended for https://www.ted.com/talks/hajer_sharief_how_to_use_family_dinner_to_teach_politics
tags appended for https://www.ted.com/talks/rahul_mehrotra_the_architectural_wonder_of_impermanent_cities
tags appended for https://www.ted.com/talks/anthony_mccarten_a_not_so_scientific_experiment_on_laughter
tags appended for https://www.ted.com/talks/brian_janosch_what_i_learned_from_writing_jokes_for_the_onion
tags appended for https://www.ted.com/talks/taika_waititi_why_humor_is_key_to_creativity
tags appended for https://www.ted.com/talks/douglas_thomas_how_a_typeface_helped_launch_apollo
tags appended for https://www.ted.com/talks/ramon_glazov_ancient_rome_s_most_notorious_doctor
tags appended for https://www.ted.com/talks/elizabeth_howell_how_

tags appended for https://www.ted.com/talks/brendan_constantine_the_opposites_game
tags appended for https://www.ted.com/talks/miguel_alcubierre_como_podriamos_viajar_mas_rapido_que_la_velocidad_de_la_luz
tags appended for https://www.ted.com/talks/holley_moyes_how_darkness_influences_imagination
tags appended for https://www.ted.com/talks/dragana_rogulja_how_artificial_light_affects_our_health
tags appended for https://www.ted.com/talks/diane_knutson_why_we_need_darkness
tags appended for https://www.ted.com/talks/bjarke_ingels_floating_cities_the_lego_house_and_other_architectural_forms_of_the_future
tags appended for https://www.ted.com/talks/william_butler_yeats_the_second_coming
tags appended for https://www.ted.com/talks/roger_hanlon_the_amazing_brains_and_morphing_skin_of_octopuses_and_other_cephalopods
tags appended for https://www.ted.com/talks/reniqua_allen_the_story_we_tell_about_millennials_and_who_we_leave_out
tags appended for https://www.ted.com/talks/andrea_gibson_the_n

tags appended for https://www.ted.com/talks/tomas_chor_turbulence_one_of_the_great_unsolved_mysteries_of_physics
tags appended for https://www.ted.com/talks/leah_georges_how_generational_stereotypes_hold_us_back_at_work
tags appended for https://www.ted.com/talks/david_r_liu_can_we_cure_genetic_diseases_by_rewriting_dna
tags appended for https://www.ted.com/talks/romain_lacombe_a_personal_air_quality_tracker_that_lets_you_know_what_you_re_breathing
tags appended for https://www.ted.com/talks/danielle_n_lee_how_hip_hop_helps_us_understand_science
tags appended for https://www.ted.com/talks/sheperd_doeleman_inside_the_black_hole_image_that_made_history
tags appended for https://www.ted.com/talks/jack_dorsey_how_twitter_needs_to_change
tags appended for https://www.ted.com/talks/sarvesh_subramanian_global_warming_the_app
tags appended for https://www.ted.com/talks/carole_cadwalladr_facebook_s_role_in_brexit_and_the_threat_to_democracy
tags appended for https://www.ted.com/talks/christoph_

tags appended for https://www.ted.com/talks/keith_lowe_why_we_need_to_stop_obsessing_over_world_war_ii
tags appended for https://www.ted.com/talks/juliet_brophy_how_a_new_species_of_ancestors_is_changing_our_theory_of_human_evolution
tags appended for https://www.ted.com/talks/jordana_moore_saggese_the_chaotic_brilliance_of_artist_jean_michel_basquiat
tags appended for https://www.ted.com/talks/farida_nabourema_is_your_country_at_risk_of_becoming_a_dictatorship_here_s_how_to_know
tags appended for https://www.ted.com/talks/pat_crowley_farming_insects_to_save_the_world
tags appended for https://www.ted.com/talks/karl_skjonnemand_the_self_assembling_computer_chips_of_the_future
tags appended for https://www.ted.com/talks/daniel_finkel_can_you_solve_the_jail_break_riddle
tags appended for https://www.ted.com/talks/lindy_lou_isonhood_a_juror_s_reflections_on_the_death_penalty
tags appended for https://www.ted.com/talks/ayana_jamieson_and_moya_bailey_why_should_you_read_sci_fi_superstar_oct

In [5]:
len(talk_tags)

500

In [6]:
for index in range(500, 1000):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/yannick_roudaut_how_today_s_truths_shape_tomorrow_s_possibilities
tags appended for https://www.ted.com/talks/michael_benton_mass_extinctions_and_the_future_of_life_on_earth
tags appended for https://www.ted.com/talks/lindiwe_mazibuko_why_the_african_diaspora_is_crucial_to_the_continent_s_future
tags appended for https://www.ted.com/talks/juan_enriquez_the_age_of_genetic_wonder_nov_2019
tags appended for https://www.ted.com/talks/anne_gaskett_the_sexual_deception_of_orchids
tags appended for https://www.ted.com/talks/aja_monet_and_phillip_agnew_a_love_story_about_the_power_of_art_as_organizing
tags appended for https://www.ted.com/talks/glenn_cantave_vr_ar_for_social_change
tags appended for https://www.ted.com/talks/steven_petrow_3_ways_to_practice_civility
tags appended for https://www.ted.com/talks/christina_greer_notes_of_a_native_son_the_world_according_to_james_baldwin
tags appended for https://www.ted.com/talks/dropbox_how_one_team_tur

tags appended for https://www.ted.com/talks/alex_gendler_can_you_solve_the_multiplying_rabbits_riddle
tags appended for https://www.ted.com/talks/karissa_sanbonmatsu_the_biology_of_gender_from_dna_to_the_brain
tags appended for https://www.ted.com/talks/danae_wolfe_ew_to_awe_your_view_of_bugs_may_never_be_the_same
tags appended for https://www.ted.com/talks/renzo_vitale_what_should_electric_cars_sound_like
tags appended for https://www.ted.com/talks/chiki_sarkar_how_india_s_smartphone_revolution_is_creating_a_new_generation_of_readers_and_writers
tags appended for https://www.ted.com/talks/brooke_buddemeier_and_jessica_s_wieder_can_you_survive_nuclear_fallout
tags appended for https://www.ted.com/talks/martin_danoesastro_what_are_you_willing_to_give_up_to_change_the_way_we_work
tags appended for https://www.ted.com/talks/scott_a_mellor_how_thor_got_his_hammer
tags appended for https://www.ted.com/talks/paula_stone_williams_and_jonathan_williams_the_story_of_a_parent_s_transition_and_a_

tags appended for https://www.ted.com/talks/jonathan_butterworth_what_s_the_smallest_thing_in_the_universe
tags appended for https://www.ted.com/talks/dawn_wacek_a_librarian_s_case_against_overdue_book_fines
tags appended for https://www.ted.com/talks/teresa_bejan_is_civility_a_sham
tags appended for https://www.ted.com/talks/sana_amanat_the_importance_of_diversity_in_the_comic_book_universe
tags appended for https://www.ted.com/talks/stan_lee_what_makes_a_superhero
tags appended for https://www.ted.com/talks/michael_uslan_how_i_brought_batman_to_hollywood_and_academia
tags appended for https://www.ted.com/talks/g_willow_wilson_the_story_behind_marvel_s_muslim_american_superheroine
tags appended for https://www.ted.com/talks/alex_gendler_the_myth_of_sisyphus
tags appended for https://www.ted.com/talks/mark_robinson_and_alex_gendler_history_vs_henry_viii
tags appended for https://www.ted.com/talks/aj_jacobs_my_journey_to_thank_all_the_people_responsible_for_my_morning_coffee
tags append

tags appended for https://www.ted.com/talks/elena_chavez_quezada_we_need_to_help_women_close_the_retirement_savings_gap
tags appended for https://www.ted.com/talks/adam_carroll_what_playing_monopoly_with_real_money_taught_me_about_my_kids_and_humanity
tags appended for https://www.ted.com/talks/leila_seth_why_i_defend_women_s_inheritance_rights
tags appended for https://www.ted.com/talks/deandrea_salvador_how_we_can_make_energy_more_affordable_for_low_income_families
tags appended for https://www.ted.com/talks/kim_preshoff_what_s_a_smartphone_made_of
tags appended for https://www.ted.com/talks/liv_boeree_3_lessons_on_decision_making_from_a_poker_champion
tags appended for https://www.ted.com/talks/bence_nanay_the_end_of_history_illusion
tags appended for https://www.ted.com/talks/isadora_kosofsky_intimate_photos_of_a_senior_love_triangle
tags appended for https://www.ted.com/talks/asali_devan_ecclesiastes_chasms
tags appended for https://www.ted.com/talks/kym_worthy_what_happened_when_

tags appended for https://www.ted.com/talks/elizabeth_cox_what_is_imposter_syndrome_and_how_can_you_combat_it
tags appended for https://www.ted.com/talks/nora_atkinson_why_art_thrives_at_burning_man
tags appended for https://www.ted.com/talks/dread_scott_how_art_can_shape_america_s_conversation_about_freedom
tags appended for https://www.ted.com/talks/elizabeth_cawein_how_to_build_a_thriving_music_scene_in_your_city
tags appended for https://www.ted.com/talks/mary_lou_jepsen_how_we_can_use_light_to_see_deep_inside_our_bodies_and_brains
tags appended for https://www.ted.com/talks/tanya_khovanova_can_you_solve_the_leonardo_da_vinci_riddle
tags appended for https://www.ted.com/talks/leticia_gasca_don_t_fail_fast_fail_mindfully
tags appended for https://www.ted.com/talks/leticia_gasca_una_nueva_manera_de_hablar_del_fracaso_en_los_negocios
tags appended for https://www.ted.com/talks/janet_stovall_how_to_get_serious_about_diversity_and_inclusion_in_the_workplace
tags appended for https://www

tags appended for https://www.ted.com/talks/alison_ledgerwood_a_simple_trick_to_improve_positive_thinking
tags appended for https://www.ted.com/talks/keiichiro_hirano_love_others_to_love_yourself
tags appended for https://www.ted.com/talks/lizzie_velasquez_how_do_you_define_yourself
tags appended for https://www.ted.com/talks/emily_nagoski_the_keys_to_a_happier_healthier_sex_life
tags appended for https://www.ted.com/talks/penny_chisholm_the_tiny_creature_that_secretly_powers_the_planet
tags appended for https://www.ted.com/talks/rodin_lyasoff_how_autonomous_flying_taxis_could_change_the_way_you_travel
tags appended for https://www.ted.com/talks/gordon_hamilton_can_you_solve_the_mondrian_squares_riddle
tags appended for https://www.ted.com/talks/travis_rieder_the_agony_of_opioid_withdrawal_and_what_doctors_should_tell_patients_about_it
tags appended for https://www.ted.com/talks/elizabeth_cox_what_causes_hallucinations
tags appended for https://www.ted.com/talks/andrew_youn_what_if_we_

tags appended for https://www.ted.com/talks/robert_neuwirth_the_age_old_sharing_economies_of_africa_and_why_we_should_scale_them
tags appended for https://www.ted.com/talks/kirsty_duncan_scientists_must_be_free_to_learn_to_speak_and_to_challenge
tags appended for https://www.ted.com/talks/rola_hallam_the_doctors_nurses_and_aid_workers_rebuilding_syria
tags appended for https://www.ted.com/talks/sarah_montana_the_real_risk_of_forgiveness_and_why_it_s_worth_it
tags appended for https://www.ted.com/talks/kate_raworth_a_healthy_economy_should_be_designed_to_thrive_not_grow
tags appended for https://www.ted.com/talks/emily_nagoski_the_truth_about_unwanted_arousal
tags appended for https://www.ted.com/talks/thandiswa_mazwai_iyeza_zabalaza
tags appended for https://www.ted.com/talks/lb_hannahs_what_it_s_like_to_be_a_transgender_dad
tags appended for https://www.ted.com/talks/simone_giertz_why_you_should_make_useless_things
tags appended for https://www.ted.com/talks/sarah_murray_a_playful_sol

In [7]:
len(talk_tags)

1000

In [8]:
for index in range(1000, 1500):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
tags appended for https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
tags appended for https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
tags appended for https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
tags appended for https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
tags appended for https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
tags appended for https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment
tags appended for https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell
tags appended for https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees
tags appended for https://www.ted.com/talks/mother_lon

tags appended for https://www.ted.com/talks/dustin_schroeder_how_we_look_kilometers_below_the_antarctic_ice_sheet
tags appended for https://www.ted.com/talks/shawn_martinbrough_how_one_black_panther_comic_artist_is_changing_perceptions
tags appended for https://www.ted.com/talks/wendy_suzuki_the_brain_changing_benefits_of_exercise
tags appended for https://www.ted.com/talks/chuck_nice_a_funny_look_at_the_unintended_consequences_of_technology
tags appended for https://www.ted.com/talks/raymond_tang_be_humble_and_other_lessons_from_the_philosophy_of_water
tags appended for https://www.ted.com/talks/ilona_stengel_the_role_of_human_emotions_in_science_and_research
tags appended for https://www.ted.com/talks/howard_c_stevenson_how_to_resolve_racially_stressful_situations
tags appended for https://www.ted.com/talks/tapiwa_chiwewe_you_don_t_have_to_be_an_expert_to_solve_big_problems
tags appended for https://www.ted.com/talks/robert_hakiza_refugees_want_empowerment_not_handouts
tags appended 

tags appended for https://www.ted.com/talks/john_cary_how_architecture_can_create_dignity_for_all
tags appended for https://www.ted.com/talks/su_kahumbu_how_we_can_help_hungry_kids_one_text_at_a_time
tags appended for https://www.ted.com/talks/karen_lloyd_this_deep_sea_mystery_is_changing_our_understanding_of_life
tags appended for https://www.ted.com/talks/guy_winch_how_to_fix_a_broken_heart
tags appended for https://www.ted.com/talks/stuart_duncan_how_i_use_minecraft_to_help_kids_with_autism
tags appended for https://www.ted.com/talks/peter_ouko_from_death_row_to_law_graduate
tags appended for https://www.ted.com/talks/danielle_wood_6_space_technologies_we_can_use_to_improve_life_on_earth
tags appended for https://www.ted.com/talks/mwende_freequency_katwiwa_black_life_at_the_intersection_of_birth_and_death
tags appended for https://www.ted.com/talks/david_hooker_who_s_teaching_us_to_read_and_write_visuals
tags appended for https://www.ted.com/talks/anjali_kumar_my_failed_mission_to_f

tags appended for https://www.ted.com/talks/tal_zaks_rewriting_the_genetic_code_a_cancer_cure_in_the_making_dec_2017
tags appended for https://www.ted.com/talks/joseph_mcgill_digging_up_the_past
tags appended for https://www.ted.com/talks/nadine_hachach_haram_how_augmented_reality_could_change_the_future_of_surgery
tags appended for https://www.ted.com/talks/devita_davison_how_urban_agriculture_is_transforming_detroit
tags appended for https://www.ted.com/talks/dao_nguyen_what_makes_something_go_viral
tags appended for https://www.ted.com/talks/alastair_gray_how_fake_handbags_fund_terrorism_and_organized_crime
tags appended for https://www.ted.com/talks/martina_flor_the_secret_language_of_letter_design
tags appended for https://www.ted.com/talks/dan_gartenberg_the_brain_benefits_of_deep_sleep_and_how_to_get_more_of_it
tags appended for https://www.ted.com/talks/wolf_a_marriage_of_opposites_krump_meets_mozart
tags appended for https://www.ted.com/talks/philipp_gerbert_the_basics_of_ai_f

tags appended for https://www.ted.com/talks/levon_biss_mind_blowing_magnified_portraits_of_insects
tags appended for https://www.ted.com/talks/prumsodun_ok_the_magic_of_khmer_classical_dance
tags appended for https://www.ted.com/talks/gabriela_gonzalez_how_ligo_discovered_gravitational_waves_and_what_might_be_next
tags appended for https://www.ted.com/talks/helen_pearson_lessons_from_the_longest_study_on_human_development
tags appended for https://www.ted.com/talks/euna_lee_what_i_learned_as_a_prisoner_in_north_korea
tags appended for https://www.ted.com/talks/anna_heringer_the_warmth_and_wisdom_of_mud_buildings
tags appended for https://www.ted.com/talks/julio_gil_future_tech_will_give_you_the_benefits_of_city_life_anywhere
tags appended for https://www.ted.com/talks/nabila_alibhai_why_people_of_different_faiths_are_painting_their_houses_of_worship_yellow
tags appended for https://www.ted.com/talks/mei_lin_neo_the_fascinating_secret_lives_of_giant_clams
tags appended for https://www.t

tags appended for https://www.ted.com/talks/tricia_wang_the_human_insights_missing_from_big_data
tags appended for https://www.ted.com/talks/anil_seth_your_brain_hallucinates_your_conscious_reality
tags appended for https://www.ted.com/talks/kate_marvel_can_clouds_buy_us_more_time_to_solve_climate_change
tags appended for https://www.ted.com/talks/adam_alter_why_our_screens_make_us_less_happy
tags appended for https://www.ted.com/talks/liz_hajek_what_rivers_can_tell_us_about_the_earth_s_history
tags appended for https://www.ted.com/talks/jorge_ramos_why_journalists_have_an_obligation_to_challenge_power
tags appended for https://www.ted.com/talks/rabbi_lord_jonathan_sacks_how_we_can_face_the_future_without_fear_together
tags appended for https://www.ted.com/talks/manu_prakash_lifesaving_scientific_tools_made_of_paper
tags appended for https://www.ted.com/talks/luma_mufleh_don_t_feel_sorry_for_refugees_believe_in_them
tags appended for https://www.ted.com/talks/cheyenne_cochrane_a_celebr

tags appended for https://www.ted.com/talks/gretchen_carlson_david_brooks_political_common_ground_in_a_polarized_united_states
tags appended for https://www.ted.com/talks/casey_brown_know_your_worth_and_then_ask_for_it
tags appended for https://www.ted.com/talks/lisa_winer_can_you_solve_the_virus_riddle
tags appended for https://www.ted.com/talks/emtithal_mahmoud_a_young_poet_tells_the_story_of_darfur
tags appended for https://www.ted.com/talks/so_percussion_music_for_wood_and_strings
tags appended for https://www.ted.com/talks/moshe_szyf_how_early_life_experience_is_written_into_dna
tags appended for https://www.ted.com/talks/michael_botticelli_addiction_is_a_disease_we_should_treat_it_like_one
tags appended for https://www.ted.com/talks/katie_hinde_what_we_don_t_know_about_mother_s_milk
tags appended for https://www.ted.com/talks/ashton_cofer_a_plan_to_recycle_the_unrecyclable
tags appended for https://www.ted.com/talks/mona_chalabi_3_ways_to_spot_a_bad_statistic
tags appended for ht

In [9]:
len(talk_tags)

1500

In [10]:
for index in range(1500, 2000):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/karina_galperin_should_we_simplify_spelling
tags appended for https://www.ted.com/talks/deanna_pucciarelli_the_history_of_chocolate
tags appended for https://www.ted.com/talks/peggy_orenstein_what_young_women_believe_about_their_own_sexual_pleasure
tags appended for https://www.ted.com/talks/carrie_nugent_adventures_of_an_asteroid_hunter
tags appended for https://www.ted.com/talks/caitlin_doughty_a_burial_practice_that_nourishes_the_planet
tags appended for https://www.ted.com/talks/amy_adkins_the_myth_of_icarus_and_daedalus
tags appended for https://www.ted.com/talks/john_koenig_beautiful_new_words_to_describe_obscure_emotions
tags appended for https://www.ted.com/talks/joy_buolamwini_how_i_m_fighting_bias_in_algorithms
tags appended for https://www.ted.com/talks/jude_kelly_why_women_should_tell_the_stories_of_humanity
tags appended for https://www.ted.com/talks/caroline_paul_to_raise_brave_girls_encourage_adventure
tags appended for https:/

tags appended for https://www.ted.com/talks/judd_a_schorr_can_you_solve_the_airplane_riddle
tags appended for https://www.ted.com/talks/danny_dorling_maps_that_show_us_who_we_are_not_just_where_we_are
tags appended for https://www.ted.com/talks/alicia_garza_patrisse_cullors_and_opal_tometi_an_interview_with_the_founders_of_black_lives_matter
tags appended for https://www.ted.com/talks/joe_lassiter_we_need_nuclear_power_to_solve_climate_change
tags appended for https://www.ted.com/talks/chand_john_what_s_the_fastest_way_to_alphabetize_your_bookshelf
tags appended for https://www.ted.com/talks/adam_galinsky_how_to_speak_up_for_yourself
tags appended for https://www.ted.com/talks/juan_enriquez_what_will_humans_look_like_in_100_years
tags appended for https://www.ted.com/talks/sandi_toksvig_a_political_party_for_women_s_equality
tags appended for https://www.ted.com/talks/roger_antonsen_math_is_the_hidden_secret_to_understanding_the_world
tags appended for https://www.ted.com/talks/victor_

tags appended for https://www.ted.com/talks/sal_khan_let_s_teach_for_mastery_not_test_scores
tags appended for https://www.ted.com/talks/james_veitch_the_agony_of_trying_to_unsubscribe
tags appended for https://www.ted.com/talks/jonathan_tepperman_the_risky_politics_of_progress
tags appended for https://www.ted.com/talks/kio_stark_why_you_should_talk_to_strangers
tags appended for https://www.ted.com/talks/christopher_bell_bring_on_the_female_superheroes
tags appended for https://www.ted.com/talks/akshita_agarwal_the_paradox_of_value
tags appended for https://www.ted.com/talks/julia_bacha_how_women_wage_conflict_without_violence
tags appended for https://www.ted.com/talks/vanessa_ruiz_the_spellbinding_art_of_human_anatomy
tags appended for https://www.ted.com/talks/don_tapscott_how_the_blockchain_is_changing_money_and_business
tags appended for https://www.ted.com/talks/laura_boushnak_the_deadly_legacy_of_cluster_bombs
tags appended for https://www.ted.com/talks/timothy_ihrig_what_we_c

tags appended for https://www.ted.com/talks/patrickjmt_the_origin_of_countless_conspiracy_theories
tags appended for https://www.ted.com/talks/juno_mac_the_laws_that_sex_workers_really_want
tags appended for https://www.ted.com/talks/sebastian_junger_our_lonely_society_makes_it_hard_to_come_home_from_war
tags appended for https://www.ted.com/talks/laura_indolfi_good_news_in_the_fight_against_pancreatic_cancer
tags appended for https://www.ted.com/talks/kang_lee_can_you_really_tell_if_a_kid_is_lying
tags appended for https://www.ted.com/talks/sara_garofalo_the_psychology_behind_irrational_decisions
tags appended for https://www.ted.com/talks/sangeeta_bhatia_this_tiny_particle_could_roam_your_body_to_find_tumors
tags appended for https://www.ted.com/talks/sanford_biggers_an_artist_s_unflinching_look_at_racial_violence
tags appended for https://www.ted.com/talks/kanawat_senanan_how_computer_memory_works
tags appended for https://www.ted.com/talks/uri_hasson_this_is_your_brain_on_communica

tags appended for https://www.ted.com/talks/allan_adams_what_the_discovery_of_gravitational_waves_means
tags appended for https://www.ted.com/talks/shonda_rhimes_my_year_of_saying_yes_to_everything
tags appended for https://www.ted.com/talks/angela_koine_flynn_the_science_of_skin_color
tags appended for https://www.ted.com/talks/celeste_headlee_10_ways_to_have_a_better_conversation
tags appended for https://www.ted.com/talks/jocelyne_bloch_the_brain_may_be_able_to_repair_itself_with_help
tags appended for https://www.ted.com/talks/dorothy_roberts_the_problem_with_race_based_medicine
tags appended for https://www.ted.com/talks/skye_c_cleary_why_do_we_love_a_philosophical_inquiry
tags appended for https://www.ted.com/talks/mike_velings_the_case_for_fish_farming
tags appended for https://www.ted.com/talks/gregory_heyworth_how_i_m_discovering_the_secrets_of_ancient_texts
tags appended for https://www.ted.com/talks/sean_follmer_shape_shifting_tech_will_change_work_as_we_know_it
tags appende

tags appended for https://www.ted.com/talks/suzanne_case_could_moby_dick_prevent_the_next_financial_crisis
tags appended for https://www.ted.com/talks/stephen_lawrence_the_future_of_reading_it_s_fast
tags appended for https://www.ted.com/talks/siawn_ou_the_art_of_letting_go_of_the_floor
tags appended for https://www.ted.com/talks/sahil_sethi_can_0_7_of_corporate_profits_fund_the_world_s_biggest_problems
tags appended for https://www.ted.com/talks/kirsten_hall_women_in_business_entirely_unremarkable
tags appended for https://www.ted.com/talks/katherine_lucas_in_defense_of_extroverts
tags appended for https://www.ted.com/talks/james_bickerton_creating_music_on_the_go
tags appended for https://www.ted.com/talks/chris_vogtherr_a_rallying_cry_for_office_shorts
tags appended for https://www.ted.com/talks/charly_haversat_perfectionism_holds_us_back_here_s_why
tags appended for https://www.ted.com/talks/b_e_a_t_nyc_b_e_a_t_nyc_performs_at_ted_statestreet
tags appended for https://www.ted.com/t

tags appended for https://www.ted.com/talks/alan_eustace_i_leapt_from_the_stratosphere_here_s_how_i_did_it
tags appended for https://www.ted.com/talks/jim_simons_the_mathematician_who_cracked_wall_street
tags appended for https://www.ted.com/talks/jamie_bartlett_how_the_mysterious_dark_net_is_going_mainstream
tags appended for https://www.ted.com/talks/alex_gendler_can_you_solve_the_bridge_riddle
tags appended for https://www.ted.com/talks/elizabeth_nyamayaro_an_invitation_to_men_who_want_a_better_world_for_women
tags appended for https://www.ted.com/talks/wendy_freedman_this_telescope_might_show_us_the_beginning_of_the_universe
tags appended for https://www.ted.com/talks/yves_morieux_how_too_many_rules_at_work_keep_you_from_getting_things_done
tags appended for https://www.ted.com/talks/robin_murphy_these_robots_come_to_the_rescue_after_a_disaster
tags appended for https://www.ted.com/talks/sarthak_sinha_why_do_some_people_go_bald
tags appended for https://www.ted.com/talks/seth_berkl

In [11]:
for index in range(2000, 2500):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
tags appended for https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
tags appended for https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
tags appended for https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
tags appended for https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
tags appended for https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
tags appended for https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
tags appended for https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america
tags appended for https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power
tags appended for https://www.ted.com/talks/murat_dalkilinc_the

tags appended for https://www.ted.com/talks/eleanor_nelsen_how_to_unboil_an_egg
tags appended for https://www.ted.com/talks/clint_smith_how_to_raise_a_black_son_in_america
tags appended for https://www.ted.com/talks/chris_milk_how_virtual_reality_can_create_the_ultimate_empathy_machine
tags appended for https://www.ted.com/talks/emma_bryce_what_really_happens_to_the_plastic_you_throw_away
tags appended for https://www.ted.com/talks/jedidah_isler_how_i_fell_in_love_with_quasars_blazars_and_our_incredible_universe
tags appended for https://www.ted.com/talks/gary_haugen_the_hidden_reason_for_poverty_the_world_needs_to_address_now
tags appended for https://www.ted.com/talks/nathalie_cabrol_how_mars_might_hold_the_secret_to_the_origin_of_life
tags appended for https://www.ted.com/talks/paul_tudor_jones_ii_why_we_need_to_rethink_capitalism
tags appended for https://www.ted.com/talks/takaharu_tezuka_the_best_kindergarten_you_ve_ever_seen
tags appended for https://www.ted.com/talks/kailash_sat

tags appended for https://www.ted.com/talks/joshua_harvey_the_evolution_of_the_human_eye
tags appended for https://www.ted.com/talks/fredy_peccerelli_a_forensic_anthropologist_who_brings_closure_for_the_disappeared
tags appended for https://www.ted.com/talks/aziz_abu_sarah_for_more_tolerance_we_need_more_tourism
tags appended for https://www.ted.com/talks/larry_lagerstrom_einstein_s_miracle_year
tags appended for https://www.ted.com/talks/daniele_quercia_happy_maps
tags appended for https://www.ted.com/talks/shai_marcu_the_benefits_of_a_good_night_s_sleep
tags appended for https://www.ted.com/talks/asha_de_vos_why_you_should_care_about_whale_poo
tags appended for https://www.ted.com/talks/michael_rubinstein_see_invisible_motion_hear_silent_sounds
tags appended for https://www.ted.com/talks/anthony_hazard_the_atlantic_slave_trade_what_too_few_textbooks_told_you
tags appended for https://www.ted.com/talks/stefan_gross_selbeck_business_model_innovation_beating_yourself_at_your_own_game
ta

tags appended for https://www.ted.com/talks/ameenah_gurib_fakim_humble_plants_that_hide_surprising_secrets
tags appended for https://www.ted.com/talks/alessandra_orofino_it_s_our_city_let_s_fix_it
tags appended for https://www.ted.com/talks/jeremy_heimans_what_new_power_looks_like
tags appended for https://www.ted.com/talks/natalya_st_clair_the_unexpected_math_behind_van_gogh_s_starry_night
tags appended for https://www.ted.com/talks/debra_jarvis_yes_i_survived_cancer_but_that_doesn_t_define_me
tags appended for https://www.ted.com/talks/frans_lanting_photos_that_give_voice_to_the_animal_kingdom
tags appended for https://www.ted.com/talks/sergei_lupashin_a_flying_camera_on_a_leash
tags appended for https://www.ted.com/talks/jeff_dekofsky_is_math_discovered_or_invented
tags appended for https://www.ted.com/talks/kimberley_motley_how_i_defend_the_rule_of_law
tags appended for https://www.ted.com/talks/marc_abrahams_a_science_award_that_makes_you_laugh_then_think
tags appended for https:/

tags appended for https://www.ted.com/talks/janet_iwasa_how_animations_can_help_scientists_test_a_hypothesis
tags appended for https://www.ted.com/talks/hubertus_knabe_the_dark_secrets_of_a_surveillance_state
tags appended for https://www.ted.com/talks/margaret_gould_stewart_how_giant_websites_design_for_you_and_a_billion_others_too
tags appended for https://www.ted.com/talks/shai_reshef_an_ultra_low_cost_college_degree
tags appended for https://www.ted.com/talks/anita_collins_how_playing_an_instrument_benefits_your_brain
tags appended for https://www.ted.com/talks/ze_frank_are_you_human
tags appended for https://www.ted.com/talks/lieven_scheire_how_quantum_mechanics_explains_global_warming
tags appended for https://www.ted.com/talks/heather_barnett_what_humans_can_learn_from_semi_intelligent_slime
tags appended for https://www.ted.com/talks/shih_chieh_huang_sculptures_that_d_be_at_home_in_the_deep_sea
tags appended for https://www.ted.com/talks/nikolai_begg_a_tool_to_fix_one_of_the_mo

tags appended for https://www.ted.com/talks/matt_anticole_what_s_the_difference_between_accuracy_and_precision
tags appended for https://www.ted.com/talks/david_brooks_should_you_live_for_your_resume_or_your_eulogy
tags appended for https://www.ted.com/talks/gabby_giffords_and_mark_kelly_be_passionate_be_courageous_be_your_best
tags appended for https://www.ted.com/talks/david_sengeh_the_sore_problem_of_prosthetic_limbs
tags appended for https://www.ted.com/talks/louie_schwartzberg_hidden_miracles_of_the_natural_world
tags appended for https://www.ted.com/talks/george_zaidan_why_is_ketchup_so_hard_to_pour
tags appended for https://www.ted.com/talks/christopher_emdin_teach_teachers_how_to_create_magic
tags appended for https://www.ted.com/talks/amanda_burden_how_public_spaces_make_cities_work
tags appended for https://www.ted.com/talks/lawrence_lessig_the_unstoppable_walk_to_political_reform
tags appended for https://www.ted.com/talks/jennifer_golbeck_your_social_media_likes_expose_more

tags appended for https://www.ted.com/talks/sheryl_sandberg_so_we_leaned_in_now_what
tags appended for https://www.ted.com/talks/mark_kendall_demo_a_needle_free_vaccine_patch_that_s_safer_and_way_cheaper
tags appended for https://www.ted.com/talks/rose_eveleth_the_mystery_of_motion_sickness
tags appended for https://www.ted.com/talks/harish_manwani_profit_s_not_always_the_point
tags appended for https://www.ted.com/talks/ryan_holladay_to_hear_this_music_you_have_to_be_there_literally
tags appended for https://www.ted.com/talks/kate_messner_how_to_build_a_fictional_world
tags appended for https://www.ted.com/talks/frederic_kaplan_how_to_build_an_information_time_machine
tags appended for https://www.ted.com/talks/sandra_aamodt_why_dieting_doesn_t_usually_work


In [12]:
len(talk_tags)

2500

In [13]:
for index in range(2500, 3000):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/nicole_avena_how_sugar_affects_the_brain
tags appended for https://www.ted.com/talks/roger_stein_a_bold_new_way_to_fund_drug_research
tags appended for https://www.ted.com/talks/suzanne_talhouk_don_t_kill_your_language
tags appended for https://www.ted.com/talks/maysoon_zayid_i_got_99_problems_palsy_is_just_one
tags appended for https://www.ted.com/talks/emma_bryce_should_we_eat_bugs
tags appended for https://www.ted.com/talks/mick_cornett_how_an_obese_town_lost_a_million_pounds
tags appended for https://www.ted.com/talks/diana_nyad_never_ever_give_up
tags appended for https://www.ted.com/talks/paul_piff_does_money_make_you_mean
tags appended for https://www.ted.com/talks/rose_eveleth_how_do_we_smell
tags appended for https://www.ted.com/talks/krista_donaldson_the_80_prosthetic_knee_that_s_changing_lives
tags appended for https://www.ted.com/talks/andrew_solomon_depression_the_secret_we_share
tags appended for https://www.ted.com/talks/marco_

tags appended for https://www.ted.com/talks/elizabeth_loftus_how_reliable_is_your_memory
tags appended for https://www.ted.com/talks/benjamin_barber_why_mayors_should_rule_the_world
tags appended for https://www.ted.com/talks/lucianne_walkowicz_light_waves_visible_and_invisible
tags appended for https://www.ted.com/talks/andras_forgacs_leather_and_meat_without_killing_animals
tags appended for https://www.ted.com/talks/eric_berlow_and_sean_gourley_mapping_ideas_worth_spreading
tags appended for https://www.ted.com/talks/marla_spivak_why_bees_are_disappearing
tags appended for https://www.ted.com/talks/james_lyne_everyday_cybercrime_and_what_you_can_do_about_it
tags appended for https://www.ted.com/talks/apollo_robbins_the_art_of_misdirection
tags appended for https://www.ted.com/talks/sonia_shah_3_reasons_we_still_haven_t_gotten_rid_of_malaria
tags appended for https://www.ted.com/talks/ron_mccallum_how_technology_allowed_me_to_read
tags appended for https://www.ted.com/talks/jake_bart

tags appended for https://www.ted.com/talks/lisa_bu_how_books_can_open_your_mind
tags appended for https://www.ted.com/talks/k_erica_dodge_gyotaku_the_ancient_japanese_art_of_printing_fish
tags appended for https://www.ted.com/talks/hendrik_poinar_bring_back_the_woolly_mammoth
tags appended for https://www.ted.com/talks/brian_jones_what_on_earth_is_spin
tags appended for https://www.ted.com/talks/jackson_katz_violence_against_women_it_s_a_men_s_issue
tags appended for https://www.ted.com/talks/ron_shaneyfelt_is_space_trying_to_kill_us
tags appended for https://www.ted.com/talks/paola_antonelli_why_i_brought_pac_man_to_moma
tags appended for https://www.ted.com/talks/ji_hae_park_the_violin_and_my_dark_night_of_the_soul
tags appended for https://www.ted.com/talks/alastair_parvin_architecture_for_the_people_by_the_people
tags appended for https://www.ted.com/talks/george_zaidan_what_is_fat
tags appended for https://www.ted.com/talks/judy_macdonald_johnston_prepare_for_a_good_end_of_life
t

tags appended for https://www.ted.com/talks/cameron_paterson_the_infamous_and_ingenious_ho_chi_minh_trail
tags appended for https://www.ted.com/talks/bono_the_good_news_on_poverty_yes_there_s_good_news
tags appended for https://www.ted.com/talks/christopher_warner_what_is_verbal_irony
tags appended for https://www.ted.com/talks/stewart_brand_the_dawn_of_de_extinction_are_you_ready
tags appended for https://www.ted.com/talks/david_anderson_your_brain_is_more_than_a_bag_of_chemicals
tags appended for https://www.ted.com/talks/dan_pallotta_the_way_we_think_about_charity_is_dead_wrong
tags appended for https://www.ted.com/talks/shane_koyczan_to_this_day_for_the_bullied_and_beautiful
tags appended for https://www.ted.com/talks/peggy_andover_the_difference_between_classical_and_operant_conditioning
tags appended for https://www.ted.com/talks/kakenya_ntaiya_a_girl_who_demanded_school
tags appended for https://www.ted.com/talks/ron_finley_a_guerrilla_gardener_in_south_central_la
tags appended 

tags appended for https://www.ted.com/talks/leah_buechley_how_to_sketch_with_electronics
tags appended for https://www.ted.com/talks/hannah_brencher_love_letters_to_strangers
tags appended for https://www.ted.com/talks/arunachalam_muruganantham_how_i_started_a_sanitary_napkin_revolution
tags appended for https://www.ted.com/talks/julie_burstein_4_lessons_in_creativity
tags appended for https://www.ted.com/talks/jeff_hancock_the_future_of_lying
tags appended for https://www.ted.com/talks/ariel_anbar_a_needle_in_countless_haystacks_finding_habitable_worlds
tags appended for https://www.ted.com/talks/georgette_mulheir_the_tragedy_of_orphanages
tags appended for https://www.ted.com/talks/gary_greenberg_the_beautiful_nano_details_of_our_world
tags appended for https://www.ted.com/talks/jake_wood_a_new_mission_for_veterans_disaster_relief
tags appended for https://www.ted.com/talks/matt_killingsworth_want_to_be_happier_stay_in_the_moment
tags appended for https://www.ted.com/talks/faith_jege

tags appended for https://www.ted.com/talks/tracy_chevalier_finding_the_story_inside_the_painting
tags appended for https://www.ted.com/talks/malte_spitz_your_phone_company_is_watching
tags appended for https://www.ted.com/talks/james_stavridis_a_navy_admiral_s_thoughts_on_global_security
tags appended for https://www.ted.com/talks/vinay_venkatraman_technology_crafts_for_the_digitally_underserved
tags appended for https://www.ted.com/talks/john_graham_cumming_the_greatest_machine_that_never_was
tags appended for https://www.ted.com/talks/neil_harbisson_i_listen_to_color
tags appended for https://www.ted.com/talks/matt_mills_image_recognition_that_triggers_augmented_reality
tags appended for https://www.ted.com/talks/baba_shiv_sometimes_it_s_good_to_give_up_the_driver_s_seat
tags appended for https://www.ted.com/talks/dennis_wildfogel_how_big_is_infinity
tags appended for https://www.ted.com/talks/jamie_drummond_let_s_crowdsource_the_world_s_goals
tags appended for https://www.ted.com/t

In [14]:
for index in range(3000, 3500):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
tags appended for https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
tags appended for https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
tags appended for https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
tags appended for https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
tags appended for https://www.ted.com/talks/hans_rosling_religions_and_babies
tags appended for https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
tags appended for https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen
tags appended for https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief
tags appended for https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out
tags appended for https://www.ted.com/talks/carl_schoonover_how_to_look_inside_the_brain
tags appended for h

tags appended for https://www.ted.com/talks/shilo_shiv_suleman_using_tech_to_enable_dreaming
tags appended for https://www.ted.com/talks/chris_bliss_comedy_is_translation
tags appended for https://www.ted.com/talks/neil_macgregor_2600_years_of_history_in_one_object
tags appended for https://www.ted.com/talks/lucianne_walkowicz_look_up_for_a_change
tags appended for https://www.ted.com/talks/garth_lenz_the_true_cost_of_oil
tags appended for https://www.ted.com/talks/paul_conneally_how_mobile_phones_power_disaster_relief
tags appended for https://www.ted.com/talks/simon_berrow_how_do_you_save_a_shark_you_know_nothing_about
tags appended for https://www.ted.com/talks/lucien_engelen_crowdsource_your_health
tags appended for https://www.ted.com/talks/inara_george_family_tree
tags appended for https://www.ted.com/talks/jenna_mccarthy_what_you_don_t_know_about_marriage
tags appended for https://www.ted.com/talks/tyrone_hayes_penelope_jagessar_chaffer_the_toxic_baby
tags appended for https://w

tags appended for https://www.ted.com/talks/charlie_todd_the_shared_experience_of_absurdity
tags appended for https://www.ted.com/talks/allan_jones_a_map_of_the_brain
tags appended for https://www.ted.com/talks/ben_kacyra_ancient_wonders_captured_in_3d
tags appended for https://www.ted.com/talks/aparna_rao_high_tech_art_with_a_sense_of_humor
tags appended for https://www.ted.com/talks/martin_hanczyc_the_line_between_life_and_not_life
tags appended for https://www.ted.com/talks/sandra_fisher_martins_the_right_to_understand
tags appended for https://www.ted.com/talks/paul_lewis_how_mobile_phones_helped_solve_two_murders
tags appended for https://www.ted.com/talks/marco_tempest_augmented_reality_techno_magic
tags appended for https://www.ted.com/talks/daniel_wolpert_the_real_reason_for_brains
tags appended for https://www.ted.com/talks/anna_mracek_dietrich_a_plane_you_can_drive
tags appended for https://www.ted.com/talks/paul_zak_trust_morality_and_oxytocin
tags appended for https://www.t

tags appended for https://www.ted.com/talks/maya_beiser_a_cello_with_many_voices
tags appended for https://www.ted.com/talks/onyx_ashanti_this_is_beatjazz
tags appended for https://www.ted.com/talks/camille_seaman_haunting_photos_of_polar_ice
tags appended for https://www.ted.com/talks/steve_keil_a_manifesto_for_play_for_bulgaria_and_beyond
tags appended for https://www.ted.com/talks/shea_hembrey_how_i_became_100_artists
tags appended for https://www.ted.com/talks/daniel_kraft_medicine_s_future_there_s_an_app_for_that
tags appended for https://www.ted.com/talks/jd_schramm_break_the_silence_for_suicide_attempt_survivors
tags appended for https://www.ted.com/talks/alice_dreger_is_anatomy_destiny
tags appended for https://www.ted.com/talks/paul_romer_the_world_s_first_charter_city
tags appended for https://www.ted.com/talks/janet_echelman_taking_imagination_seriously
tags appended for https://www.ted.com/talks/jack_horner_building_a_dinosaur_from_a_chicken
tags appended for https://www.te

tags appended for https://www.ted.com/talks/lisa_gansky_the_future_of_business_is_the_mesh
tags appended for https://www.ted.com/talks/jacqueline_novogratz_inspiring_a_life_of_immersion
tags appended for https://www.ted.com/talks/patricia_kuhl_the_linguistic_genius_of_babies
tags appended for https://www.ted.com/talks/krista_tippett_reconnecting_with_compassion
tags appended for https://www.ted.com/talks/geert_chatrou_a_whistleblower_you_haven_t_heard
tags appended for https://www.ted.com/talks/michael_pawlyn_using_nature_s_genius_in_architecture
tags appended for https://www.ted.com/talks/hawa_abdi_deqo_mohamed_mother_and_daughter_doctor_heroes
tags appended for https://www.ted.com/talks/cynthia_breazeal_the_rise_of_personal_robots
tags appended for https://www.ted.com/talks/nigel_marsh_how_to_make_work_life_balance_work
tags appended for https://www.ted.com/talks/suheir_hammad_poems_of_war_peace_women_power
tags appended for https://www.ted.com/talks/christopher_mcdougall_are_we_born

tags appended for https://www.ted.com/talks/eben_bayer_are_mushrooms_the_new_plastic
tags appended for https://www.ted.com/talks/mechai_viravaidya_how_mr_condom_made_thailand_a_better_place_for_life_and_love
tags appended for https://www.ted.com/talks/inge_missmahl_bringing_peace_to_the_minds_of_afghanistan
tags appended for https://www.ted.com/talks/sebastian_seung_i_am_my_connectome
tags appended for https://www.ted.com/talks/gary_wolf_the_quantified_self
tags appended for https://www.ted.com/talks/tim_birkhead_the_wisdom_of_birds
tags appended for https://www.ted.com/talks/julian_treasure_shh_sound_health_in_8_steps
tags appended for https://www.ted.com/talks/fabian_hemmert_the_shape_shifting_future_of_the_mobile_phone
tags appended for https://www.ted.com/talks/annie_lennox_why_i_am_an_hiv_aids_activist
tags appended for https://www.ted.com/talks/mitchell_besser_mothers_helping_mothers_fight_hiv
tags appended for https://www.ted.com/talks/steven_johnson_where_good_ideas_come_from
t

In [15]:
len(talk_tags)

3500

In [16]:
for index in range(3500, 4000):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/diane_j_savino_the_case_for_same_sex_marriage
tags appended for https://www.ted.com/talks/sheryl_wudunn_our_century_s_greatest_injustice
tags appended for https://www.ted.com/talks/jason_clay_how_big_brands_can_help_save_biodiversity
tags appended for https://www.ted.com/talks/lewis_pugh_my_mind_shifting_everest_swim
tags appended for https://www.ted.com/talks/laurie_santos_a_monkey_economy_as_irrational_as_ours
tags appended for https://www.ted.com/talks/john_delaney_wiring_an_interactive_ocean
tags appended for https://www.ted.com/talks/jeff_bezos_gifts_vs_choices
tags appended for https://www.ted.com/talks/susan_shaw_the_oil_spill_s_toxic_trade_off
tags appended for https://www.ted.com/talks/sheena_iyengar_the_art_of_choosing
tags appended for https://www.ted.com/talks/kevin_stone_the_bio_future_of_joint_replacement
tags appended for https://www.ted.com/talks/tan_le_a_headset_that_reads_your_brainwaves
tags appended for https://www.ted.com

tags appended for https://www.ted.com/talks/derek_sivers_how_to_start_a_movement
tags appended for https://www.ted.com/talks/kirk_citron_and_now_the_real_news
tags appended for https://www.ted.com/talks/shukla_bose_teaching_one_child_at_a_time
tags appended for https://www.ted.com/talks/kevin_bales_how_to_combat_modern_slavery
tags appended for https://www.ted.com/talks/patsy_rodenburg_why_i_do_theater
tags appended for https://www.ted.com/talks/robert_gupta
tags appended for https://www.ted.com/talks/joel_levine_why_we_need_to_go_back_to_mars
tags appended for https://www.ted.com/talks/alan_siegel_let_s_simplify_legal_jargon
tags appended for https://www.ted.com/talks/juliana_machado_ferreira_the_fight_to_end_rare_animal_trafficking_in_brazil
tags appended for https://www.ted.com/talks/sam_harris_science_can_answer_moral_questions
tags appended for https://www.ted.com/talks/shekhar_kapur_we_are_the_stories_we_tell_ourselves
tags appended for https://www.ted.com/talks/ken_kamler_medica

tags appended for https://www.ted.com/talks/magnus_larsson_turning_dunes_into_architecture
tags appended for https://www.ted.com/talks/rob_hopkins_transition_to_a_world_without_oil
tags appended for https://www.ted.com/talks/hans_rosling_asia_s_rise_how_and_when
tags appended for https://www.ted.com/talks/tom_wujec_learn_to_use_the_13th_century_astrolabe
tags appended for https://www.ted.com/talks/devdutt_pattanaik_east_vs_west_the_myths_that_mystify
tags appended for https://www.ted.com/talks/fields_wicker_miurin_learning_from_leadership_s_missing_manual
tags appended for https://www.ted.com/talks/mathieu_lehanneur_science_inspired_design
tags appended for https://www.ted.com/talks/pranav_mistry_the_thrilling_potential_of_sixthsense_technology
tags appended for https://www.ted.com/talks/cynthia_schneider_the_surprising_spread_of_idol_tv
tags appended for https://www.ted.com/talks/edward_burtynsky_photographing_the_landscape_of_oil
tags appended for https://www.ted.com/talks/rachel_pik

tags appended for https://www.ted.com/talks/robert_full_learning_from_the_gecko_s_tail
tags appended for https://www.ted.com/talks/nancy_etcoff_happiness_and_its_surprises
tags appended for https://www.ted.com/talks/john_la_grou_a_plug_for_smart_power_outlets
tags appended for https://www.ted.com/talks/kevin_surace_eco_friendly_drywall
tags appended for https://www.ted.com/talks/pete_alcorn_the_world_in_2200
tags appended for https://www.ted.com/talks/felix_dennis_odes_to_vice_and_consequences
tags appended for https://www.ted.com/talks/yann_arthus_bertrand_a_wide_angle_view_of_fragile_earth
tags appended for https://www.ted.com/talks/ray_kurzweil_a_university_for_the_coming_singularity
tags appended for https://www.ted.com/talks/liz_coleman_a_call_to_reinvent_liberal_arts_education
tags appended for https://www.ted.com/talks/kaki_king_playing_pink_noise_on_guitar
tags appended for https://www.ted.com/talks/jonathan_drori_why_we_re_storing_billions_of_seeds
tags appended for https://ww

tags appended for https://www.ted.com/talks/woody_norris_hypersonic_sound_and_other_inventions
tags appended for https://www.ted.com/talks/sherwin_nuland_the_extraordinary_power_of_ordinary_people
tags appended for https://www.ted.com/talks/barry_schuler_genomics_101
tags appended for https://www.ted.com/talks/jamais_cascio_tools_for_a_better_world
tags appended for https://www.ted.com/talks/david_carson_design_and_discovery
tags appended for https://www.ted.com/talks/paula_scher_great_design_is_serious_not_solemn
tags appended for https://www.ted.com/talks/joseph_pine_what_consumers_want
tags appended for https://www.ted.com/talks/peter_reinhart_the_art_and_craft_of_bread
tags appended for https://www.ted.com/talks/scott_mccloud_the_visual_magic_of_comics
tags appended for https://www.ted.com/talks/rob_forbes_ways_of_seeing
tags appended for https://www.ted.com/talks/greg_lynn_organic_algorithms_in_architecture
tags appended for https://www.ted.com/talks/paul_moller_my_dream_of_a_flyi

tags appended for https://www.ted.com/talks/john_q_walker_great_piano_performances_recreated
tags appended for https://www.ted.com/talks/nellie_mckay_the_dog_song
tags appended for https://www.ted.com/talks/ian_dunbar_dog_friendly_dog_training
tags appended for https://www.ted.com/talks/lennart_green_close_up_card_magic_with_a_twist
tags appended for https://www.ted.com/talks/david_griffin_how_photography_connects_us
tags appended for https://www.ted.com/talks/spencer_wells_a_family_tree_for_humanity
tags appended for https://www.ted.com/talks/patricia_burchat_shedding_light_on_dark_matter
tags appended for https://www.ted.com/talks/bruno_bowden_rufus_cappadocia_watch_me_fold_origami_blindfolded
tags appended for https://www.ted.com/talks/robert_lang_the_math_and_magic_of_origami
tags appended for https://www.ted.com/talks/kwabena_boahen_a_computer_that_works_like_the_brain
tags appended for https://www.ted.com/talks/kevin_kelly_the_next_5_000_days_of_the_web
tags appended for https://

In [17]:
len(talk_tags)

4000

In [18]:
len(talk_df['url'])

4244

In [19]:
for index in range(4000, len(talk_df['url'])):
    link = "https://www.ted.com" + talk_df['url'][index]
    try:
        page = requests.get(link)
        content = BS(page.content, 'html.parser')
        talk_tags.append([x['content'] for x in content.find_all('meta', property = 'og:video:tag')])
        print('tags appended for ' + link)
        time.sleep(2)
    except:
        print('NO TAGS APPENDED @ ' + link)
        talk_tags.append('NO TAGS APPENDED')

tags appended for https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
tags appended for https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
tags appended for https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
tags appended for https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
tags appended for https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
tags appended for https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
tags appended for https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
tags appended for https://www.ted.com/talks/rokia_traore_kounandi
tags appended for https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq
tags appended for https://www.ted.com/talks/susan_blackmore_memes_and_temes
tags appended for https://www.ted.com/talks/paul_collier_the_bottom_billion
tags appended for https://www.ted.com/talks/seyi_oyesola_a_hospital_to

tags appended for https://www.ted.com/talks/richard_branson_life_at_30_000_feet
tags appended for https://www.ted.com/talks/kenichi_ebina_my_magic_moves
tags appended for https://www.ted.com/talks/carolyn_porco_this_is_saturn
tags appended for https://www.ted.com/talks/paul_maccready_a_flight_on_solar_wings
tags appended for https://www.ted.com/talks/stephen_petranek_10_ways_the_world_could_end
tags appended for https://www.ted.com/talks/john_maeda_designing_for_simplicity
tags appended for https://www.ted.com/talks/zeresenay_alemseged_the_search_for_humanity_s_roots
tags appended for https://www.ted.com/talks/deborah_scranton_an_iraq_war_movie_crowd_sourced_from_soldiers
tags appended for https://www.ted.com/talks/steven_pinker_the_surprising_decline_in_violence
tags appended for https://www.ted.com/talks/steven_pinker_what_our_language_habits_reveal
tags appended for https://www.ted.com/talks/theo_jansen_my_creations_a_new_form_of_life
tags appended for https://www.ted.com/talks/andr

tags appended for https://www.ted.com/talks/robert_wright_progress_is_not_a_zero_sum_game
tags appended for https://www.ted.com/talks/martin_rees_is_this_our_final_century
tags appended for https://www.ted.com/talks/phil_borges_photos_of_endangered_cultures
tags appended for https://www.ted.com/talks/wade_davis_dreams_from_endangered_cultures
tags appended for https://www.ted.com/talks/bjorn_lomborg_global_priorities_bigger_than_climate_change
tags appended for https://www.ted.com/talks/robert_neuwirth_the_hidden_world_of_shadow_cities
tags appended for https://www.ted.com/talks/rives_if_i_controlled_the_internet
tags appended for https://www.ted.com/talks/dean_ornish_the_killer_american_diet_that_s_sweeping_the_planet
tags appended for https://www.ted.com/talks/richard_st_john_8_secrets_of_success
tags appended for https://www.ted.com/talks/peter_gabriel_fight_injustice_with_raw_video
tags appended for https://www.ted.com/talks/ray_kurzweil_the_accelerating_power_of_technology
tags ap

In [20]:
len(talk_tags)

4244

In [22]:
for i in range(0, len(talk_tags)):
    if talk_tags[i] == 'NO TAGS APPENDED':
        print(i)

1270


In [23]:
talk_df['tags'] = talk_tags

In [25]:
for index in range(0, len(talk_df)):
    if talk_df['tags'][index] == 'NO TAGS APPENDED':
        url = talk_df['url'][index]
        link = "https://www.ted.com" + url
        print(link)
        try:
            page = requests.get(link)
            content = BS(page.content, 'html.parser')
            talk_df['tags'][index] = [x['content'] for x in content.find_all('meta', property = 'og:video:tag')]
            print('appended tags for ' + link)
            links_appended += 1
            time.sleep(3)
        except:
            print('unable to append description @ ' + link)

https://www.ted.com/talks/scott_galloway_how_amazon_apple_facebook_and_google_manipulate_our_emotions
appended tags for https://www.ted.com/talks/scott_galloway_how_amazon_apple_facebook_and_google_manipulate_our_emotions
unable to append description @ https://www.ted.com/talks/scott_galloway_how_amazon_apple_facebook_and_google_manipulate_our_emotions


In [27]:
(talk_df.tags == 'NO TAGS APPENDED').value_counts()

False    4244
Name: tags, dtype: int64

In [28]:
talk_df.to_csv('talks_with_tags.csv')

In [2]:
talk_df = pd.read_csv('talks_with_tags.csv', index_col = 0)

In [3]:
talk_df.head()

,date,speaker,title,url,length,summ,tags
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so..."
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'..."
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a..."
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ..."
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',..."


# Get Views

In [65]:
talk_df['views'] = '--'

In [67]:
def get_views(start, end):
    for index in range(start, end):
        try:
            # Get link, parse using BeautifulSoup
            url = talk_df['url'][index]
            link = "https://www.ted.com" + url
            print(link)
            page = requests.get(link)
            content = BS(page.content, 'html.parser')

            # Get span class, find views
            for x in content.find_all('span'):
                if 'views' in x.text:
                    pattern = re.compile(r'\s+')
                    no_ws = re.sub(pattern, '', x.text).strip('views')
                    if no_ws != '':
                        talk_df['views'][index] = int(no_ws.replace(',', '')) # remove commas, cast as int
            time.sleep(2)
        except:
            print('NO VIEWS')

In [68]:
get_views(0, 500)

https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator
https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol
https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women
https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear
https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success
https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time
https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism
https://www.ted.com/talks/nick_bostrom_how_civilization_could_destroy_itself_and_4_ways_we_could_prevent_it
https://www.ted.com/talks/jasmine_crowe_what_we_re_getting_wrong_in_the_fight_to_end_hunger
https://www.ted.com/talks/sinan_aral_how_we_can_protect_truth_in_the_age_of_misinformation
https://www.ted.com/talks/dan_kwartler_why_should_you_read_dune_by_frank_herbert
https://www.ted.com/talks/enrico_ramirez_ruiz_your_bod

https://www.ted.com/talks/alyssa_loorya_what_i_learned_from_digging_under_new_york_city_s_streets
https://www.ted.com/talks/ella_al_shamahi_why_archaeology_needs_to_transcend_borders
https://www.ted.com/talks/rose_m_mutiso_how_to_bring_affordable_sustainable_electricity_to_africa
https://www.ted.com/talks/andrew_nemr_the_sounds_and_sights_of_tap_dance
https://www.ted.com/talks/hiromi_ozaki_how_i_bring_myth_and_magic_to_life
https://www.ted.com/talks/alexis_gambis_why_we_need_more_real_science_in_fiction
https://www.ted.com/talks/beau_lotto_and_cirque_du_soleil_how_we_experience_awe_and_why_it_matters
https://www.ted.com/talks/abigail_spanberger_how_to_connect_with_people_who_are_different_than_you
https://www.ted.com/talks/claire_wardle_how_you_can_help_transform_the_internet_into_a_place_of_trust
https://www.ted.com/talks/adrienne_mayor_the_greek_myth_of_talos_the_first_robot
https://www.ted.com/talks/ashwin_naidu_the_link_between_fishing_cats_and_mangrove_forest_conservation
https://

https://www.ted.com/talks/anthony_veneziale_stumbling_towards_intimacy_an_improvised_ted_talk
https://www.ted.com/talks/emily_f_rothman_how_porn_changes_the_way_teens_think_about_sex
https://www.ted.com/talks/iseult_gillespie_the_secret_student_resistance_to_hitler
https://www.ted.com/talks/jochen_wegner_what_happened_when_we_paired_up_thousands_of_strangers_to_talk_politics
https://www.ted.com/talks/haaziq_kazi_cleaning_our_oceans_a_big_plan_for_a_big_problem
https://www.ted.com/talks/asmeret_asefaw_berhe_a_climate_change_solution_that_s_right_under_our_feet
https://www.ted.com/talks/yeonmi_park_what_i_learned_about_freedom_after_escaping_north_korea
https://www.ted.com/talks/lee_thomas_how_i_help_people_understand_vitiligo
https://www.ted.com/talks/anne_f_broadbridge_the_rise_and_fall_of_the_mongol_empire
https://www.ted.com/talks/dessa_can_we_choose_to_fall_out_of_love_nov_2019
https://www.ted.com/talks/kelly_wanser_emergency_medicine_for_our_climate_fever
https://www.ted.com/talks/

https://www.ted.com/talks/rob_reid_how_synthetic_biology_could_wipe_out_humanity_and_how_we_can_stop_it
https://www.ted.com/talks/ayana_elizabeth_johnson_and_megan_davis_could_underwater_farms_help_fight_climate_change
https://www.ted.com/talks/kaitlyn_sadtler_your_body_vs_implants
https://www.ted.com/talks/david_baker_5_challenges_we_could_solve_by_designing_new_proteins
https://www.ted.com/talks/steve_simpson_how_ocean_noise_destroys_marine_ecosystems
https://www.ted.com/talks/michael_tubbs_the_political_power_of_being_a_good_neighbor
https://www.ted.com/talks/emeli_sande_you_are_not_alone_extraordinary_being_shine
https://www.ted.com/talks/daniel_lismore_my_life_as_a_work_of_art
https://www.ted.com/talks/lucy_clayton_all_the_rage_why_dressing_up_helps_us_speak_out
https://www.ted.com/talks/truls_nord_tactile_photographs_that_display_worlds_of_light_shadow_and_mood
https://www.ted.com/talks/stefen_chow_the_everyday_beauty_of_playgrounds
https://www.ted.com/talks/robert_frost_the_road

https://www.ted.com/talks/joanne_chory_how_supercharged_plants_could_slow_climate_change
https://www.ted.com/talks/iseult_gillespie_why_is_this_painting_so_shocking
https://www.ted.com/talks/michele_wucker_why_we_ignore_obvious_problems_and_how_to_act_on_them
https://www.ted.com/talks/yana_buhrer_tavanier_how_to_recover_from_activism_burnout
https://www.ted.com/talks/hannah_gadsby_three_ideas_three_contradictions_or_not
https://www.ted.com/talks/oliver_elfenbaum_how_does_the_stock_market_work
https://www.ted.com/talks/elizabeth_dunn_helping_others_makes_us_happier_but_it_matters_how_we_do_it
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_cuddly_duddly_fuddly_wuddly_riddle
https://www.ted.com/talks/kim_gorgens_the_surprising_connection_between_brain_injuries_and_crime
https://www.ted.com/talks/silvia_moreno_garcia_titan_of_terror_the_dark_imagination_of_h_p_lovecraft
https://www.ted.com/talks/sajan_saini_the_hidden_network_that_makes_the_internet_possible
https://www.ted.com/

https://www.ted.com/talks/farida_nabourema_is_your_country_at_risk_of_becoming_a_dictatorship_here_s_how_to_know
https://www.ted.com/talks/pat_crowley_farming_insects_to_save_the_world
https://www.ted.com/talks/karl_skjonnemand_the_self_assembling_computer_chips_of_the_future
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_jail_break_riddle
https://www.ted.com/talks/lindy_lou_isonhood_a_juror_s_reflections_on_the_death_penalty
https://www.ted.com/talks/ayana_jamieson_and_moya_bailey_why_should_you_read_sci_fi_superstar_octavia_e_butler
https://www.ted.com/talks/peggy_weil_landscapes_of_climate_change_overscapes_and_underscapes
https://www.ted.com/talks/ronald_rael_an_architect_s_subversive_reimagining_of_the_us_mexico_border_wall
https://www.ted.com/talks/dolores_huerta_how_to_overcome_apathy_and_find_your_power
https://www.ted.com/talks/christina_greer_an_unsung_hero_of_the_civil_rights_movement
https://www.ted.com/talks/ashweetha_shetty_how_education_helped_me_rewrite_my_li

In [70]:
get_views(500, 1000)

https://www.ted.com/talks/yannick_roudaut_how_today_s_truths_shape_tomorrow_s_possibilities
https://www.ted.com/talks/michael_benton_mass_extinctions_and_the_future_of_life_on_earth
https://www.ted.com/talks/lindiwe_mazibuko_why_the_african_diaspora_is_crucial_to_the_continent_s_future
https://www.ted.com/talks/juan_enriquez_the_age_of_genetic_wonder_nov_2019
https://www.ted.com/talks/anne_gaskett_the_sexual_deception_of_orchids
https://www.ted.com/talks/aja_monet_and_phillip_agnew_a_love_story_about_the_power_of_art_as_organizing
https://www.ted.com/talks/glenn_cantave_vr_ar_for_social_change
https://www.ted.com/talks/steven_petrow_3_ways_to_practice_civility
https://www.ted.com/talks/christina_greer_notes_of_a_native_son_the_world_according_to_james_baldwin
https://www.ted.com/talks/dropbox_how_one_team_turned_a_sprint_project_into_a_marathon_success
https://www.ted.com/talks/p_j_parmar_how_doctors_can_help_low_income_patients_and_still_make_a_profit
https://www.ted.com/talks/mathias

https://www.ted.com/talks/jan_rader_in_the_opioid_crisis_here_s_what_it_takes_to_save_a_life
https://www.ted.com/talks/ariana_curtis_museums_should_honor_the_everyday_not_just_the_extraordinary
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_troll_s_paradox_riddle
https://www.ted.com/talks/dana_kanze_the_real_reason_female_entrepreneurs_get_less_funding
https://www.ted.com/talks/rebecca_love_nurse_innovation_saving_the_future_of_healthcare
https://www.ted.com/talks/alex_rosenthal_the_joyful_perplexing_world_of_puzzle_hunts
https://www.ted.com/talks/darrick_hamilton_how_baby_bonds_could_help_close_the_wealth_gap
https://www.ted.com/talks/eldridge_adams_can_animals_be_deceptive
https://www.ted.com/talks/eldra_jackson_how_i_unlearned_dangerous_lessons_about_masculinity
https://www.ted.com/talks/julie_chang_instagram_filters_for_robots_optical_neural_networks
https://www.ted.com/talks/fiona_radford_from_slave_to_rebel_gladiator_the_life_of_spartacus
https://www.ted.com/talks/li_w

https://www.ted.com/talks/brian_d_avery_how_rollercoasters_affect_your_body
https://www.ted.com/talks/lauren_esposito_saving_human_kind_one_scorpion_at_a_time
https://www.ted.com/talks/rachel_wurzman_how_isolation_fuels_opioid_addiction
https://www.ted.com/talks/shane_wickes_why_i_came_out_as_a_gay_football_coach
https://www.ted.com/talks/alexis_jones_redefining_manhood_one_locker_room_talk_at_a_time
https://www.ted.com/talks/charles_c_mann_how_will_we_survive_when_the_population_hits_10_billion
https://www.ted.com/talks/a_tribe_called_red_we_are_the_halluci_nation
https://www.ted.com/talks/dennis_shasha_can_you_solve_the_stolen_rubies_riddle
https://www.ted.com/talks/elizabeth_streb_my_quest_to_defy_gravity_and_fly
https://www.ted.com/talks/darieth_chisolm_how_revenge_porn_turns_lives_upside_down
https://www.ted.com/talks/henrietta_fore_how_we_can_help_young_people_build_a_better_future
https://www.ted.com/talks/andrew_zimmerman_jones_does_time_exist
https://www.ted.com/talks/wanis_ka

https://www.ted.com/talks/hector_lanz_how_do_focus_groups_work
https://www.ted.com/talks/heba_shaheed_is_it_bad_to_hold_your_pee
https://www.ted.com/talks/emma_bryce_the_power_of_the_placebo_effect
https://www.ted.com/talks/katherine_hampsten_how_miscommunication_happens_and_how_to_avoid_it
https://www.ted.com/talks/noah_zandan_the_language_of_lying
https://www.ted.com/talks/damon_brown_how_to_choose_your_news
https://www.ted.com/talks/andrew_bastawrous_a_new_way_to_fund_health_care_for_the_most_vulnerable
https://www.ted.com/talks/trevor_maber_rethinking_thinking
https://www.ted.com/talks/alex_gendler_can_you_solve_the_rebel_supplies_riddle
https://www.ted.com/talks/pierre_barreau_how_ai_could_compose_a_personalized_soundtrack_to_your_life
https://www.ted.com/talks/mark_pollock_and_simone_george_a_love_letter_to_realism_in_a_time_of_grief
https://www.ted.com/talks/fred_krupp_let_s_launch_a_satellite_to_track_a_threatening_greenhouse_gas
https://www.ted.com/talks/sajan_saini_what_is_th

https://www.ted.com/talks/dan_gibson_how_to_build_synthetic_dna_and_send_it_across_the_internet
https://www.ted.com/talks/dan_knights_how_we_study_the_microbes_living_in_your_gut
https://www.ted.com/talks/lina_marieth_hoyos_what_is_the_coldest_thing_in_the_world
https://www.ted.com/talks/kola_masha_how_farming_could_employ_africa_s_young_workforce_and_help_build_peace
https://www.ted.com/talks/gary_liu_the_rapid_growth_of_the_chinese_internet_and_where_it_s_headed
https://www.ted.com/talks/rebecca_tarvin_why_don_t_poisonous_animals_poison_themselves
https://www.ted.com/talks/sophie_zadeh_are_there_universal_expressions_of_emotion
https://www.ted.com/talks/jakob_magolan_a_crash_course_in_organic_chemistry
https://www.ted.com/talks/jennifer_wilcox_a_new_way_to_remove_co2_from_the_atmosphere
https://www.ted.com/talks/steve_boyes_how_we_re_saving_one_of_earth_s_last_wild_places
https://www.ted.com/talks/lindsay_malloy_why_teens_confess_to_crimes_they_didn_t_commit
https://www.ted.com/talks

https://www.ted.com/talks/emily_nagoski_the_truth_about_unwanted_arousal
https://www.ted.com/talks/thandiswa_mazwai_iyeza_zabalaza
https://www.ted.com/talks/lb_hannahs_what_it_s_like_to_be_a_transgender_dad
https://www.ted.com/talks/simone_giertz_why_you_should_make_useless_things
https://www.ted.com/talks/sarah_murray_a_playful_solution_to_the_housing_crisis
https://www.ted.com/talks/erricka_bridgeford_how_baltimore_called_a_ceasefire
https://www.ted.com/talks/priya_vulchi_and_winona_guo_what_it_takes_to_be_racially_literate
https://www.ted.com/talks/kevin_etter_i_am_the_donation
https://www.ted.com/talks/frances_frei_how_to_build_and_rebuild_trust
https://www.ted.com/talks/rebecca_kleinberger_why_you_don_t_like_the_sound_of_your_own_voice
https://www.ted.com/talks/tania_douglas_to_design_better_tech_understand_context
https://www.ted.com/talks/laura_l_dunn_it_s_time_for_the_law_to_protect_victims_of_gender_violence
https://www.ted.com/talks/john_amory_how_a_male_contraceptive_pill_co

In [75]:
get_views(1000, 2000)

https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment
https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell
https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees
https://www.ted.com/talks/mother_london_ai_therapy
https://www.ted.com/talks/duncan_cowles_it_s_not_amazing_enough
https://www.ted.com/talks/aaron_duffy_lake_buckley_and_jack_foster_illusions_for_a_better_society
htt

https://www.ted.com/talks/shade_zahrai_and_aric_yegudkin_the_rhythmic_beauty_of_collaboration
https://www.ted.com/talks/will_jenkins_the_most_precious_commodity_of_all
https://www.ted.com/talks/tomer_garzberg_what_happens_when_we_take_humans_out_of_work
https://www.ted.com/talks/rach_ranton_where_are_we_trying_to_end_up
https://www.ted.com/talks/olivia_tyler_the_complex_path_to_sustainability
https://www.ted.com/talks/michael_mieni_using_aboriginal_traditions_to_teach_tech
https://www.ted.com/talks/linda_zhang_why_schools_should_teach_entrepreneurship
https://www.ted.com/talks/hayley_yeates_it_s_past_time_to_appreciate_cultural_diversity
https://www.ted.com/talks/alastair_o_neill_when_genetic_choice_becomes_personal
https://www.ted.com/talks/wale_oyejide_fashion_that_celebrates_african_strength_and_spirit
https://www.ted.com/talks/dixon_chibanda_why_i_train_grandmothers_to_treat_depression
https://www.ted.com/talks/nina_dolvik_brochmann_and_ellen_stokken_dahl_the_virginity_fraud
https:

https://www.ted.com/talks/matt_goldman_the_search_for_aha_moments
https://www.ted.com/talks/vivek_maru_how_to_put_the_power_of_law_in_people_s_hands
https://www.ted.com/talks/jacob_collier_a_one_man_musical_phenomenon
https://www.ted.com/talks/marily_oppezzo_want_to_be_more_creative_go_for_a_walk
https://www.ted.com/talks/alexis_charpentier_how_record_collectors_find_lost_music_and_preserve_our_cultural_heritage
https://www.ted.com/talks/kendall_spencer_what_can_we_learn_from_college_athletes
https://www.ted.com/talks/kevin_njabo_how_we_can_stop_africa_s_scientific_brain_drain
https://www.ted.com/talks/soyapi_mumba_medical_tech_designed_to_meet_africa_s_needs
https://www.ted.com/talks/mindy_scheier_how_adaptive_clothing_empowers_people_with_disabilities
https://www.ted.com/talks/tomas_elemans_the_inspiring_truth_in_fiction
https://www.ted.com/talks/steve_brown_how_data_is_driving_the_future_of_fashion
https://www.ted.com/talks/touria_el_glaoui_inside_africa_s_thriving_art_scene
https:/

https://www.ted.com/talks/nnedi_okorafor_sci_fi_stories_that_imagine_a_future_africa
https://www.ted.com/talks/lauren_sallan_how_to_win_at_evolution_and_survive_a_mass_extinction
https://www.ted.com/talks/tim_kruger_can_we_stop_climate_change_by_removing_co2_from_the_air
https://www.ted.com/talks/shonda_rhimes_and_cyndi_stivers_the_future_of_storytelling
https://www.ted.com/talks/zeynep_tufekci_we_re_building_a_dystopia_just_to_make_people_click_on_ads
https://www.ted.com/talks/carlos_bautista_the_awful_logic_of_land_mines_and_an_app_that_helps_people_avoid_them
https://www.ted.com/talks/martin_ford_how_we_ll_earn_money_in_a_future_without_jobs
https://www.ted.com/talks/rocio_lorenzo_how_diversity_makes_teams_more_innovative
https://www.ted.com/talks/anjan_sundaram_why_i_risked_my_life_to_expose_a_government_massacre
https://www.ted.com/talks/naomi_mcdougall_jones_what_it_s_like_to_be_a_woman_in_hollywood
https://www.ted.com/talks/mike_kinney_a_pro_wrestler_s_guide_to_confidence
https:

https://www.ted.com/talks/joseph_redmon_how_computers_learn_to_recognize_objects_instantly
https://www.ted.com/talks/francoise_mouly_the_stories_behind_the_new_yorker_s_iconic_covers
https://www.ted.com/talks/ingrid_betancourt_what_six_years_in_captivity_taught_me_about_fear_and_faith
https://www.ted.com/talks/titus_kaphar_can_art_amend_history
https://www.ted.com/talks/marc_raibert_meet_spot_the_robot_dog_that_can_run_hop_and_open_doors
https://www.ted.com/talks/kristen_marhaver_why_i_still_have_hope_for_coral_reefs
https://www.ted.com/talks/jennifer_pluznick_you_smell_with_your_body_not_just_your_nose
https://www.ted.com/talks/tristan_harris_how_a_handful_of_tech_companies_control_billions_of_minds_every_day
https://www.ted.com/talks/jimmy_lin_a_simple_new_blood_test_that_can_catch_cancer_early
https://www.ted.com/talks/grace_kim_how_cohousing_can_make_us_happier_and_live_longer
https://www.ted.com/talks/susan_robinson_how_i_fail_at_being_disabled
https://www.ted.com/talks/noah_feldm

...

https://www.ted.com/talks/chand_john_what_s_the_fastest_way_to_alphabetize_your_bookshelf
https://www.ted.com/talks/adam_galinsky_how_to_speak_up_for_yourself
https://www.ted.com/talks/juan_enriquez_what_will_humans_look_like_in_100_years
https://www.ted.com/talks/sandi_toksvig_a_political_party_for_women_s_equality
https://www.ted.com/talks/roger_antonsen_math_is_the_hidden_secret_to_understanding_the_world
https://www.ted.com/talks/victor_rios_help_for_kids_the_education_system_ignores
https://www.ted.com/talks/steven_johnson_the_playful_wonderland_behind_great_inventions
https://www.ted.com/talks/bettina_warburg_how_the_blockchain_will_radically_transform_the_economy
https://www.ted.com/talks/kimberle_crenshaw_the_urgency_of_intersectionality
https://www.ted.com/talks/hector_garcia_we_train_soldiers_for_war_let_s_train_them_to_come_home_too
https://www.ted.com/talks/stephanie_sardelis_why_do_whales_sing
https://www.ted.com/talks/fawn_qiu_easy_diy_projects_for_kid_engineers
https://w

https://www.ted.com/talks/suzanne_simard_how_trees_talk_to_each_other
https://www.ted.com/talks/matt_anticole_why_the_metric_system_matters
https://www.ted.com/talks/ed_boyden_a_new_way_to_study_the_brain_s_invisible_secrets
https://www.ted.com/talks/gerard_ryle_how_the_panama_papers_journalists_broke_the_biggest_leak_in_history
https://www.ted.com/talks/el_seed_a_project_of_peace_painted_across_50_buildings
https://www.ted.com/talks/lisa_dyson_a_forgotten_space_age_technology_could_change_how_we_grow_food
https://www.ted.com/talks/adam_savage_my_love_letter_to_cosplay
https://www.ted.com/talks/shubhendu_sharma_how_to_grow_a_forest_in_your_backyard
https://www.ted.com/talks/sandra_rey_pollution_free_lights_powered_by_microbes
https://www.ted.com/talks/naufal_mukumi_the_dark_and_delicate_rumble_of_russian_piano_music
https://www.ted.com/talks/min_zhu_interconnectivity_the_new_structure_of_the_world_economy
https://www.ted.com/talks/mike_schwartz_the_potential_of_blockchain
https://www.t

https://www.ted.com/talks/danielle_feinberg_the_magic_ingredient_that_brings_pixar_movies_to_life
https://www.ted.com/talks/parag_khanna_how_megacities_are_changing_the_map_of_the_world
https://www.ted.com/talks/haley_van_dyck_how_a_start_up_in_the_white_house_is_changing_business_as_usual
https://www.ted.com/talks/adam_grant_the_surprising_habits_of_original_thinkers
https://www.ted.com/talks/nathan_s_jacobs_how_do_schools_of_fish_swim_in_harmony
https://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_how_to_avoid_them
https://www.ted.com/talks/lisa_nip_how_humans_could_evolve_to_survive_in_space
https://www.ted.com/talks/mia_nacamulli_what_would_happen_if_you_didn_t_drink_water
https://www.ted.com/talks/dan_gross_why_gun_violence_can_t_be_our_new_normal
https://www.ted.com/talks/lisa_winer_can_you_solve_the_locker_riddle
https://www.ted.com/talks/angelica_dass_the_beauty_of_human_skin_in_every_color
https://www.ted.com/talks/alex_kipman_a_futuristic_vision_of_the_age_of

https://www.ted.com/talks/kristen_marhaver_how_we_re_growing_baby_corals_to_rebuild_reefs
https://www.ted.com/talks/dan_van_der_vieren_can_you_solve_einstein_s_riddle
https://www.ted.com/talks/year_in_ideas_2015
https://www.ted.com/talks/marina_abramovic_an_art_made_of_trust_vulnerability_and_connection
https://www.ted.com/talks/regina_hartley_why_the_best_hire_might_not_have_the_perfect_resume
https://www.ted.com/talks/ann_morgan_my_year_reading_a_book_from_every_country_in_the_world
https://www.ted.com/talks/genevieve_von_petzinger_why_are_these_32_symbols_found_in_ancient_caves_all_over_europe
https://www.ted.com/talks/carl_safina_what_are_animals_thinking_and_feeling
https://www.ted.com/talks/anote_tong_my_country_will_be_underwater_soon_unless_we_work_together
https://www.ted.com/talks/bethany_rickwald_and_hayley_levitt_would_you_opt_for_a_life_with_no_pain
https://www.ted.com/talks/nonny_de_la_pena_the_future_of_news_virtual_reality
https://www.ted.com/talks/josh_luber_why_sneake

https://www.ted.com/talks/markus_lorenz_industry_4_0_how_intelligent_machines_will_transform_everything_we_know
https://www.ted.com/talks/luc_de_brabandere_reinventing_creative_thinking
https://www.ted.com/talks/joachim_horn_a_solution_for_building_a_generation_of_inventors
https://www.ted.com/talks/emma_bridgewater_how_an_entrepreneurial_potter_helped_restore_beauty_to_a_city
https://www.ted.com/talks/diana_dosik_why_we_need_to_treat_our_employees_as_thoughtfully_as_our_customers
https://www.ted.com/talks/alan_eustace_i_leapt_from_the_stratosphere_here_s_how_i_did_it
https://www.ted.com/talks/jim_simons_the_mathematician_who_cracked_wall_street
https://www.ted.com/talks/jamie_bartlett_how_the_mysterious_dark_net_is_going_mainstream
https://www.ted.com/talks/alex_gendler_can_you_solve_the_bridge_riddle
https://www.ted.com/talks/elizabeth_nyamayaro_an_invitation_to_men_who_want_a_better_world_for_women
https://www.ted.com/talks/wendy_freedman_this_telescope_might_show_us_the_beginning_o

In [79]:
get_views(2000, 3000)

https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america
https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power
https://www.ted.com/talks/murat_dalkilinc_the_benefits_of_good_posture
https://www.ted.com/talks/yuval_noah_harari_what_explains_the_rise_of_humans
https://www.ted.com/talks/el_seed_street_art_with_a_message_of_hope_and_peace

https://www.ted.com/talks/mel_rosenberg_what_causes_bad_breath
https://www.ted.com/talks/daniel_kish_how_i_use_sonar_to_navigate_the_world
https://www.ted.com/talks/alison_killing_there_s_a_better_way_to_die_and_architecture_can_help
https://www.ted.com/talks/dame_stephanie_shirley_why_do_ambitious_women_have_flat_heads
https://www.ted.com/talks/theaster_gates_how_to_revive_a_neighborhood_with_imagination_beauty_and_art
https://www.ted.com/talks/dave_isay_everyone_around_you_has_a_story_the_world_needs_to_hear
https://www.ted.com/talks/anand_giridharadas_a_tale_of_two_americas_and_the_mini_mart_where_they_collided
https://www.ted.com/talks/fei_fei_li_how_we_re_teaching_computers_to_understand_pictures
https://www.ted.com/talks/monica_lewinsky_the_price_of_shame
https://www.ted.com/talks/gil_weinberg_can_robots_be_creative
https://www.ted.com/talks/joseph_desimone_what_if_3d_printing_was_100x_faster
https://www.ted.com/talks/david_eagleman_can_we_create_new_senses_for_humans
https://www

https://www.ted.com/talks/randall_hayes_at_what_moment_are_you_dead
https://www.ted.com/talks/catherine_crump_the_small_and_surprisingly_dangerous_detail_the_police_track_about_you
https://www.ted.com/talks/thomas_hellum_the_world_s_most_boring_television_and_why_it_s_hilariously_addictive
https://www.ted.com/talks/anastasia_taylor_lind_fighters_and_mourners_of_the_ukrainian_revolution
https://www.ted.com/talks/jose_miguel_sokoloff_how_christmas_lights_helped_guerrillas_put_down_their_guns
https://www.ted.com/talks/aakash_odedra_a_dance_in_a_hurricane_of_paper_wind_and_light
https://www.ted.com/talks/barbara_natterson_horowitz_what_veterinarians_know_that_physicians_don_t
https://www.ted.com/talks/rainer_strack_the_workforce_crisis_of_2030_and_how_to_start_solving_it_now
https://www.ted.com/talks/ben_saunders_to_the_south_pole_and_back_the_hardest_105_days_of_my_life
https://www.ted.com/talks/michael_mauser_what_are_those_floaty_things_in_your_eye
https://www.ted.com/talks/oren_yakobov

https://www.ted.com/talks/uldus_bakhtiozina_wry_photos_that_turn_stereotypes_upside_down
https://www.ted.com/talks/hans_and_ola_rosling_how_not_to_be_ignorant_about_the_world
https://www.ted.com/talks/dan_barasch_a_park_underneath_the_hustle_and_bustle_of_new_york_city
https://www.ted.com/talks/zak_ebrahim_i_am_the_son_of_a_terrorist_here_s_how_i_chose_peace
https://www.ted.com/talks/colin_grant_how_our_stories_cross_over
https://www.ted.com/talks/shubhendu_sharma_an_engineer_s_vision_for_tiny_forests_everywhere
https://www.ted.com/talks/isabel_allende_how_to_live_passionately_no_matter_your_age
https://www.ted.com/talks/elizabeth_hellmuth_margulis_why_we_love_repetition_in_music
https://www.ted.com/talks/jim_holt_why_does_the_universe_exist
https://www.ted.com/talks/jill_shargaa_please_please_people_let_s_put_the_awe_back_in_awesome
https://www.ted.com/talks/sally_kohn_don_t_like_clickbait_don_t_click
https://www.ted.com/talks/meera_vijayann_find_your_voice_against_gender_violence
htt

https://www.ted.com/talks/wendy_chung_autism_what_we_know_and_what_we_don_t_know_yet
https://www.ted.com/talks/elizabeth_gilbert_success_failure_and_the_drive_to_keep_creating
https://www.ted.com/talks/alex_gendler_how_tsunamis_work
https://www.ted.com/talks/james_patten_the_best_computer_interface_maybe_your_hands
https://www.ted.com/talks/hamish_jolly_a_shark_deterrent_wetsuit_and_it_s_not_what_you_think
https://www.ted.com/talks/michel_laberge_how_synchronized_hammer_strikes_could_generate_nuclear_fusion
https://www.ted.com/talks/sarah_lewis_embrace_the_near_win
https://www.ted.com/talks/matthew_carter_my_life_in_typefaces
https://www.ted.com/talks/jeremy_kasdin_the_flower_shaped_starshade_that_might_help_us_detect_earth_like_planets
https://www.ted.com/talks/norman_spack_how_i_help_transgender_teens_become_who_they_want_to_be
https://www.ted.com/talks/jennifer_senior_for_parents_happiness_is_a_very_high_bar
https://www.ted.com/talks/matt_anticole_what_s_the_difference_between_accur

https://www.ted.com/talks/sandra_aamodt_why_dieting_doesn_t_usually_work
https://www.ted.com/talks/nicole_avena_how_sugar_affects_the_brain
https://www.ted.com/talks/roger_stein_a_bold_new_way_to_fund_drug_research
https://www.ted.com/talks/suzanne_talhouk_don_t_kill_your_language
https://www.ted.com/talks/maysoon_zayid_i_got_99_problems_palsy_is_just_one
https://www.ted.com/talks/emma_bryce_should_we_eat_bugs
https://www.ted.com/talks/mick_cornett_how_an_obese_town_lost_a_million_pounds
https://www.ted.com/talks/diana_nyad_never_ever_give_up
https://www.ted.com/talks/paul_piff_does_money_make_you_mean
https://www.ted.com/talks/rose_eveleth_how_do_we_smell
https://www.ted.com/talks/krista_donaldson_the_80_prosthetic_knee_that_s_changing_lives
https://www.ted.com/talks/andrew_solomon_depression_the_secret_we_share
https://www.ted.com/talks/marco_annunziata_welcome_to_the_age_of_the_industrial_internet
https://www.ted.com/talks/toni_griffin_a_new_vision_for_rebuilding_detroit
https://www

https://www.ted.com/talks/peter_mende_siedlecki_should_you_trust_your_first_impression
https://www.ted.com/talks/steve_ramirez_and_xu_liu_a_mouse_a_laser_beam_a_manipulated_memory
https://www.ted.com/talks/russell_foster_why_do_we_sleep
https://www.ted.com/talks/shigeru_ban_emergency_shelters_made_from_paper
https://www.ted.com/talks/margaret_heffernan_the_dangers_of_willful_blindness
https://www.ted.com/talks/derek_paravicini_and_adam_ockelford_in_the_key_of_genius
https://www.ted.com/talks/eleanor_longden_the_voices_in_my_head
https://www.ted.com/talks/saki_mafundikwa_ingenuity_and_elegance_in_ancient_african_alphabets
https://www.ted.com/talks/jinsop_lee_design_for_all_5_senses
https://www.ted.com/talks/daniel_h_cohen_for_argument_s_sake
https://www.ted.com/talks/beardyman_the_polyphonic_me
https://www.ted.com/talks/peter_van_manen_better_baby_care_thanks_to_formula_1
https://www.ted.com/talks/julie_taymor_spider_man_the_lion_king_and_life_on_the_creative_edge
https://www.ted.com/ta

https://www.ted.com/talks/robert_gordon_the_death_of_innovation_the_end_of_growth
https://www.ted.com/talks/john_mcwhorter_txtng_is_killing_language_jk
https://www.ted.com/talks/black_my_journey_to_yo_yo_mastery
https://www.ted.com/talks/andres_lozano_parkinson_s_depression_and_the_switch_that_might_turn_them_off
https://www.ted.com/talks/joshua_prager_in_search_of_the_man_who_broke_my_neck
https://www.ted.com/talks/thomas_insel_toward_a_new_understanding_of_mental_illness
https://www.ted.com/talks/colm_kelleher_what_is_zeno_s_dichotomy_paradox
https://www.ted.com/talks/rose_george_let_s_talk_crap_seriously
https://www.ted.com/talks/laura_snyder_the_philosophical_breakfast_club
https://www.ted.com/talks/eric_dishman_health_care_should_be_a_team_sport
https://www.ted.com/talks/bill_shribman_what_cameras_see_that_our_eyes_don_t
https://www.ted.com/talks/dan_ariely_what_makes_us_feel_good_about_our_work
https://www.ted.com/talks/tom_whyntie_the_beginning_of_the_universe_for_beginners
http

https://www.ted.com/talks/steven_addis_a_father_daughter_bond_one_photo_at_a_time
https://www.ted.com/talks/colm_kelleher_what_is_color
https://www.ted.com/talks/molly_crockett_beware_neuro_bunk
https://www.ted.com/talks/gina_cooke_why_is_there_a_b_in_doubt
https://www.ted.com/talks/robin_chase_excuse_me_may_i_rent_your_car
https://www.ted.com/talks/ben_saunders_why_bother_leaving_the_house
https://www.ted.com/talks/christopher_warner_situational_irony_the_opposite_of_what_you_think
https://www.ted.com/talks/marcus_byrne_the_dance_of_the_dung_beetle
https://www.ted.com/talks/maz_jobrani_a_saudi_an_indian_and_an_iranian_walk_into_a_qatari_bar
https://www.ted.com/talks/markham_nolan_how_to_separate_fact_and_fiction_online
https://www.ted.com/talks/ellen_t_hoen_pool_medical_patents_save_lives
https://www.ted.com/talks/lemon_andersen_please_don_t_take_my_air_jordans
https://www.ted.com/talks/nina_tandon_could_tissue_engineering_mean_personalized_medicine
https://www.ted.com/talks/jeff_smit

https://www.ted.com/talks/michael_anti_behind_the_great_firewall_of_china
https://www.ted.com/talks/giles_duley_when_a_reporter_becomes_the_story
https://www.ted.com/talks/noah_wilson_rich_every_city_needs_healthy_honey_bees
https://www.ted.com/talks/michael_hansmeyer_building_unimaginable_shapes
https://www.ted.com/talks/ramesh_raskar_imaging_at_a_trillion_frames_per_second
https://www.ted.com/talks/tracy_chevalier_finding_the_story_inside_the_painting
https://www.ted.com/talks/malte_spitz_your_phone_company_is_watching
https://www.ted.com/talks/james_stavridis_a_navy_admiral_s_thoughts_on_global_security
https://www.ted.com/talks/vinay_venkatraman_technology_crafts_for_the_digitally_underserved
https://www.ted.com/talks/john_graham_cumming_the_greatest_machine_that_never_was
https://www.ted.com/talks/neil_harbisson_i_listen_to_color
https://www.ted.com/talks/matt_mills_image_recognition_that_triggers_augmented_reality
https://www.ted.com/talks/baba_shiv_sometimes_it_s_good_to_give_up

In [82]:
get_views(3000, 4000)

https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
https://www.ted.com/talks/hans_rosling_religions_and_babies
https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen
https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief
https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out
https://www.ted.com/talks/carl_schoonover_how_to_look_inside_the_brain
https://www.ted.com/talks/david_kelley_how_to_build_your_creative_confidence
https://www.ted.com/talks/jean_baptiste_michel_the_mathematics_of_history
https://www.ted.com/talks/tali_sharot_the_optimism_bias
https://ww

https://www.ted.com/talks/bill_doyle_treating_cancer_with_electric_fields
https://www.ted.com/talks/peter_van_uhm_why_i_chose_a_gun
https://www.ted.com/talks/mick_mountz_what_happens_inside_those_massive_warehouses
https://www.ted.com/talks/bjarke_ingels_hedonistic_sustainability
https://www.ted.com/talks/gayle_tzemach_lemmon_women_entrepreneurs_example_not_exception
https://www.ted.com/talks/brian_goldman_doctors_make_mistakes_can_we_talk_about_that
https://www.ted.com/talks/diana_nyad_extreme_swimming_with_the_world_s_most_dangerous_jellyfish
https://www.ted.com/talks/lisa_harouni_a_primer_on_3d_printing
https://www.ted.com/talks/julian_baggini_is_there_a_real_you
https://www.ted.com/talks/bilal_bomani_plant_fuels_that_could_power_a_jet
https://www.ted.com/talks/scott_rickard_the_beautiful_math_behind_the_world_s_ugliest_music
https://www.ted.com/talks/sheena_iyengar_how_to_make_choosing_easier
https://www.ted.com/talks/clay_shirky_why_sopa_is_a_bad_idea
https://www.ted.com/talks/mik

https://www.ted.com/talks/raghava_kk_shake_up_your_story
https://www.ted.com/talks/lee_cronin_making_matter_come_alive
https://www.ted.com/talks/sarah_kaminsky_my_father_the_forger
https://www.ted.com/talks/edward_tenner_unintended_consequences
https://www.ted.com/talks/joan_halifax_compassion_and_the_true_meaning_of_empathy
https://www.ted.com/talks/skylar_tibbits_can_we_make_things_that_make_themselves
https://www.ted.com/talks/julia_bacha_pay_attention_to_nonviolence
https://www.ted.com/talks/svante_paabo_dna_clues_to_our_inner_neanderthal
https://www.ted.com/talks/dan_ariely_beware_conflicts_of_interest
https://www.ted.com/talks/marco_tempest_the_magic_of_truth_and_lies_and_ipods
https://www.ted.com/talks/lucianne_walkowicz_finding_planets_around_other_stars
https://www.ted.com/talks/jeremy_gilley_one_day_of_peace
https://www.ted.com/talks/dyan_denapoli_the_great_penguin_rescue
https://www.ted.com/talks/alex_steffen_the_shareable_future_of_cities
https://www.ted.com/talks/eve_ensle

https://www.ted.com/talks/hans_rosling_the_magic_washing_machine
https://www.ted.com/talks/sarah_kay_if_i_should_have_a_daughter
https://www.ted.com/talks/rogier_van_der_heide_why_light_needs_darkness
https://www.ted.com/talks/mark_bezos_a_life_lesson_from_a_volunteer_firefighter
https://www.ted.com/talks/janna_levin_the_sound_the_universe_makes
https://www.ted.com/talks/david_brooks_the_social_animal
https://www.ted.com/talks/rob_harmon_how_to_keep_rivers_and_streams_flowing
https://www.ted.com/talks/deb_roy_the_birth_of_a_word
https://www.ted.com/talks/sal_khan_let_s_use_video_to_reinvent_education
https://www.ted.com/talks/courtney_e_martin_this_isn_t_her_mother_s_feminism
https://www.ted.com/talks/anthony_atala_printing_a_human_kidney
https://www.ted.com/talks/bill_gates_how_state_budgets_are_breaking_us_schools
https://www.ted.com/talks/wael_ghonim_inside_the_egyptian_revolution
https://www.ted.com/talks/jr_my_wish_use_art_to_turn_the_world_inside_out
https://www.ted.com/talks/wad

https://www.ted.com/talks/inge_missmahl_bringing_peace_to_the_minds_of_afghanistan
https://www.ted.com/talks/sebastian_seung_i_am_my_connectome
https://www.ted.com/talks/gary_wolf_the_quantified_self
https://www.ted.com/talks/tim_birkhead_the_wisdom_of_birds
https://www.ted.com/talks/julian_treasure_shh_sound_health_in_8_steps
https://www.ted.com/talks/fabian_hemmert_the_shape_shifting_future_of_the_mobile_phone
https://www.ted.com/talks/annie_lennox_why_i_am_an_hiv_aids_activist
https://www.ted.com/talks/mitchell_besser_mothers_helping_mothers_fight_hiv
https://www.ted.com/talks/steven_johnson_where_good_ideas_come_from
https://www.ted.com/talks/christien_meindertsma_how_pig_parts_make_the_world_turn
https://www.ted.com/talks/caroline_phillips_hurdy_gurdy_for_beginners
https://www.ted.com/talks/nicholas_christakis_how_social_networks_predict_epidemics
https://www.ted.com/talks/jessa_gamble_our_natural_sleep_cycle_is_nothing_like_what_we_do_now
https://www.ted.com/talks/chris_anderson_

https://www.ted.com/talks/edith_widder_glowing_life_in_an_underwater_world
https://www.ted.com/talks/a_choir_as_big_as_the_internet
https://www.ted.com/talks/thelma_golden_how_art_gives_shape_to_cultural_change
https://www.ted.com/talks/catherine_mohr_the_tradeoffs_of_building_green
https://www.ted.com/talks/jonathan_klein_photos_that_changed_the_world
https://www.ted.com/talks/michael_specter_the_danger_of_science_denial
https://www.ted.com/talks/natalie_merchant_singing_old_poems_to_life
https://www.ted.com/talks/jonathan_drori_every_pollen_grain_has_a_story
https://www.ted.com/talks/dennis_hong_my_seven_species_of_robot_and_how_we_created_them
https://www.ted.com/talks/dean_kamen_the_emotion_behind_invention
https://www.ted.com/talks/elizabeth_pisani_sex_drugs_and_hiv_let_s_get_rational
https://www.ted.com/talks/jesse_schell_when_games_invade_real_life
https://www.ted.com/talks/adora_svitak_what_adults_can_learn_from_kids
https://www.ted.com/talks/derek_sivers_how_to_start_a_movemen

https://www.ted.com/talks/rachel_pike_the_science_behind_a_climate_headline
https://www.ted.com/talks/cameron_sinclair_the_refugees_of_boom_and_bust
https://www.ted.com/talks/stefana_broadbent_how_the_internet_enables_intimacy
https://www.ted.com/talks/matthew_white_the_modern_euphonium
https://www.ted.com/talks/marcus_du_sautoy_symmetry_reality_s_riddle
https://www.ted.com/talks/becky_blanton_the_year_i_was_homeless
https://www.ted.com/talks/rachel_armstrong_architecture_that_repairs_itself
https://www.ted.com/talks/david_deutsch_a_new_way_to_explain_explanation
https://www.ted.com/talks/ian_goldin_navigating_our_global_future
https://www.ted.com/talks/marc_koska_1_3m_reasons_to_re_invent_the_syringe
https://www.ted.com/talks/itay_talgam_lead_like_the_great_conductors
https://www.ted.com/talks/paul_debevec_animating_a_photo_real_digital_face
https://www.ted.com/talks/john_gerzema_the_post_crisis_consumer
https://www.ted.com/talks/julian_treasure_the_4_ways_sound_affects_us
https://www

https://www.ted.com/talks/nate_silver_does_racism_affect_how_you_vote
https://www.ted.com/talks/erik_hersman_reporting_crisis_via_texting
https://www.ted.com/talks/niels_diffrient_rethinking_the_way_we_sit_down
https://www.ted.com/talks/margaret_wertheim_the_beautiful_math_of_coral
https://www.ted.com/talks/matthew_childs_9_life_lessons_from_rock_climbing
https://www.ted.com/talks/tim_ferriss_smash_fear_learn_anything
https://www.ted.com/talks/joann_kuchera_morin_stunning_data_visualization_in_the_allosphere
https://www.ted.com/talks/gregory_stock_to_upgrade_is_human
https://www.ted.com/talks/shai_agassi_a_new_ecosystem_for_electric_cars
https://www.ted.com/talks/renny_gleeson_our_antisocial_phone_tricks
https://www.ted.com/talks/emily_levine_a_theory_of_everything
https://www.ted.com/talks/bonnie_bassler_how_bacteria_talk
https://www.ted.com/talks/bruce_bueno_de_mesquita_a_prediction_for_the_future_of_iran
https://www.ted.com/talks/nathaniel_kahn_scenes_from_my_architect
https://www.t

https://www.ted.com/talks/virginia_postrel_on_glamour
https://www.ted.com/talks/paola_antonelli_design_and_the_elastic_mind
https://www.ted.com/talks/garrett_lisi_an_8_dimensional_model_of_the_universe
https://www.ted.com/talks/james_burchfield_playing_invisible_turntables
https://www.ted.com/talks/steven_johnson_the_web_as_a_city
https://www.ted.com/talks/doris_kearns_goodwin_lessons_from_past_presidents
https://www.ted.com/talks/david_perry_are_games_better_than_life
https://www.ted.com/talks/james_nachtwey_moving_photos_of_extreme_drug_resistant_tb
https://www.ted.com/talks/liz_diller_the_blur_building_and_other_tech_empowered_architecture
https://www.ted.com/talks/noah_feldman_politics_and_religion_are_technologies
https://www.ted.com/talks/stefan_sagmeister_things_i_ve_learned_in_my_life_so_far
https://www.ted.com/talks/rodney_brooks_robots_will_invade_our_lives
https://www.ted.com/talks/steven_pinker_human_nature_and_the_blank_slate
https://www.ted.com/talks/caleb_chung_playtime_

In [84]:
get_views(4000, len(talk_df))

https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
https://www.ted.com/talks/rokia_traore_kounandi
https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq
https://www.ted.com/talks/susan_blackmore_memes_and_temes
https://www.ted.com/talks/paul_collier_the_bottom_billion
https://www.ted.com/talks/seyi_oyesola_a_hospital_tour_in_nigeria
https://www.ted.com/talks/arthur_ganson_moving_sculpture
https://www.ted.com/talks/yves_behar_designing_objects_that_tell_stories
https://www.ted.com/talks/robert_ballard_the_astonishing_hidden_world_of

https://www.ted.com/talks/david_bolinsky_visualizing_the_wonder_of_a_living_cell
https://www.ted.com/talks/rives_the_4_a_m_mystery
https://www.ted.com/talks/will_wright_spore_birth_of_a_game
https://www.ted.com/talks/emily_oster_flip_your_thinking_on_aids_in_africa
https://www.ted.com/talks/jonathan_harris_the_web_s_secret_stories
https://www.ted.com/talks/alan_russell_the_potential_of_regenerative_medicine
https://www.ted.com/talks/dan_dennett_dangerous_memes
https://www.ted.com/talks/bill_stone_inside_the_world_s_deepest_caves
https://www.ted.com/talks/hans_rosling_new_insights_on_poverty
https://www.ted.com/talks/stephen_lawler_tour_microsoft_s_virtual_earth
https://www.ted.com/talks/ethel_a_string_quartet_plays_blue_room
https://www.ted.com/talks/thomas_barnett_let_s_rethink_america_s_military_strategy
https://www.ted.com/talks/david_rockwell_a_memorial_at_ground_zero
https://www.ted.com/talks/robert_thurman_we_can_be_buddhas
https://www.ted.com/talks/anand_agarawala_rethink_the_de

https://www.ted.com/talks/dan_dennett_let_s_teach_religion_all_religion_in_schools
https://www.ted.com/talks/rick_warren_a_life_of_purpose
https://www.ted.com/talks/julia_sweeney_letting_go_of_god
https://www.ted.com/talks/joshua_prince_ramus_behind_the_design_of_seattle_s_library
https://www.ted.com/talks/hans_rosling_the_best_stats_you_ve_ever_seen
https://www.ted.com/talks/sir_ken_robinson_do_schools_kill_creativity
https://www.ted.com/talks/majora_carter_greening_the_ghetto
https://www.ted.com/talks/david_pogue_simplicity_sells
https://www.ted.com/talks/al_gore_averting_the_climate_crisis


In [88]:
talk_df.to_csv('talk_with_views.csv')

# Get Transcript

In [124]:
talk_df['transcript'] = '--'

In [129]:
def get_transcript(start, end):
    for index in range(start, end):
        try:
            # Get link, parse using BeautifulSoup
            url = talk_df['url'][index]
            link = "https://www.ted.com" + url + "/transcript"
            print(link)
            page = requests.get(link)
            content = BS(page.content, 'html.parser')

            # Get span class, find transcript
            transcript = ''
            for x in content.find_all('div', {"class": "Grid__cell flx-s:1 p-r:4"}):
                transcript += x.get_text().strip()
            talk_df['transcript'][index] = ' '.join(transcript.split())
            print('transcript added')
            
            time.sleep(2)
        except:
            print('NO TRANSCRIPT')

In [132]:
get_transcript(0, 1000)

https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator/transcript
transcript added
https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol/transcript
transcript added
https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women/transcript
transcript added
https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear/transcript
transcript added
https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success/transcript
transcript added
https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time/transcript
transcript added
https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism/transcript
transcript added
https://www.ted.com/talks/nick_bostrom_how_civilization_could_destroy_itself_and_4_ways_we_could_prevent_it/transcript
transcript added
https://www.ted.com/talks/jasmine_crowe_what_we_re_getting_wrong_in_the_fight_to_end_hunger/tr

transcript added
https://www.ted.com/talks/maria_popova_an_excerpt_from_figuring/transcript
transcript added
https://www.ted.com/talks/morley_follow_the_sound/transcript
transcript added
https://www.ted.com/talks/paul_a_kramer_our_immigration_conversation_is_broken_here_s_how_to_have_a_better_one/transcript
transcript added
https://www.ted.com/talks/laura_rovner_what_happens_to_people_in_solitary_confinement/transcript
transcript added
https://www.ted.com/talks/lucas_husted_game_theory_challenge_can_you_predict_human_behavior/transcript
transcript added
https://www.ted.com/talks/tashka_and_laura_yawanawa_the_amazon_belongs_to_humanity_let_s_protect_it_together/transcript
transcript added
https://www.ted.com/talks/david_carroll_how_i_sued_cambridge_analytica_over_my_personal_data/transcript
transcript added
https://www.ted.com/talks/bhakti_sharma_what_open_water_swimming_taught_me_about_resilience/transcript
transcript added
https://www.ted.com/talks/bill_keegan_the_taino_myth_of_the_cu

transcript added
https://www.ted.com/talks/efosa_ojomo_reducing_corruption_takes_a_specific_kind_of_investment/transcript
transcript added
https://www.ted.com/talks/david_peterson_why_language_is_humanity_s_greatest_invention/transcript
transcript added
https://www.ted.com/talks/aracelis_girmay_for_estefani_third_grade_who_made_me_a_card/transcript
transcript added
https://www.ted.com/talks/camilla_arndal_andersen_what_happens_in_your_brain_when_you_taste_food/transcript
transcript added
https://www.ted.com/talks/mitchell_katz_what_the_us_health_care_system_assumes_about_you/transcript
transcript added
https://www.ted.com/talks/densho_ugly_history_japanese_american_incarceration_camps/transcript
transcript added
https://www.ted.com/talks/moreangels_mbizah_how_community_led_conservation_can_save_wildlife/transcript
transcript added
https://www.ted.com/talks/mohammad_modarres_why_you_should_shop_at_your_local_farmers_market/transcript
transcript added
https://www.ted.com/talks/alex_rosen

transcript added
https://www.ted.com/talks/sara_valencia_botto_when_do_kids_start_to_care_about_other_people_s_opinions/transcript
transcript added
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_multiverse_rescue_mission_riddle/transcript
transcript added
https://www.ted.com/talks/chiara_decaroli_the_high_stakes_race_to_make_quantum_computers_work/transcript
transcript added
https://www.ted.com/talks/ben_longdon_how_do_viruses_jump_from_animals_to_humans/transcript
transcript added
https://www.ted.com/talks/shrabani_basu_from_pacifist_to_spy_wwii_s_surprising_secret_agent/transcript
transcript added
https://www.ted.com/talks/judy_cebra_thomas_how_turtle_shells_evolved_twice/transcript
transcript added
https://www.ted.com/talks/soraya_field_fiorio_the_murder_of_ancient_alexandria_s_greatest_scholar/transcript
transcript added
https://www.ted.com/talks/marc_bamuthi_joseph_you_have_the_rite/transcript
transcript added
https://www.ted.com/talks/margaret_heffernan_the_human_skill

https://www.ted.com/talks/ayana_elizabeth_johnson_and_megan_davis_could_underwater_farms_help_fight_climate_change/transcript
transcript added
https://www.ted.com/talks/kaitlyn_sadtler_your_body_vs_implants/transcript
transcript added
https://www.ted.com/talks/david_baker_5_challenges_we_could_solve_by_designing_new_proteins/transcript
transcript added
https://www.ted.com/talks/steve_simpson_how_ocean_noise_destroys_marine_ecosystems/transcript
transcript added
https://www.ted.com/talks/michael_tubbs_the_political_power_of_being_a_good_neighbor/transcript
transcript added
https://www.ted.com/talks/emeli_sande_you_are_not_alone_extraordinary_being_shine/transcript
transcript added
https://www.ted.com/talks/daniel_lismore_my_life_as_a_work_of_art/transcript
transcript added
https://www.ted.com/talks/lucy_clayton_all_the_rage_why_dressing_up_helps_us_speak_out/transcript
transcript added
https://www.ted.com/talks/truls_nord_tactile_photographs_that_display_worlds_of_light_shadow_and_mood/

...

https://www.ted.com/talks/melvin_sanicas_why_is_meningitis_so_dangerous/transcript
transcript added
https://www.ted.com/talks/julia_dhar_how_to_disagree_productively_and_find_common_ground/transcript
transcript added
https://www.ted.com/talks/arindam_bhattacharya_globalization_isn_t_declining_it_s_transforming/transcript
transcript added
https://www.ted.com/talks/frank_muller_pierstorff_circular_escape/transcript
transcript added
https://www.ted.com/talks/viola_llewellyn_a_new_model_of_microfinance_for_africa_and_beyond/transcript
transcript added
https://www.ted.com/talks/a_dara_dotz_an_ingenious_solution_for_aid_in_disaster_zones/transcript
transcript added
https://www.ted.com/talks/jim_whitehurst_what_i_learned_from_giving_up_everything_i_knew_as_a_leader/transcript
transcript added
https://www.ted.com/talks/francois_candelon_how_do_we_stop_the_massive_inequality_dominating_the_world_economy/transcript
transcript added
https://www.ted.com/talks/alexander_belcredi_how_a_long_forgotte

https://www.ted.com/talks/david_korins_3_ways_to_create_a_space_that_moves_you_from_a_broadway_set_designer/transcript
transcript added
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_killer_robo_ants_riddle/transcript
transcript added
https://www.ted.com/talks/ashwini_anburajan_how_cryptocurrency_can_help_start_ups_get_investment_capital/transcript
transcript added
https://www.ted.com/talks/alex_honnold_how_i_climbed_a_3_000_foot_vertical_cliff_without_ropes/transcript
transcript added
https://www.ted.com/talks/mindy_fullilove_how_we_start_envisioning_a_future_where_all_people_are_created_and_live_as_equals/transcript
transcript added
https://www.ted.com/talks/michel_dugon_the_secrets_of_spider_venom/transcript
transcript added
https://www.ted.com/talks/camille_a_brown_new_second_line/transcript
transcript added
https://www.ted.com/talks/magatte_wade_why_it_s_too_hard_to_start_a_business_in_africa_and_how_to_change_it/transcript
transcript added
https://www.ted.com/talks/jos

transcript added
https://www.ted.com/talks/andrew_bastawrous_a_new_way_to_fund_health_care_for_the_most_vulnerable/transcript
transcript added
https://www.ted.com/talks/trevor_maber_rethinking_thinking/transcript
transcript added
https://www.ted.com/talks/alex_gendler_can_you_solve_the_rebel_supplies_riddle/transcript
transcript added
https://www.ted.com/talks/pierre_barreau_how_ai_could_compose_a_personalized_soundtrack_to_your_life/transcript
transcript added
https://www.ted.com/talks/mark_pollock_and_simone_george_a_love_letter_to_realism_in_a_time_of_grief/transcript
transcript added
https://www.ted.com/talks/fred_krupp_let_s_launch_a_satellite_to_track_a_threatening_greenhouse_gas/transcript
transcript added
https://www.ted.com/talks/sajan_saini_what_is_the_universe_expanding_into/transcript
transcript added
https://www.ted.com/talks/nikki_clifton_3_ways_businesses_can_fight_sex_trafficking/transcript
transcript added
https://www.ted.com/talks/ben_cort_what_commercialization_is_do

https://www.ted.com/talks/gabriel_heras_humanizar_los_cuidados_intensivos/transcript
transcript added
https://www.ted.com/talks/carmen_jimenez_borja_las_palabras_de_la_discriminacion/transcript
transcript added
https://www.ted.com/talks/nestor_guerra_nuestras_mentiras_en_la_vida_y_en_los_negocios/transcript
transcript added
https://www.ted.com/talks/alberto_garin_a_quien_pertenece_el_pasado/transcript
transcript added
https://www.ted.com/talks/fernanda_castillo_la_voz_del_artista/transcript
transcript added
https://www.ted.com/talks/mikhail_zygar_what_the_russian_revolution_would_have_looked_like_on_social_media/transcript
transcript added
https://www.ted.com/talks/hernan_casciari_duelo_por_la_muerte_del_padre/transcript
transcript added
https://www.ted.com/talks/alba_sotorra_un_ejercito_de_mujeres/transcript
transcript added
https://www.ted.com/talks/kashmir_hill_and_surya_mattu_what_your_smart_devices_know_and_share_about_you/transcript
transcript added
https://www.ted.com/talks/peta

transcript added
https://www.ted.com/talks/greg_gage_how_you_can_make_a_fruit_fly_eat_veggies/transcript
transcript added
https://www.ted.com/talks/greg_gage_how_sound_can_hack_your_memory_while_you_sleep/transcript
transcript added
https://www.ted.com/talks/greg_gage_how_a_dragonfly_s_brain_is_designed_to_kill/transcript
transcript added
https://www.ted.com/talks/john_doerr_why_the_secret_to_success_is_setting_the_right_goals/transcript
transcript added
https://www.ted.com/talks/heidi_m_sosik_the_discoveries_awaiting_us_in_the_ocean_s_twilight_zone/transcript
transcript added
https://www.ted.com/talks/sophie_scott_why_we_should_take_laughter_more_seriously/transcript
transcript added
https://www.ted.com/talks/olga_yurkova_inside_the_fight_against_russia_s_fake_news_empire/transcript
transcript added
https://www.ted.com/talks/enric_sala_por_que_necesitamos_proteger_el_alta_mar/transcript
transcript added
https://www.ted.com/talks/enric_sala_let_s_turn_the_high_seas_into_the_world_s_lar

In [142]:
get_transcript(1000, 2000)

https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers/transcript
transcript added
https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after/transcript
transcript added
https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes/transcript
transcript added
https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all/transcript
transcript added
https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with/transcript
transcript added
https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico/transcript
transcript added
https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment/transcript
transcript added
https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell/transcript
transcript added
https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_r

transcript added
https://www.ted.com/talks/kaustav_dey_how_fashion_helps_us_express_who_we_are_and_what_we_stand_for/transcript
transcript added
https://www.ted.com/talks/petter_johansson_do_you_really_know_why_you_do_what_you_do/transcript
transcript added
https://www.ted.com/talks/marc_bamuthi_joseph_what_soccer_can_teach_us_about_freedom/transcript
transcript added
https://www.ted.com/talks/minda_dentler_what_i_learned_when_i_conquered_the_world_s_toughest_triathlon/transcript
transcript added
https://www.ted.com/talks/bill_bernat_how_to_connect_with_depressed_friends/transcript
transcript added
https://www.ted.com/talks/felice_belle_and_jennifer_murphy_how_we_became_sisters/transcript
transcript added
https://www.ted.com/talks/shameem_akhtar_to_learn_is_to_be_free/transcript
transcript added
https://www.ted.com/talks/dustin_schroeder_how_we_look_kilometers_below_the_antarctic_ice_sheet/transcript
transcript added
https://www.ted.com/talks/shawn_martinbrough_how_one_black_panther_co

https://www.ted.com/talks/erick_brethenoux_the_big_data_behind_complex_human_emotions/transcript
transcript added
https://www.ted.com/talks/bryan_kramer_why_social_media_is_reimagining_our_future/transcript
transcript added
https://www.ted.com/talks/wobbly_world_diversity_in_harmony/transcript
transcript added
https://www.ted.com/talks/villy_wang_a_business_against_racism/transcript
transcript added
https://www.ted.com/talks/tim_exile_an_instrument_anyone_can_play/transcript
transcript added
https://www.ted.com/talks/laxmi_parida_what_if_the_best_fertilizer_was_an_algorithm/transcript
transcript added
https://www.ted.com/talks/juliane_gallina_a_library_of_minds/transcript
transcript added
https://www.ted.com/talks/james_kozloski_why_the_brain_is_simpler_than_we_think/transcript
transcript added
https://www.ted.com/talks/ise_lyfe_we_are_not_mud/transcript
transcript added
https://www.ted.com/talks/bob_nease_how_to_trick_yourself_into_good_behavior/transcript
transcript added
https://www

https://www.ted.com/talks/james_orsulak_manufacturing_in_space_could_save_life_on_earth/transcript
transcript added
https://www.ted.com/talks/lana_mazahreh_3_thoughtful_ways_to_conserve_water/transcript
transcript added
https://www.ted.com/talks/lisa_feldman_barrett_you_aren_t_at_the_mercy_of_your_emotions_your_brain_creates_them/transcript
transcript added
https://www.ted.com/talks/christopher_ategeka_how_adoption_worked_for_me/transcript
transcript added
https://www.ted.com/talks/heather_lanier_good_and_bad_are_incomplete_stories_we_tell_ourselves/transcript
transcript added
https://www.ted.com/talks/christian_benimana_the_next_generation_of_african_architects_and_designers/transcript
transcript added
https://www.ted.com/talks/deb_willis_and_hank_willis_thomas_a_mother_and_son_united_by_love_and_art/transcript
transcript added
https://www.ted.com/talks/yvette_alberdingk_thijm_the_power_of_citizen_video_to_create_undeniable_truths/transcript
transcript added
https://www.ted.com/talks/

transcript added
https://www.ted.com/talks/nnedi_okorafor_sci_fi_stories_that_imagine_a_future_africa/transcript
transcript added
https://www.ted.com/talks/lauren_sallan_how_to_win_at_evolution_and_survive_a_mass_extinction/transcript
transcript added
https://www.ted.com/talks/tim_kruger_can_we_stop_climate_change_by_removing_co2_from_the_air/transcript
transcript added
https://www.ted.com/talks/shonda_rhimes_and_cyndi_stivers_the_future_of_storytelling/transcript
transcript added
https://www.ted.com/talks/zeynep_tufekci_we_re_building_a_dystopia_just_to_make_people_click_on_ads/transcript
transcript added
https://www.ted.com/talks/carlos_bautista_the_awful_logic_of_land_mines_and_an_app_that_helps_people_avoid_them/transcript
transcript added
https://www.ted.com/talks/martin_ford_how_we_ll_earn_money_in_a_future_without_jobs/transcript
transcript added
https://www.ted.com/talks/rocio_lorenzo_how_diversity_makes_teams_more_innovative/transcript
transcript added
https://www.ted.com/talk

...

transcript added
https://www.ted.com/talks/david_camarillo_why_helmets_don_t_prevent_concussions_and_what_might/transcript
transcript added
https://www.ted.com/talks/courtney_e_martin_the_new_american_dream/transcript
transcript added
https://www.ted.com/talks/krystian_aparta_one_of_the_most_difficult_words_to_translate/transcript
transcript added
https://www.ted.com/talks/sal_khan_let_s_teach_for_mastery_not_test_scores/transcript
transcript added
https://www.ted.com/talks/james_veitch_the_agony_of_trying_to_unsubscribe/transcript
transcript added
https://www.ted.com/talks/jonathan_tepperman_the_risky_politics_of_progress/transcript
transcript added
https://www.ted.com/talks/kio_stark_why_you_should_talk_to_strangers/transcript
transcript added
https://www.ted.com/talks/christopher_bell_bring_on_the_female_superheroes/transcript
transcript added
https://www.ted.com/talks/akshita_agarwal_the_paradox_of_value/transcript
transcript added
https://www.ted.com/talks/julia_bacha_how_women_wa

transcript added
https://www.ted.com/talks/stephen_wilkes_the_passing_of_time_caught_in_a_single_photo/transcript
transcript added
https://www.ted.com/talks/norman_lear_an_entertainment_icon_on_living_a_life_of_meaning/transcript
transcript added
https://www.ted.com/talks/samantha_nutt_the_real_harm_of_the_global_arms_trade/transcript
transcript added
https://www.ted.com/talks/dennis_shasha_can_you_solve_the_control_room_riddle/transcript
transcript added
https://www.ted.com/talks/sue_desmond_hellmann_a_smarter_more_precise_way_to_think_about_public_health/transcript
transcript added
https://www.ted.com/talks/adam_driver_my_journey_from_marine_to_actor/transcript
transcript added
https://www.ted.com/talks/joseph_ravenell_how_barbershops_can_keep_men_healthy/transcript
transcript added
https://www.ted.com/talks/zaria_forman_drawings_that_show_the_beauty_and_fragility_of_earth/transcript
transcript added
https://www.ted.com/talks/mariano_sigman_your_words_may_predict_your_future_mental_h

transcript added
https://www.ted.com/talks/joe_gebbia_how_airbnb_designs_for_trust/transcript
transcript added
https://www.ted.com/talks/casey_gerald_the_gospel_of_doubt/transcript
transcript added
https://www.ted.com/talks/tshering_tobgay_this_country_isn_t_just_carbon_neutral_it_s_carbon_negative/transcript
transcript added
https://www.ted.com/talks/mileha_soneji_simple_hacks_for_life_with_parkinson_s/transcript
transcript added
https://www.ted.com/talks/laura_robinson_the_secrets_i_find_on_the_mysterious_ocean_floor/transcript
transcript added
https://www.ted.com/talks/reshma_saujani_teach_girls_bravery_not_perfection/transcript
transcript added
https://www.ted.com/talks/travis_kalanick_uber_s_plan_to_get_more_people_into_fewer_cars/transcript
transcript added
https://www.ted.com/talks/alexander_betts_our_refugee_system_is_failing_here_s_how_we_can_fix_it/transcript
transcript added
https://www.ted.com/talks/russ_altman_what_really_happens_when_you_mix_medications/transcript
transcr

transcript added
https://www.ted.com/talks/jessica_shortall_the_us_needs_paid_family_leave_for_the_sake_of_its_future/transcript
transcript added
https://www.ted.com/talks/kristen_marhaver_how_we_re_growing_baby_corals_to_rebuild_reefs/transcript
transcript added
https://www.ted.com/talks/dan_van_der_vieren_can_you_solve_einstein_s_riddle/transcript
transcript added
https://www.ted.com/talks/year_in_ideas_2015/transcript
transcript added
https://www.ted.com/talks/marina_abramovic_an_art_made_of_trust_vulnerability_and_connection/transcript
transcript added
https://www.ted.com/talks/regina_hartley_why_the_best_hire_might_not_have_the_perfect_resume/transcript
transcript added
https://www.ted.com/talks/ann_morgan_my_year_reading_a_book_from_every_country_in_the_world/transcript
transcript added
https://www.ted.com/talks/genevieve_von_petzinger_why_are_these_32_symbols_found_in_ancient_caves_all_over_europe/transcript
transcript added
https://www.ted.com/talks/carl_safina_what_are_animals

transcript added
https://www.ted.com/talks/mac_stone_stunning_photos_of_the_endangered_everglades/transcript
transcript added
https://www.ted.com/talks/taiye_selasi_don_t_ask_where_i_m_from_ask_where_i_m_a_local/transcript
transcript added
https://www.ted.com/talks/samuel_cohen_alzheimer_s_is_not_normal_aging_and_we_can_cure_it/transcript
transcript added
https://www.ted.com/talks/robin_morgan_4_powerful_poems_about_parkinson_s_and_growing_older/transcript
transcript added
https://www.ted.com/talks/mary_robinson_why_climate_change_is_a_threat_to_human_rights/transcript
transcript added
https://www.ted.com/talks/conor_heffernan_the_treadmill_s_dark_and_twisted_past/transcript
transcript added
https://www.ted.com/talks/frances_larson_why_public_beheadings_get_millions_of_views/transcript
transcript added
https://www.ted.com/talks/sakena_yacoobi_how_i_stopped_the_taliban_from_shutting_down_my_school/transcript
transcript added
https://www.ted.com/talks/scott_dinsmore_how_to_find_work_you_

In [145]:
get_transcript(2000, 3000)

https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket/transcript
transcript added
https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really/transcript
transcript added
https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge/transcript
transcript added
https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s/transcript
transcript added
https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers/transcript
transcript added
https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties/transcript
transcript added
https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry/transcript
transcript added
https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america/transcript
transcript added
https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power/t

transcript added
https://www.ted.com/talks/abe_davis_new_video_technology_that_reveals_an_object_s_hidden_properties/transcript
transcript added
https://www.ted.com/talks/pamela_ronald_the_case_for_engineering_our_food/transcript
transcript added
https://www.ted.com/talks/alice_goffman_how_we_re_priming_some_kids_for_college_and_others_for_prison/transcript
transcript added
https://www.ted.com/talks/sophie_scott_why_we_laugh/transcript
transcript added
https://www.ted.com/talks/greg_gage_how_to_control_someone_else_s_arm_with_your_brain/transcript
transcript added
https://www.ted.com/talks/nick_bostrom_what_happens_when_our_computers_get_smarter_than_we_are/transcript
transcript added
https://www.ted.com/talks/nizar_ibrahim_how_we_unearthed_the_spinosaurus/transcript
transcript added
https://www.ted.com/talks/eleanor_nelsen_how_to_unboil_an_egg/transcript
transcript added
https://www.ted.com/talks/clint_smith_how_to_raise_a_black_son_in_america/transcript
transcript added
https://www.t

transcript added
https://www.ted.com/talks/bassam_tariq_the_beauty_and_diversity_of_muslim_life/transcript
transcript added
https://www.ted.com/talks/wilton_l_virgo_how_does_your_smartphone_know_your_location/transcript
transcript added
https://www.ted.com/talks/khadija_gbla_my_mother_s_strange_definition_of_empowerment/transcript
transcript added
https://www.ted.com/talks/severine_autesserre_to_solve_mass_violence_look_to_locals/transcript
transcript added
https://www.ted.com/talks/miguel_nicolelis_brain_to_brain_communication_has_arrived_how_we_did_it/transcript
transcript added
https://www.ted.com/talks/morgana_bailey_the_danger_of_hiding_who_you_are/transcript
transcript added
https://www.ted.com/talks/joe_madiath_better_toilets_better_life/transcript
transcript added
https://www.ted.com/talks/sarah_bergbreiter_why_i_make_robots_the_size_of_a_grain_of_rice/transcript
transcript added
https://www.ted.com/talks/matthieu_ricard_how_to_let_altruism_be_your_guide/transcript
transcript a

https://www.ted.com/talks/john_o_leary_the_importance_of_good_conversation_and_how_to_have_it/transcript
transcript added
https://www.ted.com/talks/john_bolton_the_power_of_imagination_lessons_from_shakespeare/transcript
transcript added
https://www.ted.com/talks/harry_baker_the_power_of_self_acceptance/transcript
transcript added
https://www.ted.com/talks/christina_balch_selfies_and_seeing_ourselves_one_artist_s_look_in_the_mirror/transcript
transcript added
https://www.ted.com/talks/ashwini_mrinal_bhagat_what_i_learned_from_32_grueling_interviews/transcript
transcript added
https://www.ted.com/talks/alexander_mclean_restoring_hope_and_dignity_to_the_justice_system/transcript
transcript added
https://www.ted.com/talks/abbey_williams_my_ghost_town_road_trip/transcript
transcript added
https://www.ted.com/talks/will_marshall_tiny_satellites_show_us_the_earth_as_it_changes_in_near_real_time/transcript
transcript added
https://www.ted.com/talks/david_grady_how_to_save_the_world_or_at_leas

transcript added
https://www.ted.com/talks/hans_and_ola_rosling_how_not_to_be_ignorant_about_the_world/transcript
transcript added
https://www.ted.com/talks/dan_barasch_a_park_underneath_the_hustle_and_bustle_of_new_york_city/transcript
transcript added
https://www.ted.com/talks/zak_ebrahim_i_am_the_son_of_a_terrorist_here_s_how_i_chose_peace/transcript
transcript added
https://www.ted.com/talks/colin_grant_how_our_stories_cross_over/transcript
transcript added
https://www.ted.com/talks/shubhendu_sharma_an_engineer_s_vision_for_tiny_forests_everywhere/transcript
transcript added
https://www.ted.com/talks/isabel_allende_how_to_live_passionately_no_matter_your_age/transcript
transcript added
https://www.ted.com/talks/elizabeth_hellmuth_margulis_why_we_love_repetition_in_music/transcript
transcript added
https://www.ted.com/talks/jim_holt_why_does_the_universe_exist/transcript
transcript added
https://www.ted.com/talks/jill_shargaa_please_please_people_let_s_put_the_awe_back_in_awesome/tr

...

transcript added
https://www.ted.com/talks/kelly_mcgonigal_how_to_make_stress_your_friend/transcript
transcript added
https://www.ted.com/talks/andrew_vanden_heuvel_the_moon_illusion/transcript
transcript added
https://www.ted.com/talks/adam_spencer_why_i_fell_in_love_with_monster_prime_numbers/transcript
transcript added
https://www.ted.com/talks/michael_molina_what_is_deja_vu_what_is_deja_vu/transcript
transcript added
https://www.ted.com/talks/may_el_khalil_making_peace_is_a_marathon/transcript
transcript added
https://www.ted.com/talks/peter_mende_siedlecki_should_you_trust_your_first_impression/transcript
transcript added
https://www.ted.com/talks/steve_ramirez_and_xu_liu_a_mouse_a_laser_beam_a_manipulated_memory/transcript
transcript added
https://www.ted.com/talks/russell_foster_why_do_we_sleep/transcript
transcript added
https://www.ted.com/talks/shigeru_ban_emergency_shelters_made_from_paper/transcript
transcript added
https://www.ted.com/talks/margaret_heffernan_the_dangers_o

transcript added
https://www.ted.com/talks/paola_antonelli_why_i_brought_pac_man_to_moma/transcript
transcript added
https://www.ted.com/talks/ji_hae_park_the_violin_and_my_dark_night_of_the_soul/transcript
transcript added
https://www.ted.com/talks/alastair_parvin_architecture_for_the_people_by_the_people/transcript
transcript added
https://www.ted.com/talks/george_zaidan_what_is_fat/transcript
transcript added
https://www.ted.com/talks/judy_macdonald_johnston_prepare_for_a_good_end_of_life/transcript
transcript added
https://www.ted.com/talks/phil_hansen_embrace_the_shake/transcript
transcript added
https://www.ted.com/talks/david_j_malan_what_s_an_algorithm/transcript
transcript added
https://www.ted.com/talks/peter_singer_the_why_and_how_of_effective_altruism/transcript
transcript added
https://www.ted.com/talks/sergey_brin_why_google_glass/transcript
transcript added
https://www.ted.com/talks/jay_silver_hack_a_banana_make_a_keyboard/transcript
transcript added
https://www.ted.com/

transcript added
https://www.ted.com/talks/elon_musk_the_mind_behind_tesla_spacex_solarcity/transcript
transcript added
https://www.ted.com/talks/danny_hillis_the_internet_could_crash_we_need_a_plan_b/transcript
transcript added
https://www.ted.com/talks/catarina_mota_play_with_smart_materials/transcript
transcript added
https://www.ted.com/talks/cameron_paterson_the_infamous_and_ingenious_ho_chi_minh_trail/transcript
transcript added
https://www.ted.com/talks/bono_the_good_news_on_poverty_yes_there_s_good_news/transcript
transcript added
https://www.ted.com/talks/christopher_warner_what_is_verbal_irony/transcript
transcript added
https://www.ted.com/talks/stewart_brand_the_dawn_of_de_extinction_are_you_ready/transcript
transcript added
https://www.ted.com/talks/david_anderson_your_brain_is_more_than_a_bag_of_chemicals/transcript
transcript added
https://www.ted.com/talks/dan_pallotta_the_way_we_think_about_charity_is_dead_wrong/transcript
transcript added
https://www.ted.com/talks/sha

https://www.ted.com/talks/ludwick_marishane_a_bath_without_water/transcript
transcript added
https://www.ted.com/talks/bobby_ghosh_why_global_jihad_is_losing/transcript
transcript added
https://www.ted.com/talks/paolo_cardini_forget_multitasking_try_monotasking/transcript
transcript added
https://www.ted.com/talks/munir_virani_why_i_love_vultures/transcript
transcript added
https://www.ted.com/talks/janine_shepherd_a_broken_body_isn_t_a_broken_person/transcript
transcript added
https://www.ted.com/talks/jonas_eliasson_how_to_solve_traffic_jams/transcript
transcript added
https://www.ted.com/talks/ernesto_sirolli_want_to_help_someone_shut_up_and_listen/transcript
transcript added
https://www.ted.com/talks/candy_chang_before_i_die_i_want_to/transcript
transcript added
https://www.ted.com/talks/louie_schwartzberg_nature_beauty_gratitude/transcript
transcript added
https://www.ted.com/talks/sleepy_man_banjo_boys_teen_wonders_play_bluegrass/transcript
transcript added
https://www.ted.com/ta

transcript added
https://www.ted.com/talks/jon_ronson_strange_answers_to_the_psychopath_test/transcript
transcript added
https://www.ted.com/talks/caitria_morgan_o_neill_how_to_step_up_in_the_face_of_disaster/transcript
transcript added
https://www.ted.com/talks/ivan_krastev_can_democracy_exist_without_trust/transcript
transcript added
https://www.ted.com/talks/mark_forsyth_what_s_a_snollygoster_a_short_lesson_in_political_speak/transcript
transcript added
https://www.ted.com/talks/lisa_kristine_photos_that_bear_witness_to_modern_slavery/transcript
transcript added
https://www.ted.com/talks/kirby_ferguson_embrace_the_remix/transcript
transcript added
https://www.ted.com/talks/pam_warhurst_how_we_can_eat_our_landscapes/transcript
transcript added
https://www.ted.com/talks/max_little_a_test_for_parkinson_s_with_a_phone_call/transcript
transcript added
https://www.ted.com/talks/margaret_heffernan_dare_to_disagree/transcript
transcript added
https://www.ted.com/talks/scilla_elworthy_fighti

In [146]:
get_transcript(3000, 4000)

https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human/transcript
transcript added
https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables/transcript
transcript added
https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes/transcript
transcript added
https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws/transcript
transcript added
https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire/transcript
transcript added
https://www.ted.com/talks/hans_rosling_religions_and_babies/transcript
transcript added
https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore/transcript
transcript added
https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen/transcript
transcript added
https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief/transcript
transcript added
https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out/transcript
transcript added
https://

transcript added
https://www.ted.com/talks/vijay_kumar_robots_that_fly_and_cooperate/transcript
transcript added
https://www.ted.com/talks/paul_gilding_the_earth_is_full/transcript
transcript added
https://www.ted.com/talks/peter_diamandis_abundance_is_our_future/transcript
transcript added
https://www.ted.com/talks/daniel_pauly_the_ocean_s_shifting_baseline/transcript
transcript added
https://www.ted.com/talks/paul_snelgrove_a_census_of_the_ocean/transcript
transcript added
https://www.ted.com/talks/kevin_allocca_why_videos_go_viral/transcript
transcript added
https://www.ted.com/talks/jeffrey_kluger_the_sibling_bond/transcript
transcript added
https://www.ted.com/talks/avi_rubin_all_your_devices_can_be_hacked/transcript
transcript added
https://www.ted.com/talks/tan_le_my_immigration_story/transcript
transcript added
https://www.ted.com/talks/shlomo_benartzi_saving_for_tomorrow_tomorrow/transcript
transcript added
https://www.ted.com/talks/shilo_shiv_suleman_using_tech_to_enable_drea

transcript added
https://www.ted.com/talks/kathryn_schulz_don_t_regret_regret/transcript
transcript added
https://www.ted.com/talks/charles_limb_building_the_musical_muscle/transcript
transcript added
https://www.ted.com/talks/john_bohannon_dance_vs_powerpoint_a_modest_proposal/transcript
transcript added
https://www.ted.com/talks/annie_murphy_paul_what_we_learn_before_we_re_born/transcript
transcript added
https://www.ted.com/talks/damon_horowitz_philosophy_in_prison/transcript
transcript added
https://www.ted.com/talks/amy_purdy_living_beyond_limits/transcript
transcript added
https://www.ted.com/talks/britta_riley_a_garden_in_my_apartment/transcript
transcript added
https://www.ted.com/talks/joe_sabia_the_technology_of_storytelling/transcript
transcript added
https://www.ted.com/talks/peter_fankhauser_meet_rezero_the_dancing_ballbot/transcript
transcript added
https://www.ted.com/talks/phil_plait_how_to_defend_earth_from_asteroids/transcript
transcript added
https://www.ted.com/talk

https://www.ted.com/talks/mark_pagel_how_language_transformed_humanity/transcript
transcript added
https://www.ted.com/talks/harald_haas_wireless_data_from_every_light_bulb/transcript
transcript added
https://www.ted.com/talks/adam_ostrow_after_your_final_status_update/transcript
transcript added
https://www.ted.com/talks/julian_treasure_5_ways_to_listen_better/transcript
transcript added
https://www.ted.com/talks/josette_sheeran_ending_hunger_now/transcript
transcript added
https://www.ted.com/talks/paul_bloom_the_origins_of_pleasure/transcript
transcript added
https://www.ted.com/talks/geoffrey_west_the_surprising_math_of_cities_and_corporations/transcript
transcript added
https://www.ted.com/talks/rory_stewart_time_to_end_the_war_in_afghanistan/transcript
transcript added
https://www.ted.com/talks/markus_fischer_a_robot_that_flies_like_a_bird/transcript
transcript added
https://www.ted.com/talks/kevin_slavin_how_algorithms_shape_our_world/transcript
transcript added
https://www.ted.

transcript added
https://www.ted.com/talks/jackson_browne_a_song_inspired_by_the_ocean/transcript
transcript added
https://www.ted.com/talks/caroline_casey_looking_past_limits/transcript
transcript added
https://www.ted.com/talks/mick_ebeling_the_invention_that_unlocked_a_locked_in_artist/transcript
transcript added
https://www.ted.com/talks/morgan_spurlock_the_greatest_ted_talk_ever_sold/transcript
transcript added
https://www.ted.com/talks/chade_meng_tan_everyday_compassion_at_google/transcript
transcript added
https://www.ted.com/talks/stanley_mcchrystal_listen_learn_then_lead/transcript
transcript added
https://www.ted.com/talks/annmarie_thomas_hands_on_science_with_squishy_circuits/transcript
transcript added
https://www.ted.com/talks/eric_whitacre_a_virtual_choir_2_000_voices_strong/transcript
transcript added
https://www.ted.com/talks/sebastian_thrun_google_s_driverless_car/transcript
transcript added
https://www.ted.com/talks/handspring_puppet_co_the_genius_puppetry_behind_war_

...

https://www.ted.com/talks/thelma_golden_how_art_gives_shape_to_cultural_change/transcript
transcript added
https://www.ted.com/talks/catherine_mohr_the_tradeoffs_of_building_green/transcript
transcript added
https://www.ted.com/talks/jonathan_klein_photos_that_changed_the_world/transcript
transcript added
https://www.ted.com/talks/michael_specter_the_danger_of_science_denial/transcript
transcript added
https://www.ted.com/talks/natalie_merchant_singing_old_poems_to_life/transcript
transcript added
https://www.ted.com/talks/jonathan_drori_every_pollen_grain_has_a_story/transcript
transcript added
https://www.ted.com/talks/dennis_hong_my_seven_species_of_robot_and_how_we_created_them/transcript
transcript added
https://www.ted.com/talks/dean_kamen_the_emotion_behind_invention/transcript
transcript added
https://www.ted.com/talks/elizabeth_pisani_sex_drugs_and_hiv_let_s_get_rational/transcript
transcript added
https://www.ted.com/talks/jesse_schell_when_games_invade_real_life/transcript
t

https://www.ted.com/talks/michael_sandel_what_s_the_right_thing_to_do/transcript
transcript added
https://www.ted.com/talks/asher_hasan_my_message_of_peace_from_pakistan/transcript
transcript added
https://www.ted.com/talks/steven_cowley_fusion_is_energy_s_future/transcript
transcript added
https://www.ted.com/talks/shaffi_mather_a_new_way_to_fight_corruption/transcript
transcript added
https://www.ted.com/talks/james_geary_metaphorically_speaking/transcript
transcript added
https://www.ted.com/talks/charles_anderson_dragonflies_that_fly_across_oceans/transcript
transcript added
https://www.ted.com/talks/alexis_ohanian_how_to_make_a_splash_in_social_media/transcript
transcript added
https://www.ted.com/talks/ryan_lobo_photographing_the_hidden_story/transcript
transcript added
https://www.ted.com/talks/loretta_napoleoni_the_intricate_economics_of_terrorism/transcript
transcript added
https://www.ted.com/talks/shereen_el_feki_pop_culture_in_the_arab_world/transcript
transcript added
http

transcript added
https://www.ted.com/talks/paul_romer_why_the_world_needs_charter_cities/transcript
transcript added
https://www.ted.com/talks/michael_pritchard_how_to_make_filthy_water_drinkable/transcript
transcript added
https://www.ted.com/talks/willard_wigan_hold_your_breath_for_micro_sculpture/transcript
transcript added
https://www.ted.com/talks/elaine_morgan_i_believe_we_evolved_from_aquatic_apes/transcript
transcript added
https://www.ted.com/talks/golan_levin_art_that_looks_back_at_you/transcript
transcript added
https://www.ted.com/talks/alain_de_botton_a_kinder_gentler_philosophy_of_success/transcript
transcript added
https://www.ted.com/talks/gordon_brown_wiring_a_web_for_global_good/transcript
transcript added
https://www.ted.com/talks/nina_jablonski_skin_color_is_an_illusion/transcript
transcript added
https://www.ted.com/talks/jim_fallon_exploring_the_mind_of_a_killer/transcript
transcript added
https://www.ted.com/talks/daniel_kraft_a_better_way_to_harvest_bone_marrow/

transcript added
https://www.ted.com/talks/john_wooden_the_difference_between_winning_and_succeeding/transcript
transcript added
https://www.ted.com/talks/david_pogue_cool_tricks_your_phone_can_do/transcript
transcript added
https://www.ted.com/talks/jacqueline_novogratz_an_escape_from_poverty/transcript
transcript added
https://www.ted.com/talks/saul_griffith_high_altitude_wind_energy_from_kites/transcript
transcript added
https://www.ted.com/talks/kamal_meattle_how_to_grow_fresh_air/transcript
transcript added
https://www.ted.com/talks/bruce_mccall_what_is_retro_futurism/transcript
transcript added
https://www.ted.com/talks/adam_savage_my_obsession_with_objects_and_the_stories_they_tell/transcript
transcript added
https://www.ted.com/talks/dan_ariely_our_buggy_moral_code/transcript
transcript added
https://www.ted.com/talks/dan_dennett_cute_sexy_sweet_funny/transcript
transcript added
https://www.ted.com/talks/tim_berners_lee_the_next_web/transcript
transcript added
https://www.ted.c

transcript added
https://www.ted.com/talks/robert_thurman_expanding_your_circle_of_compassion/transcript
transcript added
https://www.ted.com/talks/feisal_abdul_rauf_lose_your_ego_find_your_compassion/transcript
transcript added
https://www.ted.com/talks/james_forbes_compassion_at_the_dinner_table/transcript
transcript added
https://www.ted.com/talks/dayananda_saraswati_the_profound_journey_of_compassion/transcript
transcript added
https://www.ted.com/talks/jackie_tabick_the_balancing_act_of_compassion/transcript
transcript added
https://www.ted.com/talks/newton_aduaka_the_story_of_ezra/transcript
transcript added
https://www.ted.com/talks/zach_kaplan_keith_schacht_toys_and_materials_from_the_future/transcript
transcript added
https://www.ted.com/talks/rives_a_story_of_mixed_emoticons/transcript
transcript added
https://www.ted.com/talks/jared_diamond_why_do_societies_collapse/transcript
transcript added
https://www.ted.com/talks/kristen_ashburn_the_face_of_aids_in_africa/transcript
tr

In [150]:
get_transcript(4000, len(talk_df))

https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines/transcript
transcript added
https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity/transcript
transcript added
https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate/transcript
transcript added
https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art/transcript
transcript added
https://www.ted.com/talks/george_dyson_the_birth_of_the_computer/transcript
transcript added
https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language/transcript
transcript added
https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual/transcript
transcript added
https://www.ted.com/talks/rokia_traore_kounandi/transcript
transcript added
https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq/transcript
transcript added
https://www.ted.com/talks/susan_blackmore_memes_and_temes/transcript
transcript added
https://www.ted.com/talks/paul_collier_the_bot

https://www.ted.com/talks/paul_rothemund_playing_with_dna_that_self_assembles/transcript
transcript added
https://www.ted.com/talks/lawrence_lessig_laws_that_choke_creativity/transcript
transcript added
https://www.ted.com/talks/matthieu_ricard_the_habits_of_happiness/transcript
transcript added
https://www.ted.com/talks/sherwin_nuland_how_electroshock_therapy_changed_me/transcript
transcript added
https://www.ted.com/talks/eleni_gabre_madhin_a_commodities_exchange_for_ethiopia/transcript
transcript added
https://www.ted.com/talks/vs_ramachandran_3_clues_to_understanding_your_brain/transcript
transcript added
https://www.ted.com/talks/jan_chipchase_the_anthropology_of_mobile_phones/transcript
transcript added
https://www.ted.com/talks/maira_kalman_the_illustrated_woman/transcript
transcript added
https://www.ted.com/talks/hod_lipson_building_self_aware_robots/transcript
transcript added
https://www.ted.com/talks/richard_branson_life_at_30_000_feet/transcript
transcript added
https://ww

https://www.ted.com/talks/james_watson_how_we_discovered_dna/transcript
transcript added
https://www.ted.com/talks/golan_levin_software_as_art/transcript
transcript added
https://www.ted.com/talks/chris_bangle_great_cars_are_great_art/transcript
transcript added
https://www.ted.com/talks/sheila_patek_the_shrimp_with_a_kick/transcript
transcript added
https://www.ted.com/talks/dean_kamen_to_invent_is_to_give/transcript
transcript added
https://www.ted.com/talks/janine_benyus_biomimicry_s_surprising_lessons_from_nature_s_engineers/transcript
transcript added
https://www.ted.com/talks/al_seckel_visual_illusions_that_show_how_we_mis_think/transcript
transcript added
https://www.ted.com/talks/janine_benyus_biomimicry_s_surprising_lessons_from_nature_s_engineers/transcript
transcript added
https://www.ted.com/talks/paul_bennett_design_is_in_the_details/transcript
transcript added
https://www.ted.com/talks/james_nachtwey_my_wish_let_my_photographs_bear_witness/transcript
transcript added
http

In [151]:
talk_df['transcript']

0       So I've been thinking about how to explain thi...
1       This chimpanzee stumbles across a windfall of ...
2       Recently, I've been declaring to anyone who wo...
3       When we're young, we're innocently brave, and ...
4       OK, I have a question for all of us. You ready...
5       I wanted to be a psychologist since I was a te...
6       In the 4th century BCE, a banker’s son threw t...
7       Chris Anderson: Nick Bostrom. So, you have alr...
8       In June of 2017, I volunteered with a group at...
9       So, on April 23 of 2013, the Associated Press ...
10      A mother and her son trek across an endless de...
11      We are all atomically connected. Fundamentally...
12      In her Auntie An-mei’s home, Jing-Mei reluctan...
13      Now, I know it might be easy to think that mic...
14      Aquay Wunne Kesuk. Kelsey Leonard Nooweesuonk....
15      I am an ideas activist. That means I fight for...
16      William Golding was losing his faith in humani...
17      It was

In [2]:
talk_df = pd.read_csv('talks_with_transcripts.csv')

In [48]:
talk_df_2 = pd.read_csv('talks_with_transcripts.csv')

In [19]:
math.isnan(talk_df.transcript[4243])

True

In [3]:
len(talk_df[talk_df['transcript'] == ''])

0

In [26]:
def fill_transcript(start, end):
    for index in range(start, end):
        if isinstance(talk_df['transcript'][index], float):
            try:
                # Get link, parse using BeautifulSoup
                url = talk_df['url'][index]
                link = "https://www.ted.com" + url + "/transcript"
                print(link)
                page = requests.get(link)
                content = BS(page.content, 'html.parser')

                # Get span class, find transcript
                transcript = ''
                for x in content.find_all('div', {"class": "Grid__cell flx-s:1 p-r:4"}):
                    transcript += x.get_text().strip()
                talk_df['transcript'][index] = ' '.join(transcript.split())
                print('transcript added')
                print(index)

                time.sleep(2)
            except:
                print('NO TRANSCRIPT')
        elif isinstance(talk_df['transcript'][index], str):
            if talk_df['transcript'][index] == '':
                try:
                    # Get link, parse using BeautifulSoup
                    url = talk_df['url'][index]
                    link = "https://www.ted.com" + url + "/transcript"
                    print(link)
                    page = requests.get(link)
                    content = BS(page.content, 'html.parser')

                    # Get span class, find transcript
                    transcript = ''
                    for x in content.find_all('div', {"class": "Grid__cell flx-s:1 p-r:4"}):
                        transcript += x.get_text().strip()
                    talk_df['transcript'][index] = ' '.join(transcript.split())
                    print('transcript added')
                    print(index)

                    time.sleep(2)
                except:
                    print('NO TRANSCRIPT')

In [29]:
fill_transcript(0, 1000)

https://www.ted.com/talks/mike_cannon_brookes_how_you_can_use_impostor_syndrome_to_your_benefit/transcript


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


transcript added
36
https://www.ted.com/talks/jean_manuel_izaret_a_new_netflix_style_pricing_model_that_could_make_medical_treatments_affordable_for_all/transcript
transcript added
50
https://www.ted.com/talks/juniper_fitzgerald_why_we_need_to_stop_stigmatizing_mothers_who_do_sex_work/transcript
transcript added
53
https://www.ted.com/talks/dorsa_amir_how_the_industrial_revolution_changed_childhood/transcript
transcript added
54
https://www.ted.com/talks/adar_cohen_3_ways_to_lead_tough_unavoidable_conversations/transcript
transcript added
59
https://www.ted.com/talks/martha_redbone_sleep_sleep_beauty_bright/transcript
transcript added
70
https://www.ted.com/talks/maria_popova_an_excerpt_from_figuring/transcript
transcript added
71
https://www.ted.com/talks/morley_follow_the_sound/transcript
transcript added
72
https://www.ted.com/talks/david_carroll_how_i_sued_cambridge_analytica_over_my_personal_data/transcript
transcript added
77
https://www.ted.com/talks/bhakti_sharma_what_open_wate

https://www.ted.com/talks/yuko_munakata_why_most_parenting_advice_is_wrong/transcript
transcript added
377
https://www.ted.com/talks/guy_harvey_art_and_science_working_together_for_the_conservation_of_large_pelagic_fish/transcript
transcript added
381
https://www.ted.com/talks/sarvesh_subramanian_global_warming_the_app/transcript
transcript added
403
https://www.ted.com/talks/maria_golubeva_why_medieval_politics_was_nothing_like_game_of_thrones/transcript
transcript added
408
https://www.ted.com/talks/levon_biss_extreme_macro_photography_reveals_the_beauty_of_the_insect_world/transcript
transcript added
414
https://www.ted.com/talks/christopher_robichaud_how_i_used_dungeons_dragons_to_teach_ethics/transcript
transcript added
419
https://www.ted.com/talks/lunar_mining_and_the_moon_in_human_culture/transcript
transcript added
432
https://www.ted.com/talks/virginia_wotring_how_i_built_a_first_aid_kit_for_astronauts/transcript
transcript added
436
https://www.ted.com/talks/moriba_jah_why_w

transcript added
666
https://www.ted.com/talks/jason_jay_how_to_benefit_from_uncomfortable_conversations/transcript
transcript added
667
https://www.ted.com/talks/dan_finkel_can_you_solve_the_secret_werewolf_riddle/transcript
transcript added
669
https://www.ted.com/talks/mara_mintzer_how_kids_can_help_design_cities/transcript
transcript added
671
https://www.ted.com/talks/monica_palacio_why_the_district_of_columbia_should_get_an_equal_right_to_vote/transcript
transcript added
675
https://www.ted.com/talks/elizabeth_cox_is_fire_a_solid_a_liquid_or_a_gas/transcript
transcript added
676
https://www.ted.com/talks/suzie_sheehy_the_case_for_curiosity_driven_research/transcript
transcript added
677
https://www.ted.com/talks/julia_shaw_a_memory_scientist_s_advice_on_reporting_harassment_and_discrimination/transcript
transcript added
678
https://www.ted.com/talks/rusha_modi_what_causes_heartburn/transcript
transcript added
679
https://www.ted.com/talks/caroline_winterer_america_s_pursuit_of_ha

transcript added
877
https://www.ted.com/talks/lili_haydn_the_last_serenade/transcript
transcript added
880
https://www.ted.com/talks/elizabeth_cox_what_causes_hallucinations/transcript
transcript added
903
https://www.ted.com/talks/andrew_youn_what_if_we_supported_millions_of_african_farmers_in_growing_more_food/transcript
transcript added
904
https://www.ted.com/talks/jackie_gartner_schmidt_phd_your_voice_says_a_lot_about_you_whether_you_like_it_or_not/transcript
transcript added
906
https://www.ted.com/talks/ian_firth_bridges_should_be_beautiful/transcript
transcript added
907
https://www.ted.com/talks/tracey_mcnamara_canaries_in_the_coal_mine/transcript
transcript added
914
https://www.ted.com/talks/dr_robert_talisse_putting_politics_in_its_place/transcript
transcript added
926
https://www.ted.com/talks/stephen_trzeciak_how_40_seconds_of_compassion_could_save_a_life/transcript
transcript added
947
https://www.ted.com/talks/raj_panjabi_what_if_we_digitally_empowered_community_health

In [45]:
fill_transcript(1000, 2000)

https://www.ted.com/talks/mother_london_ai_therapy/transcript
transcript added
1009
https://www.ted.com/talks/duncan_cowles_it_s_not_amazing_enough/transcript
transcript added
1010
https://www.ted.com/talks/shane_griffin_chromatic/transcript
transcript added
1012
https://www.ted.com/talks/shawn_martinbrough_how_one_black_panther_comic_artist_is_changing_perceptions/transcript
transcript added
1084
https://www.ted.com/talks/rowan_fitzpatrick_and_heart_of_mind_facing_the_future_together/transcript
transcript added
1098
https://www.ted.com/talks/shade_zahrai_and_aric_yegudkin_the_rhythmic_beauty_of_collaboration/transcript
transcript added
1099
https://www.ted.com/talks/will_jenkins_the_most_precious_commodity_of_all/transcript
transcript added
1100
https://www.ted.com/talks/tomer_garzberg_what_happens_when_we_take_humans_out_of_work/transcript
transcript added
1101
https://www.ted.com/talks/rach_ranton_where_are_we_trying_to_end_up/transcript
transcript added
1102
https://www.ted.com/tal

https://www.ted.com/talks/daniela_zamudio_a_millennial_s_unexpected_secret_to_success/transcript
transcript added
1209
https://www.ted.com/talks/nicole_wilson_lessons_from_a_lifetime_of_bad_advice/transcript
transcript added
1210
https://www.ted.com/talks/dwight_stitt_one_dad_s_mission_to_rebuild_bonds_between_kids_and_their_fathers/transcript
transcript added
1211
https://www.ted.com/talks/gabriela_roa_how_a_child_s_mess_could_help_you_rediscover_your_creativity/transcript
transcript added
1212
https://www.ted.com/talks/irene_mora_for_women_in_pursuit_of_motherhood_and_a_career/transcript
transcript added
1213
https://www.ted.com/talks/andrew_millar_why_your_business_needs_dissenters/transcript
transcript added
1214
https://www.ted.com/talks/jonathan_koch_a_data_translation_toolkit_that_anyone_can_use/transcript
transcript added
1215
https://www.ted.com/talks/mahir_can_isik_how_companies_predict_fashion_trends_and_kill_individuality/transcript
transcript added
1216
https://www.ted.com

https://www.ted.com/talks/katherine_lucas_in_defense_of_extroverts/transcript
transcript added
1913
https://www.ted.com/talks/james_bickerton_creating_music_on_the_go/transcript
transcript added
1914
https://www.ted.com/talks/chris_vogtherr_a_rallying_cry_for_office_shorts/transcript
transcript added
1915
https://www.ted.com/talks/charly_haversat_perfectionism_holds_us_back_here_s_why/transcript
transcript added
1916
https://www.ted.com/talks/b_e_a_t_nyc_b_e_a_t_nyc_performs_at_ted_statestreet/transcript
transcript added
1917
https://www.ted.com/talks/johnny_staats_davey_vaughn_a_tribute_to_west_virginia_coal_miners/transcript
transcript added
1920
https://www.ted.com/talks/yami_joshi_how_dance_helps_me_deal_with_stress/transcript
transcript added
1921
https://www.ted.com/talks/scott_szwast_the_global_business_next_door/transcript
transcript added
1922
https://www.ted.com/talks/peter_harris_taking_trash_talk_to_a_whole_new_level/transcript
transcript added
1923
https://www.ted.com/talk

In [58]:
fill_transcript(2000, 3000)

https://www.ted.com/talks/laura_schulz_the_surprisingly_logical_minds_of_babies/transcript
transcript added
2049
https://www.ted.com/talks/gary_haugen_the_hidden_reason_for_poverty_the_world_needs_to_address_now/transcript
transcript added
2084
https://www.ted.com/talks/nathalie_cabrol_how_mars_might_hold_the_secret_to_the_origin_of_life/transcript
transcript added
2085
https://www.ted.com/talks/marc_kushner_why_the_buildings_of_the_future_will_be_shaped_by_you/transcript
transcript added
2116
https://www.ted.com/talks/stefan_gross_selbeck_business_model_innovation_beating_yourself_at_your_own_game/transcript
transcript added
2170
https://www.ted.com/talks/sarah_cairns_smith_finally_tech_solutions_to_economic_development/transcript
transcript added
2171
https://www.ted.com/talks/rochelle_king_the_complex_relationship_between_data_and_design_in_ux/transcript
transcript added
2172
https://www.ted.com/talks/peter_kirn_from_beethoven_to_kinect_linking_music_to_our_bodies/transcript
transcr

https://www.ted.com/talks/james_curleigh_how_to_enhance_and_expand_a_global_brand/transcript
transcript added
2483
https://www.ted.com/talks/del_harvey_protecting_privacy_at_twitter/transcript
transcript added
2484
https://www.ted.com/talks/alan_iny_reigniting_creativity_in_business/transcript
transcript added
2485
https://www.ted.com/talks/wendy_woods_collective_cooperation_in_the_social_sector/transcript
transcript added
2486
https://www.ted.com/talks/francesca_simon_carboni_how_jiu_jitsu_taught_me_to_be_a_better_employee/transcript
transcript added
2542
https://www.ted.com/talks/megan_wilkens_economics_and_art/transcript
transcript added
2543
https://www.ted.com/talks/nathan_ide_democratizing_investing/transcript
transcript added
2544
https://www.ted.com/talks/mark_halliday_bring_out_your_inner_filmmaker/transcript
transcript added
2545
https://www.ted.com/talks/jessica_donohue_the_upside_of_data/transcript
transcript added
2546
https://www.ted.com/talks/hu_liang_crowd_sourced_inves

In [63]:
fill_transcript(3000, 4000)

https://www.ted.com/talks/marco_tempest_a_magical_tale_with_augmented_reality/transcript
transcript added
3055
https://www.ted.com/talks/improv_everywhere_a_ted_speaker_s_worst_nightmare/transcript
transcript added
3077
https://www.ted.com/talks/inara_george_family_tree/transcript
transcript added
3101
https://www.ted.com/talks/bjarke_ingels_hedonistic_sustainability/transcript
transcript added
3120
https://www.ted.com/talks/brian_goldman_doctors_make_mistakes_can_we_talk_about_that/transcript
transcript added
3122
https://www.ted.com/talks/lisa_harouni_a_primer_on_3d_printing/transcript
transcript added
3124
https://www.ted.com/talks/nate_garvis_change_our_culture_change_our_world/transcript
transcript added
3141
https://www.ted.com/talks/homaro_cantu_ben_roche_cooking_as_alchemy/transcript
transcript added
3154
https://www.ted.com/talks/charlie_todd_the_shared_experience_of_absurdity/transcript
transcript added
3186
https://www.ted.com/talks/martin_hanczyc_the_line_between_life_and_n

transcript added
3517
https://www.ted.com/talks/hans_rosling_global_population_growth_box_by_box/transcript
transcript added
3518
https://www.ted.com/talks/nalini_nadkarni_life_science_in_prison/transcript
transcript added
3519
https://www.ted.com/talks/ellen_gustafson_obesity_hunger_1_global_food_issue/transcript
transcript added
3520
https://www.ted.com/talks/mitchell_joachim_don_t_build_your_home_grow_it/transcript
transcript added
3522
https://www.ted.com/talks/clay_shirky_how_cognitive_surplus_will_change_the_world/transcript
transcript added
3526
https://www.ted.com/talks/hillel_cooperman_lego_for_grownups/transcript
transcript added
3527
https://www.ted.com/talks/charles_leadbeater_education_innovation_in_the_slums/transcript
transcript added
3529
https://www.ted.com/talks/marian_bantjes_intricate_beauty_by_design/transcript
transcript added
3530
https://www.ted.com/talks/chip_conley_measuring_what_makes_life_worthwhile/transcript
transcript added
3531
https://www.ted.com/talks/

transcript added
3662
https://www.ted.com/talks/cat_laine_engineering_a_better_life_for_all/transcript
transcript added
3663
https://www.ted.com/talks/michael_sandel_what_s_the_right_thing_to_do/transcript
transcript added
3664
https://www.ted.com/talks/asher_hasan_my_message_of_peace_from_pakistan/transcript
transcript added
3665
https://www.ted.com/talks/steven_cowley_fusion_is_energy_s_future/transcript
transcript added
3666
https://www.ted.com/talks/alexis_ohanian_how_to_make_a_splash_in_social_media/transcript
transcript added
3670
https://www.ted.com/talks/loretta_napoleoni_the_intricate_economics_of_terrorism/transcript
transcript added
3672
https://www.ted.com/talks/shereen_el_feki_pop_culture_in_the_arab_world/transcript
transcript added
3673
https://www.ted.com/talks/thulasiraj_ravilla_how_low_cost_eye_care_can_be_world_class/transcript
transcript added
3674
https://www.ted.com/talks/marc_pachter_the_art_of_the_interview/transcript
transcript added
3675
https://www.ted.com/ta

In [27]:
fill_transcript(4200, len(talk_df))

https://www.ted.com/talks/julia_sweeney_letting_go_of_god/transcript


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


transcript added
4237
https://www.ted.com/talks/sir_ken_robinson_do_schools_kill_creativity/transcript
transcript added
4240


In [67]:
talk_df.to_csv('talks_with_transcripts_filled.csv')

# Get Dates (Dates were mislabeled for some in the quicklist)

In [51]:
talk_df = pd.read_csv('talks_with_transcripts_filled.csv', index_col = 0)

In [52]:
talk_df.head()

,date,speaker,title,url,length,summ,tags,views,transcript
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so...",599446,So I've been thinking about how to explain thi...
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'...",501290,This chimpanzee stumbles across a windfall of ...
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a...",461705,"Recently, I've been declaring to anyone who wo..."
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ...",880662,"When we're young, we're innocently brave, and ..."
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',...",956498,"OK, I have a question for all of us. You ready..."


In [93]:
talk_df['date_recorded'] = '--'
talk_df['upload_date'] = '--'
def get_date(start, end):
    for index in range(start, end):
        try:
            # Get link, parse using BeautifulSoup
            url = talk_df['url'][index]
            link = "https://www.ted.com" + url
            print(link)
            page = requests.get(link)
            content = BS(page.content, 'html.parser')

            # Find information about recording date
            recorded_content = content.find_all('script', {"data-spec": "q"})[0].get_text()
            recorded_str = re.search('"recorded_at":\".*?T', recorded_content)

            # Strip string, only get date, convert to datetime object
            recorded_date = recorded_str.group(0).strip('"recorded_at":').strip(' T')
            talk_df['date_recorded'][index] = datetime.datetime.strptime(recorded_date, '%Y-%m-%d').date()
            
            # Find information about upload date
            upload_content = content.find_all('meta', {'itemprop':'uploadDate'})[0]["content"]
            
            # Strip string, only get date, convert to datetime object
            upload_date = datetime.datetime.strptime(upload_content, '%Y-%m-%dT%H:%M:%S%z')
            talk_df['upload_date'][index] = upload_date.date()
            
            print('Talk ' + str(index) + ' recorded on ' + str(talk_df['date_recorded'][index]))
            print('Talk ' + str(index) + ' uploaded on ' + str(talk_df['upload_date'][index]))
            
            time.sleep(2)
        except:
            print('NO DATE')

In [96]:
get_date(0, 100)

https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator
Talk 0 recorded on 2019-09-24
Talk 0 uploaded on 2020-01-02
https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol
Talk 1 recorded on 2020-01-02
Talk 1 uploaded on 2020-01-02
https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women
Talk 2 recorded on 2019-12-04
Talk 2 uploaded on 2020-01-02
https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear
Talk 3 recorded on 2019-12-04
Talk 3 uploaded on 2019-12-23
https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success
Talk 4 recorded on 2019-12-04
Talk 4 uploaded on 2019-12-20
https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time
Talk 5 recorded on 2019-11-14
Talk 5 uploaded on 2019-12-19
https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism
Talk 6 recorded on 2019-12-19
Talk 6 uploaded on 2019-12-19
https:

Talk 55 recorded on 2019-11-14
Talk 55 uploaded on 2019-11-14
https://www.ted.com/talks/erika_pinheiro_what_s_really_happening_at_the_us_mexico_border_and_how_we_can_do_better
Talk 56 recorded on 2019-09-10
Talk 56 uploaded on 2019-11-14
https://www.ted.com/talks/sydney_jensen_how_can_we_support_the_emotional_wellbeing_of_teachers
Talk 57 recorded on 2019-10-06
Talk 57 uploaded on 2019-11-13
https://www.ted.com/talks/cady_coleman_what_it_s_like_to_live_on_the_international_space_station
Talk 58 recorded on 2019-04-15
Talk 58 uploaded on 2019-11-13
https://www.ted.com/talks/adar_cohen_3_ways_to_lead_tough_unavoidable_conversations
Talk 59 recorded on 2019-10-03
Talk 59 uploaded on 2019-11-12
https://www.ted.com/talks/craig_costello_in_the_war_for_information_will_quantum_computers_defeat_cryptographers
Talk 60 recorded on 2019-05-23
Talk 60 uploaded on 2019-11-12
https://www.ted.com/talks/peter_beck_small_rockets_are_the_next_space_revolution
Talk 61 recorded on 2019-04-15
Talk 61 uploa

In [98]:
get_date(100, 1000)

https://www.ted.com/talks/beau_lotto_and_cirque_du_soleil_how_we_experience_awe_and_why_it_matters
Talk 100 recorded on 2019-04-15
Talk 100 uploaded on 2019-10-25
https://www.ted.com/talks/abigail_spanberger_how_to_connect_with_people_who_are_different_than_you
Talk 101 recorded on 2019-03-29
Talk 101 uploaded on 2019-10-24
https://www.ted.com/talks/claire_wardle_how_you_can_help_transform_the_internet_into_a_place_of_trust
Talk 102 recorded on 2019-04-15
Talk 102 uploaded on 2019-10-24
https://www.ted.com/talks/adrienne_mayor_the_greek_myth_of_talos_the_first_robot
Talk 103 recorded on 2019-10-24
Talk 103 uploaded on 2019-10-24
https://www.ted.com/talks/ashwin_naidu_the_link_between_fishing_cats_and_mangrove_forest_conservation
Talk 104 recorded on 2019-04-15
Talk 104 uploaded on 2019-10-24
https://www.ted.com/talks/jon_m_chu_the_pride_and_power_of_representation_in_film
Talk 105 recorded on 2019-04-15
Talk 105 uploaded on 2019-10-23
https://www.ted.com/talks/abhishek_gopalka_how_moti

https://www.ted.com/talks/sam_van_aken_how_one_tree_grows_40_different_kinds_of_fruit
Talk 154 recorded on 2019-06-27
Talk 154 uploaded on 2019-09-27
https://www.ted.com/talks/amber_stuver_einstein_s_twin_paradox_explained
Talk 155 recorded on 2019-09-26
Talk 155 uploaded on 2019-09-26
https://www.ted.com/talks/will_hurd_a_wall_won_t_solve_america_s_border_problems
Talk 156 recorded on 2019-09-10
Talk 156 uploaded on 2019-09-26
https://www.ted.com/talks/yaniv_erlich_how_we_re_building_the_world_s_largest_family_tree
Talk 157 recorded on 2018-11-14
Talk 157 uploaded on 2019-09-25
https://www.ted.com/talks/laura_wright_why_should_you_read_the_god_of_small_things_by_arundhati_roy
Talk 158 recorded on 2019-09-24
Talk 158 uploaded on 2019-09-24
https://www.ted.com/talks/emily_nagoski_how_couples_can_sustain_a_strong_sexual_connection_for_a_lifetime
Talk 159 recorded on 2019-05-11
Talk 159 uploaded on 2019-09-24
https://www.ted.com/talks/hanna_ilona_harmavaara_the_myth_of_the_sampo_an_infini

https://www.ted.com/talks/joseph_gordon_levitt_how_craving_attention_makes_you_less_creative
Talk 209 recorded on 2019-04-15
Talk 209 uploaded on 2019-08-20
https://www.ted.com/talks/anita_collins_the_benefits_of_music_education
Talk 210 recorded on 2014-10-11
Talk 210 uploaded on 2019-08-19
https://www.ted.com/talks/sergio_feferovich_la_musica_de_las_ideas
Talk 211 recorded on 2012-10-24
Talk 211 uploaded on 2019-08-19
https://www.ted.com/talks/james_keblas_how_one_city_became_a_music_capital
Talk 212 recorded on 2019-05-04
Talk 212 uploaded on 2019-08-19
https://www.ted.com/talks/indre_viskontas_how_music_makes_me_a_better_neuroscientist
Talk 213 recorded on 2016-10-06
Talk 213 uploaded on 2019-08-19
https://www.ted.com/talks/sara_valencia_botto_when_do_kids_start_to_care_about_other_people_s_opinions
Talk 214 recorded on 2019-03-15
Talk 214 uploaded on 2019-08-19
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_multiverse_rescue_mission_riddle
Talk 215 recorded on 2019-08-1

Talk 263 recorded on 2015-10-04
Talk 263 uploaded on 2019-07-01
https://www.ted.com/talks/gregory_berns_what_emotions_look_like_in_a_dog_s_brain
Talk 264 recorded on 2015-04-28
Talk 264 uploaded on 2019-07-01
https://www.ted.com/talks/barbara_sherman_how_to_read_a_dog_s_body_language
Talk 265 recorded on 2017-04-01
Talk 265 uploaded on 2019-07-01
https://www.ted.com/talks/julia_espinosa_the_secret_life_of_dogs
Talk 266 recorded on 2019-03-23
Talk 266 uploaded on 2019-07-01
https://www.ted.com/talks/glenn_cantave_how_augmented_reality_is_changing_activism
Talk 267 recorded on 2018-12-05
Talk 267 uploaded on 2019-07-01
https://www.ted.com/talks/jon_gray_the_next_big_thing_is_coming_from_the_bronx_again
Talk 268 recorded on 2019-04-15
Talk 268 uploaded on 2019-06-28
https://www.ted.com/talks/harrison_scott_key_the_american_dream_value_menu
Talk 269 recorded on 2019-06-14
Talk 269 uploaded on 2019-06-27
https://www.ted.com/talks/melvin_sanicas_what_makes_tb_the_world_s_most_infectious_kill

https://www.ted.com/talks/bjarke_ingels_floating_cities_the_lego_house_and_other_architectural_forms_of_the_future
Talk 320 recorded on 2019-04-15
Talk 320 uploaded on 2019-06-03
https://www.ted.com/talks/william_butler_yeats_the_second_coming
Talk 321 recorded on 2019-05-31
Talk 321 uploaded on 2019-05-31
https://www.ted.com/talks/roger_hanlon_the_amazing_brains_and_morphing_skin_of_octopuses_and_other_cephalopods
Talk 322 recorded on 2019-04-15
Talk 322 uploaded on 2019-05-31
https://www.ted.com/talks/reniqua_allen_the_story_we_tell_about_millennials_and_who_we_leave_out
Talk 323 recorded on 2019-01-16
Talk 323 uploaded on 2019-05-30
https://www.ted.com/talks/andrea_gibson_the_nutritionist
Talk 324 recorded on 2019-05-29
Talk 324 uploaded on 2019-05-30
https://www.ted.com/talks/alex_gendler_why_should_you_read_the_master_and_margarita
Talk 325 recorded on 2019-05-30
Talk 325 uploaded on 2019-05-30
https://www.ted.com/talks/erika_hamden_what_it_takes_to_launch_a_telescope
Talk 326 rec

...

Talk 762 recorded on 2017-06-15
Talk 762 uploaded on 2018-09-14
https://www.ted.com/talks/luhan_yang_how_to_create_a_world_where_no_one_dies_waiting_for_a_transplant
Talk 763 recorded on 2018-04-10
Talk 763 uploaded on 2018-09-13
https://www.ted.com/talks/krishna_sudhir_how_do_cigarettes_affect_the_body
Talk 764 recorded on 2018-09-13
Talk 764 uploaded on 2018-09-13
https://www.ted.com/talks/ghada_wali_how_i_m_using_lego_to_teach_arabic
Talk 765 recorded on 2017-08-27
Talk 765 uploaded on 2018-09-13
https://www.ted.com/talks/bob_woodward_what_i_learned_investigating_nixon
Talk 766 recorded on 2016-10-21
Talk 766 uploaded on 2018-09-12
https://www.ted.com/talks/will_macaskill_what_are_the_most_important_moral_problems_of_our_time
Talk 767 recorded on 2018-04-10
Talk 767 uploaded on 2018-09-12
https://www.ted.com/talks/elise_legrow_you_never_can_tell_over_the_mountain_across_the_sea
Talk 768 recorded on 2018-04-10
Talk 768 uploaded on 2018-09-12
https://www.ted.com/talks/alan_lupack_is_t

https://www.ted.com/talks/elizabeth_cawein_how_to_build_a_thriving_music_scene_in_your_city
NO DATE
https://www.ted.com/talks/mary_lou_jepsen_how_we_can_use_light_to_see_deep_inside_our_bodies_and_brains
Talk 820 recorded on 2018-04-10
Talk 820 uploaded on 2018-08-24
https://www.ted.com/talks/tanya_khovanova_can_you_solve_the_leonardo_da_vinci_riddle
Talk 821 recorded on 2018-08-23
Talk 821 uploaded on 2018-08-23
https://www.ted.com/talks/leticia_gasca_don_t_fail_fast_fail_mindfully
Talk 822 recorded on 2018-06-07
Talk 822 uploaded on 2018-08-23
https://www.ted.com/talks/leticia_gasca_una_nueva_manera_de_hablar_del_fracaso_en_los_negocios
Talk 823 recorded on 2018-04-26
Talk 823 uploaded on 2018-08-23
https://www.ted.com/talks/janet_stovall_how_to_get_serious_about_diversity_and_inclusion_in_the_workplace
Talk 824 recorded on 2018-07-19
Talk 824 uploaded on 2018-08-22
https://www.ted.com/talks/jessica_pryce_to_transform_child_welfare_take_race_out_of_the_equation
NO DATE
https://www.te

Talk 874 recorded on 2018-07-01
Talk 874 uploaded on 2018-07-17
https://www.ted.com/talks/rebeca_hwang_the_power_of_diversity_within_yourself
Talk 875 recorded on 2018-04-10
Talk 875 uploaded on 2018-07-17
https://www.ted.com/talks/rebeca_hwang_el_valor_de_nuestra_diversidad_interna
Talk 876 recorded on 2018-04-26
Talk 876 uploaded on 2018-07-17
https://www.ted.com/talks/francisco_samper_las_ideas_mas_poderosas_que_las_balas
Talk 877 recorded on 2017-10-07
Talk 877 uploaded on 2018-07-16
https://www.ted.com/talks/will_marshall_the_mission_to_create_a_searchable_database_of_earth_s_surface
Talk 878 recorded on 2018-04-10
Talk 878 uploaded on 2018-07-16
https://www.ted.com/talks/renzo_piano_the_genius_behind_some_of_the_world_s_most_famous_buildings
Talk 879 recorded on 2018-04-10
Talk 879 uploaded on 2018-07-13
https://www.ted.com/talks/lili_haydn_the_last_serenade
Talk 880 recorded on 2018-04-10
Talk 880 uploaded on 2018-07-13
https://www.ted.com/talks/elizabeth_white_an_honest_look_at

https://www.ted.com/talks/max_tegmark_how_to_get_empowered_not_overpowered_by_ai
Talk 929 recorded on 2018-04-10
Talk 929 uploaded on 2018-06-13
https://www.ted.com/talks/sam_rodriques_what_we_ll_learn_about_the_brain_in_the_next_century
Talk 930 recorded on 2017-11-18
Talk 930 uploaded on 2018-06-12
https://www.ted.com/talks/iseult_gillespie_the_irish_myth_of_the_giant_s_causeway
Talk 931 recorded on 2018-06-12
Talk 931 uploaded on 2018-06-12
https://www.ted.com/talks/jason_b_rosenthal_the_journey_through_loss_and_grief
Talk 932 recorded on 2018-04-10
Talk 932 uploaded on 2018-06-12
https://www.ted.com/talks/dan_finkel_can_you_solve_the_giant_cat_army_riddle
Talk 933 recorded on 2018-06-11
Talk 933 uploaded on 2018-06-11
https://www.ted.com/talks/greg_gage_how_octopuses_battle_each_other
Talk 934 recorded on 2018-06-04
Talk 934 uploaded on 2018-06-11
https://www.ted.com/talks/greg_gage_the_real_reason_why_mosquitoes_buzz
Talk 935 recorded on 2018-06-04
Talk 935 uploaded on 2018-06-11


Talk 984 recorded on 2017-12-10
Talk 984 uploaded on 2018-05-08
https://www.ted.com/talks/erricka_bridgeford_how_baltimore_called_a_ceasefire
Talk 985 recorded on 2017-10-27
Talk 985 uploaded on 2018-05-08
https://www.ted.com/talks/priya_vulchi_and_winona_guo_what_it_takes_to_be_racially_literate
Talk 986 recorded on 2017-11-01
Talk 986 uploaded on 2018-05-07
https://www.ted.com/talks/kevin_etter_i_am_the_donation
Talk 987 recorded on 2015-09-02
Talk 987 uploaded on 2018-05-04
https://www.ted.com/talks/frances_frei_how_to_build_and_rebuild_trust
Talk 988 recorded on 2018-04-10
Talk 988 uploaded on 2018-05-04
https://www.ted.com/talks/rebecca_kleinberger_why_you_don_t_like_the_sound_of_your_own_voice
Talk 989 recorded on 2017-11-18
Talk 989 uploaded on 2018-05-03
https://www.ted.com/talks/tania_douglas_to_design_better_tech_understand_context
Talk 990 recorded on 2017-08-27
Talk 990 uploaded on 2018-05-03
https://www.ted.com/talks/laura_l_dunn_it_s_time_for_the_law_to_protect_victims_of

In [101]:
get_date(1000, 2000)

https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
Talk 1000 recorded on 2017-08-27
Talk 1000 uploaded on 2018-04-24
https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
Talk 1001 recorded on 2017-11-01
Talk 1001 uploaded on 2018-04-24
https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
Talk 1002 recorded on 2018-04-10
Talk 1002 uploaded on 2018-04-23
https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
Talk 1003 recorded on 2018-04-10
Talk 1003 uploaded on 2018-04-20
https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
Talk 1004 recorded on 2018-04-10
Talk 1004 uploaded on 2018-04-19
https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
Talk 1005 recorded on 2017-11-01
Talk 1005 uploaded on 2018-04-18
https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatm

https://www.ted.com/talks/hadi_eldebek_why_must_artists_be_poor
Talk 1056 recorded on 2016-12-13
Talk 1056 uploaded on 2018-03-15
https://www.ted.com/talks/emma_rodero_como_darle_credibilidad_a_tu_voz
Talk 1057 recorded on 2018-02-17
Talk 1057 uploaded on 2018-03-15
https://www.ted.com/talks/isabel_wilkerson_the_great_migration_and_the_power_of_a_single_decision
Talk 1058 recorded on 2017-11-01
Talk 1058 uploaded on 2018-03-15
https://www.ted.com/talks/margaret_gould_stewart_how_the_hyperlink_changed_everything
Talk 1059 recorded on 2018-03-15
Talk 1059 uploaded on 2018-03-15
https://www.ted.com/talks/david_rockwell_the_hidden_ways_stairs_shape_your_life
Talk 1060 recorded on 2018-03-15
Talk 1060 uploaded on 2018-03-15
https://www.ted.com/talks/isaac_mizrahi_how_the_button_changed_fashion
Talk 1061 recorded on 2018-03-15
Talk 1061 uploaded on 2018-03-15
https://www.ted.com/talks/paola_antonelli_the_3_000_year_history_of_the_hoodie
Talk 1062 recorded on 2018-03-15
Talk 1062 uploaded on 

Talk 1110 recorded on 2017-05-03
Talk 1110 uploaded on 2018-02-13
https://www.ted.com/talks/marco_alvera_the_surprising_ingredient_that_makes_businesses_work_better
Talk 1111 recorded on 2017-10-04
Talk 1111 uploaded on 2018-02-13
https://www.ted.com/talks/the_holladay_brothers_to_the_beat_of_light
Talk 1112 recorded on 2016-11-15
Talk 1112 uploaded on 2018-02-12
https://www.ted.com/talks/tanmay_bakshi_technology_that_tackles_the_teen_suicide_epidemic
Talk 1113 recorded on 2017-12-06
Talk 1113 uploaded on 2018-02-12
https://www.ted.com/talks/robin_hauser_can_we_protect_ai_from_our_biases
Talk 1114 recorded on 2017-12-06
Talk 1114 uploaded on 2018-02-12
https://www.ted.com/talks/rana_novack_how_we_ll_predict_the_next_refugee_crisis
Talk 1115 recorded on 2017-12-06
Talk 1115 uploaded on 2018-02-12
https://www.ted.com/talks/phil_tetlow_8_steps_to_understanding_information_and_maybe_the_universe
Talk 1116 recorded on 2017-12-06
Talk 1116 uploaded on 2018-02-12
https://www.ted.com/talks/nat

Talk 1165 recorded on 2017-10-04
Talk 1165 uploaded on 2018-02-06
https://www.ted.com/talks/guy_winch_how_to_fix_a_broken_heart
Talk 1166 recorded on 2017-04-24
Talk 1166 uploaded on 2018-02-05
https://www.ted.com/talks/stuart_duncan_how_i_use_minecraft_to_help_kids_with_autism
Talk 1167 recorded on 2017-04-22
Talk 1167 uploaded on 2018-02-02
https://www.ted.com/talks/peter_ouko_from_death_row_to_law_graduate
Talk 1168 recorded on 2017-08-27
Talk 1168 uploaded on 2018-02-02
https://www.ted.com/talks/danielle_wood_6_space_technologies_we_can_use_to_improve_life_on_earth
Talk 1169 recorded on 2017-11-16
Talk 1169 uploaded on 2018-02-01
https://www.ted.com/talks/mwende_freequency_katwiwa_black_life_at_the_intersection_of_birth_and_death
Talk 1170 recorded on 2017-11-02
Talk 1170 uploaded on 2018-02-01
https://www.ted.com/talks/david_hooker_who_s_teaching_us_to_read_and_write_visuals
Talk 1171 recorded on 2017-11-18
Talk 1171 uploaded on 2018-02-01
https://www.ted.com/talks/anjali_kumar_my

Talk 1219 recorded on 2016-09-10
Talk 1219 uploaded on 2018-01-04
https://www.ted.com/talks/james_orsulak_manufacturing_in_space_could_save_life_on_earth
Talk 1220 recorded on 2017-11-10
Talk 1220 uploaded on 2018-01-03
https://www.ted.com/talks/lana_mazahreh_3_thoughtful_ways_to_conserve_water
Talk 1221 recorded on 2017-10-04
Talk 1221 uploaded on 2018-01-03
https://www.ted.com/talks/lisa_feldman_barrett_you_aren_t_at_the_mercy_of_your_emotions_your_brain_creates_them
Talk 1222 recorded on 2017-12-06
Talk 1222 uploaded on 2018-01-02
https://www.ted.com/talks/christopher_ategeka_how_adoption_worked_for_me
Talk 1223 recorded on 2017-08-27
Talk 1223 uploaded on 2018-01-02
https://www.ted.com/talks/heather_lanier_good_and_bad_are_incomplete_stories_we_tell_ourselves
Talk 1224 recorded on 2017-10-04
Talk 1224 uploaded on 2017-12-21
https://www.ted.com/talks/christian_benimana_the_next_generation_of_african_architects_and_designers
Talk 1225 recorded on 2017-08-27
Talk 1225 uploaded on 2017

...

https://www.ted.com/talks/aditi_gupta_a_taboo_free_way_to_talk_about_periods
Talk 1771 recorded on 2015-05-29
Talk 1771 uploaded on 2016-04-21
https://www.ted.com/talks/juan_enriquez_we_can_reprogram_life_how_to_do_it_wisely
Talk 1772 recorded on 2015-11-06
Talk 1772 uploaded on 2016-04-20
https://www.ted.com/talks/marco_a_sotomayor_the_surprising_reason_you_feel_awful_when_you_re_sick
Talk 1773 recorded on 2016-04-19
Talk 1773 uploaded on 2019-03-22
https://www.ted.com/talks/joshua_prager_wisdom_from_great_writers_on_every_year_of_life
Talk 1774 recorded on 2015-03-19
Talk 1774 uploaded on 2016-04-19
https://www.ted.com/talks/chris_anderson_ted_s_secret_to_great_public_speaking
Talk 1775 recorded on 2016-03-24
Talk 1775 uploaded on 2016-04-19
https://www.ted.com/talks/derek_abbott_should_you_trust_unanimous_decisions
Talk 1776 recorded on 2016-04-18
Talk 1776 uploaded on 2019-03-15
https://www.ted.com/talks/christiana_figueres_the_inside_story_of_the_paris_climate_agreement
Talk 1777 

https://www.ted.com/talks/raffaello_d_andrea_meet_the_dazzling_flying_machines_of_the_future
Talk 1827 recorded on 2016-02-15
Talk 1827 uploaded on 2016-02-19
https://www.ted.com/talks/allan_adams_what_the_discovery_of_gravitational_waves_means
Talk 1828 recorded on 2016-02-15
Talk 1828 uploaded on 2016-02-18
https://www.ted.com/talks/shonda_rhimes_my_year_of_saying_yes_to_everything
Talk 1829 recorded on 2016-02-15
Talk 1829 uploaded on 2016-02-17
https://www.ted.com/talks/angela_koine_flynn_the_science_of_skin_color
Talk 1830 recorded on 2016-02-16
Talk 1830 uploaded on 2017-09-07
https://www.ted.com/talks/celeste_headlee_10_ways_to_have_a_better_conversation
Talk 1831 recorded on 2015-05-01
Talk 1831 uploaded on 2016-02-16
https://www.ted.com/talks/jocelyne_bloch_the_brain_may_be_able_to_repair_itself_with_help
Talk 1832 recorded on 2015-12-08
Talk 1832 uploaded on 2016-02-15
https://www.ted.com/talks/dorothy_roberts_the_problem_with_race_based_medicine
Talk 1833 recorded on 2015-11

https://www.ted.com/talks/claudio_guerra_why_the_octopus_brain_is_so_extraordinary
Talk 1882 recorded on 2015-12-03
Talk 1882 uploaded on 2019-04-01
https://www.ted.com/talks/chieko_asakawa_how_new_technology_helps_blind_people_explore_the_world
Talk 1883 recorded on 2015-10-15
Talk 1883 uploaded on 2015-12-03
https://www.ted.com/talks/jessica_shortall_the_us_needs_paid_family_leave_for_the_sake_of_its_future
Talk 1884 recorded on 2015-10-17
Talk 1884 uploaded on 2015-12-02
https://www.ted.com/talks/kristen_marhaver_how_we_re_growing_baby_corals_to_rebuild_reefs
Talk 1885 recorded on 2015-10-10
Talk 1885 uploaded on 2015-12-01
https://www.ted.com/talks/dan_van_der_vieren_can_you_solve_einstein_s_riddle
Talk 1886 recorded on 2015-11-30
Talk 1886 uploaded on 2019-03-01
https://www.ted.com/talks/year_in_ideas_2015
Talk 1887 recorded on 2015-12-01
Talk 1887 uploaded on 2015-11-30
https://www.ted.com/talks/marina_abramovic_an_art_made_of_trust_vulnerability_and_connection
Talk 1888 recorded

https://www.ted.com/talks/francesco_sauro_deep_under_the_earth_s_surface_discovering_beauty_and_science
Talk 1937 recorded on 2015-09-29
Talk 1937 uploaded on 2015-10-23
https://www.ted.com/talks/sharon_horesh_bergquist_how_stress_affects_your_body
Talk 1938 recorded on 2015-10-22
Talk 1938 uploaded on 2019-03-15
https://www.ted.com/talks/tea_uglow_an_internet_without_screens_might_look_like_this
Talk 1939 recorded on 2015-05-20
Talk 1939 uploaded on 2015-10-22
https://www.ted.com/talks/jennifer_doudna_how_crispr_lets_us_edit_our_dna
Talk 1940 recorded on 2015-09-29
Talk 1940 uploaded on 2015-10-20
https://www.ted.com/talks/ted_ed_why_do_women_have_periods
Talk 1941 recorded on 2015-10-19
Talk 1941 uploaded on 2019-03-22
https://www.ted.com/talks/will_potter_the_secret_us_prisons_you_ve_never_heard_of_before
Talk 1942 recorded on 2015-08-28
Talk 1942 uploaded on 2015-10-19
https://www.ted.com/talks/meklit_hadero_the_unexpected_beauty_of_everyday_sounds
Talk 1943 recorded on 2015-08-28


https://www.ted.com/talks/alex_gendler_can_you_solve_the_bridge_riddle
Talk 1991 recorded on 2015-09-01
Talk 1991 uploaded on 2019-02-15
https://www.ted.com/talks/elizabeth_nyamayaro_an_invitation_to_men_who_want_a_better_world_for_women
Talk 1992 recorded on 2015-05-28
Talk 1992 uploaded on 2015-09-01
https://www.ted.com/talks/wendy_freedman_this_telescope_might_show_us_the_beginning_of_the_universe
Talk 1993 recorded on 2014-10-09
Talk 1993 uploaded on 2015-08-31
https://www.ted.com/talks/yves_morieux_how_too_many_rules_at_work_keep_you_from_getting_things_done
Talk 1994 recorded on 2015-07-01
Talk 1994 uploaded on 2015-08-28
https://www.ted.com/talks/robin_murphy_these_robots_come_to_the_rescue_after_a_disaster
Talk 1995 recorded on 2015-05-28
Talk 1995 uploaded on 2015-08-27
https://www.ted.com/talks/sarthak_sinha_why_do_some_people_go_bald
Talk 1996 recorded on 2015-08-25
Talk 1996 uploaded on 2019-04-12
https://www.ted.com/talks/seth_berkley_the_troubling_reason_why_vaccines_are_

In [104]:
get_date(2000, 3000)

https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
Talk 2000 recorded on 2015-03-17
Talk 2000 uploaded on 2015-08-20
https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
Talk 2001 recorded on 2015-06-16
Talk 2001 uploaded on 2015-08-19
https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
Talk 2002 recorded on 2015-03-13
Talk 2002 uploaded on 2015-08-18
https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
Talk 2003 recorded on 2015-06-12
Talk 2003 uploaded on 2015-08-17
https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
Talk 2004 recorded on 2015-03-18
Talk 2004 uploaded on 2015-08-14
https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
Talk 2005 recorded on 2015-03-17
Talk 2005 uploaded on 2015-08-13
https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
Talk 2006 r

https://www.ted.com/talks/chris_burkard_the_joy_of_surfing_in_ice_cold_water
Talk 2055 recorded on 2015-03-19
Talk 2055 uploaded on 2015-05-22
https://www.ted.com/talks/esther_perel_rethinking_infidelity_a_talk_for_anyone_who_has_ever_loved
Talk 2056 recorded on 2015-03-19
Talk 2056 uploaded on 2015-05-21
https://www.ted.com/talks/steven_wise_chimps_have_feelings_and_thoughts_they_should_also_have_rights
Talk 2057 recorded on 2015-03-25
Talk 2057 uploaded on 2015-05-20
https://www.ted.com/talks/dr_natascha_m_santos_debunking_the_myths_of_ocd
Talk 2058 recorded on 2015-05-19
Talk 2058 uploaded on 2019-04-05
https://www.ted.com/talks/cosmin_mihaiu_physical_therapy_is_boring_play_a_game_instead
Talk 2059 recorded on 2015-03-10
Talk 2059 uploaded on 2015-05-19
https://www.ted.com/talks/jessica_smith_mansa_musa_one_of_the_wealthiest_people_who_ever_lived
Talk 2060 recorded on 2015-05-18
Talk 2060 uploaded on 2019-04-12
https://www.ted.com/talks/martine_rothblatt_my_daughter_my_wife_our_robo

https://www.ted.com/talks/janet_iwasa_why_it_s_so_hard_to_cure_hiv_aids
Talk 2110 recorded on 2015-03-16
Talk 2110 uploaded on 2017-09-07
https://www.ted.com/talks/vincent_cochetel_i_was_held_hostage_for_317_days_here_s_what_i_thought_about
Talk 2111 recorded on 2014-12-11
Talk 2111 uploaded on 2015-03-16
https://www.ted.com/talks/armand_d_angour_the_real_story_behind_archimedes_eureka
Talk 2112 recorded on 2015-03-13
Talk 2112 uploaded on 2019-03-22
https://www.ted.com/talks/linda_hill_how_to_manage_for_collective_creativity
Talk 2113 recorded on 2014-09-12
Talk 2113 uploaded on 2015-03-13
https://www.ted.com/talks/shimpei_takahashi_play_this_word_game_to_come_up_with_original_ideas
Talk 2114 recorded on 2013-05-31
Talk 2114 uploaded on 2015-03-12
https://www.ted.com/talks/ismael_nazario_what_i_learned_as_a_kid_in_jail
Talk 2115 recorded on 2014-11-01
Talk 2115 uploaded on 2015-03-11
https://www.ted.com/talks/marc_kushner_why_the_buildings_of_the_future_will_be_shaped_by_you
Talk 2116

Talk 2165 recorded on 2014-11-10
Talk 2165 uploaded on 2015-01-06
https://www.ted.com/talks/shai_marcu_the_benefits_of_a_good_night_s_sleep
Talk 2166 recorded on 2015-01-05
Talk 2166 uploaded on 2017-09-07
https://www.ted.com/talks/asha_de_vos_why_you_should_care_about_whale_poo
Talk 2167 recorded on 2014-10-06
Talk 2167 uploaded on 2015-01-05
https://www.ted.com/talks/michael_rubinstein_see_invisible_motion_hear_silent_sounds
Talk 2168 recorded on 2014-11-13
Talk 2168 uploaded on 2014-12-23
https://www.ted.com/talks/anthony_hazard_the_atlantic_slave_trade_what_too_few_textbooks_told_you
Talk 2169 recorded on 2014-12-22
Talk 2169 uploaded on 2019-04-12
https://www.ted.com/talks/stefan_gross_selbeck_business_model_innovation_beating_yourself_at_your_own_game
Talk 2170 recorded on 2014-10-21
Talk 2170 uploaded on 2017-12-01
https://www.ted.com/talks/sarah_cairns_smith_finally_tech_solutions_to_economic_development
NO DATE
https://www.ted.com/talks/rochelle_king_the_complex_relationship_b

https://www.ted.com/talks/john_o_leary_the_importance_of_good_conversation_and_how_to_have_it
Talk 2219 recorded on 2014-11-18
Talk 2219 uploaded on 2018-07-30
https://www.ted.com/talks/john_bolton_the_power_of_imagination_lessons_from_shakespeare
Talk 2220 recorded on 2014-11-18
Talk 2220 uploaded on 2018-07-30
https://www.ted.com/talks/harry_baker_the_power_of_self_acceptance
Talk 2221 recorded on 2014-11-18
Talk 2221 uploaded on 2018-07-30
https://www.ted.com/talks/christina_balch_selfies_and_seeing_ourselves_one_artist_s_look_in_the_mirror
Talk 2222 recorded on 2014-11-18
Talk 2222 uploaded on 2018-07-30
https://www.ted.com/talks/ashwini_mrinal_bhagat_what_i_learned_from_32_grueling_interviews
Talk 2223 recorded on 2014-11-18
Talk 2223 uploaded on 2018-07-30
https://www.ted.com/talks/alexander_mclean_restoring_hope_and_dignity_to_the_justice_system
Talk 2224 recorded on 2014-11-18
Talk 2224 uploaded on 2018-07-30
https://www.ted.com/talks/abbey_williams_my_ghost_town_road_trip
Talk

...

Talk 2735 recorded on 2013-02-25
Talk 2735 uploaded on 2013-04-04
https://www.ted.com/talks/lawrence_lessig_we_the_people_and_the_republic_we_must_reclaim
Talk 2736 recorded on 2013-02-27
Talk 2736 uploaded on 2013-04-03
https://www.ted.com/talks/sanjay_dastoor_a_skateboard_with_a_boost
Talk 2737 recorded on 2013-02-27
Talk 2737 uploaded on 2013-04-02
https://www.ted.com/talks/kees_moeliker_how_a_dead_duck_changed_my_life
Talk 2738 recorded on 2013-02-28
Talk 2738 uploaded on 2013-04-01
https://www.ted.com/talks/katherine_kuchenbecker_the_technology_of_touch
Talk 2739 recorded on 2012-11-17
Talk 2739 uploaded on 2013-03-29
https://www.ted.com/talks/colin_camerer_when_you_re_making_a_deal_what_s_going_on_in_your_brain
Talk 2740 recorded on 2013-01-18
Talk 2740 uploaded on 2013-03-28
https://www.ted.com/talks/wendy_hawkins_let_them_be_scientists
Talk 2741 recorded on 2013-03-27
Talk 2741 uploaded on 2018-07-30
https://www.ted.com/talks/tony_salvador_the_listening_bias
Talk 2742 recorded 

https://www.ted.com/talks/james_b_glattfelder_who_controls_the_world
Talk 2794 recorded on 2012-10-25
Talk 2794 uploaded on 2013-02-13
https://www.ted.com/talks/erik_schlangen_a_self_healing_asphalt
Talk 2795 recorded on 2012-10-05
Talk 2795 uploaded on 2013-02-12
https://www.ted.com/talks/shabana_basij_rasikh_dare_to_educate_afghan_girls
Talk 2796 recorded on 2012-12-01
Talk 2796 uploaded on 2013-02-11
https://www.ted.com/talks/edi_rama_take_back_your_city_with_paint
Talk 2797 recorded on 2012-05-12
Talk 2797 uploaded on 2013-02-08
https://www.ted.com/talks/lee_cronin_print_your_own_medicine
Talk 2798 recorded on 2012-06-26
Talk 2798 uploaded on 2013-02-07
https://www.ted.com/talks/cesar_kuriyama_one_second_every_day
Talk 2799 recorded on 2012-03-02
Talk 2799 uploaded on 2013-02-06
https://www.ted.com/talks/tyler_dewitt_hey_science_teachers_make_it_fun
Talk 2800 recorded on 2012-11-17
Talk 2800 uploaded on 2013-02-05
https://www.ted.com/talks/zahra_langhi_why_libya_s_revolution_didn_t

Talk 2853 recorded on 2012-06-07
Talk 2853 uploaded on 2012-11-21
https://www.ted.com/talks/amos_winter_the_cheap_all_terrain_wheelchair
Talk 2854 recorded on 2012-06-22
Talk 2854 uploaded on 2012-11-20
https://www.ted.com/talks/daphne_bavelier_your_brain_on_video_games
Talk 2855 recorded on 2012-06-22
Talk 2855 uploaded on 2012-11-19
https://www.ted.com/talks/david_binder_the_arts_festival_revolution
NO DATE
https://www.ted.com/talks/leah_buechley_how_to_sketch_with_electronics
Talk 2857 recorded on 2011-11-19
Talk 2857 uploaded on 2012-11-15
https://www.ted.com/talks/hannah_brencher_love_letters_to_strangers
Talk 2858 recorded on 2012-06-07
Talk 2858 uploaded on 2012-11-14
https://www.ted.com/talks/arunachalam_muruganantham_how_i_started_a_sanitary_napkin_revolution
NO DATE
https://www.ted.com/talks/julie_burstein_4_lessons_in_creativity
Talk 2860 recorded on 2012-02-29
Talk 2860 uploaded on 2012-11-12
https://www.ted.com/talks/jeff_hancock_the_future_of_lying
Talk 2861 recorded on 2

https://www.ted.com/talks/wayne_mcgregor_a_choreographer_s_creative_process_in_real_time
Talk 2912 recorded on 2012-06-27
Talk 2912 uploaded on 2012-09-14
https://www.ted.com/talks/susan_solomon_the_promise_of_research_with_stem_cells
Talk 2913 recorded on 2012-06-28
Talk 2913 uploaded on 2012-09-13
https://www.ted.com/talks/leslie_t_chang_the_voices_of_china_s_workers
Talk 2914 recorded on 2012-06-26
Talk 2914 uploaded on 2012-09-12
https://www.ted.com/talks/vikram_patel_mental_health_for_all_by_involving_all
Talk 2915 recorded on 2012-06-27
Talk 2915 uploaded on 2012-09-11
https://www.ted.com/talks/scott_fraser_why_eyewitnesses_get_it_wrong
Talk 2916 recorded on 2012-05-04
Talk 2916 uploaded on 2012-09-10
https://www.ted.com/talks/kent_larson_brilliant_designs_to_fit_more_people_in_every_city
Talk 2917 recorded on 2012-06-22
Talk 2917 uploaded on 2012-09-09
https://www.ted.com/talks/jonathan_trent_energy_from_floating_algae_pods
Talk 2918 recorded on 2012-06-27
Talk 2918 uploaded on 

https://www.ted.com/talks/massimo_banzi_how_arduino_is_open_sourcing_imagination
Talk 2970 recorded on 2012-06-26
Talk 2970 uploaded on 2012-06-27
https://www.ted.com/talks/rives_reinventing_the_encyclopedia_game
Talk 2971 recorded on 2012-04-19
Talk 2971 uploaded on 2012-06-26
https://www.ted.com/talks/e_o_wilson_advice_to_a_young_scientist
Talk 2972 recorded on 2012-04-19
Talk 2972 uploaded on 2012-06-25
https://www.ted.com/talks/nirmalya_kumar_india_s_invisible_innovation
Talk 2973 recorded on 2012-04-27
Talk 2973 uploaded on 2012-06-24
https://www.ted.com/talks/jon_nguyen_tour_the_solar_system_from_home
Talk 2974 recorded on 2011-12-03
Talk 2974 uploaded on 2012-06-23
https://www.ted.com/talks/george_zaidan_how_do_pain_relievers_work
Talk 2975 recorded on 2012-06-22
Talk 2975 uploaded on 2018-11-26
https://www.ted.com/talks/wolfgang_kessling_how_to_air_condition_outdoor_spaces
Talk 2976 recorded on 2012-04-17
Talk 2976 uploaded on 2012-06-22
https://www.ted.com/talks/peter_norvig_t

In [105]:
get_date(3000, 4000)

https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
Talk 3000 recorded on 2012-02-04
Talk 3000 uploaded on 2012-05-27
https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
Talk 3001 recorded on 2012-03-12
Talk 3001 uploaded on 2012-05-26
https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
Talk 3002 recorded on 2012-02-29
Talk 3002 uploaded on 2012-05-25
https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
Talk 3003 recorded on 2012-04-18
Talk 3003 uploaded on 2012-05-24
https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
Talk 3004 recorded on 2012-03-01
Talk 3004 uploaded on 2012-05-23
https://www.ted.com/talks/hans_rosling_religions_and_babies
Talk 3005 recorded on 2012-04-16
Talk 3005 uploaded on 2012-05-22
https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
Talk 3006 recorded on 2012-02-27
Talk 3006 uploaded on 2012-05-21
https://www.ted.com/talks/melissa_garren_the_sea_w

https://www.ted.com/talks/jer_thorp_make_data_more_human
Talk 3061 recorded on 2011-11-12
Talk 3061 uploaded on 2012-03-24
https://www.ted.com/talks/billy_collins_everyday_moments_caught_in_time
Talk 3062 recorded on 2012-02-28
Talk 3062 uploaded on 2012-03-23
https://www.ted.com/talks/taylor_wilson_yup_i_built_a_nuclear_fusion_reactor
Talk 3063 recorded on 2012-03-01
Talk 3063 uploaded on 2012-03-22
https://www.ted.com/talks/noel_bairey_merz_the_single_biggest_health_threat_women_face
Talk 3064 recorded on 2011-12-01
Talk 3064 uploaded on 2012-03-21
https://www.ted.com/talks/t_boone_pickens_let_s_transform_energy_with_natural_gas
Talk 3065 recorded on 2012-03-01
Talk 3065 uploaded on 2012-03-19
https://www.ted.com/talks/mark_raymond_victims_of_the_city
Talk 3066 recorded on 2011-11-11
Talk 3066 uploaded on 2012-03-18
https://www.ted.com/talks/scott_summit_beautiful_artificial_limbs
Talk 3067 recorded on 2011-11-19
Talk 3067 uploaded on 2012-03-17
https://www.ted.com/talks/brene_brown_

Talk 3121 recorded on 2011-12-02
Talk 3121 uploaded on 2012-01-27
https://www.ted.com/talks/brian_goldman_doctors_make_mistakes_can_we_talk_about_that
Talk 3122 recorded on 2011-11-11
Talk 3122 uploaded on 2012-01-25
https://www.ted.com/talks/diana_nyad_extreme_swimming_with_the_world_s_most_dangerous_jellyfish
Talk 3123 recorded on 2011-10-28
Talk 3123 uploaded on 2012-01-24
https://www.ted.com/talks/lisa_harouni_a_primer_on_3d_printing
Talk 3124 recorded on 2011-11-10
Talk 3124 uploaded on 2012-01-23
https://www.ted.com/talks/julian_baggini_is_there_a_real_you
Talk 3125 recorded on 2011-11-03
Talk 3125 uploaded on 2012-01-22
https://www.ted.com/talks/bilal_bomani_plant_fuels_that_could_power_a_jet
NO DATE
https://www.ted.com/talks/scott_rickard_the_beautiful_math_behind_the_world_s_ugliest_music
Talk 3127 recorded on 2011-09-13
Talk 3127 uploaded on 2012-01-20
https://www.ted.com/talks/sheena_iyengar_how_to_make_choosing_easier
Talk 3128 recorded on 2011-11-09
Talk 3128 uploaded on 2

https://www.ted.com/talks/alexander_tsiaras_conception_to_birth_visualized
Talk 3183 recorded on 2010-12-02
Talk 3183 uploaded on 2011-11-14
https://www.ted.com/talks/michael_nielsen_open_science_now
Talk 3184 recorded on 2011-03-03
Talk 3184 uploaded on 2011-11-13
https://www.ted.com/talks/roger_mcnamee_6_ways_to_save_the_internet
Talk 3185 recorded on 2011-03-02
Talk 3185 uploaded on 2011-11-12
https://www.ted.com/talks/charlie_todd_the_shared_experience_of_absurdity
Talk 3186 recorded on 2011-05-11
Talk 3186 uploaded on 2011-11-11
https://www.ted.com/talks/allan_jones_a_map_of_the_brain
Talk 3187 recorded on 2011-07-13
Talk 3187 uploaded on 2011-11-10
https://www.ted.com/talks/ben_kacyra_ancient_wonders_captured_in_3d
Talk 3188 recorded on 2011-07-14
Talk 3188 uploaded on 2011-11-09
https://www.ted.com/talks/aparna_rao_high_tech_art_with_a_sense_of_humor
Talk 3189 recorded on 2011-07-14
Talk 3189 uploaded on 2011-11-08
https://www.ted.com/talks/martin_hanczyc_the_line_between_life_a

https://www.ted.com/talks/marco_tempest_the_magic_of_truth_and_lies_and_ipods
Talk 3244 recorded on 2011-07-14
Talk 3244 uploaded on 2011-08-12
https://www.ted.com/talks/lucianne_walkowicz_finding_planets_around_other_stars
Talk 3245 recorded on 2011-07-11
Talk 3245 uploaded on 2011-08-11
https://www.ted.com/talks/jeremy_gilley_one_day_of_peace
Talk 3246 recorded on 2011-07-15
Talk 3246 uploaded on 2011-08-10
https://www.ted.com/talks/dyan_denapoli_the_great_penguin_rescue
Talk 3247 recorded on 2011-06-28
Talk 3247 uploaded on 2011-08-09
https://www.ted.com/talks/alex_steffen_the_shareable_future_of_cities
Talk 3248 recorded on 2011-07-12
Talk 3248 uploaded on 2011-08-08
https://www.ted.com/talks/eve_ensler_suddenly_my_body
Talk 3249 recorded on 2010-12-04
Talk 3249 uploaded on 2011-08-05
https://www.ted.com/talks/philip_zimbardo_the_demise_of_guys
Talk 3250 recorded on 2011-03-01
Talk 3250 uploaded on 2011-08-05
https://www.ted.com/talks/jessica_green_are_we_filtering_the_wrong_microb

...

https://www.ted.com/talks/william_kamkwamba_how_i_harnessed_the_wind
Talk 3724 recorded on 2009-07-23
Talk 3724 uploaded on 2009-09-23
https://www.ted.com/talks/evgeny_morozov_how_the_net_aids_dictatorships
Talk 3725 recorded on 2009-07-23
Talk 3725 uploaded on 2009-09-22
https://www.ted.com/talks/jonathan_zittrain_the_web_as_random_acts_of_kindness
Talk 3726 recorded on 2009-07-22
Talk 3726 uploaded on 2009-09-21
https://www.ted.com/talks/imogen_heap_wait_it_out
Talk 3727 recorded on 2009-07-22
Talk 3727 uploaded on 2009-09-18
https://www.ted.com/talks/oliver_sacks_what_hallucination_reveals_about_our_minds
Talk 3728 recorded on 2009-02-05
Talk 3728 uploaded on 2009-09-17
https://www.ted.com/talks/john_lloyd_an_inventory_of_the_invisible
Talk 3729 recorded on 2009-07-23
Talk 3729 uploaded on 2009-09-16
https://www.ted.com/talks/bjarke_ingels_3_warp_speed_architecture_tales
Talk 3730 recorded on 2009-07-23
Talk 3730 uploaded on 2009-09-15
https://www.ted.com/talks/misha_glenny_the_real

Talk 3783 recorded on 2009-02-07
Talk 3783 uploaded on 2009-06-02
https://www.ted.com/talks/liz_coleman_a_call_to_reinvent_liberal_arts_education
Talk 3784 recorded on 2009-02-07
Talk 3784 uploaded on 2009-06-01
https://www.ted.com/talks/kaki_king_playing_pink_noise_on_guitar
Talk 3785 recorded on 2008-03-01
Talk 3785 uploaded on 2009-05-29
https://www.ted.com/talks/jonathan_drori_why_we_re_storing_billions_of_seeds
Talk 3786 recorded on 2009-02-06
Talk 3786 uploaded on 2009-05-28
https://www.ted.com/talks/jay_walker_the_world_s_english_mania
Talk 3787 recorded on 2009-02-05
Talk 3787 uploaded on 2009-05-27
https://www.ted.com/talks/michelle_obama_a_passionate_personal_case_for_education
Talk 3788 recorded on 2009-04-04
Talk 3788 uploaded on 2009-05-27
https://www.ted.com/talks/joachim_de_posada_don_t_eat_the_marshmallow
Talk 3789 recorded on 2009-02-05
Talk 3789 uploaded on 2009-05-25
https://www.ted.com/talks/yves_behar_a_supercharged_motorcycle_design
Talk 3790 recorded on 2009-02-0

https://www.ted.com/talks/nalini_nadkarni_conserving_the_canopy
Talk 3847 recorded on 2009-02-06
Talk 3847 uploaded on 2009-03-04
https://www.ted.com/talks/willie_smits_how_to_restore_a_rainforest
Talk 3848 recorded on 2009-02-06
Talk 3848 uploaded on 2009-03-03
https://www.ted.com/talks/brenda_laurel_why_not_make_video_games_for_girls
Talk 3849 recorded on 1998-02-02
Talk 3849 uploaded on 2009-03-02
https://www.ted.com/talks/evan_williams_the_voices_of_twitter_users
Talk 3850 recorded on 2009-02-05
Talk 3850 uploaded on 2009-02-27
https://www.ted.com/talks/miru_kim_my_underground_art_explorations
Talk 3851 recorded on 2008-12-12
Talk 3851 uploaded on 2009-02-26
https://www.ted.com/talks/richard_pyle_a_dive_into_the_reef_s_twilight_zone
Talk 3852 recorded on 2004-02-02
Talk 3852 uploaded on 2009-02-25
https://www.ted.com/talks/charles_moore_seas_of_plastic
Talk 3853 recorded on 2009-02-05
Talk 3853 uploaded on 2009-02-24
https://www.ted.com/talks/ed_ulbrich_how_benjamin_button_got_his_

Talk 3908 recorded on 2008-05-07
Talk 3908 uploaded on 2008-11-12
https://www.ted.com/talks/samantha_power_a_complicated_hero_in_the_war_on_dictatorship
Talk 3909 recorded on 2008-02-02
Talk 3909 uploaded on 2008-11-11
https://www.ted.com/talks/lee_smolin_science_and_democracy
Talk 3910 recorded on 2003-02-02
Talk 3910 uploaded on 2008-11-10
https://www.ted.com/talks/luca_turin_the_science_of_scent
Talk 3911 recorded on 2005-02-02
Talk 3911 uploaded on 2008-11-07
https://www.ted.com/talks/tim_brown_tales_of_creativity_and_play
Talk 3912 recorded on 2008-05-07
Talk 3912 uploaded on 2008-11-06
https://www.ted.com/talks/john_francis_walk_the_earth_my_17_year_vow_of_silence
Talk 3913 recorded on 2008-02-02
Talk 3913 uploaded on 2008-11-05
https://www.ted.com/talks/james_surowiecki_the_power_and_the_danger_of_online_crowds
Talk 3914 recorded on 2005-02-02
Talk 3914 uploaded on 2008-11-04
https://www.ted.com/talks/graham_hawkes_a_flight_through_the_ocean
Talk 3915 recorded on 2005-02-28
Talk

Talk 3970 recorded on 2007-06-06
Talk 3970 uploaded on 2008-08-18
https://www.ted.com/talks/patricia_burchat_shedding_light_on_dark_matter
Talk 3971 recorded on 2008-02-02
Talk 3971 uploaded on 2008-08-17
https://www.ted.com/talks/bruno_bowden_rufus_cappadocia_watch_me_fold_origami_blindfolded
Talk 3972 recorded on 2008-02-02
Talk 3972 uploaded on 2008-08-01
https://www.ted.com/talks/robert_lang_the_math_and_magic_of_origami
Talk 3973 recorded on 2008-02-02
Talk 3973 uploaded on 2008-07-30
https://www.ted.com/talks/kwabena_boahen_a_computer_that_works_like_the_brain
NO DATE
https://www.ted.com/talks/kevin_kelly_the_next_5_000_days_of_the_web
Talk 3975 recorded on 2007-12-12
Talk 3975 uploaded on 2008-07-28
https://www.ted.com/talks/reed_kroloff_a_tour_of_modern_architecture
Talk 3976 recorded on 2003-02-02
Talk 3976 uploaded on 2008-07-28
https://www.ted.com/talks/jonathan_harris_the_web_as_art
Talk 3977 recorded on 2007-12-12
Talk 3977 uploaded on 2008-07-24
https://www.ted.com/talks/

In [106]:
get_date(4000, len(talk_df))

https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
Talk 4000 recorded on 2007-02-02
Talk 4000 uploaded on 2008-06-23
https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
Talk 4001 recorded on 2002-02-02
Talk 4001 uploaded on 2008-06-19
https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
Talk 4002 recorded on 2008-03-03
Talk 4002 uploaded on 2008-06-16
https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
Talk 4003 recorded on 2008-02-02
Talk 4003 uploaded on 2008-06-15
https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
Talk 4004 recorded on 2003-03-03
Talk 4004 uploaded on 2008-06-15
https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
Talk 4005 recorded on 2007-03-03
Talk 4005 uploaded on 2008-06-11
https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
Talk 4006 recorded on 2008-02-02
Talk 4006 uploaded on 2008-06-10
https://www.ted.com/talks/rokia_traore_kounand

https://www.ted.com/talks/alison_jackson_an_unusual_glimpse_at_celebrity
Talk 4061 recorded on 2005-07-07
Talk 4061 uploaded on 2008-01-28
https://www.ted.com/talks/david_pogue_the_music_wars
Talk 4062 recorded on 2007-03-03
Talk 4062 uploaded on 2008-01-24
https://www.ted.com/talks/bernie_dunlap_the_life_long_learner
Talk 4063 recorded on 2007-03-03
Talk 4063 uploaded on 2008-01-23
https://www.ted.com/talks/bill_strickland_rebuilding_a_neighborhood_with_beauty_dignity_hope
Talk 4064 recorded on 2002-02-02
Talk 4064 uploaded on 2008-01-20
https://www.ted.com/talks/raul_midon_tembererana
Talk 4065 recorded on 2007-03-03
Talk 4065 uploaded on 2008-01-18
https://www.ted.com/talks/frank_gehry_a_master_architect_asks_now_what
Talk 4066 recorded on 2002-02-28
Talk 4066 uploaded on 2008-01-17
https://www.ted.com/talks/paola_antonelli_treat_design_as_art
Talk 4067 recorded on 2007-03-03
Talk 4067 uploaded on 2008-01-15
https://www.ted.com/talks/david_gallo_underwater_astonishments
Talk 4068 re

https://www.ted.com/talks/emily_oster_flip_your_thinking_on_aids_in_africa
Talk 4122 recorded on 2007-03-03
Talk 4122 uploaded on 2007-07-12
https://www.ted.com/talks/jonathan_harris_the_web_s_secret_stories
Talk 4123 recorded on 2007-03-03
Talk 4123 uploaded on 2007-07-08
https://www.ted.com/talks/alan_russell_the_potential_of_regenerative_medicine
Talk 4124 recorded on 2006-02-02
Talk 4124 uploaded on 2007-07-04
https://www.ted.com/talks/dan_dennett_dangerous_memes
Talk 4125 recorded on 2002-02-02
Talk 4125 uploaded on 2007-07-02
https://www.ted.com/talks/bill_stone_inside_the_world_s_deepest_caves
Talk 4126 recorded on 2007-03-03
Talk 4126 uploaded on 2007-06-27
https://www.ted.com/talks/hans_rosling_new_insights_on_poverty
Talk 4127 recorded on 2007-03-03
Talk 4127 uploaded on 2007-06-25
https://www.ted.com/talks/stephen_lawler_tour_microsoft_s_virtual_earth
Talk 4128 recorded on 2007-03-03
Talk 4128 uploaded on 2007-06-20
https://www.ted.com/talks/ethel_a_string_quartet_plays_blue

NO DATE
https://www.ted.com/talks/robert_wright_progress_is_not_a_zero_sum_game
NO DATE
https://www.ted.com/talks/martin_rees_is_this_our_final_century
Talk 4190 recorded on 2005-07-14
Talk 4190 uploaded on 2007-01-17
https://www.ted.com/talks/phil_borges_photos_of_endangered_cultures
NO DATE
https://www.ted.com/talks/wade_davis_dreams_from_endangered_cultures
NO DATE
https://www.ted.com/talks/bjorn_lomborg_global_priorities_bigger_than_climate_change
NO DATE
https://www.ted.com/talks/robert_neuwirth_the_hidden_world_of_shadow_cities
NO DATE
https://www.ted.com/talks/rives_if_i_controlled_the_internet
NO DATE
https://www.ted.com/talks/dean_ornish_the_killer_american_diet_that_s_sweeping_the_planet
Talk 4196 recorded on 2006-02-23
Talk 4196 uploaded on 2006-12-14
https://www.ted.com/talks/richard_st_john_8_secrets_of_success
Talk 4197 recorded on 2005-02-23
Talk 4197 uploaded on 2006-12-14
https://www.ted.com/talks/peter_gabriel_fight_injustice_with_raw_video
NO DATE
https://www.ted.com

In [115]:
talk_df.to_csv('talks_with_dates.csv')

# Get biographical info about speakers

In [116]:
talk_df['occupation'] = '--'
talk_df['bio'] = '--'

def get_bio(start, end):
    for index in range(start, end):
        try:
            # Get the url, request HTML
            url = talk_df['url'][index]
            link = "https://www.ted.com" + url
            print(link)
            page = requests.get(link)
            content = BS(page.content, 'html.parser')
            
            # Find all scripts, look for initial data about the speakers
            for x in content.find_all('script'):
                if '__INITIAL_DATA__' in x.get_text():
                    text = x.get_text()
                    matched = re.search('"speakers":\[\{.*?\}\]', text)
                    all_info = matched.group(0).strip('"speakers":[').strip(']')
                    all_info_json = json.loads(all_info)
                    
                    # Get the description (titles) of the speakers, short bios
                    if 'description' in all_info_json.keys():
                        talk_df['occupation'][index] = all_info_json['description']
                    if 'whotheyare' in all_info_json.keys():
                        talk_df['bio'][index] = all_info_json['whotheyare']
                        
            time.sleep(2)
        except:
            print('NO INFO')

In [121]:
get_bio(0,1000)

https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator
https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol
https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women
https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear
https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success
https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time
https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism
https://www.ted.com/talks/nick_bostrom_how_civilization_could_destroy_itself_and_4_ways_we_could_prevent_it
NO INFO
https://www.ted.com/talks/jasmine_crowe_what_we_re_getting_wrong_in_the_fight_to_end_hunger
https://www.ted.com/talks/sinan_aral_how_we_can_protect_truth_in_the_age_of_misinformation
https://www.ted.com/talks/dan_kwartler_why_should_you_read_dune_by_frank_herbert
https://www.ted.com/talks/enrico_ramirez_ruiz_

https://www.ted.com/talks/alyssa_loorya_what_i_learned_from_digging_under_new_york_city_s_streets
https://www.ted.com/talks/ella_al_shamahi_why_archaeology_needs_to_transcend_borders
https://www.ted.com/talks/rose_m_mutiso_how_to_bring_affordable_sustainable_electricity_to_africa
https://www.ted.com/talks/andrew_nemr_the_sounds_and_sights_of_tap_dance
https://www.ted.com/talks/hiromi_ozaki_how_i_bring_myth_and_magic_to_life
https://www.ted.com/talks/alexis_gambis_why_we_need_more_real_science_in_fiction
https://www.ted.com/talks/beau_lotto_and_cirque_du_soleil_how_we_experience_awe_and_why_it_matters
NO INFO
https://www.ted.com/talks/abigail_spanberger_how_to_connect_with_people_who_are_different_than_you
https://www.ted.com/talks/claire_wardle_how_you_can_help_transform_the_internet_into_a_place_of_trust
https://www.ted.com/talks/adrienne_mayor_the_greek_myth_of_talos_the_first_robot
https://www.ted.com/talks/ashwin_naidu_the_link_between_fishing_cats_and_mangrove_forest_conservation


https://www.ted.com/talks/andrew_marantz_inside_the_bizarre_world_of_internet_trolls_and_propagandists
https://www.ted.com/talks/anthony_veneziale_stumbling_towards_intimacy_an_improvised_ted_talk
https://www.ted.com/talks/emily_f_rothman_how_porn_changes_the_way_teens_think_about_sex
https://www.ted.com/talks/iseult_gillespie_the_secret_student_resistance_to_hitler
https://www.ted.com/talks/jochen_wegner_what_happened_when_we_paired_up_thousands_of_strangers_to_talk_politics
https://www.ted.com/talks/haaziq_kazi_cleaning_our_oceans_a_big_plan_for_a_big_problem
https://www.ted.com/talks/asmeret_asefaw_berhe_a_climate_change_solution_that_s_right_under_our_feet
https://www.ted.com/talks/yeonmi_park_what_i_learned_about_freedom_after_escaping_north_korea
https://www.ted.com/talks/lee_thomas_how_i_help_people_understand_vitiligo
https://www.ted.com/talks/anne_f_broadbridge_the_rise_and_fall_of_the_mongol_empire
https://www.ted.com/talks/dessa_can_we_choose_to_fall_out_of_love_nov_2019
htt

https://www.ted.com/talks/suleika_jaouad_what_almost_dying_taught_me_about_living
https://www.ted.com/talks/graham_baird_how_do_crystals_work
https://www.ted.com/talks/rob_reid_how_synthetic_biology_could_wipe_out_humanity_and_how_we_can_stop_it
https://www.ted.com/talks/ayana_elizabeth_johnson_and_megan_davis_could_underwater_farms_help_fight_climate_change
https://www.ted.com/talks/kaitlyn_sadtler_your_body_vs_implants
https://www.ted.com/talks/david_baker_5_challenges_we_could_solve_by_designing_new_proteins
https://www.ted.com/talks/steve_simpson_how_ocean_noise_destroys_marine_ecosystems
https://www.ted.com/talks/michael_tubbs_the_political_power_of_being_a_good_neighbor
https://www.ted.com/talks/emeli_sande_you_are_not_alone_extraordinary_being_shine
https://www.ted.com/talks/daniel_lismore_my_life_as_a_work_of_art
https://www.ted.com/talks/lucy_clayton_all_the_rage_why_dressing_up_helps_us_speak_out
https://www.ted.com/talks/truls_nord_tactile_photographs_that_display_worlds_of_

https://www.ted.com/talks/guy_harvey_art_and_science_working_together_for_the_conservation_of_large_pelagic_fish
https://www.ted.com/talks/es_devlin_mind_blowing_stage_sculptures_that_fuse_music_and_technology
https://www.ted.com/talks/joanne_chory_how_supercharged_plants_could_slow_climate_change
https://www.ted.com/talks/iseult_gillespie_why_is_this_painting_so_shocking
https://www.ted.com/talks/michele_wucker_why_we_ignore_obvious_problems_and_how_to_act_on_them
https://www.ted.com/talks/yana_buhrer_tavanier_how_to_recover_from_activism_burnout
https://www.ted.com/talks/hannah_gadsby_three_ideas_three_contradictions_or_not
https://www.ted.com/talks/oliver_elfenbaum_how_does_the_stock_market_work
https://www.ted.com/talks/elizabeth_dunn_helping_others_makes_us_happier_but_it_matters_how_we_do_it
https://www.ted.com/talks/daniel_finkel_can_you_solve_the_cuddly_duddly_fuddly_wuddly_riddle
https://www.ted.com/talks/kim_gorgens_the_surprising_connection_between_brain_injuries_and_crime
h

...

NO INFO
https://www.ted.com/talks/gunjan_bhardwaj_how_blockchain_and_ai_can_help_us_decipher_medicine_s_big_data
https://www.ted.com/talks/george_blair_west_3_ways_to_build_a_happy_marriage_and_avoid_divorce
https://www.ted.com/talks/helen_m_farrell_the_truth_about_electroconvulsive_therapy_ect
https://www.ted.com/talks/tony_luciani_a_mother_and_son_s_photographic_journey_through_dementia
https://www.ted.com/talks/matt_mullenweg_why_working_from_home_is_good_for_business
https://www.ted.com/talks/maryn_mckenna_how_antibiotics_changed_our_food_and_how_we_ll_change_it_back
https://www.ted.com/talks/shohini_ghose_a_beginner_s_guide_to_quantum_computing
https://www.ted.com/talks/alex_gendler_can_you_solve_the_multiplying_rabbits_riddle
https://www.ted.com/talks/karissa_sanbonmatsu_the_biology_of_gender_from_dna_to_the_brain
https://www.ted.com/talks/danae_wolfe_ew_to_awe_your_view_of_bugs_may_never_be_the_same
https://www.ted.com/talks/renzo_vitale_what_should_electric_cars_sound_like
http

https://www.ted.com/talks/alex_edmans_what_to_trust_in_a_post_truth_world
https://www.ted.com/talks/jason_jay_how_to_benefit_from_uncomfortable_conversations
https://www.ted.com/talks/tamas_kocsis_the_case_for_a_decentralized_internet
https://www.ted.com/talks/dan_finkel_can_you_solve_the_secret_werewolf_riddle
https://www.ted.com/talks/michael_green_the_global_goals_we_ve_made_progress_on_and_the_ones_we_haven_t
https://www.ted.com/talks/mara_mintzer_how_kids_can_help_design_cities
https://www.ted.com/talks/franklin_leonard_how_i_accidentally_changed_the_way_movies_get_made_nov_2018
https://www.ted.com/talks/rene_laufer_how_far_would_you_have_to_go_to_escape_gravity
https://www.ted.com/talks/ozlem_cekic_why_i_have_coffee_with_people_who_send_me_hate_mail
https://www.ted.com/talks/monica_palacio_why_the_district_of_columbia_should_get_an_equal_right_to_vote
https://www.ted.com/talks/elizabeth_cox_is_fire_a_solid_a_liquid_or_a_gas
https://www.ted.com/talks/suzie_sheehy_the_case_for_curi

https://www.ted.com/talks/simona_francese_your_fingerprints_reveal_more_than_you_think
https://www.ted.com/talks/the_soul_rebels_rebelosis_rebel_rock_rebel_on_that_level
https://www.ted.com/talks/tom_griffiths_3_ways_to_make_better_decisions_by_thinking_like_a_computer
https://www.ted.com/talks/luhan_yang_how_to_create_a_world_where_no_one_dies_waiting_for_a_transplant
https://www.ted.com/talks/krishna_sudhir_how_do_cigarettes_affect_the_body
https://www.ted.com/talks/ghada_wali_how_i_m_using_lego_to_teach_arabic
https://www.ted.com/talks/bob_woodward_what_i_learned_investigating_nixon
https://www.ted.com/talks/will_macaskill_what_are_the_most_important_moral_problems_of_our_time
https://www.ted.com/talks/elise_legrow_you_never_can_tell_over_the_mountain_across_the_sea
https://www.ted.com/talks/alan_lupack_is_there_any_truth_to_the_king_arthur_legends
https://www.ted.com/talks/john_bidden_not_about_me
https://www.ted.com/talks/ruchi_shah_how_small_business_can_benefit_from_village_bank

https://www.ted.com/talks/elizabeth_cox_a_day_in_the_life_of_an_ancient_egyptian_doctor
https://www.ted.com/talks/stephen_webb_where_are_all_the_aliens
https://www.ted.com/talks/pablo_santa_cruz_el_impacto_del_marketing_en_los_medicamentos_que_tomamos
https://www.ted.com/talks/karla_souza_fracaso_tras_fracaso
https://www.ted.com/talks/lucas_raspall_reescribe_el_cuento_de_tu_vida
https://www.ted.com/talks/daniel_molina_la_verdad_es_mentira
https://www.ted.com/talks/juan_lopez_aranguren_la_importancia_de_la_improvisacion
https://www.ted.com/talks/gabriel_heras_humanizar_los_cuidados_intensivos
https://www.ted.com/talks/carmen_jimenez_borja_las_palabras_de_la_discriminacion
https://www.ted.com/talks/nestor_guerra_nuestras_mentiras_en_la_vida_y_en_los_negocios
https://www.ted.com/talks/alberto_garin_a_quien_pertenece_el_pasado
https://www.ted.com/talks/fernanda_castillo_la_voz_del_artista
https://www.ted.com/talks/mikhail_zygar_what_the_russian_revolution_would_have_looked_like_on_social_m

https://www.ted.com/talks/raj_panjabi_what_if_we_digitally_empowered_community_health_workers
https://www.ted.com/talks/hugh_herr_how_we_ll_become_cyborgs_and_extend_human_potential
https://www.ted.com/talks/anushka_naiknaware_a_teen_scientist_s_invention_to_help_wounds_heal
https://www.ted.com/talks/susan_emmett_this_simple_test_can_help_kids_hear_better
https://www.ted.com/talks/gene_luen_yang_comics_belong_in_the_classroom
https://www.ted.com/talks/jeremy_forbes_how_to_start_a_conversation_about_suicide
https://www.ted.com/talks/baroness_beeban_kidron_why_we_must_protect_the_digital_rights_of_children
https://www.ted.com/talks/madhumita_murgia_how_data_brokers_sell_your_identity
https://www.ted.com/talks/marta_peirano_the_surveillance_device_you_carry_around_all_day
https://www.ted.com/talks/stefaan_verhulst_using_corporate_data_to_improve_our_lives
https://www.ted.com/talks/gaspard_koenig_do_we_really_own_our_bodies
https://www.ted.com/talks/amy_edmondson_how_to_turn_a_group_of_str

In [122]:
get_bio(1000,2000)

https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
NO INFO
https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment
https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell
https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees
https://www.ted.com/talks/mother_london_ai_therapy
https://www.ted.com/talks/duncan_cowles_it_s_not_amazing_enough
https://www.ted.com/talks/aaron_duffy_lake_buckley_and_jack_foster_illusions_for_a_better_soc

https://www.ted.com/talks/rowan_fitzpatrick_and_heart_of_mind_facing_the_future_together
NO INFO
https://www.ted.com/talks/shade_zahrai_and_aric_yegudkin_the_rhythmic_beauty_of_collaboration
NO INFO
https://www.ted.com/talks/will_jenkins_the_most_precious_commodity_of_all
https://www.ted.com/talks/tomer_garzberg_what_happens_when_we_take_humans_out_of_work
https://www.ted.com/talks/rach_ranton_where_are_we_trying_to_end_up
https://www.ted.com/talks/olivia_tyler_the_complex_path_to_sustainability
https://www.ted.com/talks/michael_mieni_using_aboriginal_traditions_to_teach_tech
https://www.ted.com/talks/linda_zhang_why_schools_should_teach_entrepreneurship
https://www.ted.com/talks/hayley_yeates_it_s_past_time_to_appreciate_cultural_diversity
https://www.ted.com/talks/alastair_o_neill_when_genetic_choice_becomes_personal
https://www.ted.com/talks/wale_oyejide_fashion_that_celebrates_african_strength_and_spirit
https://www.ted.com/talks/dixon_chibanda_why_i_train_grandmothers_to_treat_dep

https://www.ted.com/talks/scott_williams_the_hidden_role_informal_caregivers_play_in_health_care
https://www.ted.com/talks/michelle_knox_talk_about_your_death_while_you_re_still_healthy
https://www.ted.com/talks/matt_goldman_the_search_for_aha_moments
https://www.ted.com/talks/vivek_maru_how_to_put_the_power_of_law_in_people_s_hands
https://www.ted.com/talks/jacob_collier_a_one_man_musical_phenomenon
https://www.ted.com/talks/marily_oppezzo_want_to_be_more_creative_go_for_a_walk
https://www.ted.com/talks/alexis_charpentier_how_record_collectors_find_lost_music_and_preserve_our_cultural_heritage
https://www.ted.com/talks/kendall_spencer_what_can_we_learn_from_college_athletes
https://www.ted.com/talks/kevin_njabo_how_we_can_stop_africa_s_scientific_brain_drain
https://www.ted.com/talks/soyapi_mumba_medical_tech_designed_to_meet_africa_s_needs
https://www.ted.com/talks/mindy_scheier_how_adaptive_clothing_empowers_people_with_disabilities
https://www.ted.com/talks/tomas_elemans_the_inspir

https://www.ted.com/talks/gretchen_carlson_how_we_can_end_sexual_harassment_at_work
https://www.ted.com/talks/amel_karboul_the_global_learning_crisis_and_what_to_do_about_it
https://www.ted.com/talks/giulia_enders_the_surprisingly_charming_science_of_your_gut
https://www.ted.com/talks/nnedi_okorafor_sci_fi_stories_that_imagine_a_future_africa
https://www.ted.com/talks/lauren_sallan_how_to_win_at_evolution_and_survive_a_mass_extinction
https://www.ted.com/talks/tim_kruger_can_we_stop_climate_change_by_removing_co2_from_the_air
https://www.ted.com/talks/shonda_rhimes_and_cyndi_stivers_the_future_of_storytelling
NO INFO
https://www.ted.com/talks/zeynep_tufekci_we_re_building_a_dystopia_just_to_make_people_click_on_ads
https://www.ted.com/talks/carlos_bautista_the_awful_logic_of_land_mines_and_an_app_that_helps_people_avoid_them
https://www.ted.com/talks/martin_ford_how_we_ll_earn_money_in_a_future_without_jobs
https://www.ted.com/talks/rocio_lorenzo_how_diversity_makes_teams_more_innovati

https://www.ted.com/talks/ashton_applewhite_let_s_end_ageism
https://www.ted.com/talks/anjan_chatterjee_how_your_brain_decides_what_is_beautiful
https://www.ted.com/talks/tom_gruber_how_ai_can_enhance_our_memory_work_and_social_lives
https://www.ted.com/talks/joseph_redmon_how_computers_learn_to_recognize_objects_instantly
https://www.ted.com/talks/francoise_mouly_the_stories_behind_the_new_yorker_s_iconic_covers
https://www.ted.com/talks/ingrid_betancourt_what_six_years_in_captivity_taught_me_about_fear_and_faith
https://www.ted.com/talks/titus_kaphar_can_art_amend_history
https://www.ted.com/talks/marc_raibert_meet_spot_the_robot_dog_that_can_run_hop_and_open_doors
https://www.ted.com/talks/kristen_marhaver_why_i_still_have_hope_for_coral_reefs
https://www.ted.com/talks/jennifer_pluznick_you_smell_with_your_body_not_just_your_nose
https://www.ted.com/talks/tristan_harris_how_a_handful_of_tech_companies_control_billions_of_minds_every_day
https://www.ted.com/talks/jimmy_lin_a_simple_n

...

https://www.ted.com/talks/danny_dorling_maps_that_show_us_who_we_are_not_just_where_we_are
https://www.ted.com/talks/alicia_garza_patrisse_cullors_and_opal_tometi_an_interview_with_the_founders_of_black_lives_matter
NO INFO
https://www.ted.com/talks/joe_lassiter_we_need_nuclear_power_to_solve_climate_change
https://www.ted.com/talks/chand_john_what_s_the_fastest_way_to_alphabetize_your_bookshelf
https://www.ted.com/talks/adam_galinsky_how_to_speak_up_for_yourself
https://www.ted.com/talks/juan_enriquez_what_will_humans_look_like_in_100_years
https://www.ted.com/talks/sandi_toksvig_a_political_party_for_women_s_equality
https://www.ted.com/talks/roger_antonsen_math_is_the_hidden_secret_to_understanding_the_world
https://www.ted.com/talks/victor_rios_help_for_kids_the_education_system_ignores
https://www.ted.com/talks/steven_johnson_the_playful_wonderland_behind_great_inventions
https://www.ted.com/talks/bettina_warburg_how_the_blockchain_will_radically_transform_the_economy
https://www.

https://www.ted.com/talks/molly_winter_the_taboo_secret_to_better_health
https://www.ted.com/talks/martin_reeves_how_to_build_a_business_that_lasts_100_years
https://www.ted.com/talks/anthony_goldbloom_the_jobs_we_ll_lose_to_machines_and_the_ones_we_won_t
https://www.ted.com/talks/john_cameron_why_do_we_hiccup
https://www.ted.com/talks/suzanne_simard_how_trees_talk_to_each_other
https://www.ted.com/talks/matt_anticole_why_the_metric_system_matters
https://www.ted.com/talks/ed_boyden_a_new_way_to_study_the_brain_s_invisible_secrets
https://www.ted.com/talks/gerard_ryle_how_the_panama_papers_journalists_broke_the_biggest_leak_in_history
https://www.ted.com/talks/el_seed_a_project_of_peace_painted_across_50_buildings
https://www.ted.com/talks/lisa_dyson_a_forgotten_space_age_technology_could_change_how_we_grow_food
https://www.ted.com/talks/adam_savage_my_love_letter_to_cosplay
https://www.ted.com/talks/shubhendu_sharma_how_to_grow_a_forest_in_your_backyard
https://www.ted.com/talks/sandr

https://www.ted.com/talks/hugh_evans_what_does_it_mean_to_be_a_citizen_of_the_world
https://www.ted.com/talks/linus_torvalds_the_mind_behind_linux
https://www.ted.com/talks/robert_palmer_the_panama_papers_exposed_a_huge_global_problem_what_s_next
https://www.ted.com/talks/tabetha_boyajian_the_most_mysterious_star_in_the_universe
https://www.ted.com/talks/danielle_feinberg_the_magic_ingredient_that_brings_pixar_movies_to_life
https://www.ted.com/talks/parag_khanna_how_megacities_are_changing_the_map_of_the_world
https://www.ted.com/talks/haley_van_dyck_how_a_start_up_in_the_white_house_is_changing_business_as_usual
https://www.ted.com/talks/adam_grant_the_surprising_habits_of_original_thinkers
https://www.ted.com/talks/nathan_s_jacobs_how_do_schools_of_fish_swim_in_harmony
https://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_how_to_avoid_them
https://www.ted.com/talks/lisa_nip_how_humans_could_evolve_to_survive_in_space
https://www.ted.com/talks/mia_nacamulli_what_would

https://www.ted.com/talks/guillaume_nery_the_exhilarating_peace_of_freediving
https://www.ted.com/talks/claudio_guerra_why_the_octopus_brain_is_so_extraordinary
https://www.ted.com/talks/chieko_asakawa_how_new_technology_helps_blind_people_explore_the_world
https://www.ted.com/talks/jessica_shortall_the_us_needs_paid_family_leave_for_the_sake_of_its_future
https://www.ted.com/talks/kristen_marhaver_how_we_re_growing_baby_corals_to_rebuild_reefs
https://www.ted.com/talks/dan_van_der_vieren_can_you_solve_einstein_s_riddle
https://www.ted.com/talks/year_in_ideas_2015
NO INFO
https://www.ted.com/talks/marina_abramovic_an_art_made_of_trust_vulnerability_and_connection
https://www.ted.com/talks/regina_hartley_why_the_best_hire_might_not_have_the_perfect_resume
https://www.ted.com/talks/ann_morgan_my_year_reading_a_book_from_every_country_in_the_world
https://www.ted.com/talks/genevieve_von_petzinger_why_are_these_32_symbols_found_in_ancient_caves_all_over_europe
https://www.ted.com/talks/car

https://www.ted.com/talks/sarah_willersdorf_what_brands_can_learn_from_online_dating
https://www.ted.com/talks/philip_krinks_finding_your_personal_mission_in_life
https://www.ted.com/talks/patty_mccord_hr_lessons_from_the_world_of_silicon_valley_start_ups
https://www.ted.com/talks/mathieu_lamiaux_imagining_a_new_future_for_health_systems_in_africa
https://www.ted.com/talks/markus_lorenz_industry_4_0_how_intelligent_machines_will_transform_everything_we_know
https://www.ted.com/talks/luc_de_brabandere_reinventing_creative_thinking
https://www.ted.com/talks/joachim_horn_a_solution_for_building_a_generation_of_inventors
https://www.ted.com/talks/emma_bridgewater_how_an_entrepreneurial_potter_helped_restore_beauty_to_a_city
https://www.ted.com/talks/diana_dosik_why_we_need_to_treat_our_employees_as_thoughtfully_as_our_customers
https://www.ted.com/talks/alan_eustace_i_leapt_from_the_stratosphere_here_s_how_i_did_it
https://www.ted.com/talks/jim_simons_the_mathematician_who_cracked_wall_str

In [124]:
get_bio(2000, 3000)

https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america
https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power
https://www.ted.com/talks/murat_dalkilinc_the_benefits_of_good_posture
https://www.ted.com/talks/yuval_noah_harari_what_explains_the_rise_of_humans
https://www.ted.com/talks/el_seed_street_art_with_a_message_of_hope_and_peace

https://www.ted.com/talks/mel_rosenberg_what_causes_bad_breath
https://www.ted.com/talks/daniel_kish_how_i_use_sonar_to_navigate_the_world
https://www.ted.com/talks/alison_killing_there_s_a_better_way_to_die_and_architecture_can_help
https://www.ted.com/talks/dame_stephanie_shirley_why_do_ambitious_women_have_flat_heads
https://www.ted.com/talks/theaster_gates_how_to_revive_a_neighborhood_with_imagination_beauty_and_art
https://www.ted.com/talks/dave_isay_everyone_around_you_has_a_story_the_world_needs_to_hear
https://www.ted.com/talks/anand_giridharadas_a_tale_of_two_americas_and_the_mini_mart_where_they_collided
https://www.ted.com/talks/fei_fei_li_how_we_re_teaching_computers_to_understand_pictures
https://www.ted.com/talks/monica_lewinsky_the_price_of_shame
https://www.ted.com/talks/gil_weinberg_can_robots_be_creative
https://www.ted.com/talks/joseph_desimone_what_if_3d_printing_was_100x_faster
https://www.ted.com/talks/david_eagleman_can_we_create_new_senses_for_humans
https://www

https://www.ted.com/talks/randall_hayes_at_what_moment_are_you_dead
https://www.ted.com/talks/catherine_crump_the_small_and_surprisingly_dangerous_detail_the_police_track_about_you
https://www.ted.com/talks/thomas_hellum_the_world_s_most_boring_television_and_why_it_s_hilariously_addictive
https://www.ted.com/talks/anastasia_taylor_lind_fighters_and_mourners_of_the_ukrainian_revolution
https://www.ted.com/talks/jose_miguel_sokoloff_how_christmas_lights_helped_guerrillas_put_down_their_guns
https://www.ted.com/talks/aakash_odedra_a_dance_in_a_hurricane_of_paper_wind_and_light
https://www.ted.com/talks/barbara_natterson_horowitz_what_veterinarians_know_that_physicians_don_t
https://www.ted.com/talks/rainer_strack_the_workforce_crisis_of_2030_and_how_to_start_solving_it_now
https://www.ted.com/talks/ben_saunders_to_the_south_pole_and_back_the_hardest_105_days_of_my_life
https://www.ted.com/talks/michael_mauser_what_are_those_floaty_things_in_your_eye
https://www.ted.com/talks/oren_yakobov

https://www.ted.com/talks/rishi_manchanda_what_makes_us_get_sick_look_upstream
https://www.ted.com/talks/uldus_bakhtiozina_wry_photos_that_turn_stereotypes_upside_down
https://www.ted.com/talks/hans_and_ola_rosling_how_not_to_be_ignorant_about_the_world
NO INFO
https://www.ted.com/talks/dan_barasch_a_park_underneath_the_hustle_and_bustle_of_new_york_city
https://www.ted.com/talks/zak_ebrahim_i_am_the_son_of_a_terrorist_here_s_how_i_chose_peace
https://www.ted.com/talks/colin_grant_how_our_stories_cross_over
https://www.ted.com/talks/shubhendu_sharma_an_engineer_s_vision_for_tiny_forests_everywhere
https://www.ted.com/talks/isabel_allende_how_to_live_passionately_no_matter_your_age
https://www.ted.com/talks/elizabeth_hellmuth_margulis_why_we_love_repetition_in_music
https://www.ted.com/talks/jim_holt_why_does_the_universe_exist
https://www.ted.com/talks/jill_shargaa_please_please_people_let_s_put_the_awe_back_in_awesome
https://www.ted.com/talks/sally_kohn_don_t_like_clickbait_don_t_cli

https://www.ted.com/talks/wendy_chung_autism_what_we_know_and_what_we_don_t_know_yet
https://www.ted.com/talks/elizabeth_gilbert_success_failure_and_the_drive_to_keep_creating
https://www.ted.com/talks/alex_gendler_how_tsunamis_work
https://www.ted.com/talks/james_patten_the_best_computer_interface_maybe_your_hands
https://www.ted.com/talks/hamish_jolly_a_shark_deterrent_wetsuit_and_it_s_not_what_you_think
https://www.ted.com/talks/michel_laberge_how_synchronized_hammer_strikes_could_generate_nuclear_fusion
https://www.ted.com/talks/sarah_lewis_embrace_the_near_win
https://www.ted.com/talks/matthew_carter_my_life_in_typefaces
https://www.ted.com/talks/jeremy_kasdin_the_flower_shaped_starshade_that_might_help_us_detect_earth_like_planets
https://www.ted.com/talks/norman_spack_how_i_help_transgender_teens_become_who_they_want_to_be
https://www.ted.com/talks/jennifer_senior_for_parents_happiness_is_a_very_high_bar
https://www.ted.com/talks/matt_anticole_what_s_the_difference_between_accur

https://www.ted.com/talks/frederic_kaplan_how_to_build_an_information_time_machine
https://www.ted.com/talks/sandra_aamodt_why_dieting_doesn_t_usually_work
https://www.ted.com/talks/nicole_avena_how_sugar_affects_the_brain
https://www.ted.com/talks/roger_stein_a_bold_new_way_to_fund_drug_research
https://www.ted.com/talks/suzanne_talhouk_don_t_kill_your_language
https://www.ted.com/talks/maysoon_zayid_i_got_99_problems_palsy_is_just_one
https://www.ted.com/talks/emma_bryce_should_we_eat_bugs
https://www.ted.com/talks/mick_cornett_how_an_obese_town_lost_a_million_pounds
https://www.ted.com/talks/diana_nyad_never_ever_give_up
https://www.ted.com/talks/paul_piff_does_money_make_you_mean
https://www.ted.com/talks/rose_eveleth_how_do_we_smell
https://www.ted.com/talks/krista_donaldson_the_80_prosthetic_knee_that_s_changing_lives
https://www.ted.com/talks/andrew_solomon_depression_the_secret_we_share
https://www.ted.com/talks/marco_annunziata_welcome_to_the_age_of_the_industrial_internet
htt

https://www.ted.com/talks/may_el_khalil_making_peace_is_a_marathon
https://www.ted.com/talks/peter_mende_siedlecki_should_you_trust_your_first_impression
https://www.ted.com/talks/steve_ramirez_and_xu_liu_a_mouse_a_laser_beam_a_manipulated_memory
NO INFO
https://www.ted.com/talks/russell_foster_why_do_we_sleep
https://www.ted.com/talks/shigeru_ban_emergency_shelters_made_from_paper
https://www.ted.com/talks/margaret_heffernan_the_dangers_of_willful_blindness
https://www.ted.com/talks/derek_paravicini_and_adam_ockelford_in_the_key_of_genius
NO INFO
https://www.ted.com/talks/eleanor_longden_the_voices_in_my_head
https://www.ted.com/talks/saki_mafundikwa_ingenuity_and_elegance_in_ancient_african_alphabets
https://www.ted.com/talks/jinsop_lee_design_for_all_5_senses
https://www.ted.com/talks/daniel_h_cohen_for_argument_s_sake
https://www.ted.com/talks/beardyman_the_polyphonic_me
https://www.ted.com/talks/peter_van_manen_better_baby_care_thanks_to_formula_1
https://www.ted.com/talks/julie_t

https://www.ted.com/talks/erik_brynjolfsson_the_key_to_growth_race_with_the_machines
https://www.ted.com/talks/robert_gordon_the_death_of_innovation_the_end_of_growth
https://www.ted.com/talks/john_mcwhorter_txtng_is_killing_language_jk
https://www.ted.com/talks/black_my_journey_to_yo_yo_mastery
https://www.ted.com/talks/andres_lozano_parkinson_s_depression_and_the_switch_that_might_turn_them_off
https://www.ted.com/talks/joshua_prager_in_search_of_the_man_who_broke_my_neck
https://www.ted.com/talks/thomas_insel_toward_a_new_understanding_of_mental_illness
https://www.ted.com/talks/colm_kelleher_what_is_zeno_s_dichotomy_paradox
https://www.ted.com/talks/rose_george_let_s_talk_crap_seriously
https://www.ted.com/talks/laura_snyder_the_philosophical_breakfast_club
https://www.ted.com/talks/eric_dishman_health_care_should_be_a_team_sport
https://www.ted.com/talks/bill_shribman_what_cameras_see_that_our_eyes_don_t
https://www.ted.com/talks/dan_ariely_what_makes_us_feel_good_about_our_work
h

https://www.ted.com/talks/adam_davidson_what_we_learned_from_teetering_on_the_fiscal_cliff
https://www.ted.com/talks/steven_addis_a_father_daughter_bond_one_photo_at_a_time
https://www.ted.com/talks/colm_kelleher_what_is_color
https://www.ted.com/talks/molly_crockett_beware_neuro_bunk
https://www.ted.com/talks/gina_cooke_why_is_there_a_b_in_doubt
https://www.ted.com/talks/robin_chase_excuse_me_may_i_rent_your_car
https://www.ted.com/talks/ben_saunders_why_bother_leaving_the_house
https://www.ted.com/talks/christopher_warner_situational_irony_the_opposite_of_what_you_think
https://www.ted.com/talks/marcus_byrne_the_dance_of_the_dung_beetle
https://www.ted.com/talks/maz_jobrani_a_saudi_an_indian_and_an_iranian_walk_into_a_qatari_bar
https://www.ted.com/talks/markham_nolan_how_to_separate_fact_and_fiction_online
https://www.ted.com/talks/ellen_t_hoen_pool_medical_patents_save_lives
https://www.ted.com/talks/lemon_andersen_please_don_t_take_my_air_jordans
https://www.ted.com/talks/nina_tan

https://www.ted.com/talks/daphne_koller_what_we_re_learning_from_online_education
https://www.ted.com/talks/stephen_ritz_a_teacher_growing_green_in_the_south_bronx
https://www.ted.com/talks/michael_anti_behind_the_great_firewall_of_china
https://www.ted.com/talks/giles_duley_when_a_reporter_becomes_the_story
https://www.ted.com/talks/noah_wilson_rich_every_city_needs_healthy_honey_bees
https://www.ted.com/talks/michael_hansmeyer_building_unimaginable_shapes
https://www.ted.com/talks/ramesh_raskar_imaging_at_a_trillion_frames_per_second
https://www.ted.com/talks/tracy_chevalier_finding_the_story_inside_the_painting
https://www.ted.com/talks/malte_spitz_your_phone_company_is_watching
https://www.ted.com/talks/james_stavridis_a_navy_admiral_s_thoughts_on_global_security
https://www.ted.com/talks/vinay_venkatraman_technology_crafts_for_the_digitally_underserved
https://www.ted.com/talks/john_graham_cumming_the_greatest_machine_that_never_was
https://www.ted.com/talks/neil_harbisson_i_liste

In [126]:
get_bio(3000, 4000)

https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
https://www.ted.com/talks/hans_rosling_religions_and_babies
https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen
https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief
https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out
https://www.ted.com/talks/carl_schoonover_how_to_look_inside_the_brain
https://www.ted.com/talks/david_kelley_how_to_build_your_creative_confidence
https://www.ted.com/talks/jean_baptiste_michel_the_mathematics_of_history
https://www.ted.com/talks/tali_sharot_the_optimism_bias
https://ww

https://www.ted.com/talks/shawn_achor_the_happy_secret_to_better_work
https://www.ted.com/talks/bill_doyle_treating_cancer_with_electric_fields
https://www.ted.com/talks/peter_van_uhm_why_i_chose_a_gun
https://www.ted.com/talks/mick_mountz_what_happens_inside_those_massive_warehouses
https://www.ted.com/talks/bjarke_ingels_hedonistic_sustainability
https://www.ted.com/talks/gayle_tzemach_lemmon_women_entrepreneurs_example_not_exception
https://www.ted.com/talks/brian_goldman_doctors_make_mistakes_can_we_talk_about_that
https://www.ted.com/talks/diana_nyad_extreme_swimming_with_the_world_s_most_dangerous_jellyfish
https://www.ted.com/talks/lisa_harouni_a_primer_on_3d_printing
https://www.ted.com/talks/julian_baggini_is_there_a_real_you
https://www.ted.com/talks/bilal_bomani_plant_fuels_that_could_power_a_jet
https://www.ted.com/talks/scott_rickard_the_beautiful_math_behind_the_world_s_ugliest_music
https://www.ted.com/talks/sheena_iyengar_how_to_make_choosing_easier
https://www.ted.com/

https://www.ted.com/talks/misha_glenny_hire_the_hackers
https://www.ted.com/talks/yasheng_huang_does_democracy_stifle_economic_growth
https://www.ted.com/talks/raghava_kk_shake_up_your_story
https://www.ted.com/talks/lee_cronin_making_matter_come_alive
https://www.ted.com/talks/sarah_kaminsky_my_father_the_forger
https://www.ted.com/talks/edward_tenner_unintended_consequences
https://www.ted.com/talks/joan_halifax_compassion_and_the_true_meaning_of_empathy
https://www.ted.com/talks/skylar_tibbits_can_we_make_things_that_make_themselves
https://www.ted.com/talks/julia_bacha_pay_attention_to_nonviolence
https://www.ted.com/talks/svante_paabo_dna_clues_to_our_inner_neanderthal
https://www.ted.com/talks/dan_ariely_beware_conflicts_of_interest
https://www.ted.com/talks/marco_tempest_the_magic_of_truth_and_lies_and_ipods
https://www.ted.com/talks/lucianne_walkowicz_finding_planets_around_other_stars
https://www.ted.com/talks/jeremy_gilley_one_day_of_peace
https://www.ted.com/talks/dyan_denap

https://www.ted.com/talks/paul_root_wolpe_it_s_time_to_question_bio_engineering
https://www.ted.com/talks/isabel_behncke_evolution_s_gift_of_play_from_bonobo_apes_to_humans
https://www.ted.com/talks/hans_rosling_the_magic_washing_machine
https://www.ted.com/talks/sarah_kay_if_i_should_have_a_daughter
https://www.ted.com/talks/rogier_van_der_heide_why_light_needs_darkness
https://www.ted.com/talks/mark_bezos_a_life_lesson_from_a_volunteer_firefighter
https://www.ted.com/talks/janna_levin_the_sound_the_universe_makes
https://www.ted.com/talks/david_brooks_the_social_animal
https://www.ted.com/talks/rob_harmon_how_to_keep_rivers_and_streams_flowing
https://www.ted.com/talks/deb_roy_the_birth_of_a_word
https://www.ted.com/talks/sal_khan_let_s_use_video_to_reinvent_education
https://www.ted.com/talks/courtney_e_martin_this_isn_t_her_mother_s_feminism
https://www.ted.com/talks/anthony_atala_printing_a_human_kidney
https://www.ted.com/talks/bill_gates_how_state_budgets_are_breaking_us_schools

https://www.ted.com/talks/tim_jackson_an_economic_reality_check
https://www.ted.com/talks/eben_bayer_are_mushrooms_the_new_plastic
https://www.ted.com/talks/mechai_viravaidya_how_mr_condom_made_thailand_a_better_place_for_life_and_love
https://www.ted.com/talks/inge_missmahl_bringing_peace_to_the_minds_of_afghanistan
https://www.ted.com/talks/sebastian_seung_i_am_my_connectome
https://www.ted.com/talks/gary_wolf_the_quantified_self
https://www.ted.com/talks/tim_birkhead_the_wisdom_of_birds
https://www.ted.com/talks/julian_treasure_shh_sound_health_in_8_steps
https://www.ted.com/talks/fabian_hemmert_the_shape_shifting_future_of_the_mobile_phone
https://www.ted.com/talks/annie_lennox_why_i_am_an_hiv_aids_activist
https://www.ted.com/talks/mitchell_besser_mothers_helping_mothers_fight_hiv
https://www.ted.com/talks/steven_johnson_where_good_ideas_come_from
https://www.ted.com/talks/christien_meindertsma_how_pig_parts_make_the_world_turn
https://www.ted.com/talks/caroline_phillips_hurdy_gur

https://www.ted.com/talks/tom_wujec_build_a_tower_build_a_team
https://www.ted.com/talks/frederick_balagadde_bio_lab_on_a_microchip
https://www.ted.com/talks/james_randi_homeopathy_quackery_and_fraud
https://www.ted.com/talks/edith_widder_glowing_life_in_an_underwater_world
https://www.ted.com/talks/a_choir_as_big_as_the_internet
https://www.ted.com/talks/thelma_golden_how_art_gives_shape_to_cultural_change
https://www.ted.com/talks/catherine_mohr_the_tradeoffs_of_building_green
https://www.ted.com/talks/jonathan_klein_photos_that_changed_the_world
https://www.ted.com/talks/michael_specter_the_danger_of_science_denial
https://www.ted.com/talks/natalie_merchant_singing_old_poems_to_life
https://www.ted.com/talks/jonathan_drori_every_pollen_grain_has_a_story
https://www.ted.com/talks/dennis_hong_my_seven_species_of_robot_and_how_we_created_them
https://www.ted.com/talks/dean_kamen_the_emotion_behind_invention
https://www.ted.com/talks/elizabeth_pisani_sex_drugs_and_hiv_let_s_get_rational

https://www.ted.com/talks/pranav_mistry_the_thrilling_potential_of_sixthsense_technology
https://www.ted.com/talks/cynthia_schneider_the_surprising_spread_of_idol_tv
https://www.ted.com/talks/edward_burtynsky_photographing_the_landscape_of_oil
https://www.ted.com/talks/rachel_pike_the_science_behind_a_climate_headline
https://www.ted.com/talks/cameron_sinclair_the_refugees_of_boom_and_bust
https://www.ted.com/talks/stefana_broadbent_how_the_internet_enables_intimacy
https://www.ted.com/talks/matthew_white_the_modern_euphonium
https://www.ted.com/talks/marcus_du_sautoy_symmetry_reality_s_riddle
https://www.ted.com/talks/becky_blanton_the_year_i_was_homeless
https://www.ted.com/talks/rachel_armstrong_architecture_that_repairs_itself
https://www.ted.com/talks/david_deutsch_a_new_way_to_explain_explanation
https://www.ted.com/talks/ian_goldin_navigating_our_global_future
https://www.ted.com/talks/marc_koska_1_3m_reasons_to_re_invent_the_syringe
https://www.ted.com/talks/itay_talgam_lead_li

https://www.ted.com/talks/sarah_jones_a_one_woman_global_village
https://www.ted.com/talks/michael_merzenich_growing_evidence_of_brain_plasticity
https://www.ted.com/talks/alex_tabarrok_how_ideas_trump_crises
https://www.ted.com/talks/ben_katchor_comics_of_bygone_new_york
https://www.ted.com/talks/nate_silver_does_racism_affect_how_you_vote
https://www.ted.com/talks/erik_hersman_reporting_crisis_via_texting
https://www.ted.com/talks/niels_diffrient_rethinking_the_way_we_sit_down
https://www.ted.com/talks/margaret_wertheim_the_beautiful_math_of_coral
https://www.ted.com/talks/matthew_childs_9_life_lessons_from_rock_climbing
https://www.ted.com/talks/tim_ferriss_smash_fear_learn_anything
https://www.ted.com/talks/joann_kuchera_morin_stunning_data_visualization_in_the_allosphere
https://www.ted.com/talks/gregory_stock_to_upgrade_is_human
https://www.ted.com/talks/shai_agassi_a_new_ecosystem_for_electric_cars
https://www.ted.com/talks/renny_gleeson_our_antisocial_phone_tricks
https://www.t

https://www.ted.com/talks/mihaly_csikszentmihalyi_flow_the_secret_to_happiness
https://www.ted.com/talks/paul_maccready_nature_vs_humans
https://www.ted.com/talks/john_hodgman_aliens_love_where_are_they
https://www.ted.com/talks/dean_ornish_healing_through_diet
https://www.ted.com/talks/virginia_postrel_on_glamour
https://www.ted.com/talks/paola_antonelli_design_and_the_elastic_mind
https://www.ted.com/talks/garrett_lisi_an_8_dimensional_model_of_the_universe
https://www.ted.com/talks/james_burchfield_playing_invisible_turntables
https://www.ted.com/talks/steven_johnson_the_web_as_a_city
https://www.ted.com/talks/doris_kearns_goodwin_lessons_from_past_presidents
https://www.ted.com/talks/david_perry_are_games_better_than_life
https://www.ted.com/talks/james_nachtwey_moving_photos_of_extreme_drug_resistant_tb
https://www.ted.com/talks/liz_diller_the_blur_building_and_other_tech_empowered_architecture
https://www.ted.com/talks/noah_feldman_politics_and_religion_are_technologies
https://w

In [127]:
get_bio(4000, len(talk_df))

https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
https://www.ted.com/talks/rokia_traore_kounandi
https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq
https://www.ted.com/talks/susan_blackmore_memes_and_temes
https://www.ted.com/talks/paul_collier_the_bottom_billion
https://www.ted.com/talks/seyi_oyesola_a_hospital_tour_in_nigeria
https://www.ted.com/talks/arthur_ganson_moving_sculpture
https://www.ted.com/talks/yves_behar_designing_objects_that_tell_stories
https://www.ted.com/talks/robert_ballard_the_astonishing_hidden_world_of

https://www.ted.com/talks/david_bolinsky_visualizing_the_wonder_of_a_living_cell
https://www.ted.com/talks/rives_the_4_a_m_mystery
https://www.ted.com/talks/will_wright_spore_birth_of_a_game
https://www.ted.com/talks/emily_oster_flip_your_thinking_on_aids_in_africa
https://www.ted.com/talks/jonathan_harris_the_web_s_secret_stories
https://www.ted.com/talks/alan_russell_the_potential_of_regenerative_medicine
https://www.ted.com/talks/dan_dennett_dangerous_memes
https://www.ted.com/talks/bill_stone_inside_the_world_s_deepest_caves
https://www.ted.com/talks/hans_rosling_new_insights_on_poverty
https://www.ted.com/talks/stephen_lawler_tour_microsoft_s_virtual_earth
https://www.ted.com/talks/ethel_a_string_quartet_plays_blue_room
https://www.ted.com/talks/thomas_barnett_let_s_rethink_america_s_military_strategy
https://www.ted.com/talks/david_rockwell_a_memorial_at_ground_zero
https://www.ted.com/talks/robert_thurman_we_can_be_buddhas
https://www.ted.com/talks/anand_agarawala_rethink_the_de

https://www.ted.com/talks/dan_dennett_let_s_teach_religion_all_religion_in_schools
https://www.ted.com/talks/rick_warren_a_life_of_purpose
https://www.ted.com/talks/julia_sweeney_letting_go_of_god
https://www.ted.com/talks/joshua_prince_ramus_behind_the_design_of_seattle_s_library
https://www.ted.com/talks/hans_rosling_the_best_stats_you_ve_ever_seen
https://www.ted.com/talks/sir_ken_robinson_do_schools_kill_creativity
https://www.ted.com/talks/majora_carter_greening_the_ghetto
https://www.ted.com/talks/david_pogue_simplicity_sells
https://www.ted.com/talks/al_gore_averting_the_climate_crisis


In [130]:
talk_df.to_csv('has_bios.csv')

# Get Comments

In [5]:
talk_df['comments'] = '--'
def get_comments(start, end):
    for index in range(start, end):
        try:
            # Get the url, request HTML
            url = talk_df['url'][index]
            link = "https://www.ted.com" + url
            print(link)
            print(index)
            page = requests.get(link)
            content = BS(page.content, 'html.parser')
            
            # Find all scripts, look for initial data about the speakers
            for x in content.find_all('script'):
                if '__INITIAL_DATA__' in x.get_text():
                    text = x.get_text()
                    matched = re.search('"comments":\{.*?\}', text)
                    comments = matched.group(0).strip('"comments":').strip()
                    comments_json = json.loads(comments)
                    
                    # Get the description (titles) of the speakers, short bios
                    if 'count' in comments_json.keys():
                        talk_df['comments'][index] = int(comments_json['count'])
            
            time.sleep(2)
        except:
            print('NO COMMENTS')

In [6]:
get_comments(0, 1000)

https://www.ted.com/talks/ipsita_dasgupta_to_challenge_the_status_quo_find_a_co_conspirator
0
https://www.ted.com/talks/rod_phillips_a_brief_history_of_alcohol
1
NO COMMENTS
https://www.ted.com/talks/pat_mitchell_dangerous_times_call_for_dangerous_women
2
https://www.ted.com/talks/cara_e_yar_khan_the_beautiful_balance_between_courage_and_fear
3
https://www.ted.com/talks/valorie_kondos_field_why_winning_doesn_t_always_equal_success
4
https://www.ted.com/talks/guy_winch_how_to_turn_off_work_thoughts_during_your_free_time
5
https://www.ted.com/talks/william_d_desmond_the_philosophy_of_cynicism
6
NO COMMENTS
https://www.ted.com/talks/nick_bostrom_how_civilization_could_destroy_itself_and_4_ways_we_could_prevent_it
7
https://www.ted.com/talks/jasmine_crowe_what_we_re_getting_wrong_in_the_fight_to_end_hunger
8
https://www.ted.com/talks/sinan_aral_how_we_can_protect_truth_in_the_age_of_misinformation
9
https://www.ted.com/talks/dan_kwartler_why_should_you_read_dune_by_frank_herbert
10
NO COMM

https://www.ted.com/talks/jacob_soboroff_why_do_americans_vote_on_tuesdays
86
NO COMMENTS
https://www.ted.com/talks/toby_kiers_lessons_from_fungi_on_markets_and_economics
87
https://www.ted.com/talks/hans_block_and_moritz_riesewieck_the_price_of_a_clean_internet
88
https://www.ted.com/talks/leila_pirhaji_the_medical_potential_of_ai_and_metabolites
89
https://www.ted.com/talks/matthew_borths_claws_vs_nails
90
NO COMMENTS
https://www.ted.com/talks/michael_s_a_graziano_how_close_are_we_to_uploading_our_minds
91
NO COMMENTS
https://www.ted.com/talks/chris_fisher_why_we_should_archive_everything_on_the_planet
92
https://www.ted.com/talks/alisa_kazarina_human_history_at_the_intersection_science_and_archaeology
93
NO COMMENTS
https://www.ted.com/talks/alyssa_loorya_what_i_learned_from_digging_under_new_york_city_s_streets
94
NO COMMENTS
https://www.ted.com/talks/ella_al_shamahi_why_archaeology_needs_to_transcend_borders
95
NO COMMENTS
https://www.ted.com/talks/rose_m_mutiso_how_to_bring_affor

https://www.ted.com/talks/alex_gendler_can_you_solve_the_secret_sauce_riddle
172
NO COMMENTS
https://www.ted.com/talks/emmett_shear_what_streaming_means_for_the_future_of_entertainment
173
https://www.ted.com/talks/nick_hanauer_the_dirty_secret_of_capitalism_and_a_new_way_forward
174
https://www.ted.com/talks/raj_jayadev_community_powered_criminal_justice_reform
175
https://www.ted.com/talks/alex_gendler_a_brief_history_of_chess
176
NO COMMENTS
https://www.ted.com/talks/jacqueline_woodson_what_reading_slowly_taught_me_about_writing
177
https://www.ted.com/talks/danielle_citron_how_deepfakes_undermine_truth_and_threaten_democracy
178
https://www.ted.com/talks/sandeep_jauhar_how_your_emotions_change_the_shape_of_your_heart
179
https://www.ted.com/talks/iseult_gillespie_why_should_you_read_midnight_s_children
180
NO COMMENTS
https://www.ted.com/talks/carl_june_a_living_drug_that_could_change_the_way_we_treat_cancer
181
https://www.ted.com/talks/clint_smith_ode_to_the_only_black_kid_in_the

https://www.ted.com/talks/jasmine_cho_cookies_as_a_form_of_activism
258
NO COMMENTS
https://www.ted.com/talks/natalie_fratto_3_ways_to_measure_your_adaptability_and_how_to_improve_it
259
https://www.ted.com/talks/julius_maada_bio_a_vision_for_the_future_of_sierra_leone
260
https://www.ted.com/talks/moriba_jah_the_world_s_first_crowdsourced_space_traffic_monitoring_system
261
https://www.ted.com/talks/camille_defrenne_and_suzanne_simard_the_secret_language_of_trees
262
NO COMMENTS
https://www.ted.com/talks/megan_parker_how_dogs_help_with_conservation_efforts
263
NO COMMENTS
https://www.ted.com/talks/gregory_berns_what_emotions_look_like_in_a_dog_s_brain
264
NO COMMENTS
https://www.ted.com/talks/barbara_sherman_how_to_read_a_dog_s_body_language
265
NO COMMENTS
https://www.ted.com/talks/julia_espinosa_the_secret_life_of_dogs
266
NO COMMENTS
https://www.ted.com/talks/glenn_cantave_how_augmented_reality_is_changing_activism
267
https://www.ted.com/talks/jon_gray_the_next_big_thing_is_coming

NO COMMENTS
https://www.ted.com/talks/jarrell_daniels_what_prosecutors_and_incarcerated_people_can_learn_from_each_other
345
https://www.ted.com/talks/america_ferrera_my_identity_is_a_superpower_not_an_obstacle
346
https://www.ted.com/talks/austin_eubanks_what_surviving_the_columbine_shooting_taught_me_about_pain
347
NO COMMENTS
https://www.ted.com/talks/liz_powers_a_new_way_to_define_self_worth
348
NO COMMENTS
https://www.ted.com/talks/janne_willems_why_i_ask_strangers_to_draw_their_memories
349
NO COMMENTS
https://www.ted.com/talks/wendell_pierce_theater_healed_my_community_after_hurricane_katrina
350
NO COMMENTS
https://www.ted.com/talks/winnie_harlow_how_i_define_beauty
351
NO COMMENTS
https://www.ted.com/talks/jess_thom_how_i_turned_my_tourette_s_tics_into_art
352
NO COMMENTS
https://www.ted.com/talks/hamdi_ulukaya_the_anti_ceo_playbook
353
https://www.ted.com/talks/bruce_friedrich_the_next_global_agricultural_revolution
354
https://www.ted.com/talks/jeff_leek_and_lucy_mcgowan_can

...

NO COMMENTS
https://www.ted.com/talks/eldra_jackson_how_i_unlearned_dangerous_lessons_about_masculinity
603
https://www.ted.com/talks/julie_chang_instagram_filters_for_robots_optical_neural_networks
604
https://www.ted.com/talks/fiona_radford_from_slave_to_rebel_gladiator_the_life_of_spartacus
605
https://www.ted.com/talks/li_wei_tan_the_fascinating_science_of_bubbles_from_soap_to_champagne
606
https://www.ted.com/talks/michael_c_bush_this_is_what_makes_employees_happy_at_work
607
https://www.ted.com/talks/katharine_hayhoe_the_most_important_thing_you_can_do_to_fight_climate_change_talk_about_it
608
https://www.ted.com/talks/paul_s_kindstedt_a_brie_f_history_of_cheese
609
NO COMMENTS
https://www.ted.com/talks/madame_gandhi_and_amber_galloway_gallego_music_with_a_message_should_be_accessible
610
https://www.ted.com/talks/douglas_rushkoff_how_to_be_team_human_in_the_digital_future
611
https://www.ted.com/talks/carla_harris_how_to_find_the_person_who_can_help_you_get_ahead_at_work
612
htt

https://www.ted.com/talks/brian_d_avery_how_rollercoasters_affect_your_body
689
https://www.ted.com/talks/lauren_esposito_saving_human_kind_one_scorpion_at_a_time
690
NO COMMENTS
https://www.ted.com/talks/rachel_wurzman_how_isolation_fuels_opioid_addiction
691
https://www.ted.com/talks/shane_wickes_why_i_came_out_as_a_gay_football_coach
692
NO COMMENTS
https://www.ted.com/talks/alexis_jones_redefining_manhood_one_locker_room_talk_at_a_time
693
NO COMMENTS
https://www.ted.com/talks/charles_c_mann_how_will_we_survive_when_the_population_hits_10_billion
694
https://www.ted.com/talks/a_tribe_called_red_we_are_the_halluci_nation
695
https://www.ted.com/talks/dennis_shasha_can_you_solve_the_stolen_rubies_riddle
696
NO COMMENTS
https://www.ted.com/talks/elizabeth_streb_my_quest_to_defy_gravity_and_fly
697
https://www.ted.com/talks/darieth_chisolm_how_revenge_porn_turns_lives_upside_down
698
https://www.ted.com/talks/henrietta_fore_how_we_can_help_young_people_build_a_better_future
699
https:/

https://www.ted.com/talks/derek_banta_what_if_our_data_could_be_protected_online
777
NO COMMENTS
https://www.ted.com/talks/ryan_martin_the_upside_of_anger
778
https://www.ted.com/talks/benedetta_berti_did_the_global_response_to_9_11_make_us_safer
779
https://www.ted.com/talks/noa_kageyama_and_pen_pen_chen_how_to_stay_calm_under_pressure
780
NO COMMENTS
https://www.ted.com/talks/dan_kwartler_what_causes_headaches
781
NO COMMENTS
https://www.ted.com/talks/emma_bryce_how_does_the_immune_system_work
782
NO COMMENTS
https://www.ted.com/talks/alex_gendler_how_does_impeachment_work
783
NO COMMENTS
https://www.ted.com/talks/ayana_elizabeth_johnson_and_jennifer_jacquet_will_the_ocean_ever_run_out_of_fish
784
NO COMMENTS
https://www.ted.com/talks/elizabeth_waters_the_left_brain_vs_right_brain_myth
785
NO COMMENTS
https://www.ted.com/talks/brandon_rodriguez_the_power_of_creative_constraints
786
NO COMMENTS
https://www.ted.com/talks/hector_lanz_how_do_focus_groups_work
787
https://www.ted.com/talk

NO COMMENTS
https://www.ted.com/talks/carmen_jimenez_borja_las_palabras_de_la_discriminacion
865
NO COMMENTS
https://www.ted.com/talks/nestor_guerra_nuestras_mentiras_en_la_vida_y_en_los_negocios
866
NO COMMENTS
https://www.ted.com/talks/alberto_garin_a_quien_pertenece_el_pasado
867
https://www.ted.com/talks/fernanda_castillo_la_voz_del_artista
868
NO COMMENTS
https://www.ted.com/talks/mikhail_zygar_what_the_russian_revolution_would_have_looked_like_on_social_media
869
https://www.ted.com/talks/hernan_casciari_duelo_por_la_muerte_del_padre
870
NO COMMENTS
https://www.ted.com/talks/alba_sotorra_un_ejercito_de_mujeres
871
https://www.ted.com/talks/kashmir_hill_and_surya_mattu_what_your_smart_devices_know_and_share_about_you
872
https://www.ted.com/talks/peta_greenfield_alex_gendler_history_vs_augustus
873
NO COMMENTS
https://www.ted.com/talks/dr_julia_shaw_what_to_do_if_you_have_been_harassed_at_work
874
https://www.ted.com/talks/rebeca_hwang_the_power_of_diversity_within_yourself
875
ht

https://www.ted.com/talks/tobacco_brown_what_gardening_taught_me_about_life
953
https://www.ted.com/talks/raj_panjabi_what_if_we_digitally_empowered_community_health_workers
954
NO COMMENTS
https://www.ted.com/talks/hugh_herr_how_we_ll_become_cyborgs_and_extend_human_potential
955
https://www.ted.com/talks/anushka_naiknaware_a_teen_scientist_s_invention_to_help_wounds_heal
956
https://www.ted.com/talks/susan_emmett_this_simple_test_can_help_kids_hear_better
957
https://www.ted.com/talks/gene_luen_yang_comics_belong_in_the_classroom
958
https://www.ted.com/talks/jeremy_forbes_how_to_start_a_conversation_about_suicide
959
https://www.ted.com/talks/baroness_beeban_kidron_why_we_must_protect_the_digital_rights_of_children
960
NO COMMENTS
https://www.ted.com/talks/madhumita_murgia_how_data_brokers_sell_your_identity
961
NO COMMENTS
https://www.ted.com/talks/marta_peirano_the_surveillance_device_you_carry_around_all_day
962
NO COMMENTS
https://www.ted.com/talks/stefaan_verhulst_using_corpora

In [7]:
get_comments(1000, 2000)

https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
1000
https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
1001
https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
1002
https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
1003
https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
1004
https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
1005
https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment
1006
https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell
1007
https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees
1008
https://www.ted.com/talks/mother_london_ai_therapy
1009
NO COMMENTS
https://www.ted.com/talks/duncan_cowles_it_s_not_amazing_enough
1010
NO COMMENTS
https://www.ted.com/ta

https://www.ted.com/talks/tiffany_kagure_mugo_and_siphumeze_khundayi_how_to_have_a_healthier_positive_relationship_to_sex
1092
https://www.ted.com/talks/nilay_kulkarni_a_life_saving_invention_that_prevents_human_stampedes
1093
https://www.ted.com/talks/jason_shen_looking_for_a_job_highlight_your_ability_not_your_experience
1094
https://www.ted.com/talks/margaret_mitchell_how_we_can_build_ai_to_help_humans_not_hurt_us
1095
https://www.ted.com/talks/tanya_menon_the_secret_to_great_opportunities_the_person_you_haven_t_met_yet
1096
https://www.ted.com/talks/amit_kalra_3_creative_ways_to_fix_fashion_s_waste_problem
1097
https://www.ted.com/talks/rowan_fitzpatrick_and_heart_of_mind_facing_the_future_together
1098
NO COMMENTS
https://www.ted.com/talks/shade_zahrai_and_aric_yegudkin_the_rhythmic_beauty_of_collaboration
1099
NO COMMENTS
https://www.ted.com/talks/will_jenkins_the_most_precious_commodity_of_all
1100
NO COMMENTS
https://www.ted.com/talks/tomer_garzberg_what_happens_when_we_take_hu

NO COMMENTS
https://www.ted.com/talks/christian_schroter_how_we_re_tracking_medical_donations_across_the_world
1178
https://www.ted.com/talks/carl_vause_meet_the_robot_designed_like_an_octopus_tentacle
1179
NO COMMENTS
https://www.ted.com/talks/bijan_zakeri_from_flesh_eating_bacteria_to_molecular_superglue
1180
NO COMMENTS
https://www.ted.com/talks/mike_gil_could_fish_social_networks_help_us_save_coral_reefs
1181
https://www.ted.com/talks/susan_david_the_gift_and_power_of_emotional_courage
1182
https://www.ted.com/talks/fredros_okumu_why_i_study_the_most_dangerous_animal_on_earth_mosquitoes
1183
https://www.ted.com/talks/amar_inamdar_the_thrilling_potential_for_off_grid_solar_energy
1184
https://www.ted.com/talks/leila_takayama_what_s_it_like_to_be_a_robot
1185
https://www.ted.com/talks/david_katz_the_surprising_solution_to_ocean_plastic
1186
https://www.ted.com/talks/george_steinmetz_photos_of_africa_taken_from_a_flying_lawn_chair
1187
https://www.ted.com/talks/edsel_salvana_the_dange

NO COMMENTS
https://www.ted.com/talks/leah_chase_and_pat_mitchell_an_interview_with_the_queen_of_creole_cuisine
1266
https://www.ted.com/talks/mariano_sigman_and_dan_ariely_how_can_groups_make_good_decisions
1267
https://www.ted.com/talks/crystal_martin_no_hard_work_does_not_speak_for_itself
1268
NO COMMENTS
https://www.ted.com/talks/sarah_corbett_activism_needs_introverts
1269
https://www.ted.com/talks/scott_galloway_how_amazon_apple_facebook_and_google_manipulate_our_emotions
1270
https://www.ted.com/talks/niti_bhan_the_hidden_opportunities_of_the_informal_economy
1271
https://www.ted.com/talks/per_espen_stoknes_how_to_transform_apocalypse_fatigue_into_action_on_global_warming
1272
https://www.ted.com/talks/kayla_briet_why_do_i_make_art_to_build_time_capsules_for_my_heritage
1273
https://www.ted.com/talks/christen_reighter_i_don_t_want_children_stop_telling_me_i_ll_change_my_mind
1274
https://www.ted.com/talks/beth_malone_how_my_dad_s_dementia_changed_my_idea_of_death_and_life
1275
h

https://www.ted.com/talks/caitlin_quattromani_and_lauran_arledge_how_our_friendship_survives_our_opposing_politics
1352
https://www.ted.com/talks/olutimehin_adegbeye_who_belongs_in_a_city
1353
https://www.ted.com/talks/benjamin_grant_what_it_feels_like_to_see_earth_from_space
1354
https://www.ted.com/talks/tomas_saraceno_would_you_live_in_a_floating_city_in_the_sky
1355
https://www.ted.com/talks/ray_dalio_how_to_build_a_company_where_the_best_ideas_win
1356
https://www.ted.com/talks/erin_marie_saltman_how_young_people_join_violent_extremist_groups_and_how_to_stop_them
1357
https://www.ted.com/talks/daan_roosegaarde_a_smog_vacuum_cleaner_and_other_magical_city_designs
1358
https://www.ted.com/talks/chetan_bhatt_dare_to_refuse_the_origin_myths_that_claim_who_you_are
1359
https://www.ted.com/talks/chance_coughenour_how_your_pictures_can_help_reclaim_lost_history
1360
https://www.ted.com/talks/jennifer_granick_how_the_us_government_spies_on_people_who_protest_including_you
1361
https://www

...

https://www.ted.com/talks/todd_coleman_a_temporary_tattoo_that_brings_hospital_care_to_the_home
1628
https://www.ted.com/talks/rachel_botsman_we_ve_stopped_trusting_institutions_and_started_trusting_strangers
1629
https://www.ted.com/talks/mel_rosenberg_what_causes_cavities
1630
NO COMMENTS
https://www.ted.com/talks/trevor_copp_and_jeff_fox_ballroom_dance_that_breaks_gender_roles
1631
https://www.ted.com/talks/jim_hemerling_5_ways_to_lead_in_an_era_of_constant_change
1632
https://www.ted.com/talks/melissa_walker_art_can_heal_ptsd_s_invisible_wounds
1633
https://www.ted.com/talks/pico_iyer_the_beauty_of_what_we_ll_never_know
1634
https://www.ted.com/talks/ione_wells_how_we_talk_about_sexual_assault_online
1635
https://www.ted.com/talks/john_mcwhorter_4_reasons_to_learn_a_new_language
1636
https://www.ted.com/talks/rainn_wilson_ideas_worth_dating
1637
https://www.ted.com/talks/amy_adkins_3_tips_to_boost_your_confidence
1638
NO COMMENTS
https://www.ted.com/talks/isaac_lidsky_what_reality_

NO COMMENTS
https://www.ted.com/talks/keolu_fox_why_genetic_research_must_be_more_diverse
1722
https://www.ted.com/talks/gill_hicks_i_survived_a_terrorist_attack_here_s_what_i_learned
1723
https://www.ted.com/talks/tristan_harris_how_better_tech_could_protect_us_from_distraction
1724
https://www.ted.com/talks/benedetta_berti_and_evelien_borgman_what_does_it_mean_to_be_a_refugee_nov_2018
1725
NO COMMENTS
https://www.ted.com/talks/chris_milk_the_birth_of_virtual_reality_as_an_art_form
1726
https://www.ted.com/talks/andrew_pelling_this_scientist_makes_ears_out_of_apples
1727
https://www.ted.com/talks/jamila_raqib_the_secret_to_effective_nonviolent_resistance
1728
https://www.ted.com/talks/andrew_youn_3_reasons_why_we_can_win_the_fight_against_poverty
1729
https://www.ted.com/talks/negin_farsad_a_highly_scientific_taxonomy_of_haters
1730
https://www.ted.com/talks/monica_menesini_why_do_our_bodies_age
1731
NO COMMENTS
https://www.ted.com/talks/sajay_samuel_how_college_loans_exploit_students

https://www.ted.com/talks/tshering_tobgay_this_country_isn_t_just_carbon_neutral_it_s_carbon_negative
1812
https://www.ted.com/talks/mileha_soneji_simple_hacks_for_life_with_parkinson_s
1813
https://www.ted.com/talks/laura_robinson_the_secrets_i_find_on_the_mysterious_ocean_floor
1814
https://www.ted.com/talks/reshma_saujani_teach_girls_bravery_not_perfection
1815
https://www.ted.com/talks/travis_kalanick_uber_s_plan_to_get_more_people_into_fewer_cars
1816
https://www.ted.com/talks/alexander_betts_our_refugee_system_is_failing_here_s_how_we_can_fix_it
1817
https://www.ted.com/talks/russ_altman_what_really_happens_when_you_mix_medications
1818
https://www.ted.com/talks/magda_sayeg_how_yarn_bombing_grew_into_a_worldwide_movement
1819
https://www.ted.com/talks/thomas_peschak_dive_into_an_ocean_photographer_s_world
1820
https://www.ted.com/talks/ivan_coyote_why_we_need_gender_neutral_bathrooms
1821
https://www.ted.com/talks/mary_bassett_why_your_doctor_should_care_about_social_justice
1822

https://www.ted.com/talks/kaki_king_a_musical_escape_into_a_world_of_light_and_color
1903
https://www.ted.com/talks/harald_haas_forget_wi_fi_meet_the_new_li_fi_internet
1904
https://www.ted.com/talks/patricia_medici_the_coolest_animal_you_know_nothing_about_and_how_we_can_save_it
1905
https://www.ted.com/talks/jeffrey_siegel_what_makes_muscles_grow
1906
NO COMMENTS
https://www.ted.com/talks/suzanne_duncan_the_dark_side_of_storytelling
1907
NO COMMENTS
https://www.ted.com/talks/suzanne_case_could_moby_dick_prevent_the_next_financial_crisis
1908
NO COMMENTS
https://www.ted.com/talks/stephen_lawrence_the_future_of_reading_it_s_fast
1909
NO COMMENTS
https://www.ted.com/talks/siawn_ou_the_art_of_letting_go_of_the_floor
1910
NO COMMENTS
https://www.ted.com/talks/sahil_sethi_can_0_7_of_corporate_profits_fund_the_world_s_biggest_problems
1911
NO COMMENTS
https://www.ted.com/talks/kirsten_hall_women_in_business_entirely_unremarkable
1912
NO COMMENTS
https://www.ted.com/talks/katherine_lucas_in_

https://www.ted.com/talks/jim_simons_the_mathematician_who_cracked_wall_street
1989
https://www.ted.com/talks/jamie_bartlett_how_the_mysterious_dark_net_is_going_mainstream
1990
https://www.ted.com/talks/alex_gendler_can_you_solve_the_bridge_riddle
1991
NO COMMENTS
https://www.ted.com/talks/elizabeth_nyamayaro_an_invitation_to_men_who_want_a_better_world_for_women
1992
https://www.ted.com/talks/wendy_freedman_this_telescope_might_show_us_the_beginning_of_the_universe
1993
https://www.ted.com/talks/yves_morieux_how_too_many_rules_at_work_keep_you_from_getting_things_done
1994
https://www.ted.com/talks/robin_murphy_these_robots_come_to_the_rescue_after_a_disaster
1995
https://www.ted.com/talks/sarthak_sinha_why_do_some_people_go_bald
1996
NO COMMENTS
https://www.ted.com/talks/seth_berkley_the_troubling_reason_why_vaccines_are_made_too_late_if_they_re_made_at_all
1997
https://www.ted.com/talks/jim_al_khalili_how_quantum_biology_might_explain_life_s_biggest_questions
1998
https://www.ted.c

In [9]:
get_comments(2000, 3000)

https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
2000
https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
2001
https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
2002
https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
2003
https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
2004
https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
2005
https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
2006
NO COMMENTS
https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america
2007
https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power
2008
https://www.ted.com/talks/murat_dalkilinc_the_benefits_of_good_posture
2009
NO COMMENTS
https://www.ted.com/talks/yuval_noah_harari_what_explains_the_rise_of_humans
201

https://www.ted.com/talks/barat_ali_batoor_my_desperate_journey_with_a_human_smuggler
2089
https://www.ted.com/talks/fred_jansen_how_to_land_on_a_comet
2090
https://www.ted.com/talks/dan_ariely_how_equal_do_we_want_the_world_to_be_you_d_be_surprised
2091
https://www.ted.com/talks/eduardo_saenz_de_cabezon_math_is_forever
2092
https://www.ted.com/talks/bel_pesce_5_ways_to_kill_your_dreams
2093
https://www.ted.com/talks/bill_gates_the_next_outbreak_we_re_not_ready
2094
https://www.ted.com/talks/boniface_mwangi_the_day_i_stood_up_alone
2095
https://www.ted.com/talks/kevin_rudd_are_china_and_the_us_doomed_to_conflict
2096
https://www.ted.com/talks/mel_rosenberg_what_causes_bad_breath
2097
NO COMMENTS
https://www.ted.com/talks/daniel_kish_how_i_use_sonar_to_navigate_the_world
2098
https://www.ted.com/talks/alison_killing_there_s_a_better_way_to_die_and_architecture_can_help
2099
https://www.ted.com/talks/dame_stephanie_shirley_why_do_ambitious_women_have_flat_heads
2100
https://www.ted.com/t

https://www.ted.com/talks/karalee_close_will_healthcare_embrace_digital_or_will_we_die_waiting
2180
NO COMMENTS
https://www.ted.com/talks/julia_kloiber_let_s_build_better_digital_tools_for_our_cities
2181
NO COMMENTS
https://www.ted.com/talks/jay_cousins_the_future_of_economic_success_is_collaborative
2182
NO COMMENTS
https://www.ted.com/talks/dario_gil_the_future_of_expertise
2183
NO COMMENTS
https://www.ted.com/talks/carey_kolaja_a_vision_for_truly_secure_and_seamless_transactions
2184
NO COMMENTS
https://www.ted.com/talks/antonella_mei_pochtler_how_better_transportation_can_reinvigorate_a_city
2185
https://www.ted.com/talks/anil_raj_micro_power_plants_to_bring_electricity_to_millions
2186
NO COMMENTS
https://www.ted.com/talks/alison_sander_megatrends_5_tips_on_the_art_and_science_of_trend_tracking
2187
NO COMMENTS
https://www.ted.com/talks/alexander_tuerpitz_virtualizing_employment_in_saudi_arabia
2188
NO COMMENTS
https://www.ted.com/talks/mundano_trash_cart_superheroes
2189
https:/

https://www.ted.com/talks/courtney_stephens_a_brief_history_of_melancholy
2267
NO COMMENTS
https://www.ted.com/talks/nancy_kanwisher_a_neural_portrait_of_the_human_mind
2268
https://www.ted.com/talks/gail_reed_where_to_train_the_world_s_doctors_cuba
2269
https://www.ted.com/talks/susan_colantuono_the_career_advice_you_probably_didn_t_get
2270
https://www.ted.com/talks/francis_de_los_reyes_sanitation_is_a_basic_human_right
2271
https://www.ted.com/talks/moshe_safdie_how_to_reinvent_the_apartment_building
2272
https://www.ted.com/talks/matthew_o_reilly_am_i_dying_the_honest_answer
2273
https://www.ted.com/talks/eman_mohammed_the_courage_to_tell_a_hidden_story
2274
https://www.ted.com/talks/kenneth_cukier_big_data_is_better_data
2275
https://www.ted.com/talks/lord_nicholas_stern_the_state_of_the_climate_and_what_we_might_do_about_it
2276
https://www.ted.com/talks/antonio_donato_nobre_the_magic_of_the_amazon_a_river_that_flows_invisibly_all_around_us
2277
https://www.ted.com/talks/avi_reic

https://www.ted.com/talks/yoruba_richen_what_the_gay_rights_movement_learned_from_the_civil_rights_movement
2362
https://www.ted.com/talks/robert_full_the_secrets_of_nature_s_grossest_creatures_channeled_into_robots
2363
https://www.ted.com/talks/stephen_burt_why_people_need_poetry
2364
https://www.ted.com/talks/dan_gilbert_the_psychology_of_your_future_self
2365
https://www.ted.com/talks/karen_d_davis_how_does_your_brain_respond_to_pain
2366
NO COMMENTS
https://www.ted.com/talks/ray_kurzweil_get_ready_for_hybrid_thinking
2367
https://www.ted.com/talks/sting_how_i_started_writing_songs_again
2368
https://www.ted.com/talks/stephen_friend_the_hunt_for_unexpected_genetic_heroes
2369
https://www.ted.com/talks/kitra_cahana_a_glimpse_of_life_on_the_road
2370
https://www.ted.com/talks/jon_mooallem_how_the_teddy_bear_taught_us_compassion
2371
https://www.ted.com/talks/sebastian_junger_why_veterans_miss_war
2372
https://www.ted.com/talks/wes_moore_how_to_talk_to_veterans_about_war
2373
https://

...

https://www.ted.com/talks/holly_morris_why_stay_in_chernobyl_because_it_s_home
2554
https://www.ted.com/talks/abha_dawesar_life_in_the_digital_now
2555
https://www.ted.com/talks/mohamed_hijri_a_simple_solution_to_the_coming_phosphorus_crisis
2556
https://www.ted.com/talks/mariana_mazzucato_government_investor_risk_taker_innovator
2557
https://www.ted.com/talks/xavier_vilalta_architecture_at_home_in_its_community
2558
https://www.ted.com/talks/gian_giudice_why_our_universe_might_exist_on_a_knife_edge
2559
https://www.ted.com/talks/parul_sehgal_an_ode_to_envy
2560
https://www.ted.com/talks/charles_robertson_africa_s_next_boom
2561
https://www.ted.com/talks/steve_howard_let_s_go_all_in_on_selling_sustainability
2562
https://www.ted.com/talks/hetain_patel_who_am_i_think_again
2563
https://www.ted.com/talks/alessandro_acquisti_what_will_a_future_without_secrets_look_like
2564
https://www.ted.com/talks/iwan_baan_ingenious_homes_in_unexpected_places
2565
https://www.ted.com/talks/amanda_benne

NO COMMENTS
https://www.ted.com/talks/joy_lin_if_superpowers_were_real_super_speed
2653
NO COMMENTS
https://www.ted.com/talks/joy_lin_if_superpowers_were_real_body_mass
2654
NO COMMENTS
https://www.ted.com/talks/michael_archer_how_we_ll_resurrect_the_gastric_brooding_frog_the_tasmanian_tiger
2655
https://www.ted.com/talks/bob_mankoff_anatomy_of_a_new_yorker_cartoon
2656
https://www.ted.com/talks/marjee_chmiel_and_trevor_owens_is_there_a_center_of_the_universe
2657
NO COMMENTS
https://www.ted.com/talks/peter_attia_is_the_obesity_crisis_hiding_a_bigger_problem
2658
https://www.ted.com/talks/scott_gass_how_big_is_the_ocean
2659
NO COMMENTS
https://www.ted.com/talks/lesley_hazleton_the_doubt_essential_to_faith
2660
https://www.ted.com/talks/camille_seaman_photos_from_a_storm_chaser
2661
https://www.ted.com/talks/paul_pholeros_how_to_reduce_poverty_fix_homes
2662
https://www.ted.com/talks/joseph_kim_the_family_i_lost_in_north_korea_and_the_family_i_gained
2663
https://www.ted.com/talks/juli

NO COMMENTS
https://www.ted.com/talks/gary_niekerk_chips_conflict_and_the_congo
2752
NO COMMENTS
https://www.ted.com/talks/erika_debenedictis_strategies_for_fuel_efficient_space_travel
2753
NO COMMENTS
https://www.ted.com/talks/achin_bhowmik_interactive_experiences_with_perceptual_computing
2754
NO COMMENTS
https://www.ted.com/talks/steve_brown_why_machines_must_make_us_better_humans
2755
https://www.ted.com/talks/richard_turere_my_invention_that_made_peace_with_lions
2756
https://www.ted.com/talks/jeff_dekofsky_euclid_s_puzzling_parallel_postulate
2757
NO COMMENTS
https://www.ted.com/talks/mark_shaw_one_very_dry_demo
2758
https://www.ted.com/talks/jessica_green_we_re_covered_in_germs_let_s_design_for_that
2759
https://www.ted.com/talks/eric_whitacre_virtual_choir_live
2760
https://www.ted.com/talks/francis_collins_we_need_better_drugs_now
2761
https://www.ted.com/talks/mark_changizi_why_do_we_see_illusions
2762
NO COMMENTS
https://www.ted.com/talks/hyeonseo_lee_my_escape_from_north_ko

https://www.ted.com/talks/amos_winter_the_cheap_all_terrain_wheelchair
2854
https://www.ted.com/talks/daphne_bavelier_your_brain_on_video_games
2855
https://www.ted.com/talks/david_binder_the_arts_festival_revolution
2856
https://www.ted.com/talks/leah_buechley_how_to_sketch_with_electronics
2857
https://www.ted.com/talks/hannah_brencher_love_letters_to_strangers
2858
https://www.ted.com/talks/arunachalam_muruganantham_how_i_started_a_sanitary_napkin_revolution
2859
https://www.ted.com/talks/julie_burstein_4_lessons_in_creativity
2860
https://www.ted.com/talks/jeff_hancock_the_future_of_lying
2861
https://www.ted.com/talks/ariel_anbar_a_needle_in_countless_haystacks_finding_habitable_worlds
2862
NO COMMENTS
https://www.ted.com/talks/georgette_mulheir_the_tragedy_of_orphanages
2863
https://www.ted.com/talks/gary_greenberg_the_beautiful_nano_details_of_our_world
2864
https://www.ted.com/talks/jake_wood_a_new_mission_for_veterans_disaster_relief
2865
https://www.ted.com/talks/matt_killing

https://www.ted.com/talks/mina_bissell_experiments_that_point_to_a_new_understanding_of_cancer
2954
https://www.ted.com/talks/gabriel_barcia_colombo_capturing_memories_in_video_art
2955
https://www.ted.com/talks/todd_humphreys_how_to_fool_a_gps
2956
https://www.ted.com/talks/scott_gass_the_survival_of_the_sea_turtle
2957
NO COMMENTS
https://www.ted.com/talks/jared_ficklin_new_ways_to_see_music_with_color_and_fire
2958
https://www.ted.com/talks/marc_goodman_a_vision_of_crimes_in_the_future
2959
https://www.ted.com/talks/chris_gerdes_the_future_race_car_150mph_and_no_driver
2960
https://www.ted.com/talks/jonathan_eisen_meet_your_microbes
2961
https://www.ted.com/talks/jane_mcgonigal_the_game_that_can_give_you_10_extra_years_of_life
2962
https://www.ted.com/talks/usman_riaz_preston_reed_a_young_guitarist_meets_his_hero
2963
https://www.ted.com/talks/cesar_harada_a_novel_idea_for_cleaning_up_oil_spills
2964
https://www.ted.com/talks/raghava_kk_what_s_your_200_year_plan
2965
https://www.ted

In [10]:
get_comments(3000, 4000)

https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
3000
https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
3001
https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
3002
https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
3003
https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
3004
https://www.ted.com/talks/hans_rosling_religions_and_babies
3005
https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
3006
https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen
3007
https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief
3008
https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out
3009
https://www.ted.com/talks/carl_schoonover_how_to_look_inside_the_brain
3010
https://www.ted.com/talks/david_kelley_how_to_build_your_creative_confidence
3011
https://www.ted.com/talks/jean_baptiste_michel_the_mathematics_of_history
3012
h

https://www.ted.com/talks/stephen_coleman_non_lethal_weapons_a_moral_hazard
3109
https://www.ted.com/talks/neil_burgess_how_your_brain_tells_you_where_you_are
3110
https://www.ted.com/talks/nancy_duarte_the_secret_structure_of_great_talks
3111
https://www.ted.com/talks/aleph_molinari_let_s_bridge_the_digital_divide
3112
https://www.ted.com/talks/mike_degruy_hooked_by_an_octopus
3113
https://www.ted.com/talks/danny_hillis_back_to_the_future_of_1994
3114
https://www.ted.com/talks/erica_frenkel_the_universal_anesthesia_machine
3115
https://www.ted.com/talks/shawn_achor_the_happy_secret_to_better_work
3116
https://www.ted.com/talks/bill_doyle_treating_cancer_with_electric_fields
3117
https://www.ted.com/talks/peter_van_uhm_why_i_chose_a_gun
3118
https://www.ted.com/talks/mick_mountz_what_happens_inside_those_massive_warehouses
3119
https://www.ted.com/talks/bjarke_ingels_hedonistic_sustainability
3120
https://www.ted.com/talks/gayle_tzemach_lemmon_women_entrepreneurs_example_not_exception


https://www.ted.com/talks/danielle_de_niese_a_flirtatious_aria
3219
https://www.ted.com/talks/ben_goldacre_battling_bad_science
3220
https://www.ted.com/talks/jarreth_merz_filming_democracy_in_ghana
3221
https://www.ted.com/talks/geoff_mulgan_a_short_intro_to_the_studio_school
3222
https://www.ted.com/talks/abraham_verghese_a_doctor_s_touch
3223
https://www.ted.com/talks/sunni_brown_doodlers_unite
3224
https://www.ted.com/talks/elizabeth_murchison_fighting_a_contagious_cancer
3225
https://www.ted.com/talks/amy_lockwood_selling_condoms_in_the_congo
3226
https://www.ted.com/talks/jean_baptiste_michel_erez_lieberman_aiden_what_we_learned_from_5_million_books
3227
https://www.ted.com/talks/niall_ferguson_the_6_killer_apps_of_prosperity
3228
https://www.ted.com/talks/sasha_dichter
3229
https://www.ted.com/talks/lauren_zalaznick_the_conscience_of_television
3230
https://www.ted.com/talks/richard_resnick_welcome_to_the_genomic_revolution
3231
https://www.ted.com/talks/kate_hartman_the_art_of_

https://www.ted.com/talks/kathryn_schulz_on_being_wrong
3328
https://www.ted.com/talks/sam_richards_a_radical_experiment_in_empathy
3329
https://www.ted.com/talks/susan_lim_transplant_cells_not_organs
3330
https://www.ted.com/talks/marcin_jakubowski_open_sourced_blueprints_for_civilization
3331
https://www.ted.com/talks/roger_ebert_remaking_my_voice
3332
https://www.ted.com/talks/dave_meslin_the_antidote_to_apathy
3333
https://www.ted.com/talks/david_christian_the_history_of_our_world_in_18_minutes
3334
https://www.ted.com/talks/jackson_browne_a_song_inspired_by_the_ocean
3335
https://www.ted.com/talks/caroline_casey_looking_past_limits
3336
https://www.ted.com/talks/mick_ebeling_the_invention_that_unlocked_a_locked_in_artist
3337
https://www.ted.com/talks/morgan_spurlock_the_greatest_ted_talk_ever_sold
3338
https://www.ted.com/talks/chade_meng_tan_everyday_compassion_at_google
3339
https://www.ted.com/talks/stanley_mcchrystal_listen_learn_then_lead
3340
https://www.ted.com/talks/annma

https://www.ted.com/talks/aaron_huey_america_s_native_prisoners_of_war
3435
https://www.ted.com/talks/stefan_wolff_the_path_to_ending_ethnic_conflicts
3436
https://www.ted.com/talks/emily_pilloton_teaching_design_for_change
3437
https://www.ted.com/talks/andrew_bird_a_one_man_orchestra_of_the_imagination
3438
https://www.ted.com/talks/gero_miesenboeck_re_engineering_the_brain
3439
https://www.ted.com/talks/greg_stone_saving_the_ocean_one_island_at_a_time
3440
https://www.ted.com/talks/david_bismark_e_voting_without_fraud
3441
https://www.ted.com/talks/tom_chatfield_7_ways_games_reward_the_brain
3442
https://www.ted.com/talks/miwa_matreyek_glorious_visions_in_animation_and_performance
3443
https://www.ted.com/talks/shimon_steinberg_natural_pest_control_using_bugs
3444
https://www.ted.com/talks/barton_seaver_sustainable_seafood_let_s_get_smart
3445
https://www.ted.com/talks/joseph_nye_global_power_shifts
3446
https://www.ted.com/talks/r_a_mashelkar_breakthrough_designs_for_ultra_low_cost

https://www.ted.com/talks/john_kasaona_how_poachers_became_caretakers
3540
https://www.ted.com/talks/michael_sandel_the_lost_art_of_democratic_debate
3541
https://www.ted.com/talks/adam_sadowsky_how_to_engineer_a_viral_music_video
3542
https://www.ted.com/talks/brian_cox_why_we_need_the_explorers
3543
https://www.ted.com/talks/christopher_moot_poole_the_case_for_anonymity_online
3544
https://www.ted.com/talks/brian_skerry_the_ocean_s_glory_and_horror
3545
https://www.ted.com/talks/john_underkoffler_pointing_to_the_future_of_ui
3546
https://www.ted.com/talks/lawrence_lessig_re_examining_the_remix
3547
https://www.ted.com/talks/sophie_hunger_songs_of_secrets_and_city_lights
3548
https://www.ted.com/talks/seth_berkley_hiv_and_flu_the_vaccine_strategy
3549
https://www.ted.com/talks/sharmeen_obaid_chinoy_inside_a_school_for_suicide_bombers
3550
https://www.ted.com/talks/johanna_blakley_lessons_from_fashion_s_free_culture
3551
https://www.ted.com/talks/sir_ken_robinson_bring_on_the_learning_

https://www.ted.com/talks/charles_fleischer_all_things_are_moleeds
3649
https://www.ted.com/talks/edwidge_danticat_stories_of_haiti
3650
https://www.ted.com/talks/lalitesh_katragadda_making_maps_to_fight_disaster_build_economies
3651
https://www.ted.com/talks/kiran_sethi_kids_take_charge
3652
https://www.ted.com/talks/kartick_satyanarayan_how_we_rescued_the_dancing_bears
3653
https://www.ted.com/talks/matt_weinstein_what_bernie_madoff_couldn_t_steal_from_me
3654
https://www.ted.com/talks/robert_sapolsky_the_uniqueness_of_humans
3655
https://www.ted.com/talks/randy_pausch_really_achieving_your_childhood_dreams
3656
https://www.ted.com/talks/herbie_hancock_an_all_star_set
3657
https://www.ted.com/talks/romulus_whitaker_the_real_danger_lurking_in_the_water
3658
https://www.ted.com/talks/dan_buettner_how_to_live_to_be_100
3659
https://www.ted.com/talks/nick_veasey_exposing_the_invisible
3660
https://www.ted.com/talks/vilayanur_ramachandran_the_neurons_that_shaped_civilization
3661
https://

https://www.ted.com/talks/olafur_eliasson_playing_with_space_and_light
3757
https://www.ted.com/talks/stewart_brand_4_environmental_heresies
3758
https://www.ted.com/talks/kary_mullis_a_next_gen_cure_for_killer_infections
3759
https://www.ted.com/talks/sophal_ear_escaping_the_khmer_rouge
3760
https://www.ted.com/talks/tom_wujec_3_ways_the_brain_creates_meaning
3761
https://www.ted.com/talks/eames_demetrios_the_design_genius_of_charles_ray_eames
3762
https://www.ted.com/talks/daniel_libeskind_17_words_of_architectural_inspiration
3763
https://www.ted.com/talks/gever_tulley_life_lessons_through_tinkering
3764
https://www.ted.com/talks/arthur_benjamin_teach_statistics_before_calculus
3765
https://www.ted.com/talks/ray_zahab_my_trek_to_the_south_pole
3766
https://www.ted.com/talks/katherine_fulton_you_are_the_future_of_philanthropy
3767
https://www.ted.com/talks/paul_collier_new_rules_for_rebuilding_a_broken_nation
3768
https://www.ted.com/talks/philip_zimbardo_the_psychology_of_time
3769


https://www.ted.com/talks/joe_derisi_solving_medical_mysteries
3867
https://www.ted.com/talks/aimee_mullins_changing_my_legs_and_my_mindset
3868
https://www.ted.com/talks/peter_ward_a_theory_of_earth_s_mass_extinctions
3869
https://www.ted.com/talks/woody_norris_hypersonic_sound_and_other_inventions
3870
https://www.ted.com/talks/sherwin_nuland_the_extraordinary_power_of_ordinary_people
3871
https://www.ted.com/talks/barry_schuler_genomics_101
3872
https://www.ted.com/talks/jamais_cascio_tools_for_a_better_world
3873
https://www.ted.com/talks/david_carson_design_and_discovery
3874
https://www.ted.com/talks/paula_scher_great_design_is_serious_not_solemn
3875
https://www.ted.com/talks/joseph_pine_what_consumers_want
3876
https://www.ted.com/talks/peter_reinhart_the_art_and_craft_of_bread
3877
https://www.ted.com/talks/scott_mccloud_the_visual_magic_of_comics
3878
https://www.ted.com/talks/rob_forbes_ways_of_seeing
3879
https://www.ted.com/talks/greg_lynn_organic_algorithms_in_architectur

https://www.ted.com/talks/chris_abani_on_humanity
3979
https://www.ted.com/talks/marisa_fick_jordan_the_wonder_of_zulu_wire_art
3980
https://www.ted.com/talks/martin_seligman_the_new_era_of_positive_psychology
3981
https://www.ted.com/talks/keith_barry_brain_magic
3982
https://www.ted.com/talks/aj_jacobs_my_year_of_living_biblically
3983
https://www.ted.com/talks/billy_graham_on_technology_and_faith
3984
https://www.ted.com/talks/helen_fisher_the_brain_in_love
3985
https://www.ted.com/talks/freeman_dyson_let_s_look_for_life_in_the_outer_solar_system
3986
https://www.ted.com/talks/nellie_mckay_mother_of_pearl_if_i_had_you
3987
https://www.ted.com/talks/clay_shirky_institutions_vs_collaboration
3988
https://www.ted.com/talks/david_hoffman_what_happens_when_you_lose_everything
3989
https://www.ted.com/talks/torsten_reil_animate_characters_by_evolving_them
3990
https://www.ted.com/talks/corneille_ewango_a_hero_of_the_congo_forest
3991
https://www.ted.com/talks/raul_midon_peace_on_earth
399

In [11]:
get_comments(4000, len(talk_df))

https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
4000
https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
4001
https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
4002
https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
4003
https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
4004
https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
4005
https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
4006
https://www.ted.com/talks/rokia_traore_kounandi
4007
https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq
4008
https://www.ted.com/talks/susan_blackmore_memes_and_temes
4009
https://www.ted.com/talks/paul_collier_the_bottom_billion
4010
https://www.ted.com/talks/seyi_oyesola_a_hospital_tour_in_nigeria
4011
https://www.ted.com/talks/arthur_ganson_moving_sculpture
4012
https://www.ted.com/talks/yves_behar_designing_objects_that_tell_stories
4013
ht

https://www.ted.com/talks/jacqueline_novogratz_patient_capitalism
4111
https://www.ted.com/talks/chris_abani_telling_stories_from_africa
4112
https://www.ted.com/talks/patrick_awuah_how_to_educate_leaders_liberal_arts
4113
https://www.ted.com/talks/euvin_naidoo_why_invest_in_africa
4114
https://www.ted.com/talks/william_kamkwamba_how_i_built_a_windmill
4115
https://www.ted.com/talks/ngozi_okonjo_iweala_aid_versus_trade
4116
https://www.ted.com/talks/george_ayittey_africa_s_cheetahs_versus_hippos
4117
https://www.ted.com/talks/allison_hunt_how_to_get_a_new_hip
4118
https://www.ted.com/talks/david_bolinsky_visualizing_the_wonder_of_a_living_cell
4119
https://www.ted.com/talks/rives_the_4_a_m_mystery
4120
https://www.ted.com/talks/will_wright_spore_birth_of_a_game
4121
https://www.ted.com/talks/emily_oster_flip_your_thinking_on_aids_in_africa
4122
https://www.ted.com/talks/jonathan_harris_the_web_s_secret_stories
4123
https://www.ted.com/talks/alan_russell_the_potential_of_regenerative_me

https://www.ted.com/talks/eve_ensler_happiness_in_body_and_soul
4220
https://www.ted.com/talks/helen_fisher_why_we_love_why_we_cheat
4221
https://www.ted.com/talks/mena_trott_meet_the_founder_of_the_blog_revolution
4222
https://www.ted.com/talks/ze_frank_nerdcore_comedy
4223
https://www.ted.com/talks/richard_baraniuk_the_birth_of_the_open_source_learning_revolution
4224
https://www.ted.com/talks/jimmy_wales_the_birth_of_wikipedia
4225
https://www.ted.com/talks/ross_lovegrove_organic_design_inspired_by_nature
4226
https://www.ted.com/talks/amy_smith_simple_designs_to_save_a_life
4227
https://www.ted.com/talks/sirena_huang_an_11_year_old_s_magical_violin
4228
https://www.ted.com/talks/jennifer_lin_improvising_on_piano_aged_14
4229
https://www.ted.com/talks/jeff_han_the_radical_promise_of_the_multi_touch_interface
4230
https://www.ted.com/talks/nicholas_negroponte_one_laptop_per_child
4231
https://www.ted.com/talks/cameron_sinclair_my_wish_a_call_for_open_source_architecture
4232
https://

In [14]:
talk_df.to_csv('full_data.csv')

# Get TED Events

In [4]:
talk_df = pd.read_csv('Data/full_data.csv', index_col = 0)

In [5]:
talk_df.head()

,date,speaker,title,url,length,summ,tags,views,transcript,date_recorded,upload_date,occupation,bio,comments
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so...",599446,So I've been thinking about how to explain thi...,2019-09-24,2020-01-02,"Business executive, ""co-conspirator""",Ipsita Dasgupta drives the consumption of ente...,7
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'...",501290,This chimpanzee stumbles across a windfall of ...,2020-01-02,2020-01-02,NaN,NaN,--
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a...",461705,"Recently, I've been declaring to anyone who wo...",2019-12-04,2020-01-02,Dangerous woman,Pat Mitchell is a lifelong advocate for women ...,14
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ...",880662,"When we're young, we're innocently brave, and ...",2019-12-04,2019-12-23,Human rights and disability activist,Cara E. Yar Khan is an international human rig...,17
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',...",956498,"OK, I have a question for all of us. You ready...",2019-12-04,2019-12-20,Gymnastics coach,Valorie Kondos Field is the retired head coach...,18


In [84]:
# talk_df['event'] = '--'
def get_event(start, end):
    for index in range(start, end):
        if talk_df['event'][index] == '--':
            try:
                # Get the url, request HTML
                url = talk_df['url'][index]
                link = "https://www.ted.com" + url
                print(link)
                print(index)
                page = requests.get(link)
                content = BS(page.content, 'html.parser')

                # Find all scripts, look for initial data about the speakers
                for x in content.find_all('script'):
                    if '__INITIAL_DATA__' in x.get_text():
                        text = x.get_text().split('__INITIAL_DATA__":')[1].strip(')')
                        if 'event' in json.loads(text[0:-1]).keys():
                            talk_df['event'][index] = json.loads(text[0:-1])['event']

                time.sleep(1)
            except:
                print('NO EVENT LISTED')

In [91]:
# Run again to check for additional events that failed to scrape
get_event(0, len(talk_df))

https://www.ted.com/talks/steve_truglia_a_leap_from_the_edge_of_space
3736


In [76]:
get_event(1000, 2000)

https://www.ted.com/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers
1000
https://www.ted.com/talks/clemantine_wamariya_war_and_what_comes_after
1001
https://www.ted.com/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes
1002
https://www.ted.com/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all
1003
https://www.ted.com/talks/zachary_r_wood_why_it_s_worth_listening_to_people_you_disagree_with
1004
https://www.ted.com/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico
1005
https://www.ted.com/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment
1006
https://www.ted.com/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell
1007
https://www.ted.com/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees
1008
https://www.ted.com/talks/mother_london_ai_therapy
1009
https://www.ted.com/talks/duncan_cowles_it_s_not_amazing_enough
1010
https://www.ted.com/talks/aaron_duffy_lake_buc

https://www.ted.com/talks/jason_shen_looking_for_a_job_highlight_your_ability_not_your_experience
1094
https://www.ted.com/talks/margaret_mitchell_how_we_can_build_ai_to_help_humans_not_hurt_us
1095
https://www.ted.com/talks/tanya_menon_the_secret_to_great_opportunities_the_person_you_haven_t_met_yet
1096
https://www.ted.com/talks/amit_kalra_3_creative_ways_to_fix_fashion_s_waste_problem
1097
https://www.ted.com/talks/rowan_fitzpatrick_and_heart_of_mind_facing_the_future_together
1098
https://www.ted.com/talks/shade_zahrai_and_aric_yegudkin_the_rhythmic_beauty_of_collaboration
1099
https://www.ted.com/talks/will_jenkins_the_most_precious_commodity_of_all
1100
https://www.ted.com/talks/tomer_garzberg_what_happens_when_we_take_humans_out_of_work
1101
https://www.ted.com/talks/rach_ranton_where_are_we_trying_to_end_up
1102
https://www.ted.com/talks/olivia_tyler_the_complex_path_to_sustainability
1103
https://www.ted.com/talks/michael_mieni_using_aboriginal_traditions_to_teach_tech
1104
ht

https://www.ted.com/talks/david_katz_the_surprising_solution_to_ocean_plastic
1186
https://www.ted.com/talks/george_steinmetz_photos_of_africa_taken_from_a_flying_lawn_chair
1187
https://www.ted.com/talks/edsel_salvana_the_dangerous_evolution_of_hiv
1188
https://www.ted.com/talks/wendy_woods_the_business_benefits_of_doing_good
1189
https://www.ted.com/talks/naoko_ishii_an_economic_case_for_protecting_the_planet
1190
https://www.ted.com/talks/azim_khamisa_and_ples_felix_what_comes_after_tragedy_forgiveness
1191
https://www.ted.com/talks/cleo_wade_want_to_change_the_world_start_by_being_brave_enough_to_care
1192
https://www.ted.com/talks/anna_rosling_ronnlund_see_how_the_rest_of_the_world_lives_organized_by_income
1193
https://www.ted.com/talks/bob_inglis_american_bipartisan_politics_can_be_saved_here_s_how
1194
https://www.ted.com/talks/scott_williams_the_hidden_role_informal_caregivers_play_in_health_care
1195
https://www.ted.com/talks/michelle_knox_talk_about_your_death_while_you_re_s

https://www.ted.com/talks/david_titley_how_the_military_fights_climate_change
1276
https://www.ted.com/talks/lloyd_pendleton_the_housing_first_approach_to_homelessness
1277
https://www.ted.com/talks/jackson_bird_how_to_talk_and_listen_to_transgender_people
1278
https://www.ted.com/talks/teresa_njoroge_what_i_learned_serving_time_for_a_crime_i_didn_t_commit
1279
https://www.ted.com/talks/lana_yarosh_how_technology_can_reconnect_parents_and_kids
1280
https://www.ted.com/talks/victoria_pratt_how_judges_can_show_respect
1281
https://www.ted.com/talks/robert_muggah_the_biggest_risks_facing_cities_and_some_solutions
1282
https://www.ted.com/talks/jon_bowers_we_should_aim_for_perfection_and_stop_fearing_failure
1283
https://www.ted.com/talks/david_dunning_why_incompetent_people_think_they_re_amazing
1284
https://www.ted.com/talks/jessica_mccabe_failing_at_normal_an_adhd_success_story
1285
https://www.ted.com/talks/ameenah_gurib_fakim_and_stephanie_busari_an_interview_with_mauritius_s_first_fe

https://www.ted.com/talks/wanuri_kahiu_fun_fierce_and_fantastical_african_art
1364
https://www.ted.com/talks/niki_okuk_when_workers_own_companies_the_economy_is_more_resilient
1365
https://www.ted.com/talks/laolu_senbanjo_the_sacred_art_of_the_ori
1366
https://www.ted.com/talks/carolyn_bertozzi_what_the_sugar_coating_on_your_cells_is_trying_to_tell_you
1367
https://www.ted.com/talks/robin_hanson_what_would_happen_if_we_upload_our_brains_to_computers
1368
https://www.ted.com/talks/david_whyte_a_lyrical_bridge_between_past_present_and_future
1369
https://www.ted.com/talks/iyad_rahwan_what_moral_decisions_should_driverless_cars_make
1370
https://www.ted.com/talks/cathy_o_neil_the_era_of_blind_faith_in_big_data_must_end
1371
https://www.ted.com/talks/anika_paulson_how_i_found_myself_through_music
1372
https://www.ted.com/talks/susan_pinker_the_secret_to_living_longer_may_be_your_social_life
1373
https://www.ted.com/talks/richard_j_berry_a_practical_way_to_help_the_homeless_find_work_and_sa

...

https://www.ted.com/talks/robyn_j_crook_how_do_animals_experience_pain
1553
https://www.ted.com/talks/sisonke_msimang_if_a_story_moves_you_act_on_it
1554
https://www.ted.com/talks/eleanor_nelsen_would_you_sacrifice_one_person_to_save_five
1555
https://www.ted.com/talks/alejandro_sanchez_alvarado_to_solve_old_problems_study_new_species
1556
https://www.ted.com/talks/dan_bricklin_meet_the_inventor_of_the_electronic_spreadsheet
1557
https://www.ted.com/talks/george_tulevski_the_next_step_in_nanotechnology
1558
https://www.ted.com/talks/mandy_len_catron_a_better_way_to_talk_about_love
1559
https://www.ted.com/talks/sofia_jawed_wessel_the_lies_we_tell_pregnant_women
1560
https://www.ted.com/talks/sam_kass_want_kids_to_learn_well_feed_them_well
1561
https://www.ted.com/talks/erika_gregory_the_world_doesn_t_need_more_nuclear_weapons
1562
https://www.ted.com/talks/adam_grant_are_you_a_giver_or_a_taker
1563
https://www.ted.com/talks/jennifer_lu_can_you_solve_the_counterfeit_coin_riddle
1564
htt

https://www.ted.com/talks/oded_shoseyov_how_we_re_harnessing_nature_s_hidden_superpowers
1647
https://www.ted.com/talks/camille_a_brown_a_visual_history_of_social_dance_in_25_moves
1648
https://www.ted.com/talks/j_d_vance_america_s_forgotten_working_class
1649
https://www.ted.com/talks/rebecca_mackinnon_we_can_fight_terror_without_sacrificing_our_rights
1650
https://www.ted.com/talks/sebastian_kraves_the_era_of_personal_dna_testing_is_here
1651
https://www.ted.com/talks/nadia_lopez_why_open_a_school_to_close_a_prison
1652
https://www.ted.com/talks/david_burkus_why_you_should_know_how_much_your_coworkers_get_paid
1653
https://www.ted.com/talks/nwhunter_how_do_contraceptives_work
1654
https://www.ted.com/talks/eric_liu_there_s_no_such_thing_as_not_voting
1655
https://www.ted.com/talks/abigail_marsh_why_some_people_are_more_altruistic_than_others
1656
https://www.ted.com/talks/michael_murphy_architecture_that_s_built_to_heal
1657
https://www.ted.com/talks/michael_shellenberger_how_fear_of

https://www.ted.com/talks/adam_driver_my_journey_from_marine_to_actor
1742
https://www.ted.com/talks/joseph_ravenell_how_barbershops_can_keep_men_healthy
1743
https://www.ted.com/talks/zaria_forman_drawings_that_show_the_beauty_and_fragility_of_earth
1744
https://www.ted.com/talks/mariano_sigman_your_words_may_predict_your_future_mental_health
1745
https://www.ted.com/talks/lidia_yuknavitch_the_beauty_of_being_a_misfit
1746
https://www.ted.com/talks/trevor_timm_how_free_is_our_freedom_of_the_press
1747
https://www.ted.com/talks/patrickjmt_the_origin_of_countless_conspiracy_theories
1748
https://www.ted.com/talks/juno_mac_the_laws_that_sex_workers_really_want
1749
https://www.ted.com/talks/sebastian_junger_our_lonely_society_makes_it_hard_to_come_home_from_war
1750
https://www.ted.com/talks/laura_indolfi_good_news_in_the_fight_against_pancreatic_cancer
1751
https://www.ted.com/talks/kang_lee_can_you_really_tell_if_a_kid_is_lying
1752
https://www.ted.com/talks/sara_garofalo_the_psycholog

https://www.ted.com/talks/mike_velings_the_case_for_fish_farming
1835
https://www.ted.com/talks/gregory_heyworth_how_i_m_discovering_the_secrets_of_ancient_texts
1836
https://www.ted.com/talks/sean_follmer_shape_shifting_tech_will_change_work_as_we_know_it
1837
https://www.ted.com/talks/dambisa_moyo_economic_growth_has_stalled_let_s_fix_it
1838
https://www.ted.com/talks/matthew_williams_special_olympics_let_me_be_myself_a_champion
1839
https://www.ted.com/talks/alex_gendler_history_vs_napoleon_bonaparte
1840
https://www.ted.com/talks/pardis_sabeti_how_we_ll_fight_the_next_deadly_virus
1841
https://www.ted.com/talks/judson_brewer_a_simple_way_to_break_a_bad_habit
1842
https://www.ted.com/talks/andres_ruzo_the_boiling_river_of_the_amazon
1843
https://www.ted.com/talks/dennis_shasha_can_you_solve_the_temple_riddle
1844
https://www.ted.com/talks/linda_liukas_a_delightful_way_to_teach_kids_about_computers
1845
https://www.ted.com/talks/melati_and_isabel_wijsen_our_campaign_to_ban_plastic_ba

https://www.ted.com/talks/jared_hill_how_i_leapt_from_a_responsible_no_to_an_impassioned_yes
1928
https://www.ted.com/talks/jack_levis_the_hardest_step_in_innovation_looking_foolish_in_front_of_the_crowd
1929
https://www.ted.com/talks/esther_ndichu_hunger_isn_t_a_food_issue_it_s_a_logistics_issue
1930
https://www.ted.com/talks/alan_amling_the_future_of_delivery_in_our_new_on_demand_economy
1931
https://www.ted.com/talks/daniel_levitin_how_to_stay_calm_when_you_know_you_ll_be_stressed
1932
https://www.ted.com/talks/mathias_jud_art_that_lets_you_talk_back_to_nsa_spies
1933
https://www.ted.com/talks/christine_sun_kim_the_enchanting_music_of_sign_language
1934
https://www.ted.com/talks/cesar_harada_how_i_teach_kids_to_love_science
1935
https://www.ted.com/talks/hilary_cottam_social_services_are_broken_how_we_can_fix_them
1936
https://www.ted.com/talks/francesco_sauro_deep_under_the_earth_s_surface_discovering_beauty_and_science
1937
https://www.ted.com/talks/sharon_horesh_bergquist_how_str

In [77]:
get_event(2000, 3000)

https://www.ted.com/talks/christopher_soghoian_how_to_avoid_surveillance_with_the_phone_in_your_pocket
2000
https://www.ted.com/talks/tony_wyss_coray_how_young_blood_might_help_reverse_aging_yes_really
2001
https://www.ted.com/talks/manuel_lima_a_visual_history_of_human_knowledge
2002
https://www.ted.com/talks/alix_generous_how_i_learned_to_communicate_my_inner_life_with_asperger_s
2003
https://www.ted.com/talks/patience_mthunzi_could_we_cure_hiv_with_lasers
2004
https://www.ted.com/talks/matt_kenyon_a_secret_memorial_for_civilian_casualties
2005
https://www.ted.com/talks/amy_adkins_who_am_i_a_philosophical_inquiry
2006
https://www.ted.com/talks/rich_benjamin_my_road_trip_through_the_whitest_towns_in_america
2007
https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power
2008
https://www.ted.com/talks/murat_dalkilinc_the_benefits_of_good_posture
2009
https://www.ted.com/talks/yuval_noah_harari_what_explains_the_rise_of_humans
2010
https://www.ted.com/ta

https://www.ted.com/talks/dan_ariely_how_equal_do_we_want_the_world_to_be_you_d_be_surprised
2091
https://www.ted.com/talks/eduardo_saenz_de_cabezon_math_is_forever
2092
https://www.ted.com/talks/bel_pesce_5_ways_to_kill_your_dreams
2093
https://www.ted.com/talks/bill_gates_the_next_outbreak_we_re_not_ready
2094
https://www.ted.com/talks/boniface_mwangi_the_day_i_stood_up_alone
2095
https://www.ted.com/talks/kevin_rudd_are_china_and_the_us_doomed_to_conflict
2096
https://www.ted.com/talks/mel_rosenberg_what_causes_bad_breath
2097
https://www.ted.com/talks/daniel_kish_how_i_use_sonar_to_navigate_the_world
2098
https://www.ted.com/talks/alison_killing_there_s_a_better_way_to_die_and_architecture_can_help
2099
https://www.ted.com/talks/dame_stephanie_shirley_why_do_ambitious_women_have_flat_heads
2100
https://www.ted.com/talks/theaster_gates_how_to_revive_a_neighborhood_with_imagination_beauty_and_art
2101
https://www.ted.com/talks/dave_isay_everyone_around_you_has_a_story_the_world_needs

https://www.ted.com/talks/antonella_mei_pochtler_how_better_transportation_can_reinvigorate_a_city
2185
https://www.ted.com/talks/anil_raj_micro_power_plants_to_bring_electricity_to_millions
2186
https://www.ted.com/talks/alison_sander_megatrends_5_tips_on_the_art_and_science_of_trend_tracking
2187
https://www.ted.com/talks/alexander_tuerpitz_virtualizing_employment_in_saudi_arabia
2188
https://www.ted.com/talks/mundano_trash_cart_superheroes
2189
https://www.ted.com/talks/bruno_torturra_got_a_smartphone_start_broadcasting
2190
https://www.ted.com/talks/carol_dweck_the_power_of_believing_that_you_can_improve
2191
https://www.ted.com/talks/jeremy_howard_the_wonderful_and_terrifying_implications_of_computers_that_can_learn
2192
https://www.ted.com/talks/verna_myers_how_to_overcome_our_biases_walk_boldly_toward_them
2193
https://www.ted.com/talks/dave_troy_social_maps_that_reveal_a_city_s_intersections_and_separations
2194
https://www.ted.com/talks/randall_hayes_at_what_moment_are_you_dea

https://www.ted.com/talks/lord_nicholas_stern_the_state_of_the_climate_and_what_we_might_do_about_it
2276
https://www.ted.com/talks/antonio_donato_nobre_the_magic_of_the_amazon_a_river_that_flows_invisibly_all_around_us
2277
https://www.ted.com/talks/avi_reichental_what_s_next_in_3d_printing
2278
https://www.ted.com/talks/viviana_alvarez_don_t_quit_your_job_use_it_for_good
2279
https://www.ted.com/talks/richard_neville_how_can_we_give_our_kids_a_brighter_future
2280
https://www.ted.com/talks/rania_bahaa_watch_what_you_dispose_of
2281
https://www.ted.com/talks/mike_brady_and_dion_drew_why_we_hire_unemployable_people
2282
https://www.ted.com/talks/mark_jackson_a_personal_story_about_farming_and_the_future_of_agriculture
2283
https://www.ted.com/talks/keith_weed_the_power_of_transformative_collaboration
2284
https://www.ted.com/talks/john_kluge_jr_solving_the_sanitation_crisis
2285
https://www.ted.com/talks/jack_sinclair_feeding_the_future
2286
https://www.ted.com/talks/einav_gefen_owning

https://www.ted.com/talks/chris_kluwe_how_augmented_reality_will_change_sports_and_build_empathy
2374
https://www.ted.com/talks/andrew_solomon_how_the_worst_moments_in_our_lives_make_us_who_we_are
2375
https://www.ted.com/talks/edmond_hui_how_the_heart_actually_pumps_blood
2376
https://www.ted.com/talks/jackie_savitz_save_the_oceans_feed_the_world
2377
https://www.ted.com/talks/simon_sinek_why_good_leaders_make_you_feel_safe
2378
https://www.ted.com/talks/rives_the_museum_of_four_in_the_morning
2379
https://www.ted.com/talks/tristram_wyatt_the_smelly_mystery_of_the_human_pheromone
2380
https://www.ted.com/talks/kevin_briggs_the_bridge_between_suicide_and_life
2381
https://www.ted.com/talks/colm_kelleher_the_science_of_symmetry
2382
https://www.ted.com/talks/deborah_gordon_what_ants_teach_us_about_the_brain_cancer_and_the_internet
2383
https://www.ted.com/talks/william_black_how_to_rob_a_bank_from_the_inside_that_is
2384
https://www.ted.com/talks/mark_ronson_how_sampling_transformed_mus

...

https://www.ted.com/talks/mikael_cho_the_science_of_stage_fright_and_how_to_overcome_it
2572
https://www.ted.com/talks/janette_sadik_khan_new_york_s_streets_not_so_mean_any_more
2573
https://www.ted.com/talks/michael_sandel_why_we_shouldn_t_trust_markets_with_our_civic_life
2574
https://www.ted.com/talks/michael_porter_the_case_for_letting_business_solve_social_problems
2575
https://www.ted.com/talks/jason_pontin_can_technology_solve_our_big_problems
2576
https://www.ted.com/talks/fabian_oefner_psychedelic_science
2577
https://www.ted.com/talks/amy_webb_how_i_hacked_online_dating
2578
https://www.ted.com/talks/addison_anderson_the_most_groundbreaking_scientist_you_ve_never_heard_of
2579
https://www.ted.com/talks/kelli_swazey_life_that_doesn_t_end_with_death
2580
https://www.ted.com/talks/amanda_mattes_kabuki_the_people_s_dramatic_art
2581
https://www.ted.com/talks/malcolm_gladwell_the_unheard_story_of_david_and_goliath
2582
https://www.ted.com/talks/kevin_breel_confessions_of_a_depress

https://www.ted.com/talks/denise_herzing_could_we_speak_the_language_of_dolphins
2673
https://www.ted.com/talks/anas_aremeyaw_anas_how_i_named_shamed_and_jailed
2674
https://www.ted.com/talks/alex_laskey_how_behavioral_science_can_lower_your_energy_bill
2675
https://www.ted.com/talks/andrew_solomon_love_no_matter_what
2676
https://www.ted.com/talks/lisa_bu_how_books_can_open_your_mind
2677
https://www.ted.com/talks/k_erica_dodge_gyotaku_the_ancient_japanese_art_of_printing_fish
2678
https://www.ted.com/talks/hendrik_poinar_bring_back_the_woolly_mammoth
2679
https://www.ted.com/talks/brian_jones_what_on_earth_is_spin
2680
https://www.ted.com/talks/jackson_katz_violence_against_women_it_s_a_men_s_issue
2681
https://www.ted.com/talks/ron_shaneyfelt_is_space_trying_to_kill_us
2682
https://www.ted.com/talks/paola_antonelli_why_i_brought_pac_man_to_moma
2683
https://www.ted.com/talks/ji_hae_park_the_violin_and_my_dark_night_of_the_soul
2684
https://www.ted.com/talks/alastair_parvin_architect

https://www.ted.com/talks/edith_widder_how_we_found_the_giant_squid
2777
https://www.ted.com/talks/bill_shillito_how_to_organize_add_and_multiply_matrices
2778
https://www.ted.com/talks/allan_savory_how_to_fight_desertification_and_reverse_climate_change
2779
https://www.ted.com/talks/amanda_palmer_the_art_of_asking
2780
https://www.ted.com/talks/jennifer_granholm_a_clean_energy_proposal_race_to_the_top
2781
https://www.ted.com/talks/sugata_mitra_build_a_school_in_the_cloud
2782
https://www.ted.com/talks/bruno_maisonnier_dance_tiny_robots
2783
https://www.ted.com/talks/asha_de_vos_why_are_blue_whales_so_enormous
2784
https://www.ted.com/talks/bruce_feiler_agile_programming_for_your_family
2785
https://www.ted.com/talks/michael_dickinson_how_a_fly_flies
2786
https://www.ted.com/talks/andreas_schleicher_use_data_to_build_better_schools
2787
https://www.ted.com/talks/afra_raymond_three_myths_about_corruption
2788
https://www.ted.com/talks/keith_chen_could_your_language_affect_your_ability

https://www.ted.com/talks/ryan_merkley_online_video_annotated_remixed_and_popped
2881
https://www.ted.com/talks/heather_brooke_my_battle_to_expose_government_corruption
2882
https://www.ted.com/talks/beau_lotto_amy_o_toole_science_is_for_everyone_kids_included
2883
https://www.ted.com/talks/john_wilbanks_let_s_pool_our_medical_data
2884
https://www.ted.com/talks/eddie_obeng_smart_failure_for_a_fast_changing_world
2885
https://www.ted.com/talks/maurizio_seracini_the_secret_lives_of_paintings
2886
https://www.ted.com/talks/melissa_marshall_talk_nerdy_to_me
2887
https://www.ted.com/talks/ruby_wax_what_s_so_funny_about_mental_illness
2888
https://www.ted.com/talks/john_maeda_how_art_technology_and_design_inform_creative_leaders
2889
https://www.ted.com/talks/tim_leberecht_3_ways_to_usefully_lose_control_of_your_brand
2890
https://www.ted.com/talks/thomas_p_campbell_weaving_narratives_in_museum_galleries
2891
https://www.ted.com/talks/shimon_schocken_the_self_organizing_computer_course
2892

https://www.ted.com/talks/lz_granderson_the_myth_of_the_gay_agenda
2983
https://www.ted.com/talks/sarah_parcak_archaeology_from_space
2984
https://www.ted.com/talks/beeban_kidron_the_shared_wonder_of_film
2985
https://www.ted.com/talks/rebecca_onie_what_if_our_health_care_system_kept_us_healthy
2986
https://www.ted.com/talks/john_hockenberry_we_are_all_designers
2987
https://www.ted.com/talks/ami_klin_a_new_way_to_diagnose_autism
2988
https://www.ted.com/talks/john_hodgman_design_explained
2989
https://www.ted.com/talks/damian_palin_mining_minerals_from_seawater
2990
https://www.ted.com/talks/terry_moore_why_is_x_the_unknown
2991
https://www.ted.com/talks/diane_kelly_what_we_didn_t_know_about_penis_anatomy
2992
https://www.ted.com/talks/juan_enriquez_will_our_kids_be_a_different_species
2993
https://www.ted.com/talks/david_birch_a_new_way_to_stop_identity_theft
2994
https://www.ted.com/talks/seth_shostak_et_is_probably_out_there_get_ready
2995
https://www.ted.com/talks/quixotic_fusion_

In [78]:
get_event(3000, 4000)

https://www.ted.com/talks/ken_goldberg_4_lessons_from_robots_about_being_human
3000
https://www.ted.com/talks/david_mackay_a_reality_check_on_renewables
3001
https://www.ted.com/talks/reggie_watts_beats_that_defy_boxes
3002
https://www.ted.com/talks/shereen_el_feki_hiv_how_to_fight_an_epidemic_of_bad_laws
3003
https://www.ted.com/talks/philippe_petit_the_journey_across_the_high_wire
3004
https://www.ted.com/talks/hans_rosling_religions_and_babies
3005
https://www.ted.com/talks/nathan_wolfe_what_s_left_to_explore
3006
https://www.ted.com/talks/melissa_garren_the_sea_we_ve_hardly_seen
3007
https://www.ted.com/talks/michael_mcdaniel_cheap_effective_shelter_for_disaster_relief
3008
https://www.ted.com/talks/jr_one_year_of_turning_the_world_inside_out
3009
https://www.ted.com/talks/carl_schoonover_how_to_look_inside_the_brain
3010
https://www.ted.com/talks/david_kelley_how_to_build_your_creative_confidence
3011
https://www.ted.com/talks/jean_baptiste_michel_the_mathematics_of_history
3012
h

https://www.ted.com/talks/stephen_coleman_non_lethal_weapons_a_moral_hazard
3109
https://www.ted.com/talks/neil_burgess_how_your_brain_tells_you_where_you_are
3110
https://www.ted.com/talks/nancy_duarte_the_secret_structure_of_great_talks
3111
https://www.ted.com/talks/aleph_molinari_let_s_bridge_the_digital_divide
3112
https://www.ted.com/talks/mike_degruy_hooked_by_an_octopus
3113
https://www.ted.com/talks/danny_hillis_back_to_the_future_of_1994
3114
https://www.ted.com/talks/erica_frenkel_the_universal_anesthesia_machine
3115
https://www.ted.com/talks/shawn_achor_the_happy_secret_to_better_work
3116
https://www.ted.com/talks/bill_doyle_treating_cancer_with_electric_fields
3117
https://www.ted.com/talks/peter_van_uhm_why_i_chose_a_gun
3118
https://www.ted.com/talks/mick_mountz_what_happens_inside_those_massive_warehouses
3119
https://www.ted.com/talks/bjarke_ingels_hedonistic_sustainability
3120
https://www.ted.com/talks/gayle_tzemach_lemmon_women_entrepreneurs_example_not_exception


https://www.ted.com/talks/danielle_de_niese_a_flirtatious_aria
3219
https://www.ted.com/talks/ben_goldacre_battling_bad_science
3220
https://www.ted.com/talks/jarreth_merz_filming_democracy_in_ghana
3221
https://www.ted.com/talks/geoff_mulgan_a_short_intro_to_the_studio_school
3222
https://www.ted.com/talks/abraham_verghese_a_doctor_s_touch
3223
https://www.ted.com/talks/sunni_brown_doodlers_unite
3224
https://www.ted.com/talks/elizabeth_murchison_fighting_a_contagious_cancer
3225
https://www.ted.com/talks/amy_lockwood_selling_condoms_in_the_congo
3226
https://www.ted.com/talks/jean_baptiste_michel_erez_lieberman_aiden_what_we_learned_from_5_million_books
3227
https://www.ted.com/talks/niall_ferguson_the_6_killer_apps_of_prosperity
3228
https://www.ted.com/talks/sasha_dichter
3229
https://www.ted.com/talks/lauren_zalaznick_the_conscience_of_television
3230
https://www.ted.com/talks/richard_resnick_welcome_to_the_genomic_revolution
3231
https://www.ted.com/talks/kate_hartman_the_art_of_

https://www.ted.com/talks/kathryn_schulz_on_being_wrong
3328
https://www.ted.com/talks/sam_richards_a_radical_experiment_in_empathy
3329
https://www.ted.com/talks/susan_lim_transplant_cells_not_organs
3330
https://www.ted.com/talks/marcin_jakubowski_open_sourced_blueprints_for_civilization
3331
https://www.ted.com/talks/roger_ebert_remaking_my_voice
3332
https://www.ted.com/talks/dave_meslin_the_antidote_to_apathy
3333
https://www.ted.com/talks/david_christian_the_history_of_our_world_in_18_minutes
3334
https://www.ted.com/talks/jackson_browne_a_song_inspired_by_the_ocean
3335
https://www.ted.com/talks/caroline_casey_looking_past_limits
3336
https://www.ted.com/talks/mick_ebeling_the_invention_that_unlocked_a_locked_in_artist
3337
https://www.ted.com/talks/morgan_spurlock_the_greatest_ted_talk_ever_sold
3338
https://www.ted.com/talks/chade_meng_tan_everyday_compassion_at_google
3339
https://www.ted.com/talks/stanley_mcchrystal_listen_learn_then_lead
3340
https://www.ted.com/talks/annma

https://www.ted.com/talks/aaron_huey_america_s_native_prisoners_of_war
3435
https://www.ted.com/talks/stefan_wolff_the_path_to_ending_ethnic_conflicts
3436
https://www.ted.com/talks/emily_pilloton_teaching_design_for_change
3437
https://www.ted.com/talks/andrew_bird_a_one_man_orchestra_of_the_imagination
3438
https://www.ted.com/talks/gero_miesenboeck_re_engineering_the_brain
3439
https://www.ted.com/talks/greg_stone_saving_the_ocean_one_island_at_a_time
3440
https://www.ted.com/talks/david_bismark_e_voting_without_fraud
3441
https://www.ted.com/talks/tom_chatfield_7_ways_games_reward_the_brain
3442
https://www.ted.com/talks/miwa_matreyek_glorious_visions_in_animation_and_performance
3443
https://www.ted.com/talks/shimon_steinberg_natural_pest_control_using_bugs
3444
https://www.ted.com/talks/barton_seaver_sustainable_seafood_let_s_get_smart
3445
https://www.ted.com/talks/joseph_nye_global_power_shifts
3446
https://www.ted.com/talks/r_a_mashelkar_breakthrough_designs_for_ultra_low_cost

https://www.ted.com/talks/john_kasaona_how_poachers_became_caretakers
3540
https://www.ted.com/talks/michael_sandel_the_lost_art_of_democratic_debate
3541
https://www.ted.com/talks/adam_sadowsky_how_to_engineer_a_viral_music_video
3542
https://www.ted.com/talks/brian_cox_why_we_need_the_explorers
3543
https://www.ted.com/talks/christopher_moot_poole_the_case_for_anonymity_online
3544
https://www.ted.com/talks/brian_skerry_the_ocean_s_glory_and_horror
3545
https://www.ted.com/talks/john_underkoffler_pointing_to_the_future_of_ui
3546
https://www.ted.com/talks/lawrence_lessig_re_examining_the_remix
3547
https://www.ted.com/talks/sophie_hunger_songs_of_secrets_and_city_lights
3548
https://www.ted.com/talks/seth_berkley_hiv_and_flu_the_vaccine_strategy
3549
https://www.ted.com/talks/sharmeen_obaid_chinoy_inside_a_school_for_suicide_bombers
3550
https://www.ted.com/talks/johanna_blakley_lessons_from_fashion_s_free_culture
3551
https://www.ted.com/talks/sir_ken_robinson_bring_on_the_learning_

https://www.ted.com/talks/charles_fleischer_all_things_are_moleeds
3649
https://www.ted.com/talks/edwidge_danticat_stories_of_haiti
3650
https://www.ted.com/talks/lalitesh_katragadda_making_maps_to_fight_disaster_build_economies
3651
https://www.ted.com/talks/kiran_sethi_kids_take_charge
3652
https://www.ted.com/talks/kartick_satyanarayan_how_we_rescued_the_dancing_bears
3653
https://www.ted.com/talks/matt_weinstein_what_bernie_madoff_couldn_t_steal_from_me
3654
https://www.ted.com/talks/robert_sapolsky_the_uniqueness_of_humans
3655
https://www.ted.com/talks/randy_pausch_really_achieving_your_childhood_dreams
3656
https://www.ted.com/talks/herbie_hancock_an_all_star_set
3657
https://www.ted.com/talks/romulus_whitaker_the_real_danger_lurking_in_the_water
3658
https://www.ted.com/talks/dan_buettner_how_to_live_to_be_100
3659
https://www.ted.com/talks/nick_veasey_exposing_the_invisible
3660
https://www.ted.com/talks/vilayanur_ramachandran_the_neurons_that_shaped_civilization
3661
https://

https://www.ted.com/talks/olafur_eliasson_playing_with_space_and_light
3757
https://www.ted.com/talks/stewart_brand_4_environmental_heresies
3758
https://www.ted.com/talks/kary_mullis_a_next_gen_cure_for_killer_infections
3759
https://www.ted.com/talks/sophal_ear_escaping_the_khmer_rouge
3760
https://www.ted.com/talks/tom_wujec_3_ways_the_brain_creates_meaning
3761
https://www.ted.com/talks/eames_demetrios_the_design_genius_of_charles_ray_eames
3762
https://www.ted.com/talks/daniel_libeskind_17_words_of_architectural_inspiration
3763
https://www.ted.com/talks/gever_tulley_life_lessons_through_tinkering
3764
https://www.ted.com/talks/arthur_benjamin_teach_statistics_before_calculus
3765
https://www.ted.com/talks/ray_zahab_my_trek_to_the_south_pole
3766
https://www.ted.com/talks/katherine_fulton_you_are_the_future_of_philanthropy
3767
https://www.ted.com/talks/paul_collier_new_rules_for_rebuilding_a_broken_nation
3768
https://www.ted.com/talks/philip_zimbardo_the_psychology_of_time
3769


https://www.ted.com/talks/joe_derisi_solving_medical_mysteries
3867
https://www.ted.com/talks/aimee_mullins_changing_my_legs_and_my_mindset
3868
https://www.ted.com/talks/peter_ward_a_theory_of_earth_s_mass_extinctions
3869
https://www.ted.com/talks/woody_norris_hypersonic_sound_and_other_inventions
3870
https://www.ted.com/talks/sherwin_nuland_the_extraordinary_power_of_ordinary_people
3871
https://www.ted.com/talks/barry_schuler_genomics_101
3872
https://www.ted.com/talks/jamais_cascio_tools_for_a_better_world
3873
https://www.ted.com/talks/david_carson_design_and_discovery
3874
https://www.ted.com/talks/paula_scher_great_design_is_serious_not_solemn
3875
https://www.ted.com/talks/joseph_pine_what_consumers_want
3876
https://www.ted.com/talks/peter_reinhart_the_art_and_craft_of_bread
3877
https://www.ted.com/talks/scott_mccloud_the_visual_magic_of_comics
3878
https://www.ted.com/talks/rob_forbes_ways_of_seeing
3879
https://www.ted.com/talks/greg_lynn_organic_algorithms_in_architectur

https://www.ted.com/talks/chris_abani_on_humanity
3979
https://www.ted.com/talks/marisa_fick_jordan_the_wonder_of_zulu_wire_art
3980
https://www.ted.com/talks/martin_seligman_the_new_era_of_positive_psychology
3981
https://www.ted.com/talks/keith_barry_brain_magic
3982
https://www.ted.com/talks/aj_jacobs_my_year_of_living_biblically
3983
https://www.ted.com/talks/billy_graham_on_technology_and_faith
3984
https://www.ted.com/talks/helen_fisher_the_brain_in_love
3985
https://www.ted.com/talks/freeman_dyson_let_s_look_for_life_in_the_outer_solar_system
3986
https://www.ted.com/talks/nellie_mckay_mother_of_pearl_if_i_had_you
3987
https://www.ted.com/talks/clay_shirky_institutions_vs_collaboration
3988
https://www.ted.com/talks/david_hoffman_what_happens_when_you_lose_everything
3989
https://www.ted.com/talks/torsten_reil_animate_characters_by_evolving_them
3990
https://www.ted.com/talks/corneille_ewango_a_hero_of_the_congo_forest
3991
https://www.ted.com/talks/raul_midon_peace_on_earth
399

In [79]:
get_event(4000, len(talk_df))

https://www.ted.com/talks/adam_grosser_a_mobile_fridge_for_vaccines
4000
https://www.ted.com/talks/robert_full_robots_inspired_by_cockroach_ingenuity
4001
https://www.ted.com/talks/dean_ornish_your_genes_are_not_your_fate
4002
https://www.ted.com/talks/chris_jordan_turning_powerful_stats_into_art
4003
https://www.ted.com/talks/george_dyson_the_birth_of_the_computer
4004
https://www.ted.com/talks/murray_gell_mann_the_ancestor_of_language
4005
https://www.ted.com/talks/wade_davis_the_worldwide_web_of_belief_and_ritual
4006
https://www.ted.com/talks/rokia_traore_kounandi
4007
https://www.ted.com/talks/nathan_myhrvold_archeology_animal_photography_bbq
4008
https://www.ted.com/talks/susan_blackmore_memes_and_temes
4009
https://www.ted.com/talks/paul_collier_the_bottom_billion
4010
https://www.ted.com/talks/seyi_oyesola_a_hospital_tour_in_nigeria
4011
https://www.ted.com/talks/arthur_ganson_moving_sculpture
4012
https://www.ted.com/talks/yves_behar_designing_objects_that_tell_stories
4013
ht

https://www.ted.com/talks/jacqueline_novogratz_patient_capitalism
4111
https://www.ted.com/talks/chris_abani_telling_stories_from_africa
4112
https://www.ted.com/talks/patrick_awuah_how_to_educate_leaders_liberal_arts
4113
https://www.ted.com/talks/euvin_naidoo_why_invest_in_africa
4114
https://www.ted.com/talks/william_kamkwamba_how_i_built_a_windmill
4115
https://www.ted.com/talks/ngozi_okonjo_iweala_aid_versus_trade
4116
https://www.ted.com/talks/george_ayittey_africa_s_cheetahs_versus_hippos
4117
https://www.ted.com/talks/allison_hunt_how_to_get_a_new_hip
4118
https://www.ted.com/talks/david_bolinsky_visualizing_the_wonder_of_a_living_cell
4119
https://www.ted.com/talks/rives_the_4_a_m_mystery
4120
https://www.ted.com/talks/will_wright_spore_birth_of_a_game
4121
https://www.ted.com/talks/emily_oster_flip_your_thinking_on_aids_in_africa
4122
https://www.ted.com/talks/jonathan_harris_the_web_s_secret_stories
4123
https://www.ted.com/talks/alan_russell_the_potential_of_regenerative_me

https://www.ted.com/talks/eve_ensler_happiness_in_body_and_soul
4220
https://www.ted.com/talks/helen_fisher_why_we_love_why_we_cheat
4221
https://www.ted.com/talks/mena_trott_meet_the_founder_of_the_blog_revolution
4222
https://www.ted.com/talks/ze_frank_nerdcore_comedy
4223
https://www.ted.com/talks/richard_baraniuk_the_birth_of_the_open_source_learning_revolution
4224
https://www.ted.com/talks/jimmy_wales_the_birth_of_wikipedia
4225
https://www.ted.com/talks/ross_lovegrove_organic_design_inspired_by_nature
4226
https://www.ted.com/talks/amy_smith_simple_designs_to_save_a_life
4227
https://www.ted.com/talks/sirena_huang_an_11_year_old_s_magical_violin
4228
https://www.ted.com/talks/jennifer_lin_improvising_on_piano_aged_14
4229
https://www.ted.com/talks/jeff_han_the_radical_promise_of_the_multi_touch_interface
4230
https://www.ted.com/talks/nicholas_negroponte_one_laptop_per_child
4231
https://www.ted.com/talks/cameron_sinclair_my_wish_a_call_for_open_source_architecture
4232
https://

In [90]:
sum(talk_df.event == '--')

1

In [95]:
talk_df.event.to_csv('talk_events.csv', header = True)